In [1]:
import json
import os
import argparse

import numpy as np
import random
import torch
from torch import nn, optim
from tqdm.auto import tqdm
from transformers import AutoImageProcessor, AutoModelForImageClassification, AutoConfig
from transformers import AutoTokenizer, AutoModelForSequenceClassification

from pathlib import Path
from torch.utils.data import DataLoader, Subset
import sys
sys.path.append('../../lib/exlib/src')
sys.path.insert(0, '../../../github/pytorch-grad-cam/')
from exlib.modules.sop import SOPConfig, SOPTextCls, get_chained_attr, compress_masks_text, AttributionOutputSOP

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

SEED = 42
if SEED != -1:
    # Torch RNG
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    # Python RNG
    np.random.seed(SEED)
    random.seed(SEED)

In [3]:
# model paths
backbone_model_name = '../../pt_models/multirc_vanilla/best'
backbone_processor_name = 'bert-base-uncased'
# sop_config_path = 'configs/imagenet_m.json'

# data paths
# TRAIN_DATA_DIR = '../data/imagenet_m/train'
# VAL_DATA_DIR = '../data/imagenet_m/val'

# training args
batch_size = 2
lr = 0.000005
num_epochs = 20
warmup_steps = 2000
mask_batch_size = 4

# experiment args
exp_dir = '../../exps/multirc/best'

In [4]:
backbone_model = AutoModelForSequenceClassification.from_pretrained(backbone_model_name)
processor = AutoTokenizer.from_pretrained(backbone_processor_name)
backbone_config = AutoConfig.from_pretrained(backbone_model_name)

config = SOPConfig(json_file=os.path.join(exp_dir, 'config.json'),
                   projected_input_scale=2)

In [5]:
from torch.utils.data import DataLoader
from datasets import load_dataset

SENT_SEPS = [processor.convert_tokens_to_ids(processor.tokenize(token)[0]) for token in [';',',','.','?','!',';']]
SEP = processor.convert_tokens_to_ids(processor.tokenize('[SEP]')[0])
print('SEP', SEP, 'SENT_SEPS', SENT_SEPS)

def sent_seg(input_ids):
    segs = []
    count = 1
    for i, input_id in enumerate(input_ids):
        if count in [0, -1]:
            if input_id == SEP:
                count = -1
            segs.append(count)
            continue
        else:
            if input_id in SENT_SEPS:
                segs.append(count)
                count += 1
            elif input_id == SEP:
                if count > 0:
                    count = 0
                    segs.append(count)
                else:
                    segs.append(count)
                    count = -1
            else: # normal character
                segs.append(count)
    return segs

def convert_idx_masks_to_bool_text(masks):
    """
    input: masks (1, seq_len)
    output: masks_bool (num_masks, seq_len)
    """
    unique_idxs = torch.sort(torch.unique(masks)).values
    unique_idxs = unique_idxs[unique_idxs != -1]
    unique_idxs = unique_idxs[unique_idxs != 0]
    idxs = unique_idxs.view(-1, 1)
    broadcasted_masks = masks.expand(unique_idxs.shape[0], 
                                     masks.shape[1])
    masks_bool = (broadcasted_masks == idxs)
    return masks_bool

def get_mask_transform_text(num_masks_max=200, processor=None):
    def mask_transform(mask):
        seg_mask_cut_off = num_masks_max
        # print('mask 1', mask)
        # if mask.max(dim=-1) > seg_mask_cut_off:
        # import pdb; pdb.set_trace()
        if mask.max(dim=-1).values.item() > seg_mask_cut_off:
            mask_new = (mask / (mask.max(dim=-1).values / seg_mask_cut_off)).int().float() + 1
            # bsz, seq_len = mask_new.shape
            # print('mask 2', mask_new)
            # import pdb; pdb.set_trace()
            mask_new[mask == 0] = 0
            mask_new[mask == -1] = -1
            mask = mask_new
        
        if mask.dtype != torch.bool:
            if len(mask.shape) == 1:
                mask = mask.unsqueeze(0)
            # print('mask', mask.shape)
            mask_bool = convert_idx_masks_to_bool_text(mask)
        # print(mask.shape)
        bsz, seq_len = mask.shape
        mask_bool = mask_bool.float()
        
        

        if bsz < seg_mask_cut_off:
            repeat_count = seg_mask_cut_off // bsz + 1
            mask_bool = torch.cat([mask_bool] * repeat_count, dim=0)

        # add additional mask afterwards
        mask_bool_sum = torch.sum(mask_bool[:seg_mask_cut_off - 1], dim=0, keepdim=True).bool()
        if False in mask_bool_sum:
            mask_bool = mask_bool[:seg_mask_cut_off - 1]
            # import pdb; pdb.set_trace()
            compensation_mask = (1 - mask_bool_sum.int()).bool()
            compensation_mask[mask == 0] = False
            compensation_mask[mask == -1] = False
            mask_bool = torch.cat([mask_bool, compensation_mask])
        else:
            mask_bool = mask_bool[:seg_mask_cut_off]
        return mask_bool
    return mask_transform

mask_transform = get_mask_transform_text(config.num_masks_max)

def transform(batch):
    # Preprocess the image using the ViTImageProcessor
    if processor is not None:
        inputs = processor(batch['passage'], 
                           batch['query_and_answer'], 
                           padding='max_length', 
                           truncation=True, 
                           max_length=512)
        segs = [sent_seg(input_id) for input_id in inputs['input_ids']]
        inputs = {k: torch.tensor(v) for k, v in inputs.items()}
        
        segs_bool = []
        for seg in segs:
            seg_bool = mask_transform(torch.tensor(seg))
            segs_bool.append(seg_bool)
        inputs['segs'] = torch.stack(segs_bool)
        # print("inputs['segs']", inputs['segs'].shape)
        # for k, v in inputs.items():
        #     print(k, v.shape)
        # import pdb; pdb.set_trace()
        return inputs
    else:
        return batch


# train_size, val_size = -1, -1
train_size, val_size = 100, 100

train_dataset = load_dataset('eraser_multi_rc', split='train')
train_dataset = train_dataset.map(transform, batched=True,
                            remove_columns=['passage', 
                                            'query_and_answer',
                                            'evidences'])

val_dataset = load_dataset('eraser_multi_rc', split='validation')
val_dataset = val_dataset.map(transform, batched=True,
                            remove_columns=['passage', 
                                            'query_and_answer',
                                            'evidences'])

if train_size != -1:
    train_dataset = Subset(train_dataset, list(range(train_size)))
if val_size != -1:
    val_dataset = Subset(val_dataset, list(range(val_size)))

# Create a DataLoader to batch and shuffle the data
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

SEP 102 SENT_SEPS [1025, 1010, 1012, 1029, 999, 1025]


In [6]:
from collections import namedtuple

WrappedBackboneOutput = namedtuple("WrappedBackboneOutput", 
                                  ["logits",
                                   "pooler_output"])


class WrappedBackboneModel(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
    
    def forward(self, inputs=None, **kwargs):
        outputs = self.model(inputs, output_hidden_states=True, **kwargs)
        return WrappedBackboneOutput(outputs.logits, outputs.hidden_states[-1][:,0])

In [7]:
class WrappedModel(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
    
    def forward(self, **kwargs):
        outputs = self.model(**kwargs)
        return outputs.logits

In [8]:
class WrappedBackboneModel2(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
    
    def forward(self, inputs=None, **kwargs):
        outputs = self.model(inputs, output_hidden_states=True, **kwargs)
        # import pdb; pdb.set_trace()
        return torch.softmax(outputs.logits, dim=-1)
original_model2 = WrappedBackboneModel2(backbone_model)

In [9]:
wrapped_backbone_model = WrappedBackboneModel(backbone_model)
wrapped_backbone_model = wrapped_backbone_model.to(device)
class_weights = get_chained_attr(wrapped_backbone_model, config.finetune_layers[0]).weight #.clone().to(device)
projection_layer = wrapped_backbone_model.model.bert.embeddings.word_embeddings

In [10]:
# model = SOPTextCls(config, wrapped_backbone_model, class_weights=class_weights, projection_layer=projection_layer)
# state_dict = torch.load(os.path.join(exp_dir, 'checkpoint.pth'))
# model.load_state_dict(state_dict['model'])
# model = model.to(device)
# model.eval();

In [11]:
original_model = WrappedModel(backbone_model)
original_model = original_model.to(device)

In [12]:
batch = next(iter(val_dataloader))
print(batch.keys())

dict_keys(['label', 'input_ids', 'token_type_ids', 'attention_mask', 'segs'])


In [13]:
if not isinstance(batch['input_ids'], torch.Tensor):
    inputs = torch.stack(batch['input_ids']).transpose(0, 1).to(device)
    if 'token_type_ids' in batch:
        token_type_ids = torch.stack(batch['token_type_ids']).transpose(0, 1).to(device)
    else:
        token_type_ids = None
    attention_mask = torch.stack(batch['attention_mask']).transpose(0, 1).to(device)

    concatenated_rows = [torch.stack(sublist) for sublist in batch['segs']]
    segs = torch.stack(concatenated_rows).permute(2, 0, 1).to(device).float()
    # print('segs', segs.shape)
else:
    inputs = batch['input_ids'].to(device)
    if 'token_type_ids' in batch:
        token_type_ids = batch['token_type_ids'].to(device)
    else:
        token_type_ids = None
    attention_mask = batch['attention_mask'].to(device)
    segs = batch['segs'].to(device).float()
kwargs = {
    'token_type_ids': token_type_ids,
    'attention_mask': attention_mask,
}
labels = batch['label'].to(device)

In [14]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

def show_attrs(tokens, masks, normalize=True):
    if normalize:
        masks_max = masks.max(-1).values
        masks_min = masks.min(-1).values
        # import pdb; pdb.set_trace()
        masks = (masks - masks_min) / (masks_max - masks_min)
    print_str = ""
    for i in range(len(tokens)):
        expln_val = masks[i].item()
        if expln_val == 0:
            print_str += f" {tokens[i]}"
        else:
            print_str += f"<span style='background-color:rgb({255 * (1-expln_val)},{255 * expln_val},{255 * expln_val})'> {tokens[i]}</span>"
    return print_str
tokens = processor.convert_ids_to_tokens(inputs[0])

## GradCAM

In [15]:
original_model2.model.bert.encoder.layer[-1].attention.output.LayerNorm

LayerNorm((768,), eps=1e-12, elementwise_affine=True)

In [17]:
from exlib.explainers.gradcam import *
from exlib.explainers.libs.saliency.text_wrapper import ExtraDimModelWrapper
from exlib.explainers.libs.pytorch_grad_cam.grad_cam_text import GradCAMText



# class WrappedModelGradCAM(torch.nn.Module):
#     def __init__(self, model): 
#         super(WrappedModelGradCAM, self).__init__()
#         self.model = model
#         for name, param in self.model.named_parameters():
#             param.requires_grad = True
            
#     def forward(self, x):
#         return self.model(x)
    
    
class GradCAMTextCls(FeatureAttrMethod):
    def __init__(self, model, projection_layer, target_layers, reshape_transform=None):
        # model = WrappedModelGradCAM(model)
        wrapped_model = ExtraDimModelWrapper(model).to(next(model.parameters()).device)

        super().__init__(model)
        
        self.projection_layer = projection_layer
        self.wrapped_model = wrapped_model
        for name, param in self.wrapped_model.model.named_parameters():
            param.requires_grad = True
        self.target_layers = target_layers
        with torch.enable_grad():
            self.grad_cam = GradCAMText(model=self.wrapped_model, target_layers=self.target_layers,
                                    reshape_transform=reshape_transform,
                                    use_cuda=True if torch.cuda.is_available() else False)
 
    def forward(self, x, label=None, target_func=ClassifierOutputSoftmaxTarget, mini_batch_size=16):
        grad_cam_results = []
        x_embed = self.projection_layer(x) #.unsqueeze(1)
        
        def get_attr_fn(x, t, xkwargs, **kwargs):
            gradcam = kwargs['gradcam']
            target_func = kwargs['target_func']
            x = x.clone().detach().unsqueeze(1).requires_grad_()
            if t.ndim == 1:
                t = t.unsqueeze(1)
            # print('x', x.shape)
            # print('t', t.shape)
            results = torch.tensor(gradcam(input_tensor=x, targets=[target_func(tt) for tt in t]), device=x.device)
            # print('results', results.shape)
            return results #[:,None]
        
        with torch.enable_grad():
            grad_cam_results, _ = get_explanations_in_minibatches_text(x_embed, label, get_attr_fn, mini_batch_size,
                gradcam=self.grad_cam, target_func=target_func)

        attrs = grad_cam_results
        # print('attrs', attrs.shape)

        if attrs.ndim == 3 and attrs.size(-1) == 1:
            attrs = attrs.squeeze(-1)

        return FeatureAttrOutput(attrs, attrs)

#         for i in range(len(label)):
#             with torch.enable_grad():
#                 grad_cam_result = self.grad_cam(input_tensor=x_embed[i:i+1], 
#                                                 targets=[target_func(label[i:i+1])])
#                 # import pdb; pdb.set_trace()
#                 grad_cam_result = torch.tensor(grad_cam_result)
#                 grad_cam_results.append(grad_cam_result)
#         grad_cam_results = torch.cat(grad_cam_results)

#         return FeatureAttrOutput(grad_cam_results.squeeze(1), {})

gradcam_explainer = GradCAMTextCls(original_model2, projection_layer,
                                  [original_model2.model.bert.encoder.layer[-1].attention.output.LayerNorm])
gradcam_expln = gradcam_explainer(inputs, labels) #, labels) #, x_kwargs=kwargs)
print(gradcam_expln.attributions.shape)
printmd(show_attrs(tokens, gradcam_expln.attributions[0]))

torch.Size([2, 512])


<span style='background-color:rgb(254.9099302639661,0.09006973603391089,0.09006973603391089)'> [CLS]</span> allan sat down at his<span style='background-color:rgb(254.66852472745813,0.3314752725418657,0.3314752725418657)'> desk</span><span style='background-color:rgb(254.66852472745813,0.3314752725418657,0.3314752725418657)'> and</span><span style='background-color:rgb(231.97959266602993,23.02040733397007,23.02040733397007)'> pulled</span> the<span style='background-color:rgb(236.73311479389668,18.266885206103325,18.266885206103325)'> chair</span> in<span style='background-color:rgb(254.968706523141,0.0312934768589912,0.0312934768589912)'> close</span><span style='background-color:rgb(254.52097048109863,0.47902951890137047,0.47902951890137047)'> .</span><span style='background-color:rgb(254.97904794283386,0.020952057166141458,0.020952057166141458)'> opening</span><span style='background-color:rgb(254.97904794283386,0.020952057166141458,0.020952057166141458)'> a</span><span style='background-color:rgb(254.7082412650343,0.29175873496569693,0.29175873496569693)'> side</span><span style='background-color:rgb(252.9376212414354,2.0623787585645914,2.0623787585645914)'> drawer</span> ,<span style='background-color:rgb(247.88716689683497,7.1128331031650305,7.1128331031650305)'> he</span><span style='background-color:rgb(253.16618610173464,1.8338138982653618,1.8338138982653618)'> took</span> out<span style='background-color:rgb(254.65224945859518,0.347750541404821,0.347750541404821)'> a</span> piece<span style='background-color:rgb(243.93114740028977,11.068852599710226,11.068852599710226)'> of</span><span style='background-color:rgb(254.6022291801637,0.39777081983629614,0.39777081983629614)'> paper</span><span style='background-color:rgb(251.51247375179082,3.4875262482091784,3.4875262482091784)'> and</span> his<span style='background-color:rgb(254.4720444548875,0.5279555451124907,0.5279555451124907)'> ink</span><span style='background-color:rgb(250.77103592455387,4.228964075446129,4.228964075446129)'> ##pot</span><span style='background-color:rgb(240.23075111210346,14.769248887896538,14.769248887896538)'> .</span><span style='background-color:rgb(240.23075111210346,14.769248887896538,14.769248887896538)'> after</span><span style='background-color:rgb(251.9367737090215,3.0632262909784913,3.0632262909784913)'> filling</span><span style='background-color:rgb(252.8991266945377,2.1008733054623008,2.1008733054623008)'> his</span> pen<span style='background-color:rgb(247.33268782496452,7.667312175035477,7.667312175035477)'> ,</span><span style='background-color:rgb(253.6180196190253,1.38198038097471,1.38198038097471)'> allan</span><span style='background-color:rgb(248.15558211877942,6.844417881220579,6.844417881220579)'> looked</span><span style='background-color:rgb(249.84476247802377,5.1552375219762325,5.1552375219762325)'> at</span><span style='background-color:rgb(249.47756538167596,5.522434618324041,5.522434618324041)'> his</span><span style='background-color:rgb(251.59137913025916,3.4086208697408438,3.4086208697408438)'> paper</span><span style='background-color:rgb(253.69424756616354,1.3057524338364601,1.3057524338364601)'> in</span><span style='background-color:rgb(253.2126427651383,1.7873572348617017,1.7873572348617017)'> the</span><span style='background-color:rgb(253.2126427651383,1.7873572348617017,1.7873572348617017)'> orange</span><span style='background-color:rgb(254.07731736078858,0.9226826392114162,0.9226826392114162)'> glow</span><span style='background-color:rgb(254.07731736078858,0.9226826392114162,0.9226826392114162)'> from</span> the<span style='background-color:rgb(253.98012748220935,1.0198725177906454,1.0198725177906454)'> lantern</span><span style='background-color:rgb(249.29963090457022,5.700369095429778,5.700369095429778)'> set</span><span style='background-color:rgb(254.42340908339247,0.5765909166075289,0.5765909166075289)'> back</span> in<span style='background-color:rgb(253.08007322251797,1.9199267774820328,1.9199267774820328)'> the</span><span style='background-color:rgb(251.56953813973814,3.4304618602618575,3.4304618602618575)'> desk</span><span style='background-color:rgb(248.210363779217,6.789636220782995,6.789636220782995)'> '</span><span style='background-color:rgb(254.18016347335652,0.8198365266434848,0.8198365266434848)'> s</span><span style='background-color:rgb(254.18016347335652,0.8198365266434848,0.8198365266434848)'> right</span><span style='background-color:rgb(250.33846048638225,4.661539513617754,4.661539513617754)'> -</span><span style='background-color:rgb(250.33846048638225,4.661539513617754,4.661539513617754)'> hand</span><span style='background-color:rgb(251.36531358584762,3.634686414152384,3.634686414152384)'> corner</span><span style='background-color:rgb(252.74929123464972,2.250708765350282,2.250708765350282)'> .</span><span style='background-color:rgb(253.01929358392954,1.9807064160704613,1.9807064160704613)'> his</span> pen<span style='background-color:rgb(253.1769399996847,1.8230600003153086,1.8230600003153086)'> cast</span><span style='background-color:rgb(253.1769399996847,1.8230600003153086,1.8230600003153086)'> a</span><span style='background-color:rgb(252.3567098379135,2.643290162086487,2.643290162086487)'> forbid</span><span style='background-color:rgb(252.99612622708082,2.003873772919178,2.003873772919178)'> ##ding</span> line of<span style='background-color:rgb(250.98768272437155,4.0123172756284475,4.0123172756284475)'> shadow</span><span style='background-color:rgb(250.98768272437155,4.0123172756284475,4.0123172756284475)'> slant</span><span style='background-color:rgb(254.5906482922146,0.4093517077853903,0.4093517077853903)'> ##ing</span> across the<span style='background-color:rgb(253.71162367751822,1.2883763224817812,1.2883763224817812)'> page</span><span style='background-color:rgb(253.0247861891985,1.975213810801506,1.975213810801506)'> ,</span><span style='background-color:rgb(252.60087452363223,2.3991254763677716,2.3991254763677716)'> echoing</span><span style='background-color:rgb(251.56040912959725,3.4395908704027534,3.4395908704027534)'> the</span><span style='background-color:rgb(252.3449822422117,2.6550177577883005,2.6550177577883005)'> ink</span> ##y<span style='background-color:rgb(250.1471317280084,4.8528682719916105,4.8528682719916105)'> darkness</span> crouch ##ing<span style='background-color:rgb(251.61251122131944,3.387488778680563,3.387488778680563)'> in</span><span style='background-color:rgb(253.29626630293205,1.7037336970679462,1.7037336970679462)'> the</span> edges<span style='background-color:rgb(245.9277711622417,9.072228837758303,9.072228837758303)'> of</span> the<span style='background-color:rgb(254.89173367866897,0.10826632133102976,0.10826632133102976)'> lantern</span><span style='background-color:rgb(251.9454973284155,3.054502671584487,3.054502671584487)'> '</span><span style='background-color:rgb(251.58235413953662,3.417645860463381,3.417645860463381)'> s</span><span style='background-color:rgb(253.98709535831586,1.0129046416841447,1.0129046416841447)'> struggling</span><span style='background-color:rgb(253.98709535831586,1.0129046416841447,1.0129046416841447)'> glow</span> . the<span style='background-color:rgb(250.74682504869998,4.253174951300025,4.253174951300025)'> only</span><span style='background-color:rgb(250.74682504869998,4.253174951300025,4.253174951300025)'> other</span><span style='background-color:rgb(252.40948140621185,2.590518593788147,2.590518593788147)'> illumination</span><span style='background-color:rgb(252.40948140621185,2.590518593788147,2.590518593788147)'> came</span><span style='background-color:rgb(250.90646730735898,4.09353269264102,4.09353269264102)'> from</span><span style='background-color:rgb(247.43841857649386,7.561581423506141,7.561581423506141)'> a</span> lu<span style='background-color:rgb(251.3737631449476,3.6262368550524116,3.6262368550524116)'> ##rid</span><span style='background-color:rgb(254.72242424148135,0.27757575851865113,0.27757575851865113)'> moonlight</span><span style='background-color:rgb(254.147997724358,0.8520022756420076,0.8520022756420076)'> filtered</span><span style='background-color:rgb(254.02868774835952,0.9713122516404837,0.9713122516404837)'> through</span><span style='background-color:rgb(254.02868774835952,0.9713122516404837,0.9713122516404837)'> thin</span> branches and clouds<span style='background-color:rgb(245.3341755643487,9.665824435651302,9.665824435651302)'> ,</span><span style='background-color:rgb(250.56502000428736,4.434979995712638,4.434979995712638)'> casting</span><span style='background-color:rgb(245.74552059173584,9.25447940826416,9.25447940826416)'> its</span><span style='background-color:rgb(253.8594895368442,1.140510463155806,1.140510463155806)'> bone</span> -<span style='background-color:rgb(244.79639137163758,10.203608628362417,10.203608628362417)'> pale</span><span style='background-color:rgb(253.12787893228233,1.8721210677176714,1.8721210677176714)'> glow</span><span style='background-color:rgb(254.05355462338775,0.946445376612246,0.946445376612246)'> onto</span><span style='background-color:rgb(250.40326410904527,4.596735890954733,4.596735890954733)'> the</span> pine floor<span style='background-color:rgb(234.60968233644962,20.390317663550377,20.390317663550377)'> ##boards</span><span style='background-color:rgb(251.3714680681005,3.628531931899488,3.628531931899488)'> .</span><span style='background-color:rgb(250.9953986853361,4.004601314663887,4.004601314663887)'> allan</span><span style='background-color:rgb(254.82495517877396,0.17504482122603804,0.17504482122603804)'> unfolded</span><span style='background-color:rgb(251.47110370919108,3.528896290808916,3.528896290808916)'> another</span><span style='background-color:rgb(254.00528112309985,0.9947188769001514,0.9947188769001514)'> page</span><span style='background-color:rgb(253.0942035955377,1.9057964044623077,1.9057964044623077)'> ,</span><span style='background-color:rgb(244.19220669195056,10.80779330804944,10.80779330804944)'> this</span><span style='background-color:rgb(251.54819682240486,3.4518031775951385,3.4518031775951385)'> one</span><span style='background-color:rgb(254.20611305278726,0.7938869472127408,0.7938869472127408)'> crowded</span><span style='background-color:rgb(251.2718507135287,3.7281492864713073,3.7281492864713073)'> with</span><span style='background-color:rgb(254.48785932501778,0.5121406749822199,0.5121406749822199)'> ranks</span> of<span style='background-color:rgb(253.37392368586734,1.6260763141326606,1.6260763141326606)'> letters</span> in tight<span style='background-color:rgb(248.4384584147483,6.561541585251689,6.561541585251689)'> formation</span><span style='background-color:rgb(248.4384584147483,6.561541585251689,6.561541585251689)'> from</span><span style='background-color:rgb(254.55921462737024,0.4407853726297617,0.4407853726297617)'> left</span><span style='background-color:rgb(254.5912801270606,0.4087198729394004,0.4087198729394004)'> to</span><span style='background-color:rgb(249.65242962352931,5.347570376470685,5.347570376470685)'> right</span><span style='background-color:rgb(254.18565714731812,0.8143428526818752,0.8143428526818752)'> .</span><span style='background-color:rgb(248.3542359340936,6.6457640659064054,6.6457640659064054)'> the</span><span style='background-color:rgb(250.63949410803616,4.3605058919638395,4.3605058919638395)'> lines</span> of<span style='background-color:rgb(247.89004381746054,7.109956182539463,7.109956182539463)'> letters</span> stepped into<span style='background-color:rgb(254.22820756095462,0.7717924390453845,0.7717924390453845)'> their</span> divisions , in<span style='background-color:rgb(244.00302339345217,10.996976606547832,10.996976606547832)'> the</span><span style='background-color:rgb(243.66756219416857,11.332437805831432,11.332437805831432)'> shape</span><span style='background-color:rgb(242.37725783139467,12.622742168605328,12.622742168605328)'> of</span><span style='background-color:rgb(254.97457094068523,0.02542905931477435,0.02542905931477435)'> a</span><span style='background-color:rgb(254.4986543082632,0.5013456917367876,0.5013456917367876)'> story</span><span style='background-color:rgb(253.02838518982753,1.9716148101724684,1.9716148101724684)'> '</span><span style='background-color:rgb(252.59427926503122,2.4057207349687815,2.4057207349687815)'> s</span> outline<span style='background-color:rgb(254.33900872478262,0.6609912752173841,0.6609912752173841)'> :</span><span style='background-color:rgb(254.33900872478262,0.6609912752173841,0.6609912752173841)'> the</span><span style='background-color:rgb(251.57527773175389,3.4247222682461143,3.4247222682461143)'> loose</span><span style='background-color:rgb(254.43983560195193,0.560164398048073,0.560164398048073)'> ,</span> dry<span style='background-color:rgb(254.66408852487803,0.33591147512197495,0.33591147512197495)'> skeleton</span> of<span style='background-color:rgb(249.12546107545495,5.87453892454505,5.87453892454505)'> a</span><span style='background-color:rgb(253.396425365936,1.603574634063989,1.603574634063989)'> tale</span><span style='background-color:rgb(253.57956983847544,1.420430161524564,1.420430161524564)'> lay</span><span style='background-color:rgb(252.42250663228333,2.57749336771667,2.57749336771667)'> exposed</span><span style='background-color:rgb(253.95510250004008,1.0448974999599159,1.0448974999599159)'> beneath</span><span style='background-color:rgb(251.52827971614897,3.4717202838510275,3.4717202838510275)'> their</span><span style='background-color:rgb(254.0764713124372,0.9235286875627935,0.9235286875627935)'> feet</span> , awaiting tend ##ons ,<span style='background-color:rgb(250.12635396793485,4.873646032065153,4.873646032065153)'> muscle</span><span style='background-color:rgb(254.16207911563106,0.8379208843689412,0.8379208843689412)'> and</span><span style='background-color:rgb(251.42030304763466,3.579696952365339,3.579696952365339)'> blushing</span> skin .<span style='background-color:rgb(253.28734770650044,1.7126522934995592,1.7126522934995592)'> allan</span> reviewed<span style='background-color:rgb(253.86053923051804,1.139460769481957,1.139460769481957)'> the</span><span style='background-color:rgb(254.5708331570495,0.429166842950508,0.429166842950508)'> troops</span><span style='background-color:rgb(254.9734393658582,0.026560634141787887,0.026560634141787887)'> ,</span><span style='background-color:rgb(252.18392598442733,2.816074015572667,2.816074015572667)'> all</span><span style='background-color:rgb(243.1602058187127,11.839794181287289,11.839794181287289)'> prepared</span><span style='background-color:rgb(253.71193775441498,1.2880622455850244,1.2880622455850244)'> to</span><span style='background-color:rgb(252.31131106149405,2.6886889385059476,2.6886889385059476)'> di</span><span style='background-color:rgb(254.73028319946025,0.26971680053975433,0.26971680053975433)'> ##se</span><span style='background-color:rgb(253.1864425772801,1.8135574227198958,1.8135574227198958)'> ##mba</span><span style='background-color:rgb(253.55864400509745,1.4413559949025512,1.4413559949025512)'> ##rk</span> , their<span style='background-color:rgb(252.43967696093023,2.5603230390697718,2.5603230390697718)'> task</span><span style='background-color:rgb(251.70223176945,3.297768230549991,3.297768230549991)'> to</span><span style='background-color:rgb(250.70335158146918,4.296648418530822,4.296648418530822)'> form</span><span style='background-color:rgb(253.16228133626282,1.8377186637371778,1.8377186637371778)'> the</span><span style='background-color:rgb(249.105742033571,5.894257966428995,5.894257966428995)'> tale</span> of<span style='background-color:rgb(254.4840681378264,0.515931862173602,0.515931862173602)'> a</span><span style='background-color:rgb(0.0,255.0,255.0)'> young</span> man<span style='background-color:rgb(250.2301244996488,4.769875500351191,4.769875500351191)'> returning</span> home from<span style='background-color:rgb(253.50263584172353,1.4973641582764685,1.4973641582764685)'> life</span><span style='background-color:rgb(252.40311437286437,2.5968856271356344,2.5968856271356344)'> abroad</span> to<span style='background-color:rgb(254.08708259928972,0.9129174007102847,0.9129174007102847)'> find</span><span style='background-color:rgb(254.58804528345354,0.411954716546461,0.411954716546461)'> his</span><span style='background-color:rgb(241.1409504711628,13.859049528837204,13.859049528837204)'> childhood</span> friend<span style='background-color:rgb(254.35898371855728,0.6410162814427167,0.6410162814427167)'> a</span><span style='background-color:rgb(254.10184908192605,0.8981509180739522,0.8981509180739522)'> bride</span><span style='background-color:rgb(254.10184908192605,0.8981509180739522,0.8981509180739522)'> to</span><span style='background-color:rgb(254.45755440741777,0.5424455925822258,0.5424455925822258)'> -</span><span style='background-color:rgb(254.97319573918503,0.026804260814969894,0.026804260814969894)'> be</span><span style='background-color:rgb(245.7651712372899,9.234828762710094,9.234828762710094)'> ,</span><span style='background-color:rgb(245.7651712372899,9.234828762710094,9.234828762710094)'> thus</span><span style='background-color:rgb(254.70272695936728,0.2972730406327173,0.2972730406327173)'> upset</span><span style='background-color:rgb(254.9505078598304,0.04949214016960468,0.04949214016960468)'> ##ting</span><span style='background-color:rgb(252.4648679466918,2.535132053308189,2.535132053308189)'> the</span><span style='background-color:rgb(254.47279764572158,0.5272023542784154,0.5272023542784154)'> apple</span><span style='background-color:rgb(254.38627367606387,0.6137263239361346,0.6137263239361346)'> cart</span><span style='background-color:rgb(254.38627367606387,0.6137263239361346,0.6137263239361346)'> of</span><span style='background-color:rgb(253.26834730105475,1.731652698945254,1.731652698945254)'> his</span><span style='background-color:rgb(253.26834730105475,1.731652698945254,1.731652698945254)'> life</span><span style='background-color:rgb(254.676983904792,0.3230160952080041,0.3230160952080041)'> '</span> s<span style='background-color:rgb(246.07965756207705,8.920342437922955,8.920342437922955)'> plan</span> ,<span style='background-color:rgb(254.89468160798424,0.10531839201576076,0.10531839201576076)'> clarified</span><span style='background-color:rgb(254.89468160798424,0.10531839201576076,0.10531839201576076)'> –</span><span style='background-color:rgb(252.0054949214682,2.9945050785318017,2.9945050785318017)'> of</span> course<span style='background-color:rgb(235.34582443535328,19.65417556464672,19.65417556464672)'> –</span><span style='background-color:rgb(246.1892712302506,8.810728769749403,8.810728769749403)'> by</span><span style='background-color:rgb(252.91029833257198,2.0897016674280167,2.0897016674280167)'> his</span> very<span style='background-color:rgb(253.34918380714953,1.6508161928504705,1.6508161928504705)'> time</span><span style='background-color:rgb(253.13783962279558,1.8621603772044182,1.8621603772044182)'> away</span> from her he<span style='background-color:rgb(254.7597383579705,0.24026164202950895,0.24026164202950895)'> loved</span><span style='background-color:rgb(254.77822442539036,0.22177557460963726,0.22177557460963726)'> best</span><span style='background-color:rgb(254.5379940723069,0.4620059276930988,0.4620059276930988)'> .</span><span style='background-color:rgb(248.71473209001124,6.285267909988761,6.285267909988761)'> although</span><span style='background-color:rgb(251.8616446154192,3.138355384580791,3.138355384580791)'> the</span><span style='background-color:rgb(235.72513908147812,19.27486091852188,19.27486091852188)'> concept</span><span style='background-color:rgb(254.55609342141543,0.44390657858457416,0.44390657858457416)'> was</span><span style='background-color:rgb(254.08757485100068,0.9124251489993185,0.9124251489993185)'> a</span><span style='background-color:rgb(251.63292847573757,3.3670715242624283,3.3670715242624283)'> simple</span><span style='background-color:rgb(252.9148303018883,2.085169698111713,2.085169698111713)'> one</span><span style='background-color:rgb(252.72990348748863,2.2700965125113726,2.2700965125113726)'> ,</span><span style='background-color:rgb(253.4122869023122,1.5877130976878107,1.5877130976878107)'> allan</span> thought<span style='background-color:rgb(250.44509701430798,4.554902985692024,4.554902985692024)'> it</span><span style='background-color:rgb(251.9654597947374,3.0345402052626014,3.0345402052626014)'> had</span><span style='background-color:rgb(245.103829074651,9.896170925348997,9.896170925348997)'> potential</span><span style='background-color:rgb(254.26792314858176,0.7320768514182419,0.7320768514182419)'> .</span><span style='background-color:rgb(254.93537525719148,0.06462474280851893,0.06462474280851893)'> besides</span><span style='background-color:rgb(253.89149652561173,1.1085034743882716,1.1085034743882716)'> ,</span><span style='background-color:rgb(254.8010601164424,0.19893988355761394,0.19893988355761394)'> the</span><span style='background-color:rgb(254.8010601164424,0.19893988355761394,0.19893988355761394)'> public</span><span style='background-color:rgb(253.66826895391569,1.3317310460843146,1.3317310460843146)'> liked</span><span style='background-color:rgb(254.35333229368553,0.6466677063144743,0.6466677063144743)'> a</span><span style='background-color:rgb(253.85800809133798,1.1419919086620212,1.1419919086620212)'> good</span><span style='background-color:rgb(254.68429100175854,0.3157089982414618,0.3157089982414618)'> ,</span> simple romance<span style='background-color:rgb(254.90444203736843,0.0955579626315739,0.0955579626315739)'> .</span><span style='background-color:rgb(249.6205517090857,5.379448290914297,5.379448290914297)'> perhaps</span><span style='background-color:rgb(250.26076225563884,4.739237744361162,4.739237744361162)'> this</span><span style='background-color:rgb(249.18990799225867,5.810092007741332,5.810092007741332)'> will</span><span style='background-color:rgb(251.06509240809828,3.9349075919017196,3.9349075919017196)'> be</span><span style='background-color:rgb(248.48540822044015,6.514591779559851,6.514591779559851)'> more</span><span style='background-color:rgb(245.28808118775487,9.71191881224513,9.71191881224513)'> sale</span><span style='background-color:rgb(253.32702244631946,1.6729775536805391,1.6729775536805391)'> ##able</span><span style='background-color:rgb(254.21710182563402,0.7828981743659824,0.7828981743659824)'> ,</span><span style='background-color:rgb(249.96683567762375,5.033164322376251,5.033164322376251)'> he</span> thought and<span style='background-color:rgb(252.0906920498237,2.9093079501762986,2.9093079501762986)'> began</span><span style='background-color:rgb(240.09394325315952,14.906056746840477,14.906056746840477)'> to</span> write .<span style='background-color:rgb(231.08825739473104,23.911742605268955,23.911742605268955)'> they</span> gazed<span style='background-color:rgb(253.0786330997944,1.9213669002056122,1.9213669002056122)'> at</span><span style='background-color:rgb(254.14618670591153,0.853813294088468,0.853813294088468)'> each</span><span style='background-color:rgb(245.44376833364367,9.556231666356325,9.556231666356325)'> other</span><span style='background-color:rgb(246.92662166431546,8.073378335684538,8.073378335684538)'> ,</span><span style='background-color:rgb(254.12639908958226,0.8736009104177356,0.8736009104177356)'> lost</span> in<span style='background-color:rgb(250.38110844790936,4.618891552090645,4.618891552090645)'> the</span><span style='background-color:rgb(250.24606701917946,4.753932980820537,4.753932980820537)'> rap</span> ##ture of love based<span style='background-color:rgb(253.03515725769103,1.9648427423089743,1.9648427423089743)'> so</span><span style='background-color:rgb(253.21772166760638,1.7822783323936164,1.7822783323936164)'> deeply</span><span style='background-color:rgb(253.24871719814837,1.7512828018516302,1.7512828018516302)'> within</span><span style='background-color:rgb(253.24871719814837,1.7512828018516302,1.7512828018516302)'> their</span> hearts<span style='background-color:rgb(253.84432930033654,1.1556706996634603,1.1556706996634603)'> that</span><span style='background-color:rgb(251.46175811067224,3.5382418893277645,3.5382418893277645)'> they</span><span style='background-color:rgb(206.668349057436,48.33165094256401,48.33165094256401)'> had</span> never<span style='background-color:rgb(252.64796682167798,2.352033178322017,2.352033178322017)'> seen</span> it<span style='background-color:rgb(247.96053763478994,7.039462365210056,7.039462365210056)'> before</span> .<span style='background-color:rgb(250.48256965354085,4.51743034645915,4.51743034645915)'> "</span><span style='background-color:rgb(254.0806658717338,0.9193341282662004,0.9193341282662004)'> what</span><span style='background-color:rgb(254.15029434487224,0.8497056551277637,0.8497056551277637)'> about</span> roger ? "<span style='background-color:rgb(249.55517574213445,5.444824257865548,5.444824257865548)'> she</span> asked<span style='background-color:rgb(248.95368796773255,6.046312032267451,6.046312032267451)'> ,</span> knowing that<span style='background-color:rgb(253.63082825671881,1.3691717432811856,1.3691717432811856)'> the</span><span style='background-color:rgb(253.7052141339518,1.294785866048187,1.294785866048187)'> answer</span> no longer mattered<span style='background-color:rgb(254.40690995310433,0.5930900468956679,0.5930900468956679)'> .</span> [SEP]<span style='background-color:rgb(248.01870491355658,6.981295086443424,6.981295086443424)'> name</span><span style='background-color:rgb(229.4201659038663,25.57983409613371,25.57983409613371)'> few</span><span style='background-color:rgb(254.63578862312716,0.364211376872845,0.364211376872845)'> objects</span><span style='background-color:rgb(253.47780797397718,1.5221920260228217,1.5221920260228217)'> said</span><span style='background-color:rgb(254.7748637283803,0.22513627161970362,0.22513627161970362)'> to</span><span style='background-color:rgb(251.91942787729204,3.080572122707963,3.080572122707963)'> be</span><span style='background-color:rgb(251.91942787729204,3.080572122707963,3.080572122707963)'> in</span><span style='background-color:rgb(254.56967944395728,0.4303205560427159,0.4303205560427159)'> or</span><span style='background-color:rgb(249.84176253899932,5.158237461000681,5.158237461000681)'> on</span><span style='background-color:rgb(250.56997493840754,4.43002506159246,4.43002506159246)'> allan</span> '<span style='background-color:rgb(254.69883472134825,0.3011652786517516,0.3011652786517516)'> s</span><span style='background-color:rgb(250.70765770040452,4.292342299595475,4.292342299595475)'> desk</span><span style='background-color:rgb(254.95522568518936,0.044774314810638316,0.044774314810638316)'> |</span><span style='background-color:rgb(254.95522568518936,0.044774314810638316,0.044774314810638316)'> |</span><span style='background-color:rgb(253.8924475433305,1.1075524566695094,1.1075524566695094)'> erase</span><span style='background-color:rgb(254.03790937853046,0.9620906214695424,0.9620906214695424)'> ##r</span> [SEP] [PAD]<span style='background-color:rgb(252.3524592909962,2.647540709003806,2.647540709003806)'> [PAD]</span><span style='background-color:rgb(252.3524592909962,2.647540709003806,2.647540709003806)'> [PAD]</span><span style='background-color:rgb(252.90147758089006,2.0985224191099405,2.0985224191099405)'> [PAD]</span> [PAD] [PAD] [PAD] [PAD]<span style='background-color:rgb(254.47860967135057,0.5213903286494315,0.5213903286494315)'> [PAD]</span><span style='background-color:rgb(253.0963282752782,1.9036717247217894,1.9036717247217894)'> [PAD]</span><span style='background-color:rgb(251.85855443123728,3.1414455687627196,3.1414455687627196)'> [PAD]</span><span style='background-color:rgb(254.98667795845904,0.013322041540959617,0.013322041540959617)'> [PAD]</span> [PAD]<span style='background-color:rgb(253.8476645713672,1.1523354286327958,1.1523354286327958)'> [PAD]</span><span style='background-color:rgb(253.06305916048586,1.9369408395141363,1.9369408395141363)'> [PAD]</span><span style='background-color:rgb(248.65454806946218,6.34545193053782,6.34545193053782)'> [PAD]</span> [PAD]<span style='background-color:rgb(254.7298583050724,0.2701416949275881,0.2701416949275881)'> [PAD]</span> [PAD]<span style='background-color:rgb(253.8749550632201,1.1250449367798865,1.1250449367798865)'> [PAD]</span><span style='background-color:rgb(250.7168698310852,4.283130168914795,4.283130168914795)'> [PAD]</span> [PAD]<span style='background-color:rgb(254.20330571592785,0.7966942840721458,0.7966942840721458)'> [PAD]</span><span style='background-color:rgb(254.9511858488404,0.04881415115960408,0.04881415115960408)'> [PAD]</span><span style='background-color:rgb(254.9511858488404,0.04881415115960408,0.04881415115960408)'> [PAD]</span><span style='background-color:rgb(254.935070197389,0.06492980261100456,0.06492980261100456)'> [PAD]</span><span style='background-color:rgb(254.935070197389,0.06492980261100456,0.06492980261100456)'> [PAD]</span> [PAD]<span style='background-color:rgb(250.25816604495049,4.741833955049515,4.741833955049515)'> [PAD]</span><span style='background-color:rgb(248.88799661770463,6.11200338229537,6.11200338229537)'> [PAD]</span><span style='background-color:rgb(254.18585467734374,0.8141453226562589,0.8141453226562589)'> [PAD]</span><span style='background-color:rgb(251.77815312054008,3.2218468794599175,3.2218468794599175)'> [PAD]</span><span style='background-color:rgb(246.26762302592397,8.732376974076033,8.732376974076033)'> [PAD]</span><span style='background-color:rgb(254.04638636740856,0.9536136325914413,0.9536136325914413)'> [PAD]</span><span style='background-color:rgb(254.04638636740856,0.9536136325914413,0.9536136325914413)'> [PAD]</span><span style='background-color:rgb(251.99112931732088,3.0088706826791167,3.0088706826791167)'> [PAD]</span><span style='background-color:rgb(247.8233873192221,7.176612680777907,7.176612680777907)'> [PAD]</span> [PAD] [PAD]<span style='background-color:rgb(247.38288170658052,7.61711829341948,7.61711829341948)'> [PAD]</span><span style='background-color:rgb(254.9963327041587,0.0036672958412964363,0.0036672958412964363)'> [PAD]</span><span style='background-color:rgb(252.81844966113567,2.1815503388643265,2.1815503388643265)'> [PAD]</span> [PAD]<span style='background-color:rgb(251.53007440734655,3.4699255926534534,3.4699255926534534)'> [PAD]</span><span style='background-color:rgb(246.07858791947365,8.921412080526352,8.921412080526352)'> [PAD]</span><span style='background-color:rgb(251.5349229471758,3.465077052824199,3.465077052824199)'> [PAD]</span><span style='background-color:rgb(254.63845694088377,0.3615430591162294,0.3615430591162294)'> [PAD]</span> [PAD] [PAD]<span style='background-color:rgb(243.7489148788154,11.251085121184587,11.251085121184587)'> [PAD]</span><span style='background-color:rgb(248.3160346839577,6.683965316042304,6.683965316042304)'> [PAD]</span><span style='background-color:rgb(254.6751470596064,0.3248529403936118,0.3248529403936118)'> [PAD]</span><span style='background-color:rgb(254.6751470596064,0.3248529403936118,0.3248529403936118)'> [PAD]</span><span style='background-color:rgb(254.69764879904687,0.30235120095312595,0.30235120095312595)'> [PAD]</span> [PAD]<span style='background-color:rgb(254.8265237227315,0.17347627726849169,0.17347627726849169)'> [PAD]</span> [PAD] [PAD] [PAD]<span style='background-color:rgb(254.81374607712496,0.18625392287503928,0.18625392287503928)'> [PAD]</span><span style='background-color:rgb(254.81374607712496,0.18625392287503928,0.18625392287503928)'> [PAD]</span> [PAD] [PAD]<span style='background-color:rgb(252.99544487614185,2.004555123858154,2.004555123858154)'> [PAD]</span><span style='background-color:rgb(247.88360506296158,7.116394937038422,7.116394937038422)'> [PAD]</span><span style='background-color:rgb(254.9997888063399,0.00021119366010680096,0.00021119366010680096)'> [PAD]</span><span style='background-color:rgb(254.9997888063399,0.00021119366010680096,0.00021119366010680096)'> [PAD]</span><span style='background-color:rgb(253.9258059533313,1.0741940466687083,1.0741940466687083)'> [PAD]</span><span style='background-color:rgb(253.84348574560136,1.156514254398644,1.156514254398644)'> [PAD]</span><span style='background-color:rgb(253.2233543903567,1.7766456096433103,1.7766456096433103)'> [PAD]</span><span style='background-color:rgb(253.59874467831105,1.40125532168895,1.40125532168895)'> [PAD]</span><span style='background-color:rgb(251.52795412112027,3.472045878879726,3.472045878879726)'> [PAD]</span><span style='background-color:rgb(251.52795412112027,3.472045878879726,3.472045878879726)'> [PAD]</span><span style='background-color:rgb(252.18348877038807,2.8165112296119332,2.8165112296119332)'> [PAD]</span><span style='background-color:rgb(252.06880664918572,2.931193350814283,2.931193350814283)'> [PAD]</span><span style='background-color:rgb(252.33677230775356,2.663227692246437,2.663227692246437)'> [PAD]</span><span style='background-color:rgb(245.32482996582985,9.67517003417015,9.67517003417015)'> [PAD]</span><span style='background-color:rgb(254.91400184197119,0.08599815802881494,0.08599815802881494)'> [PAD]</span><span style='background-color:rgb(254.91400184197119,0.08599815802881494,0.08599815802881494)'> [PAD]</span> [PAD] [PAD] [PAD]<span style='background-color:rgb(250.620531225577,4.379468774423003,4.379468774423003)'> [PAD]</span> [PAD] [PAD]<span style='background-color:rgb(254.9969967245579,0.0030032754420972196,0.0030032754420972196)'> [PAD]</span><span style='background-color:rgb(254.556305022561,0.44369497743900865,0.44369497743900865)'> [PAD]</span> [PAD]<span style='background-color:rgb(250.81572437658906,4.18427562341094,4.18427562341094)'> [PAD]</span><span style='background-color:rgb(251.70635098591447,3.2936490140855312,3.2936490140855312)'> [PAD]</span><span style='background-color:rgb(254.88473271019757,0.11526728980243206,0.11526728980243206)'> [PAD]</span><span style='background-color:rgb(247.8438724949956,7.156127505004406,7.156127505004406)'> [PAD]</span><span style='background-color:rgb(249.8563124332577,5.143687566742301,5.143687566742301)'> [PAD]</span><span style='background-color:rgb(247.58925005793571,7.410749942064285,7.410749942064285)'> [PAD]</span><span style='background-color:rgb(252.41422117687762,2.585778823122382,2.585778823122382)'> [PAD]</span><span style='background-color:rgb(249.07846947200596,5.921530527994037,5.921530527994037)'> [PAD]</span><span style='background-color:rgb(253.15567907178774,1.8443209282122552,1.8443209282122552)'> [PAD]</span><span style='background-color:rgb(254.82360890804557,0.1763910919544287,0.1763910919544287)'> [PAD]</span><span style='background-color:rgb(254.82360890804557,0.1763910919544287,0.1763910919544287)'> [PAD]</span> [PAD]<span style='background-color:rgb(252.15137123130262,2.848628768697381,2.848628768697381)'> [PAD]</span> [PAD] [PAD] [PAD]<span style='background-color:rgb(254.98609733922422,0.01390266077578417,0.01390266077578417)'> [PAD]</span><span style='background-color:rgb(253.9365403773263,1.0634596226736903,1.0634596226736903)'> [PAD]</span><span style='background-color:rgb(252.0712404185906,2.9287595814093947,2.9287595814093947)'> [PAD]</span> [PAD] [PAD]<span style='background-color:rgb(254.95847911588498,0.04152088411501609,0.04152088411501609)'> [PAD]</span><span style='background-color:rgb(251.42966099549085,3.570339004509151,3.570339004509151)'> [PAD]</span><span style='background-color:rgb(251.75595019944012,3.2440498005598783,3.2440498005598783)'> [PAD]</span><span style='background-color:rgb(251.75595019944012,3.2440498005598783,3.2440498005598783)'> [PAD]</span><span style='background-color:rgb(254.2938398953993,0.7061601046007127,0.7061601046007127)'> [PAD]</span> [PAD]<span style='background-color:rgb(251.36010952759534,3.639890472404659,3.639890472404659)'> [PAD]</span><span style='background-color:rgb(251.07175630051643,3.9282436994835734,3.9282436994835734)'> [PAD]</span><span style='background-color:rgb(252.98261272720993,2.0173872727900743,2.0173872727900743)'> [PAD]</span><span style='background-color:rgb(253.8743058918044,1.125694108195603,1.125694108195603)'> [PAD]</span><span style='background-color:rgb(254.80434864701238,0.1956513529876247,0.1956513529876247)'> [PAD]</span><span style='background-color:rgb(240.21291771903634,14.78708228096366,14.78708228096366)'> [PAD]</span> [PAD]<span style='background-color:rgb(251.84369722846895,3.1563027715310454,3.1563027715310454)'> [PAD]</span><span style='background-color:rgb(249.74880812689662,5.25119187310338,5.25119187310338)'> [PAD]</span><span style='background-color:rgb(254.52533992007375,0.47466007992625237,0.47466007992625237)'> [PAD]</span><span style='background-color:rgb(254.81208994489862,0.18791005510138348,0.18791005510138348)'> [PAD]</span> [PAD]<span style='background-color:rgb(254.99695701988912,0.0030429801108766696,0.0030429801108766696)'> [PAD]</span><span style='background-color:rgb(254.91587211348815,0.0841278865118511,0.0841278865118511)'> [PAD]</span><span style='background-color:rgb(249.8819605819881,5.118039418011904,5.118039418011904)'> [PAD]</span><span style='background-color:rgb(251.53669483959675,3.4633051604032516,3.4633051604032516)'> [PAD]</span><span style='background-color:rgb(252.96560507733375,2.0343949226662517,2.0343949226662517)'> [PAD]</span><span style='background-color:rgb(251.1082887277007,3.8917112722992897,3.8917112722992897)'> [PAD]</span><span style='background-color:rgb(254.8047691033571,0.19523089664289728,0.19523089664289728)'> [PAD]</span><span style='background-color:rgb(245.5885006673634,9.411499332636595,9.411499332636595)'> [PAD]</span><span style='background-color:rgb(250.23769369348884,4.762306306511164,4.762306306511164)'> [PAD]</span><span style='background-color:rgb(254.94360678072553,0.05639321927446872,0.05639321927446872)'> [PAD]</span><span style='background-color:rgb(254.34153095819056,0.6584690418094397,0.6584690418094397)'> [PAD]</span><span style='background-color:rgb(251.9068393902853,3.093160609714687,3.093160609714687)'> [PAD]</span> [PAD]<span style='background-color:rgb(254.54697616689373,0.4530238331062719,0.4530238331062719)'> [PAD]</span><span style='background-color:rgb(244.93589043617249,10.064109563827515,10.064109563827515)'> [PAD]</span><span style='background-color:rgb(254.91815178334946,0.08184821665054187,0.08184821665054187)'> [PAD]</span><span style='background-color:rgb(254.3479637161363,0.6520362838637084,0.6520362838637084)'> [PAD]</span><span style='background-color:rgb(238.04935086518526,16.95064913481474,16.95064913481474)'> [PAD]</span> [PAD]<span style='background-color:rgb(248.81982210092247,6.180177899077535,6.180177899077535)'> [PAD]</span><span style='background-color:rgb(253.20821802131832,1.7917819786816835,1.7917819786816835)'> [PAD]</span> [PAD]<span style='background-color:rgb(248.1699249241501,6.830075075849891,6.830075075849891)'> [PAD]</span><span style='background-color:rgb(254.6295536326943,0.37044636730570346,0.37044636730570346)'> [PAD]</span><span style='background-color:rgb(252.86358340177685,2.1364165982231498,2.1364165982231498)'> [PAD]</span><span style='background-color:rgb(250.89945525862277,4.1005447413772345,4.1005447413772345)'> [PAD]</span><span style='background-color:rgb(251.77197037730366,3.22802962269634,3.22802962269634)'> [PAD]</span><span style='background-color:rgb(251.19765969458967,3.8023403054103255,3.8023403054103255)'> [PAD]</span><span style='background-color:rgb(248.81876623257995,6.1812337674200535,6.1812337674200535)'> [PAD]</span> [PAD]

## FullGrad

In [27]:
# from exlib.explainers.fullgrad import *
# from exlib.explainers.libs.fullgrad.saliency.fullgradbert import FullGradBERT
# from exlib.explainers.libs.saliency.text_wrapper import ExtraDimModelWrapper

# def explain_text_cls_with_fullgrad(fullgrad, x, label, model_type='vit'):
#     """
#     Explain a classification model with Integrated Gradients.
#     """
#     assert x.size(0) == len(label)
#     if label.ndim == 1:
#         label = label.unsqueeze(1)

#     # Obtain saliency maps
#     def get_attr_fn(x, t, fullgrad):
#         x = x.clone().detach().requires_grad_()
#         return fullgrad.saliency(x, t)

#     attrs, _ = get_explanations_in_minibatches(x, label, get_attr_fn, mini_batch_size=16, show_pbar=False,
#         fullgrad=fullgrad)
    
#     # print('attrs', attrs.shape)
    
#     if attrs.ndim == 5 and attrs.size(-1) == 1:
#         attrs = attrs.squeeze(-1)
#     attrs = attrs.squeeze(1).sum(2)
#     # return FeatureAttrOutput(attrs, {})

#     return FeatureAttrOutput(attrs, {})

# class FullGradTextCls(FeatureAttrMethod):
#     """ Image classification with integrated gradients
#     """
#     def __init__(self, model, projection_layer, im_size=(1, 512, 768), model_type='bert', check_completeness=False):
#         super().__init__(model)
#         print('init fullgrad')
#         # model_type = bert
#         self.wrapped_model = ExtraDimModelWrapper(model).to(next(model.parameters()).device)
#         self.fullgrad = FullGradBERT(self.wrapped_model, im_size=im_size, check_completeness=check_completeness)
#         self.projection_layer = projection_layer
        
#         print('init fullgrad done')

#     def forward(self, x, t, return_groups=False, **kwargs):
#         if not isinstance(t, torch.Tensor):
#             t = torch.tensor(t)
#         print('start fullgrad')

#         x_embed = self.projection_layer(x).unsqueeze(1)
#         with torch.enable_grad():
#             return explain_text_cls_with_fullgrad(self.fullgrad, x_embed, t, **kwargs)

In [28]:
from exlib.explainers.fullgrad import FullGradTextCls

fullgrad_explainer = FullGradTextCls(original_model2, projection_layer)
fullgrad_expln = fullgrad_explainer(inputs, labels) #, labels) #, x_kwargs=kwargs)

init fullgrad
init fullgrad done
start fullgrad


In [29]:
fullgrad_expln.attributions.shape

torch.Size([2, 512])

In [30]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

def show_attrs(tokens, masks, normalize=True):
    if normalize:
        masks_max = masks.max(-1).values
        masks_min = masks.min(-1).values
        # import pdb; pdb.set_trace()
        masks = (masks - masks_min) / (masks_max - masks_min)
    print_str = ""
    for i in range(len(tokens)):
        expln_val = masks[i].item()
        if expln_val == 0:
            print_str += f" {tokens[i]}"
        else:
            print_str += f"<span style='background-color:rgb({255 * (1-expln_val)},{255 * expln_val},{255 * expln_val})'> {tokens[i]}</span>"
    return print_str

tokens = processor.convert_ids_to_tokens(inputs[0])
# tokens = shap_expln.explainer_output.data[0]
printmd(show_attrs(tokens, fullgrad_expln.attributions[0]))

<span style='background-color:rgb(195.41996084153652,59.58003915846348,59.58003915846348)'> [CLS]</span><span style='background-color:rgb(241.39473980292678,13.605260197073221,13.605260197073221)'> allan</span><span style='background-color:rgb(252.4326480505988,2.5673519494012,2.5673519494012)'> sat</span><span style='background-color:rgb(253.8630111166276,1.1369888833723962,1.1369888833723962)'> down</span><span style='background-color:rgb(253.1026915088296,1.8973084911704063,1.8973084911704063)'> at</span><span style='background-color:rgb(253.4287830046378,1.5712169953621924,1.5712169953621924)'> his</span><span style='background-color:rgb(250.83479080349207,4.165209196507931,4.165209196507931)'> desk</span><span style='background-color:rgb(253.5432056710124,1.4567943289875984,1.4567943289875984)'> and</span><span style='background-color:rgb(253.29722955124453,1.702770448755473,1.702770448755473)'> pulled</span><span style='background-color:rgb(252.6762156933546,2.3237843066453934,2.3237843066453934)'> the</span><span style='background-color:rgb(252.4953518109396,2.50464818906039,2.50464818906039)'> chair</span><span style='background-color:rgb(254.22100665047765,0.7789933495223522,0.7789933495223522)'> in</span><span style='background-color:rgb(254.05688235419802,0.9431176458019763,0.9431176458019763)'> close</span><span style='background-color:rgb(248.43036864884198,6.569631351158023,6.569631351158023)'> .</span><span style='background-color:rgb(252.3815407929942,2.618459207005799,2.618459207005799)'> opening</span><span style='background-color:rgb(253.57052727369592,1.429472726304084,1.429472726304084)'> a</span><span style='background-color:rgb(254.3887458590325,0.6112541409675032,0.6112541409675032)'> side</span><span style='background-color:rgb(250.6952751148492,4.30472488515079,4.30472488515079)'> drawer</span><span style='background-color:rgb(253.79594910889864,1.2040508911013603,1.2040508911013603)'> ,</span><span style='background-color:rgb(253.60220747999847,1.3977925200015306,1.3977925200015306)'> he</span><span style='background-color:rgb(254.22044065897353,0.7795593410264701,0.7795593410264701)'> took</span><span style='background-color:rgb(253.57001441996545,1.429985580034554,1.429985580034554)'> out</span><span style='background-color:rgb(253.81169772008434,1.1883022799156606,1.1883022799156606)'> a</span><span style='background-color:rgb(252.99258671700954,2.0074132829904556,2.0074132829904556)'> piece</span><span style='background-color:rgb(253.5734816151671,1.4265183848328888,1.4265183848328888)'> of</span><span style='background-color:rgb(252.5608636206016,2.4391363793984056,2.4391363793984056)'> paper</span><span style='background-color:rgb(253.59030556865036,1.4096944313496351,1.4096944313496351)'> and</span><span style='background-color:rgb(253.48640762502328,1.5135923749767244,1.5135923749767244)'> his</span><span style='background-color:rgb(251.52353092096746,3.4764690790325403,3.4764690790325403)'> ink</span><span style='background-color:rgb(252.02091995626688,2.97908004373312,2.97908004373312)'> ##pot</span><span style='background-color:rgb(249.82129731215537,5.178702687844634,5.178702687844634)'> .</span><span style='background-color:rgb(251.96680824737996,3.0331917526200414,3.0331917526200414)'> after</span><span style='background-color:rgb(252.21427329350263,2.785726706497371,2.785726706497371)'> filling</span><span style='background-color:rgb(252.76249813847244,2.237501861527562,2.237501861527562)'> his</span><span style='background-color:rgb(249.5019339490682,5.498066050931811,5.498066050931811)'> pen</span><span style='background-color:rgb(252.39361179526895,2.606388204731047,2.606388204731047)'> ,</span><span style='background-color:rgb(242.1402028016746,12.859797198325396,12.859797198325396)'> allan</span><span style='background-color:rgb(250.22872474975884,4.77127525024116,4.77127525024116)'> looked</span><span style='background-color:rgb(253.9904468972236,1.0095531027764082,1.0095531027764082)'> at</span><span style='background-color:rgb(253.19057865533978,1.8094213446602225,1.8094213446602225)'> his</span><span style='background-color:rgb(251.0343473078683,3.965652692131698,3.965652692131698)'> paper</span><span style='background-color:rgb(254.27065193653107,0.7293480634689331,0.7293480634689331)'> in</span><span style='background-color:rgb(253.76976376399398,1.2302362360060215,1.2302362360060215)'> the</span><span style='background-color:rgb(252.85477713681757,2.1452228631824255,2.1452228631824255)'> orange</span><span style='background-color:rgb(250.66422270610929,4.335777293890715,4.335777293890715)'> glow</span><span style='background-color:rgb(254.5472877798602,0.4527122201398015,0.4527122201398015)'> from</span><span style='background-color:rgb(254.18070482555777,0.8192951744422317,0.8192951744422317)'> the</span><span style='background-color:rgb(248.5935266688466,6.406473331153393,6.406473331153393)'> lantern</span><span style='background-color:rgb(254.5379179873271,0.4620820126729086,0.4620820126729086)'> set</span><span style='background-color:rgb(254.58123533637263,0.4187646636273712,0.4187646636273712)'> back</span><span style='background-color:rgb(254.64730387553573,0.35269612446427345,0.35269612446427345)'> in</span><span style='background-color:rgb(254.3114686931949,0.6885313068050891,0.6885313068050891)'> the</span><span style='background-color:rgb(250.7848709821701,4.215129017829895,4.215129017829895)'> desk</span> '<span style='background-color:rgb(254.43828801624477,0.5617119837552309,0.5617119837552309)'> s</span><span style='background-color:rgb(254.28269824013114,0.7173017598688602,0.7173017598688602)'> right</span><span style='background-color:rgb(253.5747550218366,1.4252449781633914,1.4252449781633914)'> -</span><span style='background-color:rgb(254.66407935193274,0.3359206480672583,0.3359206480672583)'> hand</span><span style='background-color:rgb(251.27634349744767,3.7236565025523305,3.7236565025523305)'> corner</span><span style='background-color:rgb(252.3811097536236,2.6188902463763952,2.6188902463763952)'> .</span><span style='background-color:rgb(253.7260954384692,1.2739045615307987,1.2739045615307987)'> his</span><span style='background-color:rgb(249.8653397988528,5.134660201147199,5.134660201147199)'> pen</span><span style='background-color:rgb(251.97663665749133,3.0233633425086737,3.0233633425086737)'> cast</span><span style='background-color:rgb(254.25387512426823,0.7461248757317662,0.7461248757317662)'> a</span><span style='background-color:rgb(249.6884692646563,5.311530735343695,5.311530735343695)'> forbid</span><span style='background-color:rgb(252.8365594893694,2.1634405106306076,2.1634405106306076)'> ##ding</span><span style='background-color:rgb(252.56469500251114,2.4353049974888563,2.4353049974888563)'> line</span><span style='background-color:rgb(254.05538309714757,0.9446169028524309,0.9446169028524309)'> of</span><span style='background-color:rgb(252.20787016209215,2.7921298379078507,2.7921298379078507)'> shadow</span><span style='background-color:rgb(245.94383670017123,9.056163299828768,9.056163299828768)'> slant</span><span style='background-color:rgb(253.64673503441736,1.353264965582639,1.353264965582639)'> ##ing</span><span style='background-color:rgb(252.77387995272875,2.2261200472712517,2.2261200472712517)'> across</span><span style='background-color:rgb(253.26999771874398,1.73000228125602,1.73000228125602)'> the</span><span style='background-color:rgb(248.7591797299683,6.240820270031691,6.240820270031691)'> page</span><span style='background-color:rgb(253.89102143235505,1.1089785676449537,1.1089785676449537)'> ,</span><span style='background-color:rgb(245.50465531647205,9.495344683527946,9.495344683527946)'> echoing</span><span style='background-color:rgb(253.63255277043208,1.3674472295679152,1.3674472295679152)'> the</span><span style='background-color:rgb(251.1559571698308,3.844042830169201,3.844042830169201)'> ink</span><span style='background-color:rgb(253.31170736812055,1.688292631879449,1.688292631879449)'> ##y</span><span style='background-color:rgb(250.72053853422403,4.279461465775967,4.279461465775967)'> darkness</span><span style='background-color:rgb(251.67271139100194,3.3272886089980602,3.3272886089980602)'> crouch</span><span style='background-color:rgb(253.84099972900003,1.159000270999968,1.159000270999968)'> ##ing</span><span style='background-color:rgb(253.99088945472613,1.0091105452738702,1.0091105452738702)'> in</span><span style='background-color:rgb(252.93244425673038,2.0675557432696223,2.0675557432696223)'> the</span><span style='background-color:rgb(250.26975732296705,4.7302426770329475,4.7302426770329475)'> edges</span><span style='background-color:rgb(253.51222403347492,1.4877759665250778,1.4877759665250778)'> of</span><span style='background-color:rgb(252.98918661195785,2.010813388042152,2.010813388042152)'> the</span><span style='background-color:rgb(246.19528060778975,8.804719392210245,8.804719392210245)'> lantern</span><span style='background-color:rgb(252.27536594029516,2.72463405970484,2.72463405970484)'> '</span><span style='background-color:rgb(250.36078001372516,4.6392199862748384,4.6392199862748384)'> s</span><span style='background-color:rgb(250.1898238621652,4.810176137834787,4.810176137834787)'> struggling</span><span style='background-color:rgb(247.05200591124594,7.947994088754058,7.947994088754058)'> glow</span><span style='background-color:rgb(247.80537106096745,7.194628939032555,7.194628939032555)'> .</span><span style='background-color:rgb(250.7934100739658,4.206589926034212,4.206589926034212)'> the</span><span style='background-color:rgb(251.8748232582584,3.1251767417415977,3.1251767417415977)'> only</span><span style='background-color:rgb(252.23861834965646,2.7613816503435373,2.7613816503435373)'> other</span><span style='background-color:rgb(245.99787645041943,9.002123549580574,9.002123549580574)'> illumination</span><span style='background-color:rgb(248.67616463452578,6.323835365474224,6.323835365474224)'> came</span><span style='background-color:rgb(249.95397099293768,5.046029007062316,5.046029007062316)'> from</span><span style='background-color:rgb(249.5326405763626,5.46735942363739,5.46735942363739)'> a</span><span style='background-color:rgb(251.28167817369103,3.7183218263089657,3.7183218263089657)'> lu</span><span style='background-color:rgb(247.1423284895718,7.85767151042819,7.85767151042819)'> ##rid</span><span style='background-color:rgb(245.8987283706665,9.101271629333496,9.101271629333496)'> moonlight</span><span style='background-color:rgb(248.22866739705205,6.77133260294795,6.77133260294795)'> filtered</span><span style='background-color:rgb(249.726644391194,5.273355608806014,5.273355608806014)'> through</span><span style='background-color:rgb(244.85285064205527,10.147149357944727,10.147149357944727)'> thin</span><span style='background-color:rgb(248.50099688395858,6.499003116041422,6.499003116041422)'> branches</span><span style='background-color:rgb(252.70431684795767,2.2956831520423293,2.2956831520423293)'> and</span><span style='background-color:rgb(250.27131381444633,4.72868618555367,4.72868618555367)'> clouds</span><span style='background-color:rgb(250.71324007585645,4.286759924143553,4.286759924143553)'> ,</span><span style='background-color:rgb(249.12281214259565,5.877187857404351,5.877187857404351)'> casting</span><span style='background-color:rgb(250.21234477870166,4.787655221298337,4.787655221298337)'> its</span><span style='background-color:rgb(249.89236489869654,5.107635101303458,5.107635101303458)'> bone</span><span style='background-color:rgb(252.88204304873943,2.1179569512605667,2.1179569512605667)'> -</span><span style='background-color:rgb(250.5229890346527,4.47701096534729,4.47701096534729)'> pale</span><span style='background-color:rgb(250.9211929421872,4.07880705781281,4.07880705781281)'> glow</span><span style='background-color:rgb(251.2213716097176,3.7786283902823925,3.7786283902823925)'> onto</span><span style='background-color:rgb(251.5879859123379,3.412014087662101,3.412014087662101)'> the</span><span style='background-color:rgb(249.26061317324638,5.739386826753616,5.739386826753616)'> pine</span><span style='background-color:rgb(252.38070507533848,2.619294924661517,2.619294924661517)'> floor</span><span style='background-color:rgb(249.48256401345134,5.517435986548662,5.517435986548662)'> ##boards</span><span style='background-color:rgb(245.09426213800907,9.905737861990929,9.905737861990929)'> .</span><span style='background-color:rgb(226.06328163295984,28.936718367040157,28.936718367040157)'> allan</span><span style='background-color:rgb(236.84355966746807,18.15644033253193,18.15644033253193)'> unfolded</span><span style='background-color:rgb(248.5227887146175,6.477211285382509,6.477211285382509)'> another</span><span style='background-color:rgb(249.03584288433194,5.964157115668058,5.964157115668058)'> page</span><span style='background-color:rgb(252.6811093557626,2.318890644237399,2.318890644237399)'> ,</span><span style='background-color:rgb(252.26803138386458,2.7319686161354184,2.7319686161354184)'> this</span><span style='background-color:rgb(251.1913061980158,3.808693801984191,3.808693801984191)'> one</span><span style='background-color:rgb(251.12129542976618,3.878704570233822,3.878704570233822)'> crowded</span><span style='background-color:rgb(254.33087015524507,0.6691298447549343,0.6691298447549343)'> with</span><span style='background-color:rgb(252.93135015293956,2.068649847060442,2.068649847060442)'> ranks</span><span style='background-color:rgb(253.75847290735692,1.2415270926430821,1.2415270926430821)'> of</span><span style='background-color:rgb(250.86846127174795,4.131538728252053,4.131538728252053)'> letters</span><span style='background-color:rgb(254.06193002709188,0.9380699729081243,0.9380699729081243)'> in</span><span style='background-color:rgb(253.6444937484339,1.355506251566112,1.355506251566112)'> tight</span><span style='background-color:rgb(252.3856471851468,2.6143528148531914,2.6143528148531914)'> formation</span><span style='background-color:rgb(254.4609462004155,0.5390537995845079,0.5390537995845079)'> from</span><span style='background-color:rgb(254.00326527189463,0.9967347281053662,0.9967347281053662)'> left</span><span style='background-color:rgb(254.57170434936415,0.42829565063584596,0.42829565063584596)'> to</span><span style='background-color:rgb(253.73975629918277,1.2602437008172274,1.2602437008172274)'> right</span><span style='background-color:rgb(250.5455735977739,4.45442640222609,4.45442640222609)'> .</span><span style='background-color:rgb(252.08631064742804,2.9136893525719643,2.9136893525719643)'> the</span><span style='background-color:rgb(252.73897098843008,2.261029011569917,2.261029011569917)'> lines</span><span style='background-color:rgb(254.1647526284214,0.8352473715785891,0.8352473715785891)'> of</span><span style='background-color:rgb(249.9418914411217,5.058108558878303,5.058108558878303)'> letters</span><span style='background-color:rgb(251.71078321058303,3.289216789416969,3.289216789416969)'> stepped</span><span style='background-color:rgb(254.42958321771584,0.5704167822841555,0.5704167822841555)'> into</span><span style='background-color:rgb(253.33477711770684,1.6652228822931647,1.6652228822931647)'> their</span><span style='background-color:rgb(251.447932315059,3.552067684940994,3.552067684940994)'> divisions</span><span style='background-color:rgb(252.71819370333105,2.2818062966689467,2.2818062966689467)'> ,</span><span style='background-color:rgb(254.08445664332248,0.9155433566775173,0.9155433566775173)'> in</span><span style='background-color:rgb(253.4535106527619,1.5464893472380936,1.5464893472380936)'> the</span><span style='background-color:rgb(251.5592518541962,3.440748145803809,3.440748145803809)'> shape</span><span style='background-color:rgb(254.5397750189295,0.46022498107049614,0.46022498107049614)'> of</span><span style='background-color:rgb(254.03284763451666,0.9671523654833436,0.9671523654833436)'> a</span><span style='background-color:rgb(251.882313368842,3.1176866311579943,3.1176866311579943)'> story</span><span style='background-color:rgb(254.7534906025976,0.24650939740240574,0.24650939740240574)'> '</span><span style='background-color:rgb(254.41800387343392,0.5819961265660822,0.5819961265660822)'> s</span><span style='background-color:rgb(249.3757707439363,5.6242292560637,5.6242292560637)'> outline</span><span style='background-color:rgb(249.72044597379863,5.279554026201367,5.279554026201367)'> :</span><span style='background-color:rgb(252.95058638323098,2.049413616769016,2.049413616769016)'> the</span><span style='background-color:rgb(251.1751865129918,3.824813487008214,3.824813487008214)'> loose</span><span style='background-color:rgb(252.9663493623957,2.033650637604296,2.033650637604296)'> ,</span><span style='background-color:rgb(250.81355231814086,4.186447681859136,4.186447681859136)'> dry</span><span style='background-color:rgb(248.32990346476436,6.670096535235643,6.670096535235643)'> skeleton</span><span style='background-color:rgb(248.76520668156445,6.23479331843555,6.23479331843555)'> of</span><span style='background-color:rgb(252.39768992643803,2.6023100735619664,2.6023100735619664)'> a</span><span style='background-color:rgb(248.6461405456066,6.353859454393387,6.353859454393387)'> tale</span><span style='background-color:rgb(251.99485193006694,3.005148069933057,3.005148069933057)'> lay</span><span style='background-color:rgb(251.54933224897832,3.450667751021683,3.450667751021683)'> exposed</span><span style='background-color:rgb(252.72527319844812,2.2747268015518785,2.2747268015518785)'> beneath</span><span style='background-color:rgb(253.08262134203687,1.917378657963127,1.917378657963127)'> their</span><span style='background-color:rgb(252.03658770304173,2.9634122969582677,2.9634122969582677)'> feet</span><span style='background-color:rgb(253.00260155461729,1.9973984453827143,1.9973984453827143)'> ,</span><span style='background-color:rgb(249.72884827293456,5.271151727065444,5.271151727065444)'> awaiting</span><span style='background-color:rgb(252.7167894411832,2.2832105588167906,2.2832105588167906)'> tend</span><span style='background-color:rgb(252.64144613407552,2.3585538659244776,2.3585538659244776)'> ##ons</span><span style='background-color:rgb(252.19969893805683,2.8003010619431734,2.8003010619431734)'> ,</span><span style='background-color:rgb(249.99596253968775,5.004037460312247,5.004037460312247)'> muscle</span><span style='background-color:rgb(253.1908628088422,1.809137191157788,1.809137191157788)'> and</span><span style='background-color:rgb(247.1818283200264,7.818171679973602,7.818171679973602)'> blushing</span><span style='background-color:rgb(245.2785455994308,9.7214544005692,9.7214544005692)'> skin</span><span style='background-color:rgb(240.4702142626047,14.529785737395287,14.529785737395287)'> .</span><span style='background-color:rgb(225.41870702058077,29.58129297941923,29.58129297941923)'> allan</span><span style='background-color:rgb(211.9009418785572,43.099058121442795,43.099058121442795)'> reviewed</span><span style='background-color:rgb(237.59634867310524,17.40365132689476,17.40365132689476)'> the</span><span style='background-color:rgb(203.8716535270214,51.12834647297859,51.12834647297859)'> troops</span><span style='background-color:rgb(241.15530705079436,13.844692949205637,13.844692949205637)'> ,</span><span style='background-color:rgb(242.86774026229978,12.132259737700224,12.132259737700224)'> all</span><span style='background-color:rgb(241.0744853876531,13.925514612346888,13.925514612346888)'> prepared</span><span style='background-color:rgb(250.38675969466567,4.61324030533433,4.61324030533433)'> to</span><span style='background-color:rgb(248.43207095749676,6.567929042503238,6.567929042503238)'> di</span><span style='background-color:rgb(250.26755344122648,4.732446558773518,4.732446558773518)'> ##se</span><span style='background-color:rgb(239.8890325985849,15.11096740141511,15.11096740141511)'> ##mba</span><span style='background-color:rgb(246.30259159952402,8.697408400475979,8.697408400475979)'> ##rk</span><span style='background-color:rgb(248.56058956123888,6.439410438761115,6.439410438761115)'> ,</span><span style='background-color:rgb(246.97364699095488,8.026353009045124,8.026353009045124)'> their</span><span style='background-color:rgb(245.54371437057853,9.456285629421473,9.456285629421473)'> task</span><span style='background-color:rgb(253.28419755678624,1.7158024432137609,1.7158024432137609)'> to</span><span style='background-color:rgb(251.12803674302995,3.871963256970048,3.871963256970048)'> form</span><span style='background-color:rgb(252.99909505527467,2.0009049447253346,2.0009049447253346)'> the</span><span style='background-color:rgb(244.87338569015265,10.126614309847355,10.126614309847355)'> tale</span><span style='background-color:rgb(253.00655476748943,1.9934452325105667,1.9934452325105667)'> of</span><span style='background-color:rgb(253.57320078648627,1.426799213513732,1.426799213513732)'> a</span><span style='background-color:rgb(252.3578096413985,2.6421903586015105,2.6421903586015105)'> young</span><span style='background-color:rgb(252.36445144750178,2.6355485524982214,2.6355485524982214)'> man</span><span style='background-color:rgb(249.93762142024934,5.062378579750657,5.062378579750657)'> returning</span><span style='background-color:rgb(252.89023992139846,2.109760078601539,2.109760078601539)'> home</span><span style='background-color:rgb(253.66481529548764,1.3351847045123577,1.3351847045123577)'> from</span><span style='background-color:rgb(252.70775471348315,2.2922452865168452,2.2922452865168452)'> life</span><span style='background-color:rgb(250.44571875594556,4.554281244054437,4.554281244054437)'> abroad</span><span style='background-color:rgb(254.1485817055218,0.8514182944782078,0.8514182944782078)'> to</span><span style='background-color:rgb(251.77892091684043,3.221079083159566,3.221079083159566)'> find</span><span style='background-color:rgb(252.3419640166685,2.6580359833315015,2.6580359833315015)'> his</span><span style='background-color:rgb(251.83955901302397,3.1604409869760275,3.1604409869760275)'> childhood</span><span style='background-color:rgb(250.70753895677626,4.292461043223739,4.292461043223739)'> friend</span><span style='background-color:rgb(250.0952098891139,4.904790110886097,4.904790110886097)'> a</span><span style='background-color:rgb(247.89392625913024,7.1060737408697605,7.1060737408697605)'> bride</span><span style='background-color:rgb(253.00585750490427,1.9941424950957298,1.9941424950957298)'> to</span><span style='background-color:rgb(250.58793182484806,4.412068175151944,4.412068175151944)'> -</span><span style='background-color:rgb(253.5052384645678,1.4947615354321897,1.4947615354321897)'> be</span><span style='background-color:rgb(252.31684926431626,2.683150735683739,2.683150735683739)'> ,</span><span style='background-color:rgb(252.2119112452492,2.788088754750788,2.788088754750788)'> thus</span><span style='background-color:rgb(250.6563124805689,4.343687519431114,4.343687519431114)'> upset</span><span style='background-color:rgb(252.34190298244357,2.658097017556429,2.658097017556429)'> ##ting</span><span style='background-color:rgb(252.355080912821,2.644919087179005,2.644919087179005)'> the</span><span style='background-color:rgb(251.34168146643788,3.6583185335621238,3.6583185335621238)'> apple</span><span style='background-color:rgb(249.6338509954512,5.366149004548788,5.366149004548788)'> cart</span><span style='background-color:rgb(252.71828941069543,2.2817105893045664,2.2817105893045664)'> of</span><span style='background-color:rgb(252.7133719995618,2.2866280004382133,2.2866280004382133)'> his</span><span style='background-color:rgb(250.35414271987975,4.645857280120254,4.645857280120254)'> life</span><span style='background-color:rgb(249.66300873085856,5.336991269141436,5.336991269141436)'> '</span><span style='background-color:rgb(253.39710707310587,1.6028929268941283,1.6028929268941283)'> s</span><span style='background-color:rgb(248.6018296983093,6.398170301690698,6.398170301690698)'> plan</span><span style='background-color:rgb(251.09912171959877,3.90087828040123,3.90087828040123)'> ,</span><span style='background-color:rgb(242.72598411887884,12.274015881121159,12.274015881121159)'> clarified</span><span style='background-color:rgb(239.23547241836786,15.764527581632137,15.764527581632137)'> –</span><span style='background-color:rgb(253.60112394439057,1.3988760556094348,1.3988760556094348)'> of</span><span style='background-color:rgb(251.05796589050442,3.94203410949558,3.94203410949558)'> course</span><span style='background-color:rgb(244.18471824377775,10.815281756222248,10.815281756222248)'> –</span><span style='background-color:rgb(253.72771783266217,1.2722821673378348,1.2722821673378348)'> by</span><span style='background-color:rgb(254.0770090429578,0.9229909570422024,0.9229909570422024)'> his</span><span style='background-color:rgb(254.21488986932673,0.7851101306732744,0.7851101306732744)'> very</span><span style='background-color:rgb(253.2361297798343,1.7638702201656997,1.7638702201656997)'> time</span><span style='background-color:rgb(254.51339520164765,0.4866047983523458,0.4866047983523458)'> away</span><span style='background-color:rgb(254.0682937949896,0.9317062050104141,0.9317062050104141)'> from</span><span style='background-color:rgb(250.41148211807013,4.588517881929874,4.588517881929874)'> her</span><span style='background-color:rgb(254.15791139239445,0.8420886076055467,0.8420886076055467)'> he</span><span style='background-color:rgb(250.71721323765814,4.282786762341857,4.282786762341857)'> loved</span><span style='background-color:rgb(252.36143060959876,2.638569390401244,2.638569390401244)'> best</span><span style='background-color:rgb(250.99609167315066,4.003908326849341,4.003908326849341)'> .</span><span style='background-color:rgb(249.28004438057542,5.7199556194245815,5.7199556194245815)'> although</span><span style='background-color:rgb(252.58973518386483,2.410264816135168,2.410264816135168)'> the</span><span style='background-color:rgb(247.76983964256942,7.230160357430577,7.230160357430577)'> concept</span><span style='background-color:rgb(252.11872813291848,2.881271867081523,2.881271867081523)'> was</span><span style='background-color:rgb(252.1095656370744,2.890434362925589,2.890434362925589)'> a</span><span style='background-color:rgb(251.0410349490121,3.9589650509878993,3.9589650509878993)'> simple</span><span style='background-color:rgb(252.43814493063837,2.561855069361627,2.561855069361627)'> one</span><span style='background-color:rgb(251.9973704824224,3.0026295175775886,3.0026295175775886)'> ,</span><span style='background-color:rgb(242.5653249397874,12.434675060212612,12.434675060212612)'> allan</span><span style='background-color:rgb(248.72113949619234,6.278860503807664,6.278860503807664)'> thought</span><span style='background-color:rgb(254.1901389474515,0.8098610525485128,0.8098610525485128)'> it</span><span style='background-color:rgb(253.87478644726798,1.1252135527320206,1.1252135527320206)'> had</span><span style='background-color:rgb(251.33547924924642,3.6645207507535815,3.6645207507535815)'> potential</span><span style='background-color:rgb(249.98311875388026,5.016881246119738,5.016881246119738)'> .</span><span style='background-color:rgb(248.0905414838344,6.909458516165614,6.909458516165614)'> besides</span><span style='background-color:rgb(253.26472253305838,1.7352774669416249,1.7352774669416249)'> ,</span><span style='background-color:rgb(252.8424631850794,2.1575368149206042,2.1575368149206042)'> the</span><span style='background-color:rgb(250.4706373438239,4.52936265617609,4.52936265617609)'> public</span><span style='background-color:rgb(240.31446726992726,14.685532730072737,14.685532730072737)'> liked</span><span style='background-color:rgb(252.0941287279129,2.905871272087097,2.905871272087097)'> a</span><span style='background-color:rgb(250.05279229022563,4.947207709774375,4.947207709774375)'> good</span><span style='background-color:rgb(252.7304247720167,2.269575227983296,2.269575227983296)'> ,</span><span style='background-color:rgb(250.91316064819694,4.086839351803064,4.086839351803064)'> simple</span><span style='background-color:rgb(241.1748361028731,13.825163897126913,13.825163897126913)'> romance</span><span style='background-color:rgb(250.66336015239358,4.3366398476064205,4.3366398476064205)'> .</span><span style='background-color:rgb(250.44839903712273,4.551600962877274,4.551600962877274)'> perhaps</span><span style='background-color:rgb(253.98362365085632,1.016376349143684,1.016376349143684)'> this</span><span style='background-color:rgb(250.58632213622332,4.413677863776684,4.413677863776684)'> will</span><span style='background-color:rgb(253.57410026947036,1.4258997305296361,1.4258997305296361)'> be</span><span style='background-color:rgb(252.92753373272717,2.07246626727283,2.07246626727283)'> more</span><span style='background-color:rgb(250.83083189092577,4.169168109074235,4.169168109074235)'> sale</span><span style='background-color:rgb(251.87907095532864,3.1209290446713567,3.1209290446713567)'> ##able</span><span style='background-color:rgb(251.87462068162858,3.125379318371415,3.125379318371415)'> ,</span><span style='background-color:rgb(249.81125160120428,5.188748398795724,5.188748398795724)'> he</span><span style='background-color:rgb(248.17377934232354,6.826220657676458,6.826220657676458)'> thought</span><span style='background-color:rgb(253.19982260931283,1.8001773906871676,1.8001773906871676)'> and</span><span style='background-color:rgb(250.1280401274562,4.871959872543812,4.871959872543812)'> began</span><span style='background-color:rgb(253.00115501973778,1.99884498026222,1.99884498026222)'> to</span><span style='background-color:rgb(238.79890244454145,16.201097555458546,16.201097555458546)'> write</span><span style='background-color:rgb(240.4693222604692,14.530677739530802,14.530677739530802)'> .</span><span style='background-color:rgb(240.38659689947963,14.613403100520372,14.613403100520372)'> they</span><span style='background-color:rgb(231.51640512049198,23.48359487950802,23.48359487950802)'> gazed</span><span style='background-color:rgb(247.39752849563956,7.602471504360437,7.602471504360437)'> at</span><span style='background-color:rgb(248.99407885037363,6.005921149626374,6.005921149626374)'> each</span><span style='background-color:rgb(252.68507610540837,2.3149238945916295,2.3149238945916295)'> other</span><span style='background-color:rgb(251.0483229579404,3.9516770420596004,3.9516770420596004)'> ,</span><span style='background-color:rgb(246.33472457528114,8.665275424718857,8.665275424718857)'> lost</span><span style='background-color:rgb(253.13580518821254,1.8641948117874563,1.8641948117874563)'> in</span><span style='background-color:rgb(252.8757947590202,2.1242052409797907,2.1242052409797907)'> the</span><span style='background-color:rgb(246.19755478575826,8.802445214241743,8.802445214241743)'> rap</span><span style='background-color:rgb(242.30621019378304,12.693789806216955,12.693789806216955)'> ##ture</span><span style='background-color:rgb(253.25388860190287,1.7461113980971277,1.7461113980971277)'> of</span><span style='background-color:rgb(245.41857758536935,9.581422414630651,9.581422414630651)'> love</span><span style='background-color:rgb(248.10997316613793,6.890026833862066,6.890026833862066)'> based</span><span style='background-color:rgb(247.40853365510702,7.591466344892979,7.591466344892979)'> so</span><span style='background-color:rgb(249.66484688222408,5.335153117775917,5.335153117775917)'> deeply</span><span style='background-color:rgb(250.8772502001375,4.122749799862504,4.122749799862504)'> within</span><span style='background-color:rgb(248.0829015187919,6.917098481208086,6.917098481208086)'> their</span><span style='background-color:rgb(236.1872797831893,18.812720216810703,18.812720216810703)'> hearts</span><span style='background-color:rgb(249.54275088384748,5.457249116152525,5.457249116152525)'> that</span><span style='background-color:rgb(250.87302910164,4.126970898360014,4.126970898360014)'> they</span><span style='background-color:rgb(250.93228264711797,4.067717352882028,4.067717352882028)'> had</span><span style='background-color:rgb(250.2271734829992,4.772826517000794,4.772826517000794)'> never</span><span style='background-color:rgb(251.8744304543361,3.1255695456638932,3.1255695456638932)'> seen</span><span style='background-color:rgb(252.92256359942257,2.077436400577426,2.077436400577426)'> it</span><span style='background-color:rgb(250.9471730981022,4.052826901897788,4.052826901897788)'> before</span><span style='background-color:rgb(230.47070503234863,24.529294967651367,24.529294967651367)'> .</span><span style='background-color:rgb(214.04879182577133,40.95120817422867,40.95120817422867)'> "</span><span style='background-color:rgb(232.88813341408968,22.11186658591032,22.11186658591032)'> what</span><span style='background-color:rgb(214.35371406376362,40.64628593623638,40.64628593623638)'> about</span><span style='background-color:rgb(203.5467253625393,51.45327463746071,51.45327463746071)'> roger</span><span style='background-color:rgb(218.4107905626297,36.5892094373703,36.5892094373703)'> ?</span><span style='background-color:rgb(206.5797682106495,48.42023178935051,48.42023178935051)'> "</span><span style='background-color:rgb(224.77120466530323,30.22879533469677,30.22879533469677)'> she</span><span style='background-color:rgb(158.24250876903534,96.75749123096466,96.75749123096466)'> asked</span><span style='background-color:rgb(217.18559950590134,37.81440049409866,37.81440049409866)'> ,</span><span style='background-color:rgb(151.01800963282585,103.98199036717415,103.98199036717415)'> knowing</span><span style='background-color:rgb(167.32263073325157,87.67736926674843,87.67736926674843)'> that</span><span style='background-color:rgb(202.33132258057594,52.66867741942406,52.66867741942406)'> the</span><span style='background-color:rgb(97.56104171276093,157.43895828723907,157.43895828723907)'> answer</span><span style='background-color:rgb(188.82466614246368,66.17533385753632,66.17533385753632)'> no</span><span style='background-color:rgb(80.79286068677902,174.20713931322098,174.20713931322098)'> longer</span><span style='background-color:rgb(0.0,255.0,255.0)'> mattered</span><span style='background-color:rgb(170.63779324293137,84.36220675706863,84.36220675706863)'> .</span><span style='background-color:rgb(212.23506554961205,42.764934450387955,42.764934450387955)'> [SEP]</span><span style='background-color:rgb(222.0897113531828,32.91028864681721,32.91028864681721)'> name</span><span style='background-color:rgb(211.28622226417065,43.71377773582935,43.71377773582935)'> few</span><span style='background-color:rgb(228.62118277698755,26.378817223012447,26.378817223012447)'> objects</span><span style='background-color:rgb(234.51085533946753,20.489144660532475,20.489144660532475)'> said</span><span style='background-color:rgb(247.38774449564517,7.612255504354835,7.612255504354835)'> to</span><span style='background-color:rgb(250.62529664486647,4.3747033551335335,4.3747033551335335)'> be</span><span style='background-color:rgb(244.1490951552987,10.85090484470129,10.85090484470129)'> in</span><span style='background-color:rgb(245.92506665736437,9.074933342635632,9.074933342635632)'> or</span><span style='background-color:rgb(233.84121917188168,21.158780828118324,21.158780828118324)'> on</span><span style='background-color:rgb(225.77475171536207,29.225248284637928,29.225248284637928)'> allan</span><span style='background-color:rgb(242.2685418650508,12.731458134949207,12.731458134949207)'> '</span><span style='background-color:rgb(240.43087782338262,14.569122176617384,14.569122176617384)'> s</span><span style='background-color:rgb(230.0220327079296,24.97796729207039,24.97796729207039)'> desk</span><span style='background-color:rgb(179.14725422859192,75.85274577140808,75.85274577140808)'> |</span><span style='background-color:rgb(150.12756541371346,104.87243458628654,104.87243458628654)'> |</span><span style='background-color:rgb(140.3754799067974,114.62452009320259,114.62452009320259)'> erase</span><span style='background-color:rgb(116.98309153318405,138.01690846681595,138.01690846681595)'> ##r</span><span style='background-color:rgb(215.7495691627264,39.2504308372736,39.2504308372736)'> [SEP]</span><span style='background-color:rgb(215.55323369801044,39.446766301989555,39.446766301989555)'> [PAD]</span><span style='background-color:rgb(223.327341042459,31.67265895754099,31.67265895754099)'> [PAD]</span><span style='background-color:rgb(233.80765177309513,21.19234822690487,21.19234822690487)'> [PAD]</span><span style='background-color:rgb(240.36029566079378,14.639704339206219,14.639704339206219)'> [PAD]</span><span style='background-color:rgb(244.03736025094986,10.96263974905014,10.96263974905014)'> [PAD]</span><span style='background-color:rgb(243.653278760612,11.346721239387989,11.346721239387989)'> [PAD]</span><span style='background-color:rgb(245.2946500852704,9.705349914729595,9.705349914729595)'> [PAD]</span><span style='background-color:rgb(246.38108303770423,8.61891696229577,8.61891696229577)'> [PAD]</span><span style='background-color:rgb(248.65595446899533,6.344045531004667,6.344045531004667)'> [PAD]</span><span style='background-color:rgb(249.049471328035,5.950528671965003,5.950528671965003)'> [PAD]</span><span style='background-color:rgb(248.39576960541308,6.604230394586921,6.604230394586921)'> [PAD]</span><span style='background-color:rgb(247.7390166465193,7.260983353480697,7.260983353480697)'> [PAD]</span><span style='background-color:rgb(247.78923807665706,7.210761923342943,7.210761923342943)'> [PAD]</span><span style='background-color:rgb(250.44609968550503,4.5539003144949675,4.5539003144949675)'> [PAD]</span><span style='background-color:rgb(249.6305684465915,5.3694315534085035,5.3694315534085035)'> [PAD]</span><span style='background-color:rgb(249.8346844688058,5.16531553119421,5.16531553119421)'> [PAD]</span><span style='background-color:rgb(249.4426908530295,5.5573091469705105,5.5573091469705105)'> [PAD]</span><span style='background-color:rgb(248.45646707341075,6.543532926589251,6.543532926589251)'> [PAD]</span><span style='background-color:rgb(248.27788710594177,6.7221128940582275,6.7221128940582275)'> [PAD]</span><span style='background-color:rgb(249.26897224970162,5.731027750298381,5.731027750298381)'> [PAD]</span><span style='background-color:rgb(249.58179663866758,5.4182033613324165,5.4182033613324165)'> [PAD]</span><span style='background-color:rgb(249.73257159814239,5.2674284018576145,5.2674284018576145)'> [PAD]</span><span style='background-color:rgb(249.2300533130765,5.769946686923504,5.769946686923504)'> [PAD]</span><span style='background-color:rgb(248.21410562843084,6.785894371569157,6.785894371569157)'> [PAD]</span><span style='background-color:rgb(249.46602017618716,5.533979823812842,5.533979823812842)'> [PAD]</span><span style='background-color:rgb(248.13750791363418,6.862492086365819,6.862492086365819)'> [PAD]</span><span style='background-color:rgb(243.98497816175222,11.015021838247776,11.015021838247776)'> [PAD]</span><span style='background-color:rgb(232.45083957910538,22.549160420894623,22.549160420894623)'> [PAD]</span><span style='background-color:rgb(232.57792945951223,22.422070540487766,22.422070540487766)'> [PAD]</span><span style='background-color:rgb(243.48795438185334,11.512045618146658,11.512045618146658)'> [PAD]</span><span style='background-color:rgb(245.23843685165048,9.761563148349524,9.761563148349524)'> [PAD]</span><span style='background-color:rgb(246.13422453403473,8.865775465965271,8.865775465965271)'> [PAD]</span><span style='background-color:rgb(245.48007918521762,9.519920814782381,9.519920814782381)'> [PAD]</span><span style='background-color:rgb(244.76105136796832,10.23894863203168,10.23894863203168)'> [PAD]</span><span style='background-color:rgb(234.06824558973312,20.931754410266876,20.931754410266876)'> [PAD]</span><span style='background-color:rgb(242.87096913903952,12.129030860960484,12.129030860960484)'> [PAD]</span><span style='background-color:rgb(248.25209219008684,6.747907809913158,6.747907809913158)'> [PAD]</span><span style='background-color:rgb(242.33330748975277,12.66669251024723,12.66669251024723)'> [PAD]</span><span style='background-color:rgb(248.08340166695416,6.91659833304584,6.91659833304584)'> [PAD]</span><span style='background-color:rgb(247.66547539271414,7.334524607285857,7.334524607285857)'> [PAD]</span><span style='background-color:rgb(249.57546000368893,5.4245399963110685,5.4245399963110685)'> [PAD]</span><span style='background-color:rgb(249.07304288819432,5.926957111805677,5.926957111805677)'> [PAD]</span><span style='background-color:rgb(250.57481445372105,4.4251855462789536,4.4251855462789536)'> [PAD]</span><span style='background-color:rgb(248.82677525281906,6.173224747180939,6.173224747180939)'> [PAD]</span><span style='background-color:rgb(249.25338548608124,5.746614513918757,5.746614513918757)'> [PAD]</span><span style='background-color:rgb(249.22550733201206,5.774492667987943,5.774492667987943)'> [PAD]</span><span style='background-color:rgb(249.49653633870184,5.503463661298156,5.503463661298156)'> [PAD]</span><span style='background-color:rgb(249.69494696706533,5.305053032934666,5.305053032934666)'> [PAD]</span><span style='background-color:rgb(249.13895035162568,5.861049648374319,5.861049648374319)'> [PAD]</span><span style='background-color:rgb(248.5571730695665,6.442826930433512,6.442826930433512)'> [PAD]</span><span style='background-color:rgb(246.32683524861932,8.673164751380682,8.673164751380682)'> [PAD]</span><span style='background-color:rgb(245.51451578736305,9.485484212636948,9.485484212636948)'> [PAD]</span><span style='background-color:rgb(242.00055744498968,12.999442555010319,12.999442555010319)'> [PAD]</span><span style='background-color:rgb(238.87717444449663,16.12282555550337,16.12282555550337)'> [PAD]</span><span style='background-color:rgb(239.99705415219069,15.002945847809315,15.002945847809315)'> [PAD]</span><span style='background-color:rgb(242.56003277376294,12.439967226237059,12.439967226237059)'> [PAD]</span><span style='background-color:rgb(247.04452126286924,7.955478737130761,7.955478737130761)'> [PAD]</span><span style='background-color:rgb(247.71518480032682,7.284815199673176,7.284815199673176)'> [PAD]</span><span style='background-color:rgb(247.22622703760862,7.7737729623913765,7.7737729623913765)'> [PAD]</span><span style='background-color:rgb(247.53004068508744,7.469959314912558,7.469959314912558)'> [PAD]</span><span style='background-color:rgb(247.0408630091697,7.959136990830302,7.959136990830302)'> [PAD]</span><span style='background-color:rgb(248.6761803086847,6.323819691315293,6.323819691315293)'> [PAD]</span><span style='background-color:rgb(249.97585924342275,5.024140756577253,5.024140756577253)'> [PAD]</span><span style='background-color:rgb(250.93910090625286,4.060899093747139,4.060899093747139)'> [PAD]</span><span style='background-color:rgb(250.62102139927447,4.378978600725532,4.378978600725532)'> [PAD]</span><span style='background-color:rgb(249.461083766073,5.5389162339270115,5.5389162339270115)'> [PAD]</span><span style='background-color:rgb(249.66399525292218,5.336004747077823,5.336004747077823)'> [PAD]</span><span style='background-color:rgb(248.85355051606894,6.146449483931065,6.146449483931065)'> [PAD]</span><span style='background-color:rgb(248.4537255205214,6.546274479478598,6.546274479478598)'> [PAD]</span><span style='background-color:rgb(246.56140331178904,8.438596688210964,8.438596688210964)'> [PAD]</span><span style='background-color:rgb(248.36723788641393,6.632762113586068,6.632762113586068)'> [PAD]</span><span style='background-color:rgb(249.07885372638702,5.921146273612976,5.921146273612976)'> [PAD]</span><span style='background-color:rgb(248.6958029307425,6.304197069257498,6.304197069257498)'> [PAD]</span><span style='background-color:rgb(249.4853934366256,5.5146065633744,5.5146065633744)'> [PAD]</span><span style='background-color:rgb(248.35339142940938,6.646608570590615,6.646608570590615)'> [PAD]</span><span style='background-color:rgb(248.55750317685306,6.442496823146939,6.442496823146939)'> [PAD]</span><span style='background-color:rgb(249.7405948676169,5.259405132383108,5.259405132383108)'> [PAD]</span><span style='background-color:rgb(249.6744537167251,5.325546283274889,5.325546283274889)'> [PAD]</span><span style='background-color:rgb(248.35071209818125,6.649287901818752,6.649287901818752)'> [PAD]</span><span style='background-color:rgb(248.94606415182352,6.053935848176479,6.053935848176479)'> [PAD]</span><span style='background-color:rgb(247.09523476660252,7.904765233397484,7.904765233397484)'> [PAD]</span><span style='background-color:rgb(245.71252221241593,9.287477787584066,9.287477787584066)'> [PAD]</span><span style='background-color:rgb(246.61753959953785,8.38246040046215,8.38246040046215)'> [PAD]</span><span style='background-color:rgb(247.77452621608973,7.2254737839102745,7.2254737839102745)'> [PAD]</span><span style='background-color:rgb(247.65497513115406,7.34502486884594,7.34502486884594)'> [PAD]</span><span style='background-color:rgb(247.8318181168288,7.168181883171201,7.168181883171201)'> [PAD]</span><span style='background-color:rgb(247.24394311197102,7.756056888028979,7.756056888028979)'> [PAD]</span><span style='background-color:rgb(247.81685451976955,7.183145480230451,7.183145480230451)'> [PAD]</span><span style='background-color:rgb(248.6421379353851,6.357862064614892,6.357862064614892)'> [PAD]</span><span style='background-color:rgb(246.68206868693233,8.317931313067675,8.317931313067675)'> [PAD]</span><span style='background-color:rgb(247.86273278295994,7.137267217040062,7.137267217040062)'> [PAD]</span><span style='background-color:rgb(246.61317268386483,8.386827316135168,8.386827316135168)'> [PAD]</span><span style='background-color:rgb(247.67512402497232,7.32487597502768,7.32487597502768)'> [PAD]</span><span style='background-color:rgb(247.20536330714822,7.794636692851782,7.794636692851782)'> [PAD]</span><span style='background-color:rgb(246.94260645657778,8.057393543422222,8.057393543422222)'> [PAD]</span><span style='background-color:rgb(247.91716628707945,7.0828337129205465,7.0828337129205465)'> [PAD]</span><span style='background-color:rgb(246.83376034721732,8.166239652782679,8.166239652782679)'> [PAD]</span><span style='background-color:rgb(248.68249556981027,6.317504430189729,6.317504430189729)'> [PAD]</span><span style='background-color:rgb(248.9200987201184,6.079901279881597,6.079901279881597)'> [PAD]</span><span style='background-color:rgb(247.44233426637948,7.557665733620524,7.557665733620524)'> [PAD]</span><span style='background-color:rgb(247.3916350118816,7.60836498811841,7.60836498811841)'> [PAD]</span><span style='background-color:rgb(246.86131931841373,8.138680681586266,8.138680681586266)'> [PAD]</span><span style='background-color:rgb(245.83902407437563,9.16097592562437,9.16097592562437)'> [PAD]</span><span style='background-color:rgb(244.09068183973432,10.909318160265684,10.909318160265684)'> [PAD]</span><span style='background-color:rgb(239.79583309963346,15.204166900366545,15.204166900366545)'> [PAD]</span><span style='background-color:rgb(240.84298755973577,14.157012440264225,14.157012440264225)'> [PAD]</span><span style='background-color:rgb(241.87301253899932,13.126987461000681,13.126987461000681)'> [PAD]</span><span style='background-color:rgb(242.6543285138905,12.345671486109495,12.345671486109495)'> [PAD]</span><span style='background-color:rgb(241.89893949776888,13.101060502231121,13.101060502231121)'> [PAD]</span><span style='background-color:rgb(244.73108997568488,10.268910024315119,10.268910024315119)'> [PAD]</span><span style='background-color:rgb(243.0891372822225,11.91086271777749,11.91086271777749)'> [PAD]</span><span style='background-color:rgb(244.06139966100454,10.938600338995457,10.938600338995457)'> [PAD]</span><span style='background-color:rgb(244.16411384940147,10.835886150598526,10.835886150598526)'> [PAD]</span><span style='background-color:rgb(241.79897731170058,13.201022688299417,13.201022688299417)'> [PAD]</span><span style='background-color:rgb(242.59892178699374,12.401078213006258,12.401078213006258)'> [PAD]</span><span style='background-color:rgb(244.27138779312372,10.728612206876278,10.728612206876278)'> [PAD]</span><span style='background-color:rgb(241.3777043670416,13.622295632958412,13.622295632958412)'> [PAD]</span><span style='background-color:rgb(183.41438725590706,71.58561274409294,71.58561274409294)'> [PAD]</span><span style='background-color:rgb(243.42156814411283,11.578431855887175,11.578431855887175)'> [PAD]</span><span style='background-color:rgb(242.68554288893938,12.31445711106062,12.31445711106062)'> [PAD]</span><span style='background-color:rgb(242.8331307694316,12.166869230568409,12.166869230568409)'> [PAD]</span><span style='background-color:rgb(238.13951052725315,16.86048947274685,16.86048947274685)'> [PAD]</span><span style='background-color:rgb(228.92175424844027,26.078245751559734,26.078245751559734)'> [PAD]</span><span style='background-color:rgb(143.9086975157261,111.09130248427391,111.09130248427391)'> [PAD]</span><span style='background-color:rgb(215.01273170113564,39.987268298864365,39.987268298864365)'> [PAD]</span><span style='background-color:rgb(230.9525666758418,24.04743332415819,24.04743332415819)'> [PAD]</span><span style='background-color:rgb(229.15760569274426,25.842394307255745,25.842394307255745)'> [PAD]</span><span style='background-color:rgb(226.8143854290247,28.185614570975304,28.185614570975304)'> [PAD]</span><span style='background-color:rgb(185.6437884271145,69.35621157288551,69.35621157288551)'> [PAD]</span><span style='background-color:rgb(227.60303881019354,27.39696118980646,27.39696118980646)'> [PAD]</span><span style='background-color:rgb(247.8610067255795,7.1389932744205,7.1389932744205)'> [PAD]</span><span style='background-color:rgb(248.9985820837319,6.00141791626811,6.00141791626811)'> [PAD]</span><span style='background-color:rgb(249.75895120762289,5.241048792377114,5.241048792377114)'> [PAD]</span><span style='background-color:rgb(244.1272092796862,10.872790720313787,10.872790720313787)'> [PAD]</span><span style='background-color:rgb(247.79743233695626,7.202567663043737,7.202567663043737)'> [PAD]</span><span style='background-color:rgb(245.29340280219913,9.706597197800875,9.706597197800875)'> [PAD]</span><span style='background-color:rgb(249.0991346631199,5.900865336880088,5.900865336880088)'> [PAD]</span><span style='background-color:rgb(249.5772164594382,5.422783540561795,5.422783540561795)'> [PAD]</span><span style='background-color:rgb(249.44041904993355,5.559580950066447,5.559580950066447)'> [PAD]</span><span style='background-color:rgb(245.54100131615996,9.458998683840036,9.458998683840036)'> [PAD]</span><span style='background-color:rgb(234.9636086449027,20.036391355097294,20.036391355097294)'> [PAD]</span><span style='background-color:rgb(240.27976563200355,14.720234367996454,14.720234367996454)'> [PAD]</span><span style='background-color:rgb(245.11063925921917,9.88936074078083,9.88936074078083)'> [PAD]</span><span style='background-color:rgb(239.12250163033605,15.877498369663954,15.877498369663954)'> [PAD]</span><span style='background-color:rgb(238.83948806673288,16.160511933267117,16.160511933267117)'> [PAD]</span><span style='background-color:rgb(230.31491339206696,24.685086607933044,24.685086607933044)'> [PAD]</span><span style='background-color:rgb(236.99708852916956,18.00291147083044,18.00291147083044)'> [PAD]</span><span style='background-color:rgb(242.01561888679862,12.98438111320138,12.98438111320138)'> [PAD]</span><span style='background-color:rgb(240.94791512936354,14.052084870636463,14.052084870636463)'> [PAD]</span><span style='background-color:rgb(246.53533386066556,8.46466613933444,8.46466613933444)'> [PAD]</span><span style='background-color:rgb(233.14030688256025,21.859693117439747,21.859693117439747)'> [PAD]</span><span style='background-color:rgb(238.2069132104516,16.793086789548397,16.793086789548397)'> [PAD]</span><span style='background-color:rgb(246.4305630326271,8.569436967372894,8.569436967372894)'> [PAD]</span><span style='background-color:rgb(244.91308595985174,10.086914040148258,10.086914040148258)'> [PAD]</span><span style='background-color:rgb(239.53584250062704,15.46415749937296,15.46415749937296)'> [PAD]</span><span style='background-color:rgb(238.26103750616312,16.73896249383688,16.73896249383688)'> [PAD]</span><span style='background-color:rgb(200.77529087662697,54.22470912337303,54.22470912337303)'> [PAD]</span>

## AGI

In [14]:
from exlib.explainers.agi import AgiTextCls
max_iter = 4
topk = 5
agi_explainer = AgiTextCls(original_model2, projection_layer,
                            max_iter=max_iter, topk=topk)
agi_expln = agi_explainer(inputs) #, labels) #, x_kwargs=kwargs)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
agi_expln.attributions.shape

torch.Size([2, 512])

In [16]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

def show_attrs(tokens, masks, normalize=True):
    if normalize:
        masks_max = masks.max(-1).values
        masks_min = masks.min(-1).values
        # import pdb; pdb.set_trace()
        masks = (masks - masks_min) / (masks_max - masks_min)
    print_str = ""
    for i in range(len(tokens)):
        expln_val = masks[i].item()
        if expln_val == 0:
            print_str += f" {tokens[i]}"
        else:
            print_str += f"<span style='background-color:rgb({255 * (1-expln_val)},{255 * expln_val},{255 * expln_val})'> {tokens[i]}</span>"
    return print_str

tokens = processor.convert_ids_to_tokens(inputs[0])
# tokens = shap_expln.explainer_output.data[0]
printmd(show_attrs(tokens, agi_expln.attributions[0]))

<span style='background-color:rgb(166.27377301454544,88.72622698545456,88.72622698545456)'> [CLS]</span><span style='background-color:rgb(244.35138205066323,10.648617949336767,10.648617949336767)'> allan</span><span style='background-color:rgb(250.58611647225916,4.413883527740836,4.413883527740836)'> sat</span><span style='background-color:rgb(239.21826409175992,15.78173590824008,15.78173590824008)'> down</span><span style='background-color:rgb(242.31904400512576,12.680955994874239,12.680955994874239)'> at</span><span style='background-color:rgb(0.0,255.0,255.0)'> his</span><span style='background-color:rgb(241.7297431267798,13.270256873220205,13.270256873220205)'> desk</span><span style='background-color:rgb(245.71000484749675,9.289995152503252,9.289995152503252)'> and</span><span style='background-color:rgb(250.2034100331366,4.796589966863394,4.796589966863394)'> pulled</span><span style='background-color:rgb(253.11194294365123,1.8880570563487709,1.8880570563487709)'> the</span><span style='background-color:rgb(252.10206127725542,2.897938722744584,2.897938722744584)'> chair</span><span style='background-color:rgb(252.45336145162582,2.546638548374176,2.546638548374176)'> in</span><span style='background-color:rgb(253.32568883663043,1.6743111633695662,1.6743111633695662)'> close</span><span style='background-color:rgb(251.88702321611345,3.112976783886552,3.112976783886552)'> .</span><span style='background-color:rgb(253.35571684408933,1.6442831559106708,1.6442831559106708)'> opening</span><span style='background-color:rgb(253.9988793572411,1.0011206427589059,1.0011206427589059)'> a</span><span style='background-color:rgb(253.51603606017306,1.4839639398269355,1.4839639398269355)'> side</span><span style='background-color:rgb(253.94337478559464,1.0566252144053578,1.0566252144053578)'> drawer</span><span style='background-color:rgb(253.7994284159504,1.2005715840496123,1.2005715840496123)'> ,</span><span style='background-color:rgb(254.4921959610656,0.5078040389344096,0.5078040389344096)'> he</span><span style='background-color:rgb(254.24086218816228,0.7591378118377179,0.7591378118377179)'> took</span><span style='background-color:rgb(253.4396580210887,1.5603419789113104,1.5603419789113104)'> out</span><span style='background-color:rgb(254.3897760787513,0.6102239212486893,0.6102239212486893)'> a</span><span style='background-color:rgb(252.88122395519167,2.118776044808328,2.118776044808328)'> piece</span><span style='background-color:rgb(218.43536004424095,36.56463995575905,36.56463995575905)'> of</span><span style='background-color:rgb(252.43466574233025,2.565334257669747,2.565334257669747)'> paper</span><span style='background-color:rgb(250.48516823910177,4.5148317608982325,4.5148317608982325)'> and</span><span style='background-color:rgb(253.31383620388806,1.6861637961119413,1.6861637961119413)'> his</span><span style='background-color:rgb(253.73207370517775,1.2679262948222458,1.2679262948222458)'> ink</span><span style='background-color:rgb(252.77279558591545,2.2272044140845537,2.2272044140845537)'> ##pot</span><span style='background-color:rgb(251.9730242388323,3.026975761167705,3.026975761167705)'> .</span><span style='background-color:rgb(237.13784627616405,17.862153723835945,17.862153723835945)'> after</span><span style='background-color:rgb(252.1576907671988,2.842309232801199,2.842309232801199)'> filling</span><span style='background-color:rgb(250.95036492682993,4.049635073170066,4.049635073170066)'> his</span><span style='background-color:rgb(248.15719085745513,6.842809142544866,6.842809142544866)'> pen</span><span style='background-color:rgb(252.16845606453717,2.8315439354628325,2.8315439354628325)'> ,</span><span style='background-color:rgb(250.9803125448525,4.019687455147505,4.019687455147505)'> allan</span><span style='background-color:rgb(252.1464039478451,2.8535960521548986,2.8535960521548986)'> looked</span><span style='background-color:rgb(252.81741896644235,2.1825810335576534,2.1825810335576534)'> at</span><span style='background-color:rgb(252.7964008692652,2.2035991307348013,2.2035991307348013)'> his</span><span style='background-color:rgb(248.76266746781766,6.237332532182336,6.237332532182336)'> paper</span><span style='background-color:rgb(246.2632399611175,8.736760038882494,8.736760038882494)'> in</span><span style='background-color:rgb(251.31591267418116,3.6840873258188367,3.6840873258188367)'> the</span><span style='background-color:rgb(250.16444597393274,4.835554026067257,4.835554026067257)'> orange</span><span style='background-color:rgb(246.9086628779769,8.091337122023106,8.091337122023106)'> glow</span><span style='background-color:rgb(250.16846235841513,4.831537641584873,4.831537641584873)'> from</span><span style='background-color:rgb(240.61258122324944,14.387418776750565,14.387418776750565)'> the</span><span style='background-color:rgb(251.25954744871706,3.7404525512829423,3.7404525512829423)'> lantern</span><span style='background-color:rgb(253.8944566855207,1.1055433144792914,1.1055433144792914)'> set</span><span style='background-color:rgb(254.11869589355774,0.8813041064422578,0.8813041064422578)'> back</span><span style='background-color:rgb(253.72956180246547,1.2704381975345314,1.2704381975345314)'> in</span><span style='background-color:rgb(250.92919958755374,4.0708004124462605,4.0708004124462605)'> the</span><span style='background-color:rgb(252.86146715283394,2.1385328471660614,2.1385328471660614)'> desk</span><span style='background-color:rgb(252.19475587829947,2.8052441217005253,2.8052441217005253)'> '</span><span style='background-color:rgb(249.9845826253295,5.0154173746705055,5.0154173746705055)'> s</span><span style='background-color:rgb(252.0601590257138,2.9398409742861986,2.9398409742861986)'> right</span><span style='background-color:rgb(251.08632685616612,3.9136731438338757,3.9136731438338757)'> -</span><span style='background-color:rgb(253.88042819453403,1.1195718054659665,1.1195718054659665)'> hand</span><span style='background-color:rgb(252.9023251729086,2.097674827091396,2.097674827091396)'> corner</span><span style='background-color:rgb(252.44295143522322,2.557048564776778,2.557048564776778)'> .</span><span style='background-color:rgb(253.08367994148284,1.916320058517158,1.916320058517158)'> his</span><span style='background-color:rgb(253.15923508722335,1.840764912776649,1.840764912776649)'> pen</span><span style='background-color:rgb(250.28388971462846,4.716110285371542,4.716110285371542)'> cast</span><span style='background-color:rgb(252.06859599798918,2.9314040020108223,2.9314040020108223)'> a</span><span style='background-color:rgb(250.815012389794,4.184987610206008,4.184987610206008)'> forbid</span><span style='background-color:rgb(253.68549390463158,1.3145060953684151,1.3145060953684151)'> ##ding</span><span style='background-color:rgb(251.27383325714618,3.7261667428538203,3.7261667428538203)'> line</span><span style='background-color:rgb(252.56059549748898,2.4394045025110245,2.4394045025110245)'> of</span><span style='background-color:rgb(251.155663873069,3.8443361269310117,3.8443361269310117)'> shadow</span><span style='background-color:rgb(253.2581962645054,1.7418037354946136,1.7418037354946136)'> slant</span><span style='background-color:rgb(252.84711294807494,2.152887051925063,2.152887051925063)'> ##ing</span><span style='background-color:rgb(253.24010116048157,1.7598988395184278,1.7598988395184278)'> across</span><span style='background-color:rgb(251.5750471316278,3.424952868372202,3.424952868372202)'> the</span><span style='background-color:rgb(251.35294596198946,3.6470540380105376,3.6470540380105376)'> page</span><span style='background-color:rgb(250.9322978463024,4.06770215369761,4.06770215369761)'> ,</span><span style='background-color:rgb(250.80195106565952,4.198048934340477,4.198048934340477)'> echoing</span><span style='background-color:rgb(251.41164996195585,3.5883500380441546,3.5883500380441546)'> the</span><span style='background-color:rgb(250.1969043072313,4.803095692768693,4.803095692768693)'> ink</span><span style='background-color:rgb(218.25887851417065,36.74112148582935,36.74112148582935)'> ##y</span><span style='background-color:rgb(245.44222086668015,9.557779133319855,9.557779133319855)'> darkness</span><span style='background-color:rgb(251.90207278355956,3.0979272164404392,3.0979272164404392)'> crouch</span><span style='background-color:rgb(232.11845431476831,22.881545685231686,22.881545685231686)'> ##ing</span><span style='background-color:rgb(248.04498192854226,6.955018071457744,6.955018071457744)'> in</span><span style='background-color:rgb(252.39530674181879,2.6046932581812143,2.6046932581812143)'> the</span><span style='background-color:rgb(249.3612787965685,5.638721203431487,5.638721203431487)'> edges</span><span style='background-color:rgb(194.09556470811367,60.90443529188633,60.90443529188633)'> of</span><span style='background-color:rgb(248.91216474585235,6.087835254147649,6.087835254147649)'> the</span><span style='background-color:rgb(251.99080490972847,3.0091950902715325,3.0091950902715325)'> lantern</span><span style='background-color:rgb(251.21891765389591,3.7810823461040854,3.7810823461040854)'> '</span><span style='background-color:rgb(251.08317848760635,3.9168215123936534,3.9168215123936534)'> s</span><span style='background-color:rgb(248.5219594091177,6.478040590882301,6.478040590882301)'> struggling</span><span style='background-color:rgb(250.64907671883702,4.350923281162977,4.350923281162977)'> glow</span><span style='background-color:rgb(249.06760870479047,5.932391295209527,5.932391295209527)'> .</span><span style='background-color:rgb(250.82928394898772,4.170716051012278,4.170716051012278)'> the</span><span style='background-color:rgb(250.1576581131667,4.84234188683331,4.84234188683331)'> only</span><span style='background-color:rgb(253.22678038151935,1.7732196184806526,1.7732196184806526)'> other</span><span style='background-color:rgb(253.21217491524294,1.7878250847570598,1.7878250847570598)'> illumination</span><span style='background-color:rgb(250.68439439870417,4.315605601295829,4.315605601295829)'> came</span><span style='background-color:rgb(251.39717392623425,3.6028260737657547,3.6028260737657547)'> from</span><span style='background-color:rgb(243.01589811220765,11.984101887792349,11.984101887792349)'> a</span><span style='background-color:rgb(253.0090657202527,1.9909342797473073,1.9909342797473073)'> lu</span><span style='background-color:rgb(253.55148412054405,1.4485158794559538,1.4485158794559538)'> ##rid</span><span style='background-color:rgb(252.9910967219621,2.0089032780379057,2.0089032780379057)'> moonlight</span><span style='background-color:rgb(253.97184285800904,1.0281571419909596,1.0281571419909596)'> filtered</span><span style='background-color:rgb(251.57396133989096,3.426038660109043,3.426038660109043)'> through</span><span style='background-color:rgb(251.1091776425019,3.8908223574981093,3.8908223574981093)'> thin</span><span style='background-color:rgb(252.2895199432969,2.7104800567030907,2.7104800567030907)'> branches</span><span style='background-color:rgb(248.33718054927886,6.662819450721145,6.662819450721145)'> and</span><span style='background-color:rgb(241.77784094586968,13.222159054130316,13.222159054130316)'> clouds</span><span style='background-color:rgb(241.70323384925723,13.29676615074277,13.29676615074277)'> ,</span><span style='background-color:rgb(250.26998341083527,4.730016589164734,4.730016589164734)'> casting</span><span style='background-color:rgb(247.04088675789535,7.9591132421046495,7.9591132421046495)'> its</span><span style='background-color:rgb(249.62428025901318,5.375719740986824,5.375719740986824)'> bone</span><span style='background-color:rgb(249.36116290278733,5.638837097212672,5.638837097212672)'> -</span><span style='background-color:rgb(250.7954443898052,4.204555610194802,4.204555610194802)'> pale</span><span style='background-color:rgb(251.13174083177,3.868259168229997,3.868259168229997)'> glow</span><span style='background-color:rgb(246.83699207380414,8.16300792619586,8.16300792619586)'> onto</span><span style='background-color:rgb(246.729431245476,8.270568754523993,8.270568754523993)'> the</span><span style='background-color:rgb(252.44282176718116,2.557178232818842,2.557178232818842)'> pine</span><span style='background-color:rgb(253.52932644076645,1.4706735592335463,1.4706735592335463)'> floor</span><span style='background-color:rgb(254.09605623339303,0.9039437666069716,0.9039437666069716)'> ##boards</span><span style='background-color:rgb(243.00610033795238,11.993899662047625,11.993899662047625)'> .</span><span style='background-color:rgb(250.7090284768492,4.290971523150802,4.290971523150802)'> allan</span><span style='background-color:rgb(252.84386032260954,2.156139677390456,2.156139677390456)'> unfolded</span><span style='background-color:rgb(235.72710167616606,19.272898323833942,19.272898323833942)'> another</span><span style='background-color:rgb(249.98785092495382,5.012149075046182,5.012149075046182)'> page</span><span style='background-color:rgb(246.03269208222628,8.967307917773724,8.967307917773724)'> ,</span><span style='background-color:rgb(247.10093351081014,7.899066489189863,7.899066489189863)'> this</span><span style='background-color:rgb(252.41953519172966,2.580464808270335,2.580464808270335)'> one</span><span style='background-color:rgb(252.13448280002922,2.865517199970782,2.865517199970782)'> crowded</span><span style='background-color:rgb(245.61056608334184,9.389433916658163,9.389433916658163)'> with</span><span style='background-color:rgb(253.07793179992586,1.9220682000741363,1.9220682000741363)'> ranks</span><span style='background-color:rgb(253.43818904366344,1.5618109563365579,1.5618109563365579)'> of</span><span style='background-color:rgb(250.07188959047198,4.928110409528017,4.928110409528017)'> letters</span><span style='background-color:rgb(253.6058631213382,1.3941368786618114,1.3941368786618114)'> in</span><span style='background-color:rgb(253.85901076253504,1.1409892374649644,1.1409892374649644)'> tight</span><span style='background-color:rgb(254.56749378936365,0.43250621063634753,0.43250621063634753)'> formation</span><span style='background-color:rgb(254.7721999080386,0.22780009196139872,0.22780009196139872)'> from</span><span style='background-color:rgb(253.39624736923724,1.6037526307627559,1.6037526307627559)'> left</span><span style='background-color:rgb(254.2501361842733,0.7498638157267123,0.7498638157267123)'> to</span><span style='background-color:rgb(252.58544117677957,2.414558823220432,2.414558823220432)'> right</span><span style='background-color:rgb(251.107598352246,3.8924016477540135,3.8924016477540135)'> .</span><span style='background-color:rgb(252.43556819390506,2.5644318060949445,2.5644318060949445)'> the</span><span style='background-color:rgb(253.0806488916278,1.9193511083722115,1.9193511083722115)'> lines</span><span style='background-color:rgb(251.19620579760522,3.8037942023947835,3.8037942023947835)'> of</span><span style='background-color:rgb(224.9851293861866,30.0148706138134,30.0148706138134)'> letters</span><span style='background-color:rgb(244.50027706101537,10.499722938984632,10.499722938984632)'> stepped</span><span style='background-color:rgb(251.85156043153256,3.148439568467438,3.148439568467438)'> into</span><span style='background-color:rgb(253.09802951524034,1.9019704847596586,1.9019704847596586)'> their</span><span style='background-color:rgb(252.9833646118641,2.01663538813591,2.01663538813591)'> divisions</span><span style='background-color:rgb(253.30258619505912,1.6974138049408793,1.6974138049408793)'> ,</span><span style='background-color:rgb(248.3029068633914,6.697093136608601,6.697093136608601)'> in</span><span style='background-color:rgb(250.10391854681075,4.896081453189254,4.896081453189254)'> the</span><span style='background-color:rgb(253.23986877920106,1.7601312207989395,1.7601312207989395)'> shape</span><span style='background-color:rgb(254.37237609992735,0.6276239000726491,0.6276239000726491)'> of</span><span style='background-color:rgb(252.86698421929032,2.133015780709684,2.133015780709684)'> a</span><span style='background-color:rgb(250.90237920172513,4.097620798274875,4.097620798274875)'> story</span><span style='background-color:rgb(251.68965895660222,3.3103410433977842,3.3103410433977842)'> '</span><span style='background-color:rgb(254.69088035286404,0.3091196471359581,0.3091196471359581)'> s</span><span style='background-color:rgb(250.7916061207652,4.208393879234791,4.208393879234791)'> outline</span><span style='background-color:rgb(251.32223363500088,3.6777663649991155,3.6777663649991155)'> :</span><span style='background-color:rgb(252.19775747973472,2.802242520265281,2.802242520265281)'> the</span><span style='background-color:rgb(252.82731506042182,2.1726849395781755,2.1726849395781755)'> loose</span><span style='background-color:rgb(252.83297105692327,2.1670289430767298,2.1670289430767298)'> ,</span><span style='background-color:rgb(254.39704591990449,0.6029540800955147,0.6029540800955147)'> dry</span><span style='background-color:rgb(253.91112888464704,1.0888711153529584,1.0888711153529584)'> skeleton</span><span style='background-color:rgb(248.64067738875747,6.359322611242533,6.359322611242533)'> of</span><span style='background-color:rgb(251.88961586449295,3.110384135507047,3.110384135507047)'> a</span><span style='background-color:rgb(253.62530632177368,1.374693678226322,1.374693678226322)'> tale</span><span style='background-color:rgb(253.8150901068002,1.1849098931998014,1.1849098931998014)'> lay</span><span style='background-color:rgb(254.8104387553758,0.1895612446242012,0.1895612446242012)'> exposed</span><span style='background-color:rgb(254.24104273784906,0.7589572621509433,0.7589572621509433)'> beneath</span><span style='background-color:rgb(254.107264385093,0.8927356149069965,0.8927356149069965)'> their</span><span style='background-color:rgb(252.7372867288068,2.2627132711932063,2.2627132711932063)'> feet</span><span style='background-color:rgb(251.7073177965358,3.29268220346421,3.29268220346421)'> ,</span><span style='background-color:rgb(254.53253560583107,0.46746439416892827,0.46746439416892827)'> awaiting</span><span style='background-color:rgb(253.75153079861775,1.2484692013822496,1.2484692013822496)'> tend</span><span style='background-color:rgb(252.5056668324396,2.4943331675603986,2.4943331675603986)'> ##ons</span><span style='background-color:rgb(250.6607297435403,4.339270256459713,4.339270256459713)'> ,</span><span style='background-color:rgb(254.0317659988068,0.9682340011931956,0.9682340011931956)'> muscle</span><span style='background-color:rgb(251.57563586253673,3.4243641374632716,3.4243641374632716)'> and</span><span style='background-color:rgb(251.2198360171169,3.7801639828830957,3.7801639828830957)'> blushing</span><span style='background-color:rgb(252.68800432328135,2.3119956767186522,2.3119956767186522)'> skin</span><span style='background-color:rgb(244.95559807866812,10.044401921331882,10.044401921331882)'> .</span><span style='background-color:rgb(249.09668711945415,5.903312880545855,5.903312880545855)'> allan</span><span style='background-color:rgb(243.45593160018325,11.544068399816751,11.544068399816751)'> reviewed</span><span style='background-color:rgb(244.05105661600828,10.948943383991718,10.948943383991718)'> the</span><span style='background-color:rgb(241.07253894209862,13.927461057901382,13.927461057901382)'> troops</span><span style='background-color:rgb(249.0942115522921,5.9057884477078915,5.9057884477078915)'> ,</span><span style='background-color:rgb(249.59569439291954,5.40430560708046,5.40430560708046)'> all</span><span style='background-color:rgb(249.80454781092703,5.195452189072967,5.195452189072967)'> prepared</span><span style='background-color:rgb(253.32091629272327,1.6790837072767317,1.6790837072767317)'> to</span><span style='background-color:rgb(251.9918033061549,3.0081966938450933,3.0081966938450933)'> di</span><span style='background-color:rgb(254.84055720095057,0.1594427990494296,0.1594427990494296)'> ##se</span><span style='background-color:rgb(253.6117894970812,1.3882105029188097,1.3882105029188097)'> ##mba</span><span style='background-color:rgb(253.17621886963025,1.8237811303697526,1.8237811303697526)'> ##rk</span><span style='background-color:rgb(251.36681450530887,3.6331854946911335,3.6331854946911335)'> ,</span><span style='background-color:rgb(251.33012984879315,3.669870151206851,3.669870151206851)'> their</span><span style='background-color:rgb(251.40674656257033,3.5932534374296665,3.5932534374296665)'> task</span><span style='background-color:rgb(252.77169530745596,2.228304692544043,2.228304692544043)'> to</span><span style='background-color:rgb(252.54737434443086,2.452625655569136,2.452625655569136)'> form</span><span style='background-color:rgb(249.18014964088798,5.819850359112024,5.819850359112024)'> the</span><span style='background-color:rgb(252.4881250737235,2.511874926276505,2.511874926276505)'> tale</span><span style='background-color:rgb(248.41299835592508,6.587001644074917,6.587001644074917)'> of</span><span style='background-color:rgb(252.95962229836732,2.0403777016326785,2.0403777016326785)'> a</span><span style='background-color:rgb(254.06056352541782,0.9394364745821804,0.9394364745821804)'> young</span><span style='background-color:rgb(253.36454353295267,1.6354564670473337,1.6354564670473337)'> man</span><span style='background-color:rgb(252.94693739153445,2.0530626084655523,2.0530626084655523)'> returning</span><span style='background-color:rgb(252.6545209949836,2.3454790050163865,2.3454790050163865)'> home</span><span style='background-color:rgb(248.83808344602585,6.161916553974152,6.161916553974152)'> from</span><span style='background-color:rgb(253.36785208666697,1.6321479133330286,1.6321479133330286)'> life</span><span style='background-color:rgb(252.9437614744529,2.056238525547087,2.056238525547087)'> abroad</span><span style='background-color:rgb(253.84204621659592,1.157953783404082,1.157953783404082)'> to</span><span style='background-color:rgb(248.4433587267995,6.556641273200512,6.556641273200512)'> find</span><span style='background-color:rgb(249.59939871914685,5.400601280853152,5.400601280853152)'> his</span><span style='background-color:rgb(253.41555401450023,1.5844459854997694,1.5844459854997694)'> childhood</span><span style='background-color:rgb(251.93993110209703,3.0600688979029655,3.0600688979029655)'> friend</span><span style='background-color:rgb(250.30402863398194,4.695971366018057,4.695971366018057)'> a</span><span style='background-color:rgb(251.5152276540175,3.484772345982492,3.484772345982492)'> bride</span><span style='background-color:rgb(251.96291630622,3.037083693780005,3.037083693780005)'> to</span><span style='background-color:rgb(252.22887923475355,2.771120765246451,2.771120765246451)'> -</span><span style='background-color:rgb(254.38814958790317,0.6118504120968282,0.6118504120968282)'> be</span><span style='background-color:rgb(251.11494597047567,3.8850540295243263,3.8850540295243263)'> ,</span><span style='background-color:rgb(248.097077133134,6.9029228668659925,6.9029228668659925)'> thus</span><span style='background-color:rgb(254.30488536832854,0.6951146316714585,0.6951146316714585)'> upset</span><span style='background-color:rgb(254.9332610714191,0.06673892858088948,0.06673892858088948)'> ##ting</span><span style='background-color:rgb(249.86557348631322,5.134426513686776,5.134426513686776)'> the</span><span style='background-color:rgb(249.69035918824375,5.309640811756253,5.309640811756253)'> apple</span><span style='background-color:rgb(249.39541426487267,5.60458573512733,5.60458573512733)'> cart</span><span style='background-color:rgb(248.92402295954525,6.075977040454745,6.075977040454745)'> of</span><span style='background-color:rgb(253.6371549172327,1.3628450827673078,1.3628450827673078)'> his</span><span style='background-color:rgb(252.58382246363908,2.4161775363609195,2.4161775363609195)'> life</span><span style='background-color:rgb(253.59284513862804,1.4071548613719642,1.4071548613719642)'> '</span><span style='background-color:rgb(253.0315200216137,1.9684799783863127,1.9684799783863127)'> s</span><span style='background-color:rgb(251.7634595464915,3.2365404535084963,3.2365404535084963)'> plan</span><span style='background-color:rgb(240.24548292160034,14.754517078399658,14.754517078399658)'> ,</span><span style='background-color:rgb(253.00736934877932,1.9926306512206793,1.9926306512206793)'> clarified</span><span style='background-color:rgb(252.1826708642766,2.817329135723412,2.817329135723412)'> –</span><span style='background-color:rgb(244.1203287988901,10.879671201109886,10.879671201109886)'> of</span><span style='background-color:rgb(253.4427422680892,1.557257731910795,1.557257731910795)'> course</span><span style='background-color:rgb(251.1025980580598,3.8974019419401884,3.8974019419401884)'> –</span><span style='background-color:rgb(252.06339288968593,2.936607110314071,2.936607110314071)'> by</span><span style='background-color:rgb(254.30469620972872,0.6953037902712822,0.6953037902712822)'> his</span><span style='background-color:rgb(254.05552202719264,0.9444779728073627,0.9444779728073627)'> very</span><span style='background-color:rgb(251.91282976884395,3.087170231156051,3.087170231156051)'> time</span><span style='background-color:rgb(253.5551096010022,1.4448903989978135,1.4448903989978135)'> away</span><span style='background-color:rgb(238.03255766630173,16.967442333698273,16.967442333698273)'> from</span><span style='background-color:rgb(245.7892875932157,9.210712406784296,9.210712406784296)'> her</span><span style='background-color:rgb(251.88962417654693,3.1103758234530687,3.1103758234530687)'> he</span><span style='background-color:rgb(246.6864707507193,8.313529249280691,8.313529249280691)'> loved</span><span style='background-color:rgb(251.76144114229828,3.238558857701719,3.238558857701719)'> best</span><span style='background-color:rgb(251.8047516932711,3.1952483067288995,3.1952483067288995)'> .</span><span style='background-color:rgb(250.4124923888594,4.587507611140609,4.587507611140609)'> although</span><span style='background-color:rgb(247.45480757206678,7.545192427933216,7.545192427933216)'> the</span><span style='background-color:rgb(251.6563240578398,3.343675942160189,3.343675942160189)'> concept</span><span style='background-color:rgb(253.00539321731776,1.99460678268224,1.99460678268224)'> was</span><span style='background-color:rgb(253.92912863753736,1.0708713624626398,1.0708713624626398)'> a</span><span style='background-color:rgb(253.71563744964078,1.2843625503592193,1.2843625503592193)'> simple</span><span style='background-color:rgb(253.3948578312993,1.605142168700695,1.605142168700695)'> one</span><span style='background-color:rgb(253.07910237461329,1.920897625386715,1.920897625386715)'> ,</span><span style='background-color:rgb(251.90042794682086,3.099572053179145,3.099572053179145)'> allan</span><span style='background-color:rgb(253.56728830374777,1.4327116962522268,1.4327116962522268)'> thought</span><span style='background-color:rgb(253.71216574218124,1.2878342578187585,1.2878342578187585)'> it</span><span style='background-color:rgb(252.25490878801793,2.7450912119820714,2.7450912119820714)'> had</span><span style='background-color:rgb(253.69148024590686,1.3085197540931404,1.3085197540931404)'> potential</span><span style='background-color:rgb(248.35264381952584,6.647356180474162,6.647356180474162)'> .</span><span style='background-color:rgb(251.02062933146954,3.979370668530464,3.979370668530464)'> besides</span><span style='background-color:rgb(252.2464010445401,2.7535989554598927,2.7535989554598927)'> ,</span><span style='background-color:rgb(253.31437814980745,1.6856218501925468,1.6856218501925468)'> the</span><span style='background-color:rgb(252.25594708230346,2.7440529176965356,2.7440529176965356)'> public</span><span style='background-color:rgb(250.48438453115523,4.515615468844771,4.515615468844771)'> liked</span><span style='background-color:rgb(253.29651530832052,1.7034846916794777,1.7034846916794777)'> a</span><span style='background-color:rgb(240.62909418717027,14.370905812829733,14.370905812829733)'> good</span><span style='background-color:rgb(251.59108108375221,3.408918916247785,3.408918916247785)'> ,</span><span style='background-color:rgb(254.9180193025677,0.08198069743230008,0.08198069743230008)'> simple</span><span style='background-color:rgb(251.22166181914508,3.7783381808549166,3.7783381808549166)'> romance</span><span style='background-color:rgb(245.13170152902603,9.868298470973969,9.868298470973969)'> .</span><span style='background-color:rgb(253.56662274571136,1.4333772542886436,1.4333772542886436)'> perhaps</span><span style='background-color:rgb(253.53618210414425,1.4638178958557546,1.4638178958557546)'> this</span><span style='background-color:rgb(249.56896947696805,5.43103052303195,5.43103052303195)'> will</span><span style='background-color:rgb(252.00162910390645,2.9983708960935473,2.9983708960935473)'> be</span><span style='background-color:rgb(252.87095049396157,2.1290495060384274,2.1290495060384274)'> more</span><span style='background-color:rgb(253.67371548665687,1.3262845133431256,1.3262845133431256)'> sale</span> ##able<span style='background-color:rgb(247.33241993933916,7.667580060660839,7.667580060660839)'> ,</span><span style='background-color:rgb(218.39361548423767,36.60638451576233,36.60638451576233)'> he</span><span style='background-color:rgb(248.50342685356736,6.496573146432638,6.496573146432638)'> thought</span><span style='background-color:rgb(253.63689000019804,1.3631099998019636,1.3631099998019636)'> and</span><span style='background-color:rgb(251.97761581745,3.022384182550013,3.022384182550013)'> began</span><span style='background-color:rgb(253.68769398657605,1.3123060134239495,1.3123060134239495)'> to</span><span style='background-color:rgb(251.23795985709876,3.762040142901242,3.762040142901242)'> write</span><span style='background-color:rgb(244.42065803334117,10.57934196665883,10.57934196665883)'> .</span><span style='background-color:rgb(247.07229017280042,7.927709827199578,7.927709827199578)'> they</span><span style='background-color:rgb(250.20473948679864,4.795260513201356,4.795260513201356)'> gazed</span><span style='background-color:rgb(248.67221902124584,6.327780978754163,6.327780978754163)'> at</span><span style='background-color:rgb(249.2014764714986,5.798523528501391,5.798523528501391)'> each</span><span style='background-color:rgb(253.09230120386928,1.9076987961307168,1.9076987961307168)'> other</span><span style='background-color:rgb(251.14433620590717,3.855663794092834,3.855663794092834)'> ,</span><span style='background-color:rgb(248.82688687182963,6.173113128170371,6.173113128170371)'> lost</span><span style='background-color:rgb(253.2545549911447,1.745445008855313,1.745445008855313)'> in</span><span style='background-color:rgb(252.6981360046193,2.3018639953806996,2.3018639953806996)'> the</span><span style='background-color:rgb(252.28395514190197,2.71604485809803,2.71604485809803)'> rap</span><span style='background-color:rgb(252.1183434035629,2.8816565964370966,2.8816565964370966)'> ##ture</span><span style='background-color:rgb(253.8708831067197,1.1291168932802975,1.1291168932802975)'> of</span><span style='background-color:rgb(250.57553736492991,4.4244626350700855,4.4244626350700855)'> love</span><span style='background-color:rgb(252.5806140108034,2.4193859891965985,2.4193859891965985)'> based</span><span style='background-color:rgb(252.73006236646324,2.269937633536756,2.269937633536756)'> so</span><span style='background-color:rgb(253.52121743839234,1.4787825616076589,1.4787825616076589)'> deeply</span><span style='background-color:rgb(253.6625976394862,1.3374023605138063,1.3374023605138063)'> within</span><span style='background-color:rgb(249.70913683064282,5.290863169357181,5.290863169357181)'> their</span><span style='background-color:rgb(252.51435910351574,2.485640896484256,2.485640896484256)'> hearts</span><span style='background-color:rgb(249.9225595034659,5.077440496534109,5.077440496534109)'> that</span><span style='background-color:rgb(253.90489840647206,1.095101593527943,1.095101593527943)'> they</span><span style='background-color:rgb(246.53311002999544,8.466889970004559,8.466889970004559)'> had</span><span style='background-color:rgb(253.00905158976093,1.9909484102390707,1.9909484102390707)'> never</span><span style='background-color:rgb(254.1766335221473,0.8233664778526872,0.8233664778526872)'> seen</span><span style='background-color:rgb(253.69065082166344,1.3093491783365607,1.3093491783365607)'> it</span><span style='background-color:rgb(253.02677882602438,1.9732211739756167,1.9732211739756167)'> before</span><span style='background-color:rgb(243.26593989506364,11.734060104936361,11.734060104936361)'> .</span><span style='background-color:rgb(237.99370665103197,17.00629334896803,17.00629334896803)'> "</span><span style='background-color:rgb(248.9670181274414,6.032981872558594,6.032981872558594)'> what</span><span style='background-color:rgb(239.40550854429603,15.594491455703974,15.594491455703974)'> about</span><span style='background-color:rgb(245.92637283727527,9.073627162724733,9.073627162724733)'> roger</span><span style='background-color:rgb(239.29011538624763,15.709884613752365,15.709884613752365)'> ?</span><span style='background-color:rgb(231.22790560126305,23.772094398736954,23.772094398736954)'> "</span><span style='background-color:rgb(245.96308076754212,9.036919232457876,9.036919232457876)'> she</span><span style='background-color:rgb(232.2216510772705,22.778348922729492,22.778348922729492)'> asked</span><span style='background-color:rgb(235.1755403727293,19.8244596272707,19.8244596272707)'> ,</span><span style='background-color:rgb(226.44040379673243,28.559596203267574,28.559596203267574)'> knowing</span><span style='background-color:rgb(220.4923378676176,34.50766213238239,34.50766213238239)'> that</span><span style='background-color:rgb(223.5893939808011,31.410606019198895,31.410606019198895)'> the</span><span style='background-color:rgb(221.72189868986607,33.278101310133934,33.278101310133934)'> answer</span><span style='background-color:rgb(235.71974907070398,19.280250929296017,19.280250929296017)'> no</span><span style='background-color:rgb(219.44080129265785,35.55919870734215,35.55919870734215)'> longer</span><span style='background-color:rgb(217.03186735510826,37.96813264489174,37.96813264489174)'> mattered</span><span style='background-color:rgb(214.18200887739658,40.817991122603416,40.817991122603416)'> .</span><span style='background-color:rgb(207.72561572492123,47.27438427507877,47.27438427507877)'> [SEP]</span><span style='background-color:rgb(246.17191471159458,8.828085288405418,8.828085288405418)'> name</span><span style='background-color:rgb(242.79335735365748,12.206642646342516,12.206642646342516)'> few</span><span style='background-color:rgb(247.10982408374548,7.89017591625452,7.89017591625452)'> objects</span><span style='background-color:rgb(248.05042181164026,6.949578188359737,6.949578188359737)'> said</span><span style='background-color:rgb(248.74392449855804,6.256075501441956,6.256075501441956)'> to</span><span style='background-color:rgb(251.4951457316056,3.5048542683944106,3.5048542683944106)'> be</span><span style='background-color:rgb(247.71440774202347,7.285592257976532,7.285592257976532)'> in</span><span style='background-color:rgb(251.37150820344687,3.628491796553135,3.628491796553135)'> or</span><span style='background-color:rgb(246.05653485283256,8.943465147167444,8.943465147167444)'> on</span><span style='background-color:rgb(248.57510620728135,6.424893792718649,6.424893792718649)'> allan</span><span style='background-color:rgb(247.70479425787926,7.295205742120743,7.295205742120743)'> '</span><span style='background-color:rgb(248.13934131525457,6.860658684745431,6.860658684745431)'> s</span><span style='background-color:rgb(246.17053348571062,8.82946651428938,8.82946651428938)'> desk</span><span style='background-color:rgb(238.41464806348085,16.585351936519146,16.585351936519146)'> |</span><span style='background-color:rgb(229.12493884563446,25.87506115436554,25.87506115436554)'> |</span><span style='background-color:rgb(235.1291847601533,19.870815239846706,19.870815239846706)'> erase</span><span style='background-color:rgb(226.26362778246403,28.736372217535973,28.736372217535973)'> ##r</span><span style='background-color:rgb(176.50061264634132,78.49938735365868,78.49938735365868)'> [SEP]</span><span style='background-color:rgb(244.29074300453067,10.709256995469332,10.709256995469332)'> [PAD]</span><span style='background-color:rgb(249.40056868828833,5.599431311711669,5.599431311711669)'> [PAD]</span><span style='background-color:rgb(250.6292379833758,4.370762016624212,4.370762016624212)'> [PAD]</span><span style='background-color:rgb(250.12486254796386,4.875137452036142,4.875137452036142)'> [PAD]</span><span style='background-color:rgb(251.17165982723236,3.828340172767639,3.828340172767639)'> [PAD]</span><span style='background-color:rgb(249.1384345293045,5.861565470695496,5.861565470695496)'> [PAD]</span><span style='background-color:rgb(253.13628123141825,1.863718768581748,1.863718768581748)'> [PAD]</span><span style='background-color:rgb(253.30230180406943,1.6976981959305704,1.6976981959305704)'> [PAD]</span><span style='background-color:rgb(253.28178848605603,1.7182115139439702,1.7182115139439702)'> [PAD]</span><span style='background-color:rgb(254.5815545489313,0.4184454510686919,0.4184454510686919)'> [PAD]</span><span style='background-color:rgb(254.18422711780295,0.8157728821970522,0.8157728821970522)'> [PAD]</span><span style='background-color:rgb(254.11691159242764,0.8830884075723588,0.8830884075723588)'> [PAD]</span><span style='background-color:rgb(254.26966992672533,0.730330073274672,0.730330073274672)'> [PAD]</span><span style='background-color:rgb(254.78723320760764,0.21276679239235818,0.21276679239235818)'> [PAD]</span><span style='background-color:rgb(254.515393122565,0.48460687743499875,0.48460687743499875)'> [PAD]</span><span style='background-color:rgb(253.15549727529287,1.8445027247071266,1.8445027247071266)'> [PAD]</span><span style='background-color:rgb(254.71830912167206,0.28169087832793593,0.28169087832793593)'> [PAD]</span><span style='background-color:rgb(252.87753934040666,2.1224606595933437,2.1224606595933437)'> [PAD]</span><span style='background-color:rgb(253.39697930496186,1.6030206950381398,1.6030206950381398)'> [PAD]</span><span style='background-color:rgb(254.22479451284744,0.7752054871525615,0.7752054871525615)'> [PAD]</span><span style='background-color:rgb(253.76978264423087,1.2302173557691276,1.2302173557691276)'> [PAD]</span><span style='background-color:rgb(254.23621295951307,0.7637870404869318,0.7637870404869318)'> [PAD]</span><span style='background-color:rgb(253.67963260039687,1.3203673996031284,1.3203673996031284)'> [PAD]</span><span style='background-color:rgb(253.48907460691407,1.5109253930859268,1.5109253930859268)'> [PAD]</span><span style='background-color:rgb(252.59749436751008,2.4025056324899197,2.4025056324899197)'> [PAD]</span><span style='background-color:rgb(251.48109836038202,3.5189016396179795,3.5189016396179795)'> [PAD]</span><span style='background-color:rgb(251.1702218418941,3.82977815810591,3.82977815810591)'> [PAD]</span><span style='background-color:rgb(252.07635280676186,2.923647193238139,2.923647193238139)'> [PAD]</span><span style='background-color:rgb(249.51778147369623,5.482218526303768,5.482218526303768)'> [PAD]</span><span style='background-color:rgb(253.38331856299192,1.6166814370080829,1.6166814370080829)'> [PAD]</span><span style='background-color:rgb(253.54922917904332,1.4507708209566772,1.4507708209566772)'> [PAD]</span><span style='background-color:rgb(253.898287829943,1.1017121700569987,1.1017121700569987)'> [PAD]</span><span style='background-color:rgb(253.1502645998262,1.849735400173813,1.849735400173813)'> [PAD]</span><span style='background-color:rgb(254.31552046327852,0.6844795367214829,0.6844795367214829)'> [PAD]</span><span style='background-color:rgb(249.90523908287287,5.094760917127132,5.094760917127132)'> [PAD]</span><span style='background-color:rgb(243.36328307166696,11.636716928333044,11.636716928333044)'> [PAD]</span><span style='background-color:rgb(225.16789197921753,29.83210802078247,29.83210802078247)'> [PAD]</span><span style='background-color:rgb(245.96382647752762,9.036173522472382,9.036173522472382)'> [PAD]</span><span style='background-color:rgb(243.16362753510475,11.836372464895248,11.836372464895248)'> [PAD]</span><span style='background-color:rgb(252.91555036325008,2.0844496367499232,2.0844496367499232)'> [PAD]</span><span style='background-color:rgb(254.4063026981894,0.5936973018106073,0.5936973018106073)'> [PAD]</span><span style='background-color:rgb(254.41100631142035,0.5889936885796487,0.5889936885796487)'> [PAD]</span><span style='background-color:rgb(253.57742794090882,1.4225720590911806,1.4225720590911806)'> [PAD]</span><span style='background-color:rgb(249.94709241203964,5.052907587960362,5.052907587960362)'> [PAD]</span><span style='background-color:rgb(253.88750543352216,1.1124945664778352,1.1124945664778352)'> [PAD]</span><span style='background-color:rgb(253.81295426515862,1.1870457348413765,1.1870457348413765)'> [PAD]</span><span style='background-color:rgb(254.01836987701245,0.9816301229875535,0.9816301229875535)'> [PAD]</span><span style='background-color:rgb(253.95053146407008,1.0494685359299183,1.0494685359299183)'> [PAD]</span><span style='background-color:rgb(250.3102892730385,4.6897107269614935,4.6897107269614935)'> [PAD]</span><span style='background-color:rgb(248.05327355861664,6.946726441383362,6.946726441383362)'> [PAD]</span><span style='background-color:rgb(240.71281414479017,14.287185855209827,14.287185855209827)'> [PAD]</span><span style='background-color:rgb(246.9882315583527,8.011768441647291,8.011768441647291)'> [PAD]</span><span style='background-color:rgb(248.36743880063295,6.632561199367046,6.632561199367046)'> [PAD]</span><span style='background-color:rgb(249.2509493418038,5.749050658196211,5.749050658196211)'> [PAD]</span><span style='background-color:rgb(248.98353821597993,6.016461784020066,6.016461784020066)'> [PAD]</span><span style='background-color:rgb(249.4361195806414,5.563880419358611,5.563880419358611)'> [PAD]</span><span style='background-color:rgb(252.8795499075204,2.1204500924795866,2.1204500924795866)'> [PAD]</span><span style='background-color:rgb(253.39197414228693,1.6080258577130735,1.6080258577130735)'> [PAD]</span><span style='background-color:rgb(254.21442825347185,0.7855717465281487,0.7855717465281487)'> [PAD]</span><span style='background-color:rgb(253.58522369759157,1.414776302408427,1.414776302408427)'> [PAD]</span><span style='background-color:rgb(252.65877344179899,2.341226558201015,2.341226558201015)'> [PAD]</span><span style='background-color:rgb(249.8689961526543,5.13100384734571,5.13100384734571)'> [PAD]</span><span style='background-color:rgb(253.84960614843294,1.1503938515670598,1.1503938515670598)'> [PAD]</span><span style='background-color:rgb(252.41242743562907,2.58757256437093,2.58757256437093)'> [PAD]</span><span style='background-color:rgb(253.1021596561186,1.8978403438813984,1.8978403438813984)'> [PAD]</span><span style='background-color:rgb(253.65556184202433,1.3444381579756737,1.3444381579756737)'> [PAD]</span><span style='background-color:rgb(253.16238535568118,1.837614644318819,1.837614644318819)'> [PAD]</span><span style='background-color:rgb(248.59420208260417,6.40579791739583,6.40579791739583)'> [PAD]</span><span style='background-color:rgb(251.47499470040202,3.5250052995979786,3.5250052995979786)'> [PAD]</span><span style='background-color:rgb(249.25327054224908,5.7467294577509165,5.7467294577509165)'> [PAD]</span><span style='background-color:rgb(252.25680773612112,2.743192263878882,2.743192263878882)'> [PAD]</span><span style='background-color:rgb(253.72298839269206,1.2770116073079407,1.2770116073079407)'> [PAD]</span><span style='background-color:rgb(249.78438751772046,5.215612482279539,5.215612482279539)'> [PAD]</span><span style='background-color:rgb(254.21595624648035,0.7840437535196543,0.7840437535196543)'> [PAD]</span><span style='background-color:rgb(253.40891090221703,1.5910890977829695,1.5910890977829695)'> [PAD]</span><span style='background-color:rgb(249.53493755310774,5.4650624468922615,5.4650624468922615)'> [PAD]</span><span style='background-color:rgb(254.37939242343418,0.620607576565817,0.620607576565817)'> [PAD]</span><span style='background-color:rgb(253.84807958034799,1.151920419652015,1.151920419652015)'> [PAD]</span><span style='background-color:rgb(252.59713338688016,2.4028666131198406,2.4028666131198406)'> [PAD]</span><span style='background-color:rgb(253.46175621030852,1.538243789691478,1.538243789691478)'> [PAD]</span><span style='background-color:rgb(252.0295146200806,2.9704853799194098,2.9704853799194098)'> [PAD]</span><span style='background-color:rgb(249.49132919311523,5.508670806884766,5.508670806884766)'> [PAD]</span><span style='background-color:rgb(254.10950673976913,0.890493260230869,0.890493260230869)'> [PAD]</span><span style='background-color:rgb(241.7759676463902,13.2240323536098,13.2240323536098)'> [PAD]</span><span style='background-color:rgb(250.54338491521776,4.456615084782243,4.456615084782243)'> [PAD]</span><span style='background-color:rgb(253.2674207445234,1.732579255476594,1.732579255476594)'> [PAD]</span><span style='background-color:rgb(252.5208781287074,2.479121871292591,2.479121871292591)'> [PAD]</span><span style='background-color:rgb(254.1785904765129,0.8214095234870911,0.8214095234870911)'> [PAD]</span><span style='background-color:rgb(253.58192001236603,1.4180799876339734,1.4180799876339734)'> [PAD]</span><span style='background-color:rgb(252.4863403569907,2.513659643009305,2.513659643009305)'> [PAD]</span><span style='background-color:rgb(251.96366652846336,3.0363334715366364,3.0363334715366364)'> [PAD]</span><span style='background-color:rgb(249.1108722332865,5.8891277667135,5.8891277667135)'> [PAD]</span><span style='background-color:rgb(253.3238871395588,1.6761128604412079,1.6761128604412079)'> [PAD]</span><span style='background-color:rgb(252.7480064285919,2.251993571408093,2.251993571408093)'> [PAD]</span><span style='background-color:rgb(252.90315400343388,2.096845996566117,2.096845996566117)'> [PAD]</span><span style='background-color:rgb(254.40260169678368,0.5973983032163233,0.5973983032163233)'> [PAD]</span><span style='background-color:rgb(251.6430188342929,3.3569811657071114,3.3569811657071114)'> [PAD]</span><span style='background-color:rgb(254.07594450633042,0.9240554936695844,0.9240554936695844)'> [PAD]</span><span style='background-color:rgb(253.58133163768798,1.4186683623120189,1.4186683623120189)'> [PAD]</span><span style='background-color:rgb(253.4483111067675,1.5516888932324946,1.5516888932324946)'> [PAD]</span><span style='background-color:rgb(249.12850328721106,5.8714967127889395,5.8714967127889395)'> [PAD]</span><span style='background-color:rgb(253.38296993169934,1.6170300683006644,1.6170300683006644)'> [PAD]</span><span style='background-color:rgb(253.7747474340722,1.2252525659278035,1.2252525659278035)'> [PAD]</span><span style='background-color:rgb(253.24130937689915,1.758690623100847,1.758690623100847)'> [PAD]</span><span style='background-color:rgb(250.92613315209746,4.073866847902536,4.073866847902536)'> [PAD]</span><span style='background-color:rgb(252.290332862176,2.709667137823999,2.709667137823999)'> [PAD]</span><span style='background-color:rgb(236.19368623942137,18.806313760578632,18.806313760578632)'> [PAD]</span><span style='background-color:rgb(253.21045527001843,1.7895447299815714,1.7895447299815714)'> [PAD]</span><span style='background-color:rgb(252.7248925063759,2.275107493624091,2.275107493624091)'> [PAD]</span><span style='background-color:rgb(253.98417509626597,1.0158249037340283,1.0158249037340283)'> [PAD]</span><span style='background-color:rgb(254.13296650280245,0.8670334971975535,0.8670334971975535)'> [PAD]</span><span style='background-color:rgb(253.79392239265144,1.2060776073485613,1.2060776073485613)'> [PAD]</span><span style='background-color:rgb(253.9985525747761,1.001447425223887,1.001447425223887)'> [PAD]</span><span style='background-color:rgb(252.15957950334996,2.84042049665004,2.84042049665004)'> [PAD]</span><span style='background-color:rgb(250.97687230445445,4.023127695545554,4.023127695545554)'> [PAD]</span><span style='background-color:rgb(253.01939392229542,1.9806060777045786,1.9806060777045786)'> [PAD]</span><span style='background-color:rgb(250.53654860705137,4.4634513929486275,4.4634513929486275)'> [PAD]</span><span style='background-color:rgb(245.77546298503876,9.224537014961243,9.224537014961243)'> [PAD]</span><span style='background-color:rgb(250.6575906369835,4.342409363016486,4.342409363016486)'> [PAD]</span><span style='background-color:rgb(250.10499198921025,4.895008010789752,4.895008010789752)'> [PAD]</span><span style='background-color:rgb(253.5015185829252,1.4984814170747995,1.4984814170747995)'> [PAD]</span><span style='background-color:rgb(243.27483901754022,11.725160982459784,11.725160982459784)'> [PAD]</span><span style='background-color:rgb(250.04954298958182,4.950457010418177,4.950457010418177)'> [PAD]</span><span style='background-color:rgb(217.80553624033928,37.19446375966072,37.19446375966072)'> [PAD]</span><span style='background-color:rgb(247.3191002290696,7.680899770930409,7.680899770930409)'> [PAD]</span><span style='background-color:rgb(249.11990102380514,5.8800989761948586,5.8800989761948586)'> [PAD]</span><span style='background-color:rgb(249.1502048727125,5.849795127287507,5.849795127287507)'> [PAD]</span><span style='background-color:rgb(243.95512506365776,11.04487493634224,11.04487493634224)'> [PAD]</span><span style='background-color:rgb(233.78650210797787,21.213497892022133,21.213497892022133)'> [PAD]</span><span style='background-color:rgb(238.1491544097662,16.850845590233803,16.850845590233803)'> [PAD]</span><span style='background-color:rgb(252.85133689641953,2.148663103580475,2.148663103580475)'> [PAD]</span><span style='background-color:rgb(245.26620101183653,9.733798988163471,9.733798988163471)'> [PAD]</span><span style='background-color:rgb(252.55301300436258,2.446986995637417,2.446986995637417)'> [PAD]</span><span style='background-color:rgb(252.53625922836363,2.463740771636367,2.463740771636367)'> [PAD]</span><span style='background-color:rgb(251.712246844545,3.2877531554549932,3.2877531554549932)'> [PAD]</span><span style='background-color:rgb(253.3445542305708,1.6554457694292068,1.6554457694292068)'> [PAD]</span><span style='background-color:rgb(245.33461444079876,9.66538555920124,9.66538555920124)'> [PAD]</span><span style='background-color:rgb(249.90722067654133,5.092779323458672,5.092779323458672)'> [PAD]</span><span style='background-color:rgb(249.0369329508394,5.9630670491606,5.9630670491606)'> [PAD]</span><span style='background-color:rgb(250.06633998826146,4.933660011738539,4.933660011738539)'> [PAD]</span><span style='background-color:rgb(250.34658112563193,4.653418874368072,4.653418874368072)'> [PAD]</span><span style='background-color:rgb(248.50114935077727,6.498850649222732,6.498850649222732)'> [PAD]</span><span style='background-color:rgb(252.335414593108,2.6645854068920016,2.6645854068920016)'> [PAD]</span><span style='background-color:rgb(241.95168256759644,13.048317432403564,13.048317432403564)'> [PAD]</span><span style='background-color:rgb(250.85583597421646,4.144164025783539,4.144164025783539)'> [PAD]</span><span style='background-color:rgb(247.7548513468355,7.245148653164506,7.245148653164506)'> [PAD]</span><span style='background-color:rgb(251.75554433371872,3.244455666281283,3.244455666281283)'> [PAD]</span><span style='background-color:rgb(250.0887117628008,4.911288237199187,4.911288237199187)'> [PAD]</span><span style='background-color:rgb(251.03460616897792,3.9653938310220838,3.9653938310220838)'> [PAD]</span><span style='background-color:rgb(243.33776269108057,11.66223730891943,11.66223730891943)'> [PAD]</span><span style='background-color:rgb(216.27991810441017,38.72008189558983,38.72008189558983)'> [PAD]</span><span style='background-color:rgb(249.66839351691306,5.331606483086944,5.331606483086944)'> [PAD]</span><span style='background-color:rgb(235.85055135190487,19.14944864809513,19.14944864809513)'> [PAD]</span><span style='background-color:rgb(250.69721681065857,4.302783189341426,4.302783189341426)'> [PAD]</span><span style='background-color:rgb(251.18588198907673,3.8141180109232664,3.8141180109232664)'> [PAD]</span><span style='background-color:rgb(186.25363290309906,68.74636709690094,68.74636709690094)'> [PAD]</span><span style='background-color:rgb(235.77450033277273,19.225499667227268,19.225499667227268)'> [PAD]</span>

## AMPE

In [15]:
from exlib.explainers.ampe import AmpeTextCls
N = 5
num_steps = 4
ampe_explainer = AmpeTextCls(original_model2, projection_layer,
                            N=N, num_steps=num_steps)
ampe_expln = ampe_explainer(inputs, labels) #, x_kwargs=kwargs)

In [16]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

def show_attrs(tokens, masks, normalize=True):
    if normalize:
        masks_max = masks.max(-1).values
        masks_min = masks.min(-1).values
        # import pdb; pdb.set_trace()
        masks = (masks - masks_min) / (masks_max - masks_min)
    print_str = ""
    for i in range(len(tokens)):
        expln_val = masks[i].item()
        if expln_val == 0:
            print_str += f" {tokens[i]}"
        else:
            print_str += f"<span style='background-color:rgb({255 * (1-expln_val)},{255 * expln_val},{255 * expln_val})'> {tokens[i]}</span>"
    return print_str

tokens = processor.convert_ids_to_tokens(inputs[0])
# tokens = shap_expln.explainer_output.data[0]
printmd(show_attrs(tokens, ampe_expln.attributions[0]))

<span style='background-color:rgb(56.21871292591095,198.78128707408905,198.78128707408905)'> [CLS]</span><span style='background-color:rgb(148.8811182975769,106.1188817024231,106.1188817024231)'> allan</span><span style='background-color:rgb(144.18402314186096,110.81597685813904,110.81597685813904)'> sat</span><span style='background-color:rgb(41.416683197021484,213.58331680297852,213.58331680297852)'> down</span><span style='background-color:rgb(0.0,255.0,255.0)'> at</span><span style='background-color:rgb(116.38299733400345,138.61700266599655,138.61700266599655)'> his</span><span style='background-color:rgb(172.73302361369133,82.26697638630867,82.26697638630867)'> desk</span><span style='background-color:rgb(95.28637737035751,159.7136226296425,159.7136226296425)'> and</span><span style='background-color:rgb(82.34373986721039,172.6562601327896,172.6562601327896)'> pulled</span><span style='background-color:rgb(126.22215896844864,128.77784103155136,128.77784103155136)'> the</span><span style='background-color:rgb(194.43168707191944,60.56831292808056,60.56831292808056)'> chair</span><span style='background-color:rgb(215.39775744080544,39.602242559194565,39.602242559194565)'> in</span><span style='background-color:rgb(212.7656576782465,42.2343423217535,42.2343423217535)'> close</span><span style='background-color:rgb(145.99277168512344,109.00722831487656,109.00722831487656)'> .</span><span style='background-color:rgb(114.17037606239319,140.8296239376068,140.8296239376068)'> opening</span><span style='background-color:rgb(162.63817369937897,92.36182630062103,92.36182630062103)'> a</span><span style='background-color:rgb(214.38733845949173,40.61266154050827,40.61266154050827)'> side</span><span style='background-color:rgb(200.2703207731247,54.729679226875305,54.729679226875305)'> drawer</span><span style='background-color:rgb(134.1172233223915,120.88277667760849,120.88277667760849)'> ,</span><span style='background-color:rgb(122.64510691165924,132.35489308834076,132.35489308834076)'> he</span><span style='background-color:rgb(134.9473039805889,120.05269601941109,120.05269601941109)'> took</span><span style='background-color:rgb(123.61674517393112,131.38325482606888,131.38325482606888)'> out</span><span style='background-color:rgb(141.9508145749569,113.0491854250431,113.0491854250431)'> a</span><span style='background-color:rgb(200.47823041677475,54.52176958322525,54.52176958322525)'> piece</span><span style='background-color:rgb(171.17508441209793,83.82491558790207,83.82491558790207)'> of</span><span style='background-color:rgb(149.7650496661663,105.2349503338337,105.2349503338337)'> paper</span><span style='background-color:rgb(169.41188022494316,85.58811977505684,85.58811977505684)'> and</span><span style='background-color:rgb(197.22722366452217,57.77277633547783,57.77277633547783)'> his</span><span style='background-color:rgb(204.55652117729187,50.44347882270813,50.44347882270813)'> ink</span><span style='background-color:rgb(203.26814331114292,51.73185668885708,51.73185668885708)'> ##pot</span><span style='background-color:rgb(188.89466598629951,66.10533401370049,66.10533401370049)'> .</span><span style='background-color:rgb(205.60884051024914,49.39115948975086,49.39115948975086)'> after</span><span style='background-color:rgb(226.0030035674572,28.9969964325428,28.9969964325428)'> filling</span><span style='background-color:rgb(216.22083887457848,38.779161125421524,38.779161125421524)'> his</span><span style='background-color:rgb(218.88270244002342,36.11729755997658,36.11729755997658)'> pen</span><span style='background-color:rgb(207.64119565486908,47.35880434513092,47.35880434513092)'> ,</span><span style='background-color:rgb(218.56802992522717,36.431970074772835,36.431970074772835)'> allan</span><span style='background-color:rgb(204.82072480022907,50.17927519977093,50.17927519977093)'> looked</span><span style='background-color:rgb(185.24803206324577,69.75196793675423,69.75196793675423)'> at</span><span style='background-color:rgb(156.87853395938873,98.12146604061127,98.12146604061127)'> his</span><span style='background-color:rgb(145.7767228782177,109.2232771217823,109.2232771217823)'> paper</span><span style='background-color:rgb(177.38180816173553,77.61819183826447,77.61819183826447)'> in</span><span style='background-color:rgb(210.9809884428978,44.0190115571022,44.0190115571022)'> the</span><span style='background-color:rgb(219.4660471379757,35.53395286202431,35.53395286202431)'> orange</span><span style='background-color:rgb(219.17375162243843,35.82624837756157,35.82624837756157)'> glow</span><span style='background-color:rgb(206.16914324462414,48.83085675537586,48.83085675537586)'> from</span><span style='background-color:rgb(203.15277010202408,51.84722989797592,51.84722989797592)'> the</span><span style='background-color:rgb(213.63478124141693,41.36521875858307,41.36521875858307)'> lantern</span><span style='background-color:rgb(147.5308683514595,107.4691316485405,107.4691316485405)'> set</span><span style='background-color:rgb(95.24020224809647,159.75979775190353,159.75979775190353)'> back</span><span style='background-color:rgb(104.84104067087173,150.15895932912827,150.15895932912827)'> in</span><span style='background-color:rgb(115.30143857002258,139.69856142997742,139.69856142997742)'> the</span><span style='background-color:rgb(161.7097543179989,93.29024568200111,93.29024568200111)'> desk</span><span style='background-color:rgb(178.1823492050171,76.81765079498291,76.81765079498291)'> '</span><span style='background-color:rgb(197.94271767139435,57.05728232860565,57.05728232860565)'> s</span><span style='background-color:rgb(193.57351832091808,61.42648167908192,61.42648167908192)'> right</span><span style='background-color:rgb(161.60852774977684,93.39147225022316,93.39147225022316)'> -</span><span style='background-color:rgb(180.32387629151344,74.67612370848656,74.67612370848656)'> hand</span><span style='background-color:rgb(218.21899585425854,36.78100414574146,36.78100414574146)'> corner</span><span style='background-color:rgb(227.94775061309338,27.052249386906624,27.052249386906624)'> .</span><span style='background-color:rgb(229.8848220705986,25.115177929401398,25.115177929401398)'> his</span><span style='background-color:rgb(236.10032334923744,18.899676650762558,18.899676650762558)'> pen</span><span style='background-color:rgb(235.3905100375414,19.60948996245861,19.60948996245861)'> cast</span><span style='background-color:rgb(225.63682481646538,29.363175183534622,29.363175183534622)'> a</span><span style='background-color:rgb(239.53881964087486,15.461180359125137,15.461180359125137)'> forbid</span><span style='background-color:rgb(243.04113160818815,11.958868391811848,11.958868391811848)'> ##ding</span><span style='background-color:rgb(235.71898721158504,19.281012788414955,19.281012788414955)'> line</span><span style='background-color:rgb(230.5587501078844,24.441249892115593,24.441249892115593)'> of</span><span style='background-color:rgb(235.01023594290018,19.98976405709982,19.98976405709982)'> shadow</span><span style='background-color:rgb(240.00560749322176,14.99439250677824,14.99439250677824)'> slant</span><span style='background-color:rgb(228.99108722805977,26.00891277194023,26.00891277194023)'> ##ing</span><span style='background-color:rgb(230.3512793406844,24.648720659315586,24.648720659315586)'> across</span><span style='background-color:rgb(229.48380298912525,25.51619701087475,25.51619701087475)'> the</span><span style='background-color:rgb(235.3569369390607,19.643063060939312,19.643063060939312)'> page</span><span style='background-color:rgb(229.74385533481836,25.256144665181637,25.256144665181637)'> ,</span><span style='background-color:rgb(242.1320417895913,12.867958210408688,12.867958210408688)'> echoing</span><span style='background-color:rgb(226.82564802467823,28.17435197532177,28.17435197532177)'> the</span><span style='background-color:rgb(232.79733918607235,22.20266081392765,22.20266081392765)'> ink</span><span style='background-color:rgb(231.20197009295225,23.79802990704775,23.79802990704775)'> ##y</span><span style='background-color:rgb(230.2232338115573,24.776766188442707,24.776766188442707)'> darkness</span><span style='background-color:rgb(227.4226339906454,27.57736600935459,27.57736600935459)'> crouch</span><span style='background-color:rgb(190.69883957505226,64.30116042494774,64.30116042494774)'> ##ing</span><span style='background-color:rgb(200.47708667814732,54.522913321852684,54.522913321852684)'> in</span><span style='background-color:rgb(221.17069266736507,33.829307332634926,33.829307332634926)'> the</span><span style='background-color:rgb(237.85927176475525,17.14072823524475,17.14072823524475)'> edges</span><span style='background-color:rgb(230.0414477661252,24.958552233874798,24.958552233874798)'> of</span><span style='background-color:rgb(235.86044792085886,19.13955207914114,19.13955207914114)'> the</span><span style='background-color:rgb(247.90429352782667,7.095706472173333,7.095706472173333)'> lantern</span><span style='background-color:rgb(238.40093839913607,16.599061600863934,16.599061600863934)'> '</span><span style='background-color:rgb(238.64980224519968,16.35019775480032,16.35019775480032)'> s</span><span style='background-color:rgb(245.28066303581,9.719336964190006,9.719336964190006)'> struggling</span><span style='background-color:rgb(238.60172722488642,16.398272775113583,16.398272775113583)'> glow</span><span style='background-color:rgb(232.28307478129864,22.716925218701363,22.716925218701363)'> .</span><span style='background-color:rgb(236.6728975251317,18.327102474868298,18.327102474868298)'> the</span><span style='background-color:rgb(233.17030057311058,21.82969942688942,21.82969942688942)'> only</span><span style='background-color:rgb(234.94898702949286,20.051012970507145,20.051012970507145)'> other</span><span style='background-color:rgb(250.7949494663626,4.205050533637404,4.205050533637404)'> illumination</span><span style='background-color:rgb(234.26676593720913,20.73323406279087,20.73323406279087)'> came</span><span style='background-color:rgb(232.16502651572227,22.834973484277725,22.834973484277725)'> from</span><span style='background-color:rgb(235.88794324547052,19.112056754529476,19.112056754529476)'> a</span><span style='background-color:rgb(245.18802115693688,9.811978843063116,9.811978843063116)'> lu</span><span style='background-color:rgb(254.7450384311378,0.2549615688621998,0.2549615688621998)'> ##rid</span><span style='background-color:rgb(249.91314218379557,5.086857816204429,5.086857816204429)'> moonlight</span> filtered<span style='background-color:rgb(231.2886605411768,23.711339458823204,23.711339458823204)'> through</span><span style='background-color:rgb(233.87608230113983,21.12391769886017,21.12391769886017)'> thin</span><span style='background-color:rgb(240.07758418098092,14.92241581901908,14.92241581901908)'> branches</span><span style='background-color:rgb(231.5146705135703,23.48532948642969,23.48532948642969)'> and</span><span style='background-color:rgb(241.82416711002588,13.175832889974117,13.175832889974117)'> clouds</span><span style='background-color:rgb(234.31446477770805,20.685535222291946,20.685535222291946)'> ,</span><span style='background-color:rgb(247.60088123381138,7.3991187661886215,7.3991187661886215)'> casting</span><span style='background-color:rgb(238.8088370114565,16.19116298854351,16.19116298854351)'> its</span><span style='background-color:rgb(251.85650111641735,3.1434988835826516,3.1434988835826516)'> bone</span><span style='background-color:rgb(239.8966122418642,15.103387758135796,15.103387758135796)'> -</span><span style='background-color:rgb(244.11246987059712,10.887530129402876,10.887530129402876)'> pale</span><span style='background-color:rgb(239.91448648273945,15.085513517260551,15.085513517260551)'> glow</span><span style='background-color:rgb(234.28396571427584,20.716034285724163,20.716034285724163)'> onto</span><span style='background-color:rgb(231.84586264193058,23.15413735806942,23.15413735806942)'> the</span><span style='background-color:rgb(235.85275523364544,19.14724476635456,19.14724476635456)'> pine</span><span style='background-color:rgb(201.60807318985462,53.39192681014538,53.39192681014538)'> floor</span><span style='background-color:rgb(206.7365401983261,48.26345980167389,48.26345980167389)'> ##boards</span><span style='background-color:rgb(212.02530540525913,42.97469459474087,42.97469459474087)'> .</span><span style='background-color:rgb(244.2808872833848,10.7191127166152,10.7191127166152)'> allan</span><span style='background-color:rgb(248.67862405255437,6.321375947445631,6.321375947445631)'> unfolded</span><span style='background-color:rgb(235.33540159463882,19.664598405361176,19.664598405361176)'> another</span><span style='background-color:rgb(238.70361875742674,16.29638124257326,16.29638124257326)'> page</span><span style='background-color:rgb(238.93451906740665,16.065480932593346,16.065480932593346)'> ,</span><span style='background-color:rgb(239.0075834468007,15.992416553199291,15.992416553199291)'> this</span><span style='background-color:rgb(234.1972277686,20.802772231400013,20.802772231400013)'> one</span><span style='background-color:rgb(241.4132866077125,13.586713392287493,13.586713392287493)'> crowded</span><span style='background-color:rgb(227.49527659267187,27.50472340732813,27.50472340732813)'> with</span><span style='background-color:rgb(239.29310772567987,15.706892274320126,15.706892274320126)'> ranks</span><span style='background-color:rgb(227.3882059380412,27.61179406195879,27.61179406195879)'> of</span><span style='background-color:rgb(229.5368044450879,25.46319555491209,25.46319555491209)'> letters</span><span style='background-color:rgb(211.82835437357426,43.17164562642574,43.17164562642574)'> in</span><span style='background-color:rgb(207.17525325715542,47.82474674284458,47.82474674284458)'> tight</span><span style='background-color:rgb(193.6187130957842,61.38128690421581,61.38128690421581)'> formation</span><span style='background-color:rgb(188.16479355096817,66.83520644903183,66.83520644903183)'> from</span><span style='background-color:rgb(222.23367422819138,32.766325771808624,32.766325771808624)'> left</span><span style='background-color:rgb(230.52617445588112,24.47382554411888,24.47382554411888)'> to</span><span style='background-color:rgb(237.0256857946515,17.97431420534849,17.97431420534849)'> right</span><span style='background-color:rgb(232.7291252464056,22.2708747535944,22.2708747535944)'> .</span><span style='background-color:rgb(236.85631178319454,18.143688216805458,18.143688216805458)'> the</span><span style='background-color:rgb(244.6603273227811,10.339672677218914,10.339672677218914)'> lines</span><span style='background-color:rgb(236.9133372232318,18.086662776768208,18.086662776768208)'> of</span><span style='background-color:rgb(240.76806128025055,14.23193871974945,14.23193871974945)'> letters</span><span style='background-color:rgb(238.39873261749744,16.601267382502556,16.601267382502556)'> stepped</span><span style='background-color:rgb(232.73832835257053,22.261671647429466,22.261671647429466)'> into</span><span style='background-color:rgb(230.05100045353174,24.948999546468258,24.948999546468258)'> their</span><span style='background-color:rgb(235.25671921670437,19.74328078329563,19.74328078329563)'> divisions</span><span style='background-color:rgb(228.67253132164478,26.327468678355217,26.327468678355217)'> ,</span><span style='background-color:rgb(233.24973721057177,21.750262789428234,21.750262789428234)'> in</span><span style='background-color:rgb(233.65223441272974,21.34776558727026,21.34776558727026)'> the</span><span style='background-color:rgb(236.9174998998642,18.082500100135803,18.082500100135803)'> shape</span><span style='background-color:rgb(234.49117999523878,20.50882000476122,20.50882000476122)'> of</span><span style='background-color:rgb(236.78308591246605,18.21691408753395,18.21691408753395)'> a</span><span style='background-color:rgb(234.73819144070148,20.261808559298515,20.261808559298515)'> story</span><span style='background-color:rgb(225.31688388437033,29.683116115629673,29.683116115629673)'> '</span><span style='background-color:rgb(232.0328715071082,22.96712849289179,22.96712849289179)'> s</span><span style='background-color:rgb(247.7033693343401,7.2966306656599045,7.2966306656599045)'> outline</span><span style='background-color:rgb(227.33221594244242,27.667784057557583,27.667784057557583)'> :</span><span style='background-color:rgb(227.8044831007719,27.195516899228096,27.195516899228096)'> the</span><span style='background-color:rgb(235.55286582559347,19.44713417440653,19.44713417440653)'> loose</span><span style='background-color:rgb(227.55190685391426,27.44809314608574,27.44809314608574)'> ,</span><span style='background-color:rgb(226.47548161447048,28.524518385529518,28.524518385529518)'> dry</span><span style='background-color:rgb(214.8609108477831,40.13908915221691,40.13908915221691)'> skeleton</span><span style='background-color:rgb(179.05692547559738,75.94307452440262,75.94307452440262)'> of</span><span style='background-color:rgb(162.05121159553528,92.94878840446472,92.94878840446472)'> a</span><span style='background-color:rgb(162.34383389353752,92.65616610646248,92.65616610646248)'> tale</span><span style='background-color:rgb(184.531679302454,70.468320697546,70.468320697546)'> lay</span><span style='background-color:rgb(227.02622596174479,27.973774038255215,27.973774038255215)'> exposed</span><span style='background-color:rgb(223.92254870384932,31.077451296150684,31.077451296150684)'> beneath</span><span style='background-color:rgb(218.49166922271252,36.50833077728748,36.50833077728748)'> their</span><span style='background-color:rgb(222.89434097707272,32.105659022927284,32.105659022927284)'> feet</span><span style='background-color:rgb(213.31025205552578,41.68974794447422,41.68974794447422)'> ,</span><span style='background-color:rgb(226.4318884536624,28.568111546337605,28.568111546337605)'> awaiting</span><span style='background-color:rgb(206.56393066048622,48.43606933951378,48.43606933951378)'> tend</span><span style='background-color:rgb(216.82709634304047,38.172903656959534,38.172903656959534)'> ##ons</span><span style='background-color:rgb(196.36298283934593,58.63701716065407,58.63701716065407)'> ,</span><span style='background-color:rgb(197.3925679922104,57.60743200778961,57.60743200778961)'> muscle</span><span style='background-color:rgb(198.2373310625553,56.76266893744469,56.76266893744469)'> and</span><span style='background-color:rgb(232.76524610817432,22.234753891825676,22.234753891825676)'> blushing</span><span style='background-color:rgb(222.6115145534277,32.388485446572304,32.388485446572304)'> skin</span><span style='background-color:rgb(219.83532652258873,35.16467347741127,35.16467347741127)'> .</span><span style='background-color:rgb(234.6472566202283,20.35274337977171,20.35274337977171)'> allan</span><span style='background-color:rgb(227.53284327685833,27.46715672314167,27.46715672314167)'> reviewed</span><span style='background-color:rgb(210.54449826478958,44.45550173521042,44.45550173521042)'> the</span><span style='background-color:rgb(215.5030269920826,39.496973007917404,39.496973007917404)'> troops</span><span style='background-color:rgb(209.30478438735008,45.69521561264992,45.69521561264992)'> ,</span><span style='background-color:rgb(216.15059964358807,38.849400356411934,38.849400356411934)'> all</span><span style='background-color:rgb(223.75678449869156,31.24321550130844,31.24321550130844)'> prepared</span><span style='background-color:rgb(215.70115216076374,39.29884783923626,39.29884783923626)'> to</span><span style='background-color:rgb(231.9560795277357,23.04392047226429,23.04392047226429)'> di</span><span style='background-color:rgb(231.2639181688428,23.736081831157207,23.736081831157207)'> ##se</span><span style='background-color:rgb(235.418695025146,19.581304974853992,19.581304974853992)'> ##mba</span><span style='background-color:rgb(234.7124022245407,20.28759777545929,20.28759777545929)'> ##rk</span><span style='background-color:rgb(213.679611235857,41.32038876414299,41.32038876414299)'> ,</span><span style='background-color:rgb(198.8634918630123,56.136508136987686,56.136508136987686)'> their</span><span style='background-color:rgb(171.86249032616615,83.13750967383385,83.13750967383385)'> task</span><span style='background-color:rgb(147.91574209928513,107.08425790071487,107.08425790071487)'> to</span><span style='background-color:rgb(170.79858541488647,84.20141458511353,84.20141458511353)'> form</span><span style='background-color:rgb(193.60751129686832,61.392488703131676,61.392488703131676)'> the</span><span style='background-color:rgb(209.6993438154459,45.3006561845541,45.3006561845541)'> tale</span><span style='background-color:rgb(196.65353044867516,58.346469551324844,58.346469551324844)'> of</span><span style='background-color:rgb(204.50146213173866,50.49853786826134,50.49853786826134)'> a</span><span style='background-color:rgb(208.69917288422585,46.300827115774155,46.300827115774155)'> young</span><span style='background-color:rgb(206.22816167771816,48.77183832228184,48.77183832228184)'> man</span><span style='background-color:rgb(207.00189895927906,47.99810104072094,47.99810104072094)'> returning</span><span style='background-color:rgb(207.31155194342136,47.688448056578636,47.688448056578636)'> home</span><span style='background-color:rgb(213.69354128837585,41.306458711624146,41.306458711624146)'> from</span><span style='background-color:rgb(222.11479000747204,32.88520999252796,32.88520999252796)'> life</span><span style='background-color:rgb(230.27796037495136,24.722039625048637,24.722039625048637)'> abroad</span><span style='background-color:rgb(221.4938349276781,33.50616507232189,33.50616507232189)'> to</span><span style='background-color:rgb(229.98784594237804,25.012154057621956,25.012154057621956)'> find</span><span style='background-color:rgb(225.3042609617114,29.695739038288593,29.695739038288593)'> his</span><span style='background-color:rgb(230.7497563585639,24.2502436414361,24.2502436414361)'> childhood</span><span style='background-color:rgb(224.13294341415167,30.86705658584833,30.86705658584833)'> friend</span><span style='background-color:rgb(215.4613470286131,39.53865297138691,39.53865297138691)'> a</span><span style='background-color:rgb(223.1989250332117,31.801074966788292,31.801074966788292)'> bride</span><span style='background-color:rgb(215.94273179769516,39.05726820230484,39.05726820230484)'> to</span><span style='background-color:rgb(213.15551675856113,41.844483241438866,41.844483241438866)'> -</span><span style='background-color:rgb(210.5328404903412,44.46715950965881,44.46715950965881)'> be</span><span style='background-color:rgb(199.49216432869434,55.507835671305656,55.507835671305656)'> ,</span><span style='background-color:rgb(205.68935059010983,49.310649409890175,49.310649409890175)'> thus</span><span style='background-color:rgb(213.67065891623497,41.32934108376503,41.32934108376503)'> upset</span><span style='background-color:rgb(205.07377982139587,49.926220178604126,49.926220178604126)'> ##ting</span><span style='background-color:rgb(201.5501756966114,53.449824303388596,53.449824303388596)'> the</span><span style='background-color:rgb(219.621424600482,35.37857539951801,35.37857539951801)'> apple</span><span style='background-color:rgb(218.43972600996494,36.56027399003506,36.56027399003506)'> cart</span><span style='background-color:rgb(198.38622607290745,56.61377392709255,56.61377392709255)'> of</span><span style='background-color:rgb(207.06974811851978,47.93025188148022,47.93025188148022)'> his</span><span style='background-color:rgb(217.87514850497246,37.12485149502754,37.12485149502754)'> life</span><span style='background-color:rgb(221.0610267519951,33.93897324800491,33.93897324800491)'> '</span><span style='background-color:rgb(222.56422609090805,32.43577390909195,32.43577390909195)'> s</span><span style='background-color:rgb(225.93058325350285,29.069416746497154,29.069416746497154)'> plan</span><span style='background-color:rgb(220.96711479127407,34.03288520872593,34.03288520872593)'> ,</span><span style='background-color:rgb(244.55336591228843,10.446634087711573,10.446634087711573)'> clarified</span><span style='background-color:rgb(233.56283850967884,21.43716149032116,21.43716149032116)'> –</span><span style='background-color:rgb(227.9964715987444,27.003528401255608,27.003528401255608)'> of</span><span style='background-color:rgb(232.30745807290077,22.692541927099228,22.692541927099228)'> course</span><span style='background-color:rgb(230.22448394447565,24.77551605552435,24.77551605552435)'> –</span><span style='background-color:rgb(207.57204316556454,47.42795683443546,47.42795683443546)'> by</span><span style='background-color:rgb(196.56939156353474,58.43060843646526,58.43060843646526)'> his</span><span style='background-color:rgb(201.20222456753254,53.79777543246746,53.79777543246746)'> very</span><span style='background-color:rgb(206.80900990962982,48.19099009037018,48.19099009037018)'> time</span><span style='background-color:rgb(217.08550907671452,37.914490923285484,37.914490923285484)'> away</span><span style='background-color:rgb(211.9266740977764,43.07332590222359,43.07332590222359)'> from</span><span style='background-color:rgb(195.7962280511856,59.20377194881439,59.20377194881439)'> her</span><span style='background-color:rgb(207.15333983302116,47.846660166978836,47.846660166978836)'> he</span><span style='background-color:rgb(218.80637973546982,36.19362026453018,36.19362026453018)'> loved</span><span style='background-color:rgb(228.5984656959772,26.40153430402279,26.40153430402279)'> best</span><span style='background-color:rgb(225.04543595016003,29.954564049839973,29.954564049839973)'> .</span><span style='background-color:rgb(223.5392575711012,31.46074242889881,31.46074242889881)'> although</span><span style='background-color:rgb(225.1224920153618,29.877507984638214,29.877507984638214)'> the</span><span style='background-color:rgb(229.82445660978556,25.175543390214443,25.175543390214443)'> concept</span><span style='background-color:rgb(217.44056962430477,37.55943037569523,37.55943037569523)'> was</span><span style='background-color:rgb(217.69216932356358,37.307830676436424,37.307830676436424)'> a</span><span style='background-color:rgb(228.68221890181303,26.31778109818697,26.31778109818697)'> simple</span><span style='background-color:rgb(226.3617841154337,28.638215884566307,28.638215884566307)'> one</span><span style='background-color:rgb(229.33961782604456,25.66038217395544,25.66038217395544)'> ,</span><span style='background-color:rgb(240.2160601504147,14.783939849585295,14.783939849585295)'> allan</span><span style='background-color:rgb(231.59497160464525,23.405028395354748,23.405028395354748)'> thought</span><span style='background-color:rgb(230.5690285563469,24.430971443653107,24.430971443653107)'> it</span><span style='background-color:rgb(227.0681833103299,27.931816689670086,27.931816689670086)'> had</span><span style='background-color:rgb(227.08228815346956,27.917711846530437,27.917711846530437)'> potential</span><span style='background-color:rgb(223.75065922737122,31.249340772628784,31.249340772628784)'> .</span><span style='background-color:rgb(227.98817664384842,27.01182335615158,27.01182335615158)'> besides</span><span style='background-color:rgb(213.75663690268993,41.243363097310066,41.243363097310066)'> ,</span><span style='background-color:rgb(209.6811731904745,45.31882680952549,45.31882680952549)'> the</span><span style='background-color:rgb(209.58171352744102,45.418286472558975,45.418286472558975)'> public</span><span style='background-color:rgb(216.57859347760677,38.42140652239323,38.42140652239323)'> liked</span><span style='background-color:rgb(211.31505891680717,43.684941083192825,43.684941083192825)'> a</span><span style='background-color:rgb(212.32341840863228,42.67658159136772,42.67658159136772)'> good</span><span style='background-color:rgb(213.9925054460764,41.00749455392361,41.00749455392361)'> ,</span><span style='background-color:rgb(221.49781711399555,33.50218288600445,33.50218288600445)'> simple</span><span style='background-color:rgb(215.87183140218258,39.12816859781742,39.12816859781742)'> romance</span><span style='background-color:rgb(195.33793084323406,59.66206915676594,59.66206915676594)'> .</span><span style='background-color:rgb(194.44729283452034,60.55270716547966,60.55270716547966)'> perhaps</span><span style='background-color:rgb(201.49908363819122,53.50091636180878,53.50091636180878)'> this</span><span style='background-color:rgb(213.32515105605125,41.674848943948746,41.674848943948746)'> will</span><span style='background-color:rgb(210.48759631812572,44.512403681874275,44.512403681874275)'> be</span><span style='background-color:rgb(202.8789871931076,52.121012806892395,52.121012806892395)'> more</span><span style='background-color:rgb(213.13622519373894,41.86377480626106,41.86377480626106)'> sale</span><span style='background-color:rgb(213.00283335149288,41.99716664850712,41.99716664850712)'> ##able</span><span style='background-color:rgb(209.6539780497551,45.346021950244904,45.346021950244904)'> ,</span><span style='background-color:rgb(212.30539977550507,42.694600224494934,42.694600224494934)'> he</span><span style='background-color:rgb(206.46601371467113,48.533986285328865,48.533986285328865)'> thought</span><span style='background-color:rgb(205.40886864066124,49.59113135933876,49.59113135933876)'> and</span><span style='background-color:rgb(215.048985555768,39.95101444423199,39.95101444423199)'> began</span><span style='background-color:rgb(213.06883960962296,41.931160390377045,41.931160390377045)'> to</span><span style='background-color:rgb(228.002315685153,26.997684314846992,26.997684314846992)'> write</span><span style='background-color:rgb(215.20155876874924,39.79844123125076,39.79844123125076)'> .</span><span style='background-color:rgb(213.08025799691677,41.91974200308323,41.91974200308323)'> they</span><span style='background-color:rgb(219.68733206391335,35.312667936086655,35.312667936086655)'> gazed</span><span style='background-color:rgb(223.0909652262926,31.90903477370739,31.90903477370739)'> at</span><span style='background-color:rgb(230.90553659945726,24.094463400542736,24.094463400542736)'> each</span><span style='background-color:rgb(233.09848822653294,21.901511773467064,21.901511773467064)'> other</span><span style='background-color:rgb(231.410050727427,23.589949272572994,23.589949272572994)'> ,</span><span style='background-color:rgb(234.05499380081892,20.94500619918108,20.94500619918108)'> lost</span><span style='background-color:rgb(225.0289011374116,29.971098862588406,29.971098862588406)'> in</span><span style='background-color:rgb(223.1247416138649,31.8752583861351,31.8752583861351)'> the</span><span style='background-color:rgb(234.90038953721523,20.099610462784767,20.099610462784767)'> rap</span><span style='background-color:rgb(240.62098922207952,14.379010777920485,14.379010777920485)'> ##ture</span><span style='background-color:rgb(230.3809880465269,24.61901195347309,24.61901195347309)'> of</span><span style='background-color:rgb(231.33338034152985,23.666619658470154,23.666619658470154)'> love</span><span style='background-color:rgb(230.1566917821765,24.843308217823505,24.843308217823505)'> based</span><span style='background-color:rgb(225.51206801086664,29.487931989133358,29.487931989133358)'> so</span><span style='background-color:rgb(234.9672906473279,20.0327093526721,20.0327093526721)'> deeply</span><span style='background-color:rgb(230.6628417223692,24.337158277630806,24.337158277630806)'> within</span><span style='background-color:rgb(228.26113689690828,26.738863103091717,26.738863103091717)'> their</span><span style='background-color:rgb(232.4607190489769,22.539280951023102,22.539280951023102)'> hearts</span><span style='background-color:rgb(215.12060031294823,39.87939968705177,39.87939968705177)'> that</span><span style='background-color:rgb(218.47574047744274,36.52425952255726,36.52425952255726)'> they</span><span style='background-color:rgb(217.72370763123035,37.276292368769646,37.276292368769646)'> had</span><span style='background-color:rgb(224.9010854959488,30.09891450405121,30.09891450405121)'> never</span><span style='background-color:rgb(229.5211910828948,25.478808917105198,25.478808917105198)'> seen</span><span style='background-color:rgb(228.4455581009388,26.554441899061203,26.554441899061203)'> it</span><span style='background-color:rgb(227.12370973080397,27.876290269196033,27.876290269196033)'> before</span><span style='background-color:rgb(218.62743213772774,36.37256786227226,36.37256786227226)'> .</span><span style='background-color:rgb(213.0599670857191,41.94003291428089,41.94003291428089)'> "</span><span style='background-color:rgb(212.33404263854027,42.66595736145973,42.66595736145973)'> what</span><span style='background-color:rgb(213.13090547919273,41.869094520807266,41.869094520807266)'> about</span><span style='background-color:rgb(224.9533687904477,30.046631209552288,30.046631209552288)'> roger</span><span style='background-color:rgb(211.16087839007378,43.839121609926224,43.839121609926224)'> ?</span><span style='background-color:rgb(176.67425572872162,78.32574427127838,78.32574427127838)'> "</span><span style='background-color:rgb(152.56541579961777,102.43458420038223,102.43458420038223)'> she</span><span style='background-color:rgb(192.66021452844143,62.33978547155857,62.33978547155857)'> asked</span><span style='background-color:rgb(207.2555809468031,47.74441905319691,47.74441905319691)'> ,</span><span style='background-color:rgb(221.82984329760075,33.170156702399254,33.170156702399254)'> knowing</span><span style='background-color:rgb(210.37710584700108,44.622894152998924,44.622894152998924)'> that</span><span style='background-color:rgb(209.13362257182598,45.86637742817402,45.86637742817402)'> the</span><span style='background-color:rgb(212.21011608839035,42.78988391160965,42.78988391160965)'> answer</span><span style='background-color:rgb(207.90019735693932,47.099802643060684,47.099802643060684)'> no</span><span style='background-color:rgb(206.33377701044083,48.666222989559174,48.666222989559174)'> longer</span><span style='background-color:rgb(206.80915050208569,48.190849497914314,48.190849497914314)'> mattered</span><span style='background-color:rgb(187.84836173057556,67.15163826942444,67.15163826942444)'> .</span><span style='background-color:rgb(165.50262719392776,89.49737280607224,89.49737280607224)'> [SEP]</span><span style='background-color:rgb(192.11813941597939,62.881860584020615,62.881860584020615)'> name</span><span style='background-color:rgb(195.0863767415285,59.91362325847149,59.91362325847149)'> few</span><span style='background-color:rgb(206.06888562440872,48.93111437559128,48.93111437559128)'> objects</span><span style='background-color:rgb(192.3098847270012,62.69011527299881,62.69011527299881)'> said</span><span style='background-color:rgb(159.58935409784317,95.41064590215683,95.41064590215683)'> to</span><span style='background-color:rgb(164.9740071594715,90.02599284052849,90.02599284052849)'> be</span><span style='background-color:rgb(153.929922580719,101.070077419281,101.070077419281)'> in</span><span style='background-color:rgb(143.09570834040642,111.90429165959358,111.90429165959358)'> or</span><span style='background-color:rgb(142.51244723796844,112.48755276203156,112.48755276203156)'> on</span><span style='background-color:rgb(177.37686082720757,77.62313917279243,77.62313917279243)'> allan</span><span style='background-color:rgb(174.11195442080498,80.88804557919502,80.88804557919502)'> '</span><span style='background-color:rgb(191.94524869322777,63.05475130677223,63.05475130677223)'> s</span><span style='background-color:rgb(209.66636538505554,45.33363461494446,45.33363461494446)'> desk</span><span style='background-color:rgb(211.84806771576405,43.151932284235954,43.151932284235954)'> |</span><span style='background-color:rgb(217.6757200062275,37.32427999377251,37.32427999377251)'> |</span><span style='background-color:rgb(227.39191833883524,27.60808166116476,27.60808166116476)'> erase</span><span style='background-color:rgb(201.86589695513248,53.134103044867516,53.134103044867516)'> ##r</span><span style='background-color:rgb(184.70598354935646,70.29401645064354,70.29401645064354)'> [SEP]</span><span style='background-color:rgb(228.11685673892498,26.88314326107502,26.88314326107502)'> [PAD]</span><span style='background-color:rgb(234.54734858125448,20.452651418745518,20.452651418745518)'> [PAD]</span><span style='background-color:rgb(237.0566750317812,17.943324968218803,17.943324968218803)'> [PAD]</span><span style='background-color:rgb(234.86797537654638,20.132024623453617,20.132024623453617)'> [PAD]</span><span style='background-color:rgb(231.49771392345428,23.502286076545715,23.502286076545715)'> [PAD]</span><span style='background-color:rgb(226.8088909238577,28.19110907614231,28.19110907614231)'> [PAD]</span><span style='background-color:rgb(220.77103771269321,34.228962287306786,34.228962287306786)'> [PAD]</span><span style='background-color:rgb(218.2472587376833,36.752741262316704,36.752741262316704)'> [PAD]</span><span style='background-color:rgb(216.21154837310314,38.78845162689686,38.78845162689686)'> [PAD]</span><span style='background-color:rgb(211.02159306406975,43.97840693593025,43.97840693593025)'> [PAD]</span><span style='background-color:rgb(217.5243703275919,37.475629672408104,37.475629672408104)'> [PAD]</span><span style='background-color:rgb(233.6281418055296,21.371858194470406,21.371858194470406)'> [PAD]</span><span style='background-color:rgb(238.08412849903107,16.915871500968933,16.915871500968933)'> [PAD]</span><span style='background-color:rgb(230.30084654688835,24.69915345311165,24.69915345311165)'> [PAD]</span><span style='background-color:rgb(229.23643816262484,25.763561837375164,25.763561837375164)'> [PAD]</span><span style='background-color:rgb(230.3870601207018,24.61293987929821,24.61293987929821)'> [PAD]</span><span style='background-color:rgb(228.97922806441784,26.02077193558216,26.02077193558216)'> [PAD]</span><span style='background-color:rgb(227.81518902629614,27.18481097370386,27.18481097370386)'> [PAD]</span><span style='background-color:rgb(227.27587636560202,27.724123634397984,27.724123634397984)'> [PAD]</span><span style='background-color:rgb(229.17181123048067,25.82818876951933,25.82818876951933)'> [PAD]</span><span style='background-color:rgb(229.12029549479485,25.879704505205154,25.879704505205154)'> [PAD]</span><span style='background-color:rgb(229.5226502045989,25.477349795401096,25.477349795401096)'> [PAD]</span><span style='background-color:rgb(231.63447998464108,23.365520015358925,23.365520015358925)'> [PAD]</span><span style='background-color:rgb(235.9465816989541,19.053418301045895,19.053418301045895)'> [PAD]</span><span style='background-color:rgb(240.91193486005068,14.088065139949322,14.088065139949322)'> [PAD]</span><span style='background-color:rgb(244.8636506125331,10.136349387466908,10.136349387466908)'> [PAD]</span><span style='background-color:rgb(243.82451562210917,11.175484377890825,11.175484377890825)'> [PAD]</span><span style='background-color:rgb(240.4161669127643,14.58383308723569,14.58383308723569)'> [PAD]</span><span style='background-color:rgb(238.3034660294652,16.6965339705348,16.6965339705348)'> [PAD]</span><span style='background-color:rgb(238.54474168270826,16.455258317291737,16.455258317291737)'> [PAD]</span><span style='background-color:rgb(241.06219874694943,13.937801253050566,13.937801253050566)'> [PAD]</span><span style='background-color:rgb(245.18145985901356,9.818540140986443,9.818540140986443)'> [PAD]</span><span style='background-color:rgb(247.7387326117605,7.261267388239503,7.261267388239503)'> [PAD]</span><span style='background-color:rgb(247.73197134956717,7.268028650432825,7.268028650432825)'> [PAD]</span><span style='background-color:rgb(245.96190948039293,9.038090519607067,9.038090519607067)'> [PAD]</span><span style='background-color:rgb(244.51811710372567,10.481882896274328,10.481882896274328)'> [PAD]</span><span style='background-color:rgb(244.8164951428771,10.183504857122898,10.183504857122898)'> [PAD]</span><span style='background-color:rgb(246.7087242566049,8.29127574339509,8.29127574339509)'> [PAD]</span><span style='background-color:rgb(247.95608712360263,7.043912876397371,7.043912876397371)'> [PAD]</span><span style='background-color:rgb(247.49382387846708,7.506176121532917,7.506176121532917)'> [PAD]</span><span style='background-color:rgb(245.48934308812022,9.510656911879778,9.510656911879778)'> [PAD]</span><span style='background-color:rgb(244.33638045564294,10.663619544357061,10.663619544357061)'> [PAD]</span><span style='background-color:rgb(243.40461060404778,11.595389395952225,11.595389395952225)'> [PAD]</span><span style='background-color:rgb(240.85363933816552,14.146360661834478,14.146360661834478)'> [PAD]</span><span style='background-color:rgb(240.54056653752923,14.45943346247077,14.45943346247077)'> [PAD]</span><span style='background-color:rgb(239.59973607212305,15.40026392787695,15.40026392787695)'> [PAD]</span><span style='background-color:rgb(239.68668775632977,15.313312243670225,15.313312243670225)'> [PAD]</span><span style='background-color:rgb(238.72693430632353,16.27306569367647,16.27306569367647)'> [PAD]</span><span style='background-color:rgb(239.62746033445,15.372539665549994,15.372539665549994)'> [PAD]</span><span style='background-color:rgb(245.59489572420716,9.405104275792837,9.405104275792837)'> [PAD]</span><span style='background-color:rgb(248.40387457050383,6.596125429496169,6.596125429496169)'> [PAD]</span><span style='background-color:rgb(246.68738175183535,8.312618248164654,8.312618248164654)'> [PAD]</span><span style='background-color:rgb(246.84285420924425,8.157145790755749,8.157145790755749)'> [PAD]</span><span style='background-color:rgb(248.06379234418273,6.93620765581727,6.93620765581727)'> [PAD]</span><span style='background-color:rgb(218.0075827986002,36.9924172013998,36.9924172013998)'> [PAD]</span><span style='background-color:rgb(160.91469258069992,94.08530741930008,94.08530741930008)'> [PAD]</span><span style='background-color:rgb(149.61122632026672,105.38877367973328,105.38877367973328)'> [PAD]</span><span style='background-color:rgb(188.6636459827423,66.33635401725769,66.33635401725769)'> [PAD]</span><span style='background-color:rgb(229.3681200966239,25.631879903376102,25.631879903376102)'> [PAD]</span><span style='background-color:rgb(239.72576390951872,15.274236090481281,15.274236090481281)'> [PAD]</span><span style='background-color:rgb(232.37666375935078,22.623336240649223,22.623336240649223)'> [PAD]</span><span style='background-color:rgb(234.88701805472374,20.11298194527626,20.11298194527626)'> [PAD]</span><span style='background-color:rgb(243.48317043855786,11.516829561442137,11.516829561442137)'> [PAD]</span><span style='background-color:rgb(246.95817042142153,8.041829578578472,8.041829578578472)'> [PAD]</span><span style='background-color:rgb(243.75875445082784,11.241245549172163,11.241245549172163)'> [PAD]</span><span style='background-color:rgb(236.45103313028812,18.548966869711876,18.548966869711876)'> [PAD]</span><span style='background-color:rgb(234.61044419556856,20.38955580443144,20.38955580443144)'> [PAD]</span><span style='background-color:rgb(237.82749596983194,17.172504030168056,17.172504030168056)'> [PAD]</span><span style='background-color:rgb(240.5582089908421,14.441791009157896,14.441791009157896)'> [PAD]</span><span style='background-color:rgb(241.4541439153254,13.545856084674597,13.545856084674597)'> [PAD]</span><span style='background-color:rgb(243.0206697061658,11.97933029383421,11.97933029383421)'> [PAD]</span><span style='background-color:rgb(244.86555621027946,10.134443789720535,10.134443789720535)'> [PAD]</span><span style='background-color:rgb(244.8636002652347,10.136399734765291,10.136399734765291)'> [PAD]</span><span style='background-color:rgb(243.92589323222637,11.074106767773628,11.074106767773628)'> [PAD]</span><span style='background-color:rgb(244.0546882711351,10.945311728864908,10.945311728864908)'> [PAD]</span><span style='background-color:rgb(245.4147844389081,9.5852155610919,9.5852155610919)'> [PAD]</span><span style='background-color:rgb(243.42062199488282,11.579378005117178,11.579378005117178)'> [PAD]</span><span style='background-color:rgb(240.52178509533405,14.478214904665947,14.478214904665947)'> [PAD]</span><span style='background-color:rgb(227.02017858624458,27.979821413755417,27.979821413755417)'> [PAD]</span><span style='background-color:rgb(207.97270126640797,47.02729873359203,47.02729873359203)'> [PAD]</span><span style='background-color:rgb(208.05550262331963,46.944497376680374,46.944497376680374)'> [PAD]</span><span style='background-color:rgb(186.08543872833252,68.91456127166748,68.91456127166748)'> [PAD]</span><span style='background-color:rgb(165.3260126709938,89.6739873290062,89.6739873290062)'> [PAD]</span><span style='background-color:rgb(184.42308112978935,70.57691887021065,70.57691887021065)'> [PAD]</span><span style='background-color:rgb(219.15196739137173,35.84803260862827,35.84803260862827)'> [PAD]</span><span style='background-color:rgb(234.05871000140905,20.941289998590946,20.941289998590946)'> [PAD]</span><span style='background-color:rgb(236.10766265541315,18.89233734458685,18.89233734458685)'> [PAD]</span><span style='background-color:rgb(235.1372441276908,19.862755872309208,19.862755872309208)'> [PAD]</span><span style='background-color:rgb(233.9553802460432,21.044619753956795,21.044619753956795)'> [PAD]</span><span style='background-color:rgb(236.41560193151236,18.584398068487644,18.584398068487644)'> [PAD]</span><span style='background-color:rgb(244.01934636756778,10.980653632432222,10.980653632432222)'> [PAD]</span><span style='background-color:rgb(244.16481871157885,10.835181288421154,10.835181288421154)'> [PAD]</span><span style='background-color:rgb(243.68271578103304,11.317284218966961,11.317284218966961)'> [PAD]</span><span style='background-color:rgb(240.9917476773262,14.008252322673798,14.008252322673798)'> [PAD]</span><span style='background-color:rgb(220.8089178800583,34.19108211994171,34.19108211994171)'> [PAD]</span><span style='background-color:rgb(207.69081339240074,47.30918660759926,47.30918660759926)'> [PAD]</span><span style='background-color:rgb(225.04031002521515,29.95968997478485,29.95968997478485)'> [PAD]</span><span style='background-color:rgb(244.17341575026512,10.826584249734879,10.826584249734879)'> [PAD]</span><span style='background-color:rgb(246.99648091569543,8.003519084304571,8.003519084304571)'> [PAD]</span><span style='background-color:rgb(244.65844167396426,10.341558326035738,10.341558326035738)'> [PAD]</span><span style='background-color:rgb(244.09433154389262,10.905668456107378,10.905668456107378)'> [PAD]</span><span style='background-color:rgb(243.244158513844,11.755841486155987,11.755841486155987)'> [PAD]</span><span style='background-color:rgb(244.7172492183745,10.28275078162551,10.28275078162551)'> [PAD]</span><span style='background-color:rgb(246.76470380276442,8.235296197235584,8.235296197235584)'> [PAD]</span><span style='background-color:rgb(246.22280348092318,8.777196519076824,8.777196519076824)'> [PAD]</span><span style='background-color:rgb(245.8167183212936,9.183281678706408,9.183281678706408)'> [PAD]</span><span style='background-color:rgb(245.76667880639434,9.233321193605661,9.233321193605661)'> [PAD]</span><span style='background-color:rgb(242.65977837145329,12.340221628546715,12.340221628546715)'> [PAD]</span><span style='background-color:rgb(235.69153748452663,19.308462515473366,19.308462515473366)'> [PAD]</span><span style='background-color:rgb(230.1385496556759,24.861450344324112,24.861450344324112)'> [PAD]</span><span style='background-color:rgb(233.74470245093107,21.255297549068928,21.255297549068928)'> [PAD]</span><span style='background-color:rgb(240.5387464351952,14.461253564804792,14.461253564804792)'> [PAD]</span><span style='background-color:rgb(242.4677680246532,12.532231975346804,12.532231975346804)'> [PAD]</span><span style='background-color:rgb(240.53737185895443,14.46262814104557,14.46262814104557)'> [PAD]</span><span style='background-color:rgb(235.3284764662385,19.6715235337615,19.6715235337615)'> [PAD]</span><span style='background-color:rgb(231.27608701586723,23.723912984132767,23.723912984132767)'> [PAD]</span><span style='background-color:rgb(229.7992278635502,25.200772136449814,25.200772136449814)'> [PAD]</span><span style='background-color:rgb(226.18571676313877,28.81428323686123,28.81428323686123)'> [PAD]</span><span style='background-color:rgb(233.92309717833996,21.076902821660042,21.076902821660042)'> [PAD]</span><span style='background-color:rgb(239.11825820803642,15.881741791963577,15.881741791963577)'> [PAD]</span><span style='background-color:rgb(238.5839517787099,16.41604822129011,16.41604822129011)'> [PAD]</span><span style='background-color:rgb(234.8833018541336,20.116698145866394,20.116698145866394)'> [PAD]</span><span style='background-color:rgb(231.722530759871,23.277469240128994,23.277469240128994)'> [PAD]</span><span style='background-color:rgb(228.77021078020334,26.229789219796658,26.229789219796658)'> [PAD]</span><span style='background-color:rgb(228.47275894135237,26.527241058647633,26.527241058647633)'> [PAD]</span><span style='background-color:rgb(229.5983611419797,25.401638858020306,25.401638858020306)'> [PAD]</span><span style='background-color:rgb(228.21634490042925,26.78365509957075,26.78365509957075)'> [PAD]</span><span style='background-color:rgb(227.47182045131922,27.528179548680782,27.528179548680782)'> [PAD]</span><span style='background-color:rgb(229.5534437522292,25.446556247770786,25.446556247770786)'> [PAD]</span><span style='background-color:rgb(230.3448462858796,24.655153714120388,24.655153714120388)'> [PAD]</span><span style='background-color:rgb(231.95239942520857,23.04760057479143,23.04760057479143)'> [PAD]</span><span style='background-color:rgb(228.66112433373928,26.33887566626072,26.33887566626072)'> [PAD]</span><span style='background-color:rgb(226.19180783629417,28.808192163705826,28.808192163705826)'> [PAD]</span><span style='background-color:rgb(224.52438320964575,30.475616790354252,30.475616790354252)'> [PAD]</span><span style='background-color:rgb(224.15575549006462,30.84424450993538,30.84424450993538)'> [PAD]</span><span style='background-color:rgb(225.60341130942106,29.396588690578938,29.396588690578938)'> [PAD]</span><span style='background-color:rgb(226.27159785479307,28.72840214520693,28.72840214520693)'> [PAD]</span><span style='background-color:rgb(225.41447784751654,29.585522152483463,29.585522152483463)'> [PAD]</span><span style='background-color:rgb(224.62593466043472,30.374065339565277,30.374065339565277)'> [PAD]</span><span style='background-color:rgb(223.4461321681738,31.55386783182621,31.55386783182621)'> [PAD]</span><span style='background-color:rgb(221.9761886447668,33.02381135523319,33.02381135523319)'> [PAD]</span><span style='background-color:rgb(209.11260209977627,45.88739790022373,45.88739790022373)'> [PAD]</span><span style='background-color:rgb(188.79039958119392,66.20960041880608,66.20960041880608)'> [PAD]</span><span style='background-color:rgb(198.46898183226585,56.531018167734146,56.531018167734146)'> [PAD]</span><span style='background-color:rgb(215.29032580554485,39.70967419445515,39.70967419445515)'> [PAD]</span><span style='background-color:rgb(226.12395487725735,28.876045122742653,28.876045122742653)'> [PAD]</span><span style='background-color:rgb(231.3917338103056,23.608266189694405,23.608266189694405)'> [PAD]</span><span style='background-color:rgb(222.52287670969963,32.47712329030037,32.47712329030037)'> [PAD]</span><span style='background-color:rgb(193.39161828160286,61.60838171839714,61.60838171839714)'> [PAD]</span><span style='background-color:rgb(176.0010838508606,78.9989161491394,78.9989161491394)'> [PAD]</span><span style='background-color:rgb(204.3777483701706,50.62225162982941,50.62225162982941)'> [PAD]</span><span style='background-color:rgb(223.59911005944014,31.400889940559864,31.400889940559864)'> [PAD]</span><span style='background-color:rgb(222.1054045110941,32.89459548890591,32.89459548890591)'> [PAD]</span><span style='background-color:rgb(217.90480971336365,37.09519028663635,37.09519028663635)'> [PAD]</span><span style='background-color:rgb(206.6879902034998,48.312009796500206,48.312009796500206)'> [PAD]</span><span style='background-color:rgb(143.33250403404236,111.66749596595764,111.66749596595764)'> [PAD]</span><span style='background-color:rgb(89.37208592891693,165.62791407108307,165.62791407108307)'> [PAD]</span>

## MFABA

In [14]:
from exlib.explainers.mfaba import MfabaTextCls

mfaba_explainer = MfabaTextCls(original_model2, projection_layer)
mfaba_expln = mfaba_explainer(inputs, labels) #, x_kwargs=kwargs)

In [15]:
inputs.shape, labels.shape, mfaba_expln.attributions.shape

(torch.Size([2, 512]), torch.Size([2]), torch.Size([2, 512]))

In [16]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

def show_attrs(tokens, masks, normalize=True):
    if normalize:
        masks_max = masks.max(-1).values
        masks_min = masks.min(-1).values
        # import pdb; pdb.set_trace()
        masks = (masks - masks_min) / (masks_max - masks_min)
    print_str = ""
    for i in range(len(tokens)):
        expln_val = masks[i].item()
        if expln_val == 0:
            print_str += f" {tokens[i]}"
        else:
            print_str += f"<span style='background-color:rgb({255 * (1-expln_val)},{255 * expln_val},{255 * expln_val})'> {tokens[i]}</span>"
    return print_str

tokens = processor.convert_ids_to_tokens(inputs[0])
# tokens = shap_expln.explainer_output.data[0]
printmd(show_attrs(tokens, mfaba_expln.attributions[0]))

<span style='background-color:rgb(167.83899262547493,87.16100737452507,87.16100737452507)'> [CLS]</span><span style='background-color:rgb(247.5893445778638,7.4106554221361876,7.4106554221361876)'> allan</span><span style='background-color:rgb(243.9376175031066,11.062382496893406,11.062382496893406)'> sat</span><span style='background-color:rgb(242.91251800954342,12.087481990456581,12.087481990456581)'> down</span><span style='background-color:rgb(242.09998480975628,12.900015190243721,12.900015190243721)'> at</span><span style='background-color:rgb(243.06207893416286,11.937921065837145,11.937921065837145)'> his</span><span style='background-color:rgb(239.67897417023778,15.32102582976222,15.32102582976222)'> desk</span><span style='background-color:rgb(240.34324027597904,14.656759724020958,14.656759724020958)'> and</span><span style='background-color:rgb(241.27192184329033,13.728078156709671,13.728078156709671)'> pulled</span><span style='background-color:rgb(239.19437762349844,15.80562237650156,15.80562237650156)'> the</span><span style='background-color:rgb(237.20606591552496,17.79393408447504,17.79393408447504)'> chair</span><span style='background-color:rgb(241.980738658458,13.019261341542006,13.019261341542006)'> in</span><span style='background-color:rgb(242.31091149151325,12.689088508486748,12.689088508486748)'> close</span><span style='background-color:rgb(238.4306376054883,16.5693623945117,16.5693623945117)'> .</span><span style='background-color:rgb(242.00872130692005,12.991278693079948,12.991278693079948)'> opening</span><span style='background-color:rgb(242.6103164255619,12.389683574438095,12.389683574438095)'> a</span><span style='background-color:rgb(241.7161607556045,13.283839244395494,13.283839244395494)'> side</span><span style='background-color:rgb(241.19959177449346,13.800408225506544,13.800408225506544)'> drawer</span><span style='background-color:rgb(238.57991829514503,16.420081704854965,16.420081704854965)'> ,</span><span style='background-color:rgb(241.02657660841942,13.973423391580582,13.973423391580582)'> he</span><span style='background-color:rgb(235.7625974714756,19.2374025285244,19.2374025285244)'> took</span><span style='background-color:rgb(242.46712490916252,12.532875090837479,12.532875090837479)'> out</span><span style='background-color:rgb(242.62286050245166,12.377139497548342,12.377139497548342)'> a</span><span style='background-color:rgb(241.81338043883443,13.186619561165571,13.186619561165571)'> piece</span><span style='background-color:rgb(240.72699688374996,14.273003116250038,14.273003116250038)'> of</span><span style='background-color:rgb(242.25255327299237,12.747446727007627,12.747446727007627)'> paper</span><span style='background-color:rgb(239.7698443941772,15.230155605822802,15.230155605822802)'> and</span><span style='background-color:rgb(242.8512624464929,12.14873755350709,12.14873755350709)'> his</span><span style='background-color:rgb(242.041284609586,12.958715390414,12.958715390414)'> ink</span><span style='background-color:rgb(242.1819046139717,12.81809538602829,12.81809538602829)'> ##pot</span><span style='background-color:rgb(239.87576846033335,15.124231539666653,15.124231539666653)'> .</span><span style='background-color:rgb(245.0766092352569,9.92339076474309,9.92339076474309)'> after</span><span style='background-color:rgb(245.01046428456903,9.989535715430975,9.989535715430975)'> filling</span><span style='background-color:rgb(242.57571833208203,12.424281667917967,12.424281667917967)'> his</span><span style='background-color:rgb(244.9247133359313,10.075286664068699,10.075286664068699)'> pen</span><span style='background-color:rgb(241.4991059526801,13.500894047319889,13.500894047319889)'> ,</span><span style='background-color:rgb(239.59765758365393,15.402342416346073,15.402342416346073)'> allan</span><span style='background-color:rgb(239.9330598860979,15.066940113902092,15.066940113902092)'> looked</span><span style='background-color:rgb(243.50118147209287,11.498818527907133,11.498818527907133)'> at</span><span style='background-color:rgb(242.37941231578588,12.620587684214115,12.620587684214115)'> his</span><span style='background-color:rgb(244.4403580762446,10.559641923755407,10.559641923755407)'> paper</span><span style='background-color:rgb(243.6816917359829,11.318308264017105,11.318308264017105)'> in</span><span style='background-color:rgb(244.53197970986366,10.468020290136337,10.468020290136337)'> the</span><span style='background-color:rgb(242.94641884043813,12.053581159561872,12.053581159561872)'> orange</span><span style='background-color:rgb(243.11134804040194,11.888651959598064,11.888651959598064)'> glow</span><span style='background-color:rgb(242.95579578727484,12.044204212725163,12.044204212725163)'> from</span><span style='background-color:rgb(242.93679490685463,12.06320509314537,12.06320509314537)'> the</span><span style='background-color:rgb(240.87953304871917,14.120466951280832,14.120466951280832)'> lantern</span><span style='background-color:rgb(242.28764723986387,12.712352760136127,12.712352760136127)'> set</span><span style='background-color:rgb(238.82206980139017,16.17793019860983,16.17793019860983)'> back</span><span style='background-color:rgb(240.84370952099562,14.156290479004383,14.156290479004383)'> in</span><span style='background-color:rgb(216.93202771246433,38.06797228753567,38.06797228753567)'> the</span><span style='background-color:rgb(235.09684089571238,19.903159104287624,19.903159104287624)'> desk</span><span style='background-color:rgb(241.7840165644884,13.215983435511589,13.215983435511589)'> '</span><span style='background-color:rgb(241.751545406878,13.248454593122005,13.248454593122005)'> s</span><span style='background-color:rgb(242.08871461451054,12.911285385489464,12.911285385489464)'> right</span><span style='background-color:rgb(242.29845860973,12.701541390269995,12.701541390269995)'> -</span><span style='background-color:rgb(242.11218880489469,12.887811195105314,12.887811195105314)'> hand</span><span style='background-color:rgb(241.64027407765388,13.359725922346115,13.359725922346115)'> corner</span><span style='background-color:rgb(241.62376016378403,13.376239836215973,13.376239836215973)'> .</span><span style='background-color:rgb(241.91494518890977,13.085054811090231,13.085054811090231)'> his</span><span style='background-color:rgb(238.80397897213697,16.196021027863026,16.196021027863026)'> pen</span><span style='background-color:rgb(242.4323064275086,12.567693572491407,12.567693572491407)'> cast</span><span style='background-color:rgb(241.37149835005403,13.628501649945974,13.628501649945974)'> a</span><span style='background-color:rgb(239.13430949673057,15.865690503269434,15.865690503269434)'> forbid</span><span style='background-color:rgb(241.8839417025447,13.11605829745531,13.11605829745531)'> ##ding</span><span style='background-color:rgb(242.56484046578407,12.435159534215927,12.435159534215927)'> line</span><span style='background-color:rgb(241.42124623060226,13.578753769397736,13.578753769397736)'> of</span><span style='background-color:rgb(242.86237685009837,12.137623149901628,12.137623149901628)'> shadow</span><span style='background-color:rgb(244.84838588163257,10.151614118367434,10.151614118367434)'> slant</span><span style='background-color:rgb(242.3109171912074,12.689082808792591,12.689082808792591)'> ##ing</span><span style='background-color:rgb(244.0626459941268,10.937354005873203,10.937354005873203)'> across</span><span style='background-color:rgb(245.20826742053032,9.79173257946968,9.79173257946968)'> the</span><span style='background-color:rgb(245.2135386876762,9.786461312323809,9.786461312323809)'> page</span><span style='background-color:rgb(238.917575776577,16.082424223423004,16.082424223423004)'> ,</span><span style='background-color:rgb(243.89220423996449,11.107795760035515,11.107795760035515)'> echoing</span><span style='background-color:rgb(245.08250081911683,9.91749918088317,9.91749918088317)'> the</span><span style='background-color:rgb(241.0873904451728,13.912609554827213,13.912609554827213)'> ink</span><span style='background-color:rgb(242.33981939032674,12.660180609673262,12.660180609673262)'> ##y</span><span style='background-color:rgb(244.38146978616714,10.618530213832855,10.618530213832855)'> darkness</span><span style='background-color:rgb(241.81133614853024,13.188663851469755,13.188663851469755)'> crouch</span><span style='background-color:rgb(243.832368850708,11.167631149291992,11.167631149291992)'> ##ing</span><span style='background-color:rgb(243.0769370868802,11.923062913119793,11.923062913119793)'> in</span><span style='background-color:rgb(243.01088523119688,11.98911476880312,11.98911476880312)'> the</span><span style='background-color:rgb(246.70672841370106,8.293271586298943,8.293271586298943)'> edges</span><span style='background-color:rgb(242.036299277097,12.963700722903013,12.963700722903013)'> of</span><span style='background-color:rgb(244.92341285571456,10.07658714428544,10.07658714428544)'> the</span><span style='background-color:rgb(244.63149731978774,10.36850268021226,10.36850268021226)'> lantern</span><span style='background-color:rgb(242.62130163609982,12.378698363900185,12.378698363900185)'> '</span><span style='background-color:rgb(241.39356376603246,13.606436233967543,13.606436233967543)'> s</span><span style='background-color:rgb(245.10160429403186,9.898395705968142,9.898395705968142)'> struggling</span><span style='background-color:rgb(248.0360248591751,6.963975140824914,6.963975140824914)'> glow</span><span style='background-color:rgb(240.35542622208595,14.644573777914047,14.644573777914047)'> .</span><span style='background-color:rgb(244.12029745057225,10.879702549427748,10.879702549427748)'> the</span><span style='background-color:rgb(243.6426716297865,11.357328370213509,11.357328370213509)'> only</span><span style='background-color:rgb(241.28671824932098,13.713281750679016,13.713281750679016)'> other</span><span style='background-color:rgb(247.52625323832035,7.473746761679649,7.473746761679649)'> illumination</span><span style='background-color:rgb(244.09539073705673,10.904609262943268,10.904609262943268)'> came</span><span style='background-color:rgb(244.5499042980373,10.45009570196271,10.45009570196271)'> from</span><span style='background-color:rgb(245.7938938960433,9.2061061039567,9.2061061039567)'> a</span><span style='background-color:rgb(242.09041882306337,12.909581176936626,12.909581176936626)'> lu</span><span style='background-color:rgb(240.00367529690266,14.996324703097343,14.996324703097343)'> ##rid</span><span style='background-color:rgb(245.25679176673293,9.743208233267069,9.743208233267069)'> moonlight</span><span style='background-color:rgb(240.15352690592408,14.846473094075918,14.846473094075918)'> filtered</span><span style='background-color:rgb(242.74673575535417,12.253264244645834,12.253264244645834)'> through</span><span style='background-color:rgb(249.68490078113973,5.315099218860269,5.315099218860269)'> thin</span><span style='background-color:rgb(246.54122449457645,8.458775505423546,8.458775505423546)'> branches</span><span style='background-color:rgb(244.3375583924353,10.662441607564688,10.662441607564688)'> and</span><span style='background-color:rgb(247.23454859107733,7.765451408922672,7.765451408922672)'> clouds</span><span style='background-color:rgb(246.68915435671806,8.310845643281937,8.310845643281937)'> ,</span><span style='background-color:rgb(244.72194101661444,10.278058983385563,10.278058983385563)'> casting</span><span style='background-color:rgb(245.95359932631254,9.046400673687458,9.046400673687458)'> its</span><span style='background-color:rgb(246.22252229601145,8.777477703988552,8.777477703988552)'> bone</span><span style='background-color:rgb(243.74831641092896,11.251683589071035,11.251683589071035)'> -</span><span style='background-color:rgb(245.0102600455284,9.989739954471588,9.989739954471588)'> pale</span><span style='background-color:rgb(246.90171400085092,8.098285999149084,8.098285999149084)'> glow</span><span style='background-color:rgb(245.43609369546175,9.56390630453825,9.56390630453825)'> onto</span><span style='background-color:rgb(242.66280395910144,12.337196040898561,12.337196040898561)'> the</span><span style='background-color:rgb(247.52001634798944,7.47998365201056,7.47998365201056)'> pine</span><span style='background-color:rgb(244.1632636450231,10.836736354976892,10.836736354976892)'> floor</span><span style='background-color:rgb(244.55454574897885,10.445454251021147,10.445454251021147)'> ##boards</span><span style='background-color:rgb(248.84764943271875,6.152350567281246,6.152350567281246)'> .</span><span style='background-color:rgb(240.6236813776195,14.376318622380495,14.376318622380495)'> allan</span><span style='background-color:rgb(237.65615366399288,17.34384633600712,17.34384633600712)'> unfolded</span><span style='background-color:rgb(242.92962849140167,12.070371508598328,12.070371508598328)'> another</span><span style='background-color:rgb(243.68675021454692,11.313249785453081,11.313249785453081)'> page</span><span style='background-color:rgb(239.56488339230418,15.435116607695818,15.435116607695818)'> ,</span><span style='background-color:rgb(242.67434488981962,12.325655110180378,12.325655110180378)'> this</span><span style='background-color:rgb(239.48950493708253,15.510495062917471,15.510495062917471)'> one</span><span style='background-color:rgb(236.52331095188856,18.47668904811144,18.47668904811144)'> crowded</span><span style='background-color:rgb(160.5587200820446,94.4412799179554,94.4412799179554)'> with</span><span style='background-color:rgb(239.44785822182894,15.552141778171062,15.552141778171062)'> ranks</span><span style='background-color:rgb(238.08053579181433,16.919464208185673,16.919464208185673)'> of</span><span style='background-color:rgb(237.43050087243319,17.569499127566814,17.569499127566814)'> letters</span><span style='background-color:rgb(240.9692215360701,14.030778463929892,14.030778463929892)'> in</span><span style='background-color:rgb(242.73399978876114,12.266000211238861,12.266000211238861)'> tight</span><span style='background-color:rgb(242.2261019423604,12.773898057639599,12.773898057639599)'> formation</span><span style='background-color:rgb(241.8380420655012,13.161957934498787,13.161957934498787)'> from</span><span style='background-color:rgb(239.9431606940925,15.056839305907488,15.056839305907488)'> left</span><span style='background-color:rgb(241.2007792107761,13.79922078922391,13.79922078922391)'> to</span><span style='background-color:rgb(241.41791285946965,13.582087140530348,13.582087140530348)'> right</span><span style='background-color:rgb(240.8867023140192,14.113297685980797,14.113297685980797)'> .</span><span style='background-color:rgb(244.2108085937798,10.789191406220198,10.789191406220198)'> the</span><span style='background-color:rgb(240.22731989622116,14.772680103778839,14.772680103778839)'> lines</span><span style='background-color:rgb(240.0074285455048,14.992571454495192,14.992571454495192)'> of</span><span style='background-color:rgb(238.0300972983241,16.969902701675892,16.969902701675892)'> letters</span><span style='background-color:rgb(240.0814153254032,14.918584674596786,14.918584674596786)'> stepped</span><span style='background-color:rgb(241.76577089354396,13.234229106456041,13.234229106456041)'> into</span><span style='background-color:rgb(242.11119705811143,12.88880294188857,12.88880294188857)'> their</span><span style='background-color:rgb(241.3625061325729,13.63749386742711,13.63749386742711)'> divisions</span><span style='background-color:rgb(237.41475071758032,17.58524928241968,17.58524928241968)'> ,</span><span style='background-color:rgb(241.74117291346192,13.258827086538076,13.258827086538076)'> in</span><span style='background-color:rgb(243.13899820670485,11.861001793295145,11.861001793295145)'> the</span><span style='background-color:rgb(241.42806021496654,13.571939785033464,13.571939785033464)'> shape</span><span style='background-color:rgb(240.75602827593684,14.243971724063158,14.243971724063158)'> of</span><span style='background-color:rgb(240.657465364784,14.342534635215998,14.342534635215998)'> a</span><span style='background-color:rgb(230.72367075830698,24.27632924169302,24.27632924169302)'> story</span><span style='background-color:rgb(240.34688618034124,14.653113819658756,14.653113819658756)'> '</span><span style='background-color:rgb(241.02275116369128,13.977248836308718,13.977248836308718)'> s</span><span style='background-color:rgb(245.21959841251373,9.780401587486267,9.780401587486267)'> outline</span><span style='background-color:rgb(238.94094832241535,16.059051677584648,16.059051677584648)'> :</span><span style='background-color:rgb(243.75041104853153,11.249588951468468,11.249588951468468)'> the</span><span style='background-color:rgb(241.2880871258676,13.711912874132395,13.711912874132395)'> loose</span><span style='background-color:rgb(237.45551872998476,17.54448127001524,17.54448127001524)'> ,</span><span style='background-color:rgb(244.5320139080286,10.467986091971397,10.467986091971397)'> dry</span><span style='background-color:rgb(245.29094148427248,9.70905851572752,9.70905851572752)'> skeleton</span><span style='background-color:rgb(243.68428414687514,11.315715853124857,11.315715853124857)'> of</span><span style='background-color:rgb(245.40159914642572,9.598400853574276,9.598400853574276)'> a</span><span style='background-color:rgb(240.84743712097406,14.152562879025936,14.152562879025936)'> tale</span><span style='background-color:rgb(243.51244121789932,11.487558782100677,11.487558782100677)'> lay</span><span style='background-color:rgb(241.6960284858942,13.303971514105797,13.303971514105797)'> exposed</span><span style='background-color:rgb(241.2039558403194,13.796044159680605,13.796044159680605)'> beneath</span><span style='background-color:rgb(241.71853847801685,13.281461521983147,13.281461521983147)'> their</span><span style='background-color:rgb(240.34752264618874,14.652477353811264,14.652477353811264)'> feet</span><span style='background-color:rgb(237.55374155938625,17.446258440613747,17.446258440613747)'> ,</span><span style='background-color:rgb(240.22374523803592,14.776254761964083,14.776254761964083)'> awaiting</span><span style='background-color:rgb(241.6146378032863,13.385362196713686,13.385362196713686)'> tend</span><span style='background-color:rgb(242.46037742123008,12.539622578769922,12.539622578769922)'> ##ons</span><span style='background-color:rgb(236.42141941934824,18.57858058065176,18.57858058065176)'> ,</span><span style='background-color:rgb(240.9641013108194,14.035898689180613,14.035898689180613)'> muscle</span><span style='background-color:rgb(238.8263388723135,16.1736611276865,16.1736611276865)'> and</span><span style='background-color:rgb(236.06591809540987,18.93408190459013,18.93408190459013)'> blushing</span><span style='background-color:rgb(233.5845961421728,21.415403857827187,21.415403857827187)'> skin</span><span style='background-color:rgb(239.98333593830466,15.016664061695337,15.016664061695337)'> .</span><span style='background-color:rgb(237.54318572580814,17.456814274191856,17.456814274191856)'> allan</span><span style='background-color:rgb(233.2973277568817,21.702672243118286,21.702672243118286)'> reviewed</span><span style='background-color:rgb(227.91040051728487,27.08959948271513,27.08959948271513)'> the</span><span style='background-color:rgb(203.54593120515347,51.454068794846535,51.454068794846535)'> troops</span><span style='background-color:rgb(216.46407142281532,38.53592857718468,38.53592857718468)'> ,</span><span style='background-color:rgb(240.70186883211136,14.298131167888641,14.298131167888641)'> all</span><span style='background-color:rgb(235.8759244903922,19.124075509607792,19.124075509607792)'> prepared</span><span style='background-color:rgb(238.3390549197793,16.6609450802207,16.6609450802207)'> to</span><span style='background-color:rgb(235.91410674154758,19.085893258452415,19.085893258452415)'> di</span><span style='background-color:rgb(238.30408159643412,16.695918403565884,16.695918403565884)'> ##se</span><span style='background-color:rgb(235.63772287219763,19.362277127802372,19.362277127802372)'> ##mba</span><span style='background-color:rgb(241.7081232368946,13.291876763105392,13.291876763105392)'> ##rk</span><span style='background-color:rgb(234.36224341392517,20.63775658607483,20.63775658607483)'> ,</span><span style='background-color:rgb(237.14695438742638,17.853045612573624,17.853045612573624)'> their</span><span style='background-color:rgb(241.43140783533454,13.56859216466546,13.56859216466546)'> task</span><span style='background-color:rgb(241.72194689512253,13.278053104877472,13.278053104877472)'> to</span><span style='background-color:rgb(242.1807361766696,12.819263823330402,12.819263823330402)'> form</span><span style='background-color:rgb(244.06891660764813,10.931083392351866,10.931083392351866)'> the</span><span style='background-color:rgb(241.7300632596016,13.269936740398407,13.269936740398407)'> tale</span><span style='background-color:rgb(241.86580907553434,13.134190924465656,13.134190924465656)'> of</span><span style='background-color:rgb(242.34893795102835,12.651062048971653,12.651062048971653)'> a</span><span style='background-color:rgb(241.783565338701,13.21643466129899,13.21643466129899)'> young</span><span style='background-color:rgb(239.887981005013,15.112018994987011,15.112018994987011)'> man</span><span style='background-color:rgb(236.74044080078602,18.25955919921398,18.25955919921398)'> returning</span><span style='background-color:rgb(242.3083447292447,12.691655270755291,12.691655270755291)'> home</span><span style='background-color:rgb(242.06902692094445,12.930973079055548,12.930973079055548)'> from</span><span style='background-color:rgb(242.45210621505976,12.547893784940243,12.547893784940243)'> life</span><span style='background-color:rgb(242.91897291317582,12.081027086824179,12.081027086824179)'> abroad</span><span style='background-color:rgb(242.60601980611682,12.39398019388318,12.39398019388318)'> to</span><span style='background-color:rgb(245.072027631104,9.927972368896008,9.927972368896008)'> find</span><span style='background-color:rgb(243.3486719056964,11.651328094303608,11.651328094303608)'> his</span><span style='background-color:rgb(241.39151852577925,13.608481474220753,13.608481474220753)'> childhood</span><span style='background-color:rgb(242.52267222851515,12.477327771484852,12.477327771484852)'> friend</span><span style='background-color:rgb(243.32486048340797,11.675139516592026,11.675139516592026)'> a</span><span style='background-color:rgb(236.75502441823483,18.244975581765175,18.244975581765175)'> bride</span><span style='background-color:rgb(242.95531511306763,12.044684886932373,12.044684886932373)'> to</span><span style='background-color:rgb(240.50802793353796,14.49197206646204,14.49197206646204)'> -</span><span style='background-color:rgb(242.00222650542855,12.997773494571447,12.997773494571447)'> be</span><span style='background-color:rgb(238.84836439043283,16.151635609567165,16.151635609567165)'> ,</span><span style='background-color:rgb(242.88623386994004,12.113766130059958,12.113766130059958)'> thus</span><span style='background-color:rgb(242.73403968662024,12.265960313379765,12.265960313379765)'> upset</span><span style='background-color:rgb(239.82701517641544,15.172984823584557,15.172984823584557)'> ##ting</span><span style='background-color:rgb(241.09903492033482,13.900965079665184,13.900965079665184)'> the</span><span style='background-color:rgb(241.54191825538874,13.458081744611263,13.458081744611263)'> apple</span><span style='background-color:rgb(243.17216092720628,11.827839072793722,11.827839072793722)'> cart</span><span style='background-color:rgb(239.60814502090216,15.391854979097843,15.391854979097843)'> of</span><span style='background-color:rgb(241.68076945468783,13.319230545312166,13.319230545312166)'> his</span><span style='background-color:rgb(242.3350658454001,12.664934154599905,12.664934154599905)'> life</span><span style='background-color:rgb(237.703204639256,17.296795360744,17.296795360744)'> '</span><span style='background-color:rgb(240.62740992754698,14.372590072453022,14.372590072453022)'> s</span><span style='background-color:rgb(239.35725588351488,15.642744116485119,15.642744116485119)'> plan</span><span style='background-color:rgb(231.4546698331833,23.54533016681671,23.54533016681671)'> ,</span><span style='background-color:rgb(239.3994716182351,15.600528381764889,15.600528381764889)'> clarified</span><span style='background-color:rgb(236.83123502880335,18.16876497119665,18.16876497119665)'> –</span><span style='background-color:rgb(242.92193485423923,12.078065145760775,12.078065145760775)'> of</span><span style='background-color:rgb(240.85456838831306,14.145431611686945,14.145431611686945)'> course</span><span style='background-color:rgb(235.88067043572664,19.119329564273357,19.119329564273357)'> –</span><span style='background-color:rgb(243.0878966487944,11.912103351205587,11.912103351205587)'> by</span><span style='background-color:rgb(238.58783707022667,16.41216292977333,16.41216292977333)'> his</span><span style='background-color:rgb(240.87116304785013,14.128836952149868,14.128836952149868)'> very</span><span style='background-color:rgb(241.17358691990376,13.826413080096245,13.826413080096245)'> time</span><span style='background-color:rgb(241.9364757835865,13.063524216413498,13.063524216413498)'> away</span><span style='background-color:rgb(241.4204102754593,13.57958972454071,13.57958972454071)'> from</span><span style='background-color:rgb(244.4628737680614,10.5371262319386,10.5371262319386)'> her</span><span style='background-color:rgb(241.84158347547054,13.158416524529457,13.158416524529457)'> he</span><span style='background-color:rgb(245.25390297174454,9.746097028255463,9.746097028255463)'> loved</span><span style='background-color:rgb(242.23648773506284,12.763512264937162,12.763512264937162)'> best</span><span style='background-color:rgb(236.58026419579983,18.419735804200172,18.419735804200172)'> .</span><span style='background-color:rgb(243.18926380947232,11.810736190527678,11.810736190527678)'> although</span><span style='background-color:rgb(240.76492549851537,14.235074501484632,14.235074501484632)'> the</span><span style='background-color:rgb(237.6892650872469,17.310734912753105,17.310734912753105)'> concept</span><span style='background-color:rgb(241.0731383599341,13.926861640065908,13.926861640065908)'> was</span><span style='background-color:rgb(234.88215811550617,20.117841884493828,20.117841884493828)'> a</span><span style='background-color:rgb(240.54731592535973,14.452684074640274,14.452684074640274)'> simple</span><span style='background-color:rgb(240.31991807743907,14.68008192256093,14.68008192256093)'> one</span><span style='background-color:rgb(235.9964521229267,19.003547877073288,19.003547877073288)'> ,</span><span style='background-color:rgb(234.25089228898287,20.749107711017132,20.749107711017132)'> allan</span><span style='background-color:rgb(239.59957933053374,15.400420669466257,15.400420669466257)'> thought</span><span style='background-color:rgb(241.3032853603363,13.696714639663696,13.696714639663696)'> it</span><span style='background-color:rgb(241.03322435170412,13.96677564829588,13.96677564829588)'> had</span><span style='background-color:rgb(243.19646252319217,11.803537476807833,11.803537476807833)'> potential</span><span style='background-color:rgb(239.11199804395437,15.888001956045628,15.888001956045628)'> .</span><span style='background-color:rgb(242.80497713014483,12.195022869855165,12.195022869855165)'> besides</span><span style='background-color:rgb(238.48690688610077,16.51309311389923,16.51309311389923)'> ,</span><span style='background-color:rgb(241.5396241284907,13.460375871509314,13.460375871509314)'> the</span><span style='background-color:rgb(240.83652220666409,14.163477793335915,14.163477793335915)'> public</span><span style='background-color:rgb(242.48782144859433,12.512178551405668,12.512178551405668)'> liked</span><span style='background-color:rgb(241.69893817976117,13.301061820238829,13.301061820238829)'> a</span><span style='background-color:rgb(238.15345387905836,16.84654612094164,16.84654612094164)'> good</span><span style='background-color:rgb(239.2208033055067,15.779196694493294,15.779196694493294)'> ,</span><span style='background-color:rgb(239.47056960314512,15.529430396854877,15.529430396854877)'> simple</span><span style='background-color:rgb(228.567457459867,26.432542540133,26.432542540133)'> romance</span><span style='background-color:rgb(236.89410645514727,18.105893544852734,18.105893544852734)'> .</span><span style='background-color:rgb(239.65718898922205,15.34281101077795,15.34281101077795)'> perhaps</span><span style='background-color:rgb(242.04422280192375,12.955777198076248,12.955777198076248)'> this</span><span style='background-color:rgb(241.72213023528457,13.277869764715433,13.277869764715433)'> will</span><span style='background-color:rgb(241.93220006302,13.067799936980009,13.067799936980009)'> be</span><span style='background-color:rgb(241.34179154410958,13.658208455890417,13.658208455890417)'> more</span><span style='background-color:rgb(240.78867137432098,14.211328625679016,14.211328625679016)'> sale</span><span style='background-color:rgb(240.4914304241538,14.508569575846195,14.508569575846195)'> ##able</span><span style='background-color:rgb(239.61772810667753,15.382271893322468,15.382271893322468)'> ,</span><span style='background-color:rgb(242.06498013809323,12.935019861906767,12.935019861906767)'> he</span><span style='background-color:rgb(242.13827250525355,12.861727494746447,12.861727494746447)'> thought</span><span style='background-color:rgb(239.1064978390932,15.893502160906792,15.893502160906792)'> and</span><span style='background-color:rgb(237.91757583618164,17.08242416381836,17.08242416381836)'> began</span><span style='background-color:rgb(237.0328788086772,17.967121191322803,17.967121191322803)'> to</span><span style='background-color:rgb(231.71996399760246,23.280036002397537,23.280036002397537)'> write</span><span style='background-color:rgb(225.69712568074465,29.302874319255352,29.302874319255352)'> .</span><span style='background-color:rgb(236.26075264066458,18.739247359335423,18.739247359335423)'> they</span><span style='background-color:rgb(223.31664271652699,31.683357283473015,31.683357283473015)'> gazed</span><span style='background-color:rgb(236.44450888037682,18.555491119623184,18.555491119623184)'> at</span><span style='background-color:rgb(237.51254227012396,17.48745772987604,17.48745772987604)'> each</span><span style='background-color:rgb(237.54427436739206,17.455725632607937,17.455725632607937)'> other</span><span style='background-color:rgb(232.19211716204882,22.807882837951183,22.807882837951183)'> ,</span><span style='background-color:rgb(239.55342320725322,15.446576792746782,15.446576792746782)'> lost</span><span style='background-color:rgb(239.59710661321878,15.402893386781216,15.402893386781216)'> in</span><span style='background-color:rgb(240.5191014893353,14.480898510664701,14.480898510664701)'> the</span><span style='background-color:rgb(240.7598584704101,14.240141529589891,14.240141529589891)'> rap</span><span style='background-color:rgb(241.82462783530354,13.175372164696455,13.175372164696455)'> ##ture</span><span style='background-color:rgb(240.16445606946945,14.835543930530548,14.835543930530548)'> of</span><span style='background-color:rgb(242.19102317467332,12.808976825326681,12.808976825326681)'> love</span><span style='background-color:rgb(239.1226640716195,15.87733592838049,15.87733592838049)'> based</span><span style='background-color:rgb(236.8973970785737,18.102602921426296,18.102602921426296)'> so</span><span style='background-color:rgb(240.88181672617793,14.11818327382207,14.11818327382207)'> deeply</span><span style='background-color:rgb(240.07866902276874,14.921330977231264,14.921330977231264)'> within</span><span style='background-color:rgb(237.40637596696615,17.593624033033848,17.593624033033848)'> their</span><span style='background-color:rgb(237.7821321040392,17.217867895960808,17.217867895960808)'> hearts</span><span style='background-color:rgb(243.38059589266777,11.61940410733223,11.61940410733223)'> that</span><span style='background-color:rgb(236.7345131188631,18.265486881136894,18.265486881136894)'> they</span><span style='background-color:rgb(238.33708852529526,16.662911474704742,16.662911474704742)'> had</span><span style='background-color:rgb(239.9599120952189,15.040087904781103,15.040087904781103)'> never</span><span style='background-color:rgb(238.57220470905304,16.42779529094696,16.42779529094696)'> seen</span><span style='background-color:rgb(240.53375635296106,14.466243647038937,14.466243647038937)'> it</span><span style='background-color:rgb(236.2349121272564,18.765087872743607,18.765087872743607)'> before</span><span style='background-color:rgb(210.48718594014645,44.512814059853554,44.512814059853554)'> .</span><span style='background-color:rgb(219.02574196457863,35.97425803542137,35.97425803542137)'> "</span><span style='background-color:rgb(231.49368803948164,23.50631196051836,23.50631196051836)'> what</span><span style='background-color:rgb(252.22591468133032,2.774085318669677,2.774085318669677)'> about</span><span style='background-color:rgb(226.56086303293705,28.43913696706295,28.43913696706295)'> roger</span><span style='background-color:rgb(253.65300552919507,1.3469944708049297,1.3469944708049297)'> ?</span><span style='background-color:rgb(204.98263031244278,50.01736968755722,50.01736968755722)'> "</span><span style='background-color:rgb(227.17606142163277,27.823938578367233,27.823938578367233)'> she</span><span style='background-color:rgb(171.1661396920681,83.8338603079319,83.8338603079319)'> asked</span> ,<span style='background-color:rgb(193.33092033863068,61.669079661369324,61.669079661369324)'> knowing</span><span style='background-color:rgb(147.78062894940376,107.21937105059624,107.21937105059624)'> that</span><span style='background-color:rgb(196.64580166339874,58.35419833660126,58.35419833660126)'> the</span><span style='background-color:rgb(119.62971031665802,135.37028968334198,135.37028968334198)'> answer</span><span style='background-color:rgb(161.22890532016754,93.77109467983246,93.77109467983246)'> no</span><span style='background-color:rgb(72.61644423007965,182.38355576992035,182.38355576992035)'> longer</span><span style='background-color:rgb(0.0,255.0,255.0)'> mattered</span><span style='background-color:rgb(167.89283573627472,87.10716426372528,87.10716426372528)'> .</span><span style='background-color:rgb(194.64185759425163,60.35814240574837,60.35814240574837)'> [SEP]</span><span style='background-color:rgb(207.6290249079466,47.37097509205341,47.37097509205341)'> name</span><span style='background-color:rgb(229.23811577260494,25.761884227395058,25.761884227395058)'> few</span><span style='background-color:rgb(216.53518460690975,38.46481539309025,38.46481539309025)'> objects</span><span style='background-color:rgb(222.17997930943966,32.82002069056034,32.82002069056034)'> said</span><span style='background-color:rgb(225.75940623879433,29.240593761205673,29.240593761205673)'> to</span><span style='background-color:rgb(227.64903344213963,27.350966557860374,27.350966557860374)'> be</span><span style='background-color:rgb(225.9975166618824,29.0024833381176,29.0024833381176)'> in</span><span style='background-color:rgb(209.00230161845684,45.99769838154316,45.99769838154316)'> or</span><span style='background-color:rgb(212.07762859761715,42.92237140238285,42.92237140238285)'> on</span><span style='background-color:rgb(229.8073974251747,25.192602574825287,25.192602574825287)'> allan</span><span style='background-color:rgb(232.8408696502447,22.159130349755287,22.159130349755287)'> '</span><span style='background-color:rgb(238.01575876772404,16.984241232275963,16.984241232275963)'> s</span><span style='background-color:rgb(233.01793444901705,21.982065550982952,21.982065550982952)'> desk</span><span style='background-color:rgb(228.34951635450125,26.650483645498753,26.650483645498753)'> |</span><span style='background-color:rgb(242.51773154363036,12.482268456369638,12.482268456369638)'> |</span><span style='background-color:rgb(230.13502344489098,24.864976555109024,24.864976555109024)'> erase</span><span style='background-color:rgb(139.7602853178978,115.2397146821022,115.2397146821022)'> ##r</span><span style='background-color:rgb(236.44309725612402,18.55690274387598,18.55690274387598)'> [SEP]</span><span style='background-color:rgb(222.73480653762817,32.265193462371826,32.265193462371826)'> [PAD]</span><span style='background-color:rgb(231.21562846004963,23.78437153995037,23.78437153995037)'> [PAD]</span><span style='background-color:rgb(228.01094312220812,26.98905687779188,26.98905687779188)'> [PAD]</span><span style='background-color:rgb(237.07619838416576,17.923801615834236,17.923801615834236)'> [PAD]</span><span style='background-color:rgb(241.5535237826407,13.446476217359304,13.446476217359304)'> [PAD]</span><span style='background-color:rgb(243.2202017493546,11.779798250645399,11.779798250645399)'> [PAD]</span><span style='background-color:rgb(238.21130767464638,16.788692325353622,16.788692325353622)'> [PAD]</span><span style='background-color:rgb(234.0790294110775,20.9209705889225,20.9209705889225)'> [PAD]</span><span style='background-color:rgb(235.3164292126894,19.6835707873106,19.6835707873106)'> [PAD]</span><span style='background-color:rgb(234.3349589779973,20.665041022002697,20.665041022002697)'> [PAD]</span><span style='background-color:rgb(235.356912240386,19.64308775961399,19.64308775961399)'> [PAD]</span><span style='background-color:rgb(233.48764814436436,21.512351855635643,21.512351855635643)'> [PAD]</span><span style='background-color:rgb(233.27397231012583,21.726027689874172,21.726027689874172)'> [PAD]</span><span style='background-color:rgb(237.55917716771364,17.440822832286358,17.440822832286358)'> [PAD]</span><span style='background-color:rgb(236.28823656588793,18.711763434112072,18.711763434112072)'> [PAD]</span><span style='background-color:rgb(237.07311864942312,17.926881350576878,17.926881350576878)'> [PAD]</span><span style='background-color:rgb(236.74848306924105,18.251516930758953,18.251516930758953)'> [PAD]</span><span style='background-color:rgb(236.05850659310818,18.941493406891823,18.941493406891823)'> [PAD]</span><span style='background-color:rgb(233.91004297882318,21.089957021176815,21.089957021176815)'> [PAD]</span><span style='background-color:rgb(235.57545561343431,19.424544386565685,19.424544386565685)'> [PAD]</span><span style='background-color:rgb(236.67209386825562,18.327906131744385,18.327906131744385)'> [PAD]</span><span style='background-color:rgb(236.95011354982853,18.04988645017147,18.04988645017147)'> [PAD]</span><span style='background-color:rgb(237.1416536718607,17.858346328139305,17.858346328139305)'> [PAD]</span><span style='background-color:rgb(236.92211855202913,18.077881447970867,18.077881447970867)'> [PAD]</span><span style='background-color:rgb(238.73016983270645,16.26983016729355,16.26983016729355)'> [PAD]</span><span style='background-color:rgb(236.8021323904395,18.19786760956049,18.19786760956049)'> [PAD]</span><span style='background-color:rgb(235.16472235322,19.835277646780014,19.835277646780014)'> [PAD]</span><span style='background-color:rgb(229.6924326941371,25.307567305862904,25.307567305862904)'> [PAD]</span><span style='background-color:rgb(239.3054067157209,15.694593284279108,15.694593284279108)'> [PAD]</span><span style='background-color:rgb(238.1816103681922,16.818389631807804,16.818389631807804)'> [PAD]</span><span style='background-color:rgb(239.1604178957641,15.839582104235888,15.839582104235888)'> [PAD]</span><span style='background-color:rgb(237.5930219516158,17.40697804838419,17.40697804838419)'> [PAD]</span><span style='background-color:rgb(239.4468237273395,15.553176272660494,15.553176272660494)'> [PAD]</span><span style='background-color:rgb(238.97626172751188,16.023738272488117,16.023738272488117)'> [PAD]</span><span style='background-color:rgb(229.06540364027023,25.934596359729767,25.934596359729767)'> [PAD]</span><span style='background-color:rgb(240.64705677330494,14.35294322669506,14.35294322669506)'> [PAD]</span><span style='background-color:rgb(240.45215288177133,14.547847118228674,14.547847118228674)'> [PAD]</span><span style='background-color:rgb(236.43818031996489,18.561819680035114,18.561819680035114)'> [PAD]</span><span style='background-color:rgb(242.8853295184672,12.114670481532812,12.114670481532812)'> [PAD]</span><span style='background-color:rgb(240.8488249965012,14.151175003498793,14.151175003498793)'> [PAD]</span><span style='background-color:rgb(239.77830559015274,15.22169440984726,15.22169440984726)'> [PAD]</span><span style='background-color:rgb(238.51234462112188,16.487655378878117,16.487655378878117)'> [PAD]</span><span style='background-color:rgb(239.75871859118342,15.241281408816576,15.241281408816576)'> [PAD]</span><span style='background-color:rgb(236.97742078453302,18.022579215466976,18.022579215466976)'> [PAD]</span><span style='background-color:rgb(237.60032325983047,17.399676740169525,17.399676740169525)'> [PAD]</span><span style='background-color:rgb(237.74377696216106,17.256223037838936,17.256223037838936)'> [PAD]</span><span style='background-color:rgb(238.49899783730507,16.50100216269493,16.50100216269493)'> [PAD]</span><span style='background-color:rgb(239.0323182195425,15.967681780457497,15.967681780457497)'> [PAD]</span><span style='background-color:rgb(238.8777045160532,16.1222954839468,16.1222954839468)'> [PAD]</span><span style='background-color:rgb(240.14894530177116,14.851054698228836,14.851054698228836)'> [PAD]</span><span style='background-color:rgb(239.72164208069444,15.278357919305563,15.278357919305563)'> [PAD]</span><span style='background-color:rgb(244.3457412533462,10.654258746653795,10.654258746653795)'> [PAD]</span><span style='background-color:rgb(244.75903747603297,10.240962523967028,10.240962523967028)'> [PAD]</span><span style='background-color:rgb(249.01522804051638,5.9847719594836235,5.9847719594836235)'> [PAD]</span><span style='background-color:rgb(244.87519059330225,10.12480940669775,10.12480940669775)'> [PAD]</span><span style='background-color:rgb(245.7940173894167,9.205982610583305,9.205982610583305)'> [PAD]</span><span style='background-color:rgb(242.56131045520306,12.438689544796944,12.438689544796944)'> [PAD]</span><span style='background-color:rgb(240.58017561212182,14.41982438787818,14.41982438787818)'> [PAD]</span><span style='background-color:rgb(238.36280174553394,16.637198254466057,16.637198254466057)'> [PAD]</span><span style='background-color:rgb(238.26808042824268,16.731919571757317,16.731919571757317)'> [PAD]</span><span style='background-color:rgb(239.20094937086105,15.799050629138947,15.799050629138947)'> [PAD]</span><span style='background-color:rgb(241.43859514966607,13.561404850333929,13.561404850333929)'> [PAD]</span><span style='background-color:rgb(239.6144849807024,15.3855150192976,15.3855150192976)'> [PAD]</span><span style='background-color:rgb(239.1647183150053,15.835281684994698,15.835281684994698)'> [PAD]</span><span style='background-color:rgb(237.96980023384094,17.030199766159058,17.030199766159058)'> [PAD]</span><span style='background-color:rgb(238.2787198573351,16.72128014266491,16.72128014266491)'> [PAD]</span><span style='background-color:rgb(236.81207835674286,18.18792164325714,18.18792164325714)'> [PAD]</span><span style='background-color:rgb(238.98045480251312,16.019545197486877,16.019545197486877)'> [PAD]</span><span style='background-color:rgb(240.05967954173684,14.940320458263159,14.940320458263159)'> [PAD]</span><span style='background-color:rgb(241.46553380414844,13.534466195851564,13.534466195851564)'> [PAD]</span><span style='background-color:rgb(239.66395262628794,15.336047373712063,15.336047373712063)'> [PAD]</span><span style='background-color:rgb(239.72952095791698,15.270479042083025,15.270479042083025)'> [PAD]</span><span style='background-color:rgb(241.2440712377429,13.7559287622571,13.7559287622571)'> [PAD]</span><span style='background-color:rgb(240.89497921988368,14.10502078011632,14.10502078011632)'> [PAD]</span><span style='background-color:rgb(241.26315381377935,13.736846186220646,13.736846186220646)'> [PAD]</span><span style='background-color:rgb(243.0416426807642,11.958357319235802,11.958357319235802)'> [PAD]</span><span style='background-color:rgb(241.25784834846854,13.742151651531458,13.742151651531458)'> [PAD]</span><span style='background-color:rgb(240.29217576608062,14.707824233919382,14.707824233919382)'> [PAD]</span><span style='background-color:rgb(241.27048172056675,13.72951827943325,13.72951827943325)'> [PAD]</span><span style='background-color:rgb(243.54424361139536,11.455756388604641,11.455756388604641)'> [PAD]</span><span style='background-color:rgb(242.29574555531144,12.704254444688559,12.704254444688559)'> [PAD]</span><span style='background-color:rgb(246.40301451086998,8.59698548913002,8.59698548913002)'> [PAD]</span><span style='background-color:rgb(242.10399834439158,12.896001655608416,12.896001655608416)'> [PAD]</span><span style='background-color:rgb(247.62594136409461,7.374058635905385,7.374058635905385)'> [PAD]</span><span style='background-color:rgb(242.04996334388852,12.950036656111479,12.950036656111479)'> [PAD]</span><span style='background-color:rgb(243.52416548877954,11.475834511220455,11.475834511220455)'> [PAD]</span><span style='background-color:rgb(244.77394787594676,10.22605212405324,10.22605212405324)'> [PAD]</span><span style='background-color:rgb(243.45410769805312,11.545892301946878,11.545892301946878)'> [PAD]</span><span style='background-color:rgb(242.36446676775813,12.635533232241869,12.635533232241869)'> [PAD]</span><span style='background-color:rgb(243.10850199311972,11.891498006880283,11.891498006880283)'> [PAD]</span><span style='background-color:rgb(242.58855309337378,12.411446906626225,12.411446906626225)'> [PAD]</span><span style='background-color:rgb(243.56286926195025,11.437130738049746,11.437130738049746)'> [PAD]</span><span style='background-color:rgb(241.42851809039712,13.57148190960288,13.57148190960288)'> [PAD]</span><span style='background-color:rgb(241.96895834058523,13.031041659414768,13.031041659414768)'> [PAD]</span><span style='background-color:rgb(237.24456924945116,17.75543075054884,17.75543075054884)'> [PAD]</span><span style='background-color:rgb(238.09464063495398,16.905359365046024,16.905359365046024)'> [PAD]</span><span style='background-color:rgb(238.88023518025875,16.11976481974125,16.11976481974125)'> [PAD]</span><span style='background-color:rgb(237.28834860026836,17.711651399731636,17.711651399731636)'> [PAD]</span><span style='background-color:rgb(238.017139993608,16.982860006392002,16.982860006392002)'> [PAD]</span><span style='background-color:rgb(237.89421278983355,17.105787210166454,17.105787210166454)'> [PAD]</span><span style='background-color:rgb(241.0218097642064,13.97819023579359,13.97819023579359)'> [PAD]</span><span style='background-color:rgb(237.15800419449806,17.841995805501938,17.841995805501938)'> [PAD]</span><span style='background-color:rgb(238.13936803489923,16.860631965100765,16.860631965100765)'> [PAD]</span><span style='background-color:rgb(236.62755645811558,18.372443541884422,18.372443541884422)'> [PAD]</span><span style='background-color:rgb(233.81864458322525,21.18135541677475,21.18135541677475)'> [PAD]</span><span style='background-color:rgb(234.85357984900475,20.146420150995255,20.146420150995255)'> [PAD]</span><span style='background-color:rgb(235.50085611641407,19.49914388358593,19.49914388358593)'> [PAD]</span><span style='background-color:rgb(237.14675299823284,17.85324700176716,17.85324700176716)'> [PAD]</span><span style='background-color:rgb(235.23231122642756,19.767688773572445,19.767688773572445)'> [PAD]</span><span style='background-color:rgb(236.01391788572073,18.98608211427927,18.98608211427927)'> [PAD]</span><span style='background-color:rgb(236.09540831297636,18.90459168702364,18.90459168702364)'> [PAD]</span><span style='background-color:rgb(238.81641380488873,16.183586195111275,16.183586195111275)'> [PAD]</span><span style='background-color:rgb(237.76697851717472,17.23302148282528,17.23302148282528)'> [PAD]</span><span style='background-color:rgb(237.4016908183694,17.59830918163061,17.59830918163061)'> [PAD]</span><span style='background-color:rgb(242.43643775582314,12.563562244176865,12.563562244176865)'> [PAD]</span><span style='background-color:rgb(237.45861936360598,17.541380636394024,17.541380636394024)'> [PAD]</span><span style='background-color:rgb(245.22433770820498,9.775662291795015,9.775662291795015)'> [PAD]</span><span style='background-color:rgb(181.9658137857914,73.0341862142086,73.0341862142086)'> [PAD]</span><span style='background-color:rgb(241.7151015624404,13.284898437559605,13.284898437559605)'> [PAD]</span><span style='background-color:rgb(236.32730226963758,18.672697730362415,18.672697730362415)'> [PAD]</span><span style='background-color:rgb(236.7750759422779,18.22492405772209,18.22492405772209)'> [PAD]</span><span style='background-color:rgb(242.01917169615626,12.980828303843737,12.980828303843737)'> [PAD]</span><span style='background-color:rgb(221.09443455934525,33.905565440654755,33.905565440654755)'> [PAD]</span><span style='background-color:rgb(248.7319370917976,6.26806290820241,6.26806290820241)'> [PAD]</span><span style='background-color:rgb(215.24913601577282,39.75086398422718,39.75086398422718)'> [PAD]</span><span style='background-color:rgb(235.03193467855453,19.968065321445465,19.968065321445465)'> [PAD]</span><span style='background-color:rgb(231.43837440758944,23.561625592410564,23.561625592410564)'> [PAD]</span><span style='background-color:rgb(240.7988453283906,14.201154671609402,14.201154671609402)'> [PAD]</span><span style='background-color:rgb(245.74179394170642,9.258206058293581,9.258206058293581)'> [PAD]</span><span style='background-color:rgb(233.0677953734994,21.932204626500607,21.932204626500607)'> [PAD]</span><span style='background-color:rgb(237.98215717077255,17.017842829227448,17.017842829227448)'> [PAD]</span><span style='background-color:rgb(242.4005895294249,12.599410470575094,12.599410470575094)'> [PAD]</span><span style='background-color:rgb(241.52684161439538,13.47315838560462,13.47315838560462)'> [PAD]</span><span style='background-color:rgb(239.86328138038516,15.13671861961484,15.13671861961484)'> [PAD]</span><span style='background-color:rgb(239.3868638947606,15.613136105239391,15.613136105239391)'> [PAD]</span><span style='background-color:rgb(235.82985766232014,19.170142337679863,19.170142337679863)'> [PAD]</span><span style='background-color:rgb(240.73570791631937,14.26429208368063,14.26429208368063)'> [PAD]</span><span style='background-color:rgb(240.6490279175341,14.350972082465887,14.350972082465887)'> [PAD]</span><span style='background-color:rgb(241.22840087860823,13.771599121391773,13.771599121391773)'> [PAD]</span><span style='background-color:rgb(239.31286096572876,15.68713903427124,15.68713903427124)'> [PAD]</span><span style='background-color:rgb(234.1368718072772,20.863128192722797,20.863128192722797)'> [PAD]</span><span style='background-color:rgb(238.6768700927496,16.323129907250404,16.323129907250404)'> [PAD]</span><span style='background-color:rgb(236.23568158596754,18.76431841403246,18.76431841403246)'> [PAD]</span><span style='background-color:rgb(240.40410635992885,14.595893640071154,14.595893640071154)'> [PAD]</span><span style='background-color:rgb(230.06247963756323,24.93752036243677,24.93752036243677)'> [PAD]</span><span style='background-color:rgb(238.03603447973728,16.963965520262718,16.963965520262718)'> [PAD]</span><span style='background-color:rgb(230.05985017865896,24.940149821341038,24.940149821341038)'> [PAD]</span><span style='background-color:rgb(233.55253726243973,21.447462737560272,21.447462737560272)'> [PAD]</span><span style='background-color:rgb(236.5467157959938,18.453284204006195,18.453284204006195)'> [PAD]</span><span style='background-color:rgb(235.49644835293293,19.50355164706707,19.50355164706707)'> [PAD]</span><span style='background-color:rgb(247.58820416405797,7.41179583594203,7.41179583594203)'> [PAD]</span><span style='background-color:rgb(226.3954997062683,28.60450029373169,28.60450029373169)'> [PAD]</span><span style='background-color:rgb(241.27350920811296,13.726490791887045,13.726490791887045)'> [PAD]</span><span style='background-color:rgb(238.34238544106483,16.657614558935165,16.657614558935165)'> [PAD]</span><span style='background-color:rgb(241.68053481727839,13.319465182721615,13.319465182721615)'> [PAD]</span><span style='background-color:rgb(241.27109253779054,13.728907462209463,13.728907462209463)'> [PAD]</span><span style='background-color:rgb(135.16026735305786,119.83973264694214,119.83973264694214)'> [PAD]</span>

## Archipelago

In [50]:
class TextXformer:
    def __init__(self, input_ids, baseline_ids):
        self.input = input_ids
        self.baseline = baseline_ids
        self.num_features = input_ids.shape[-1]
        
    def simple_xform(self, instance_repr):
        mask_indices = np.argwhere(instance_repr==True).flatten()
        id_list = list(self.baseline)
        for i in mask_indices:
            try:
                id_list[i] = self.input[i]
            except:
                import pdb; pdb.set_trace()
        return id_list

    def __call__(self, instance_repr):
        return self.simple_xform(instance_repr)

In [54]:
class BertWrapperTorch:
    def __init__(self, model, device, merge_logits=False):
        self.model = model.to(device)
        self.device = device
        self.merge_logits = merge_logits

    def get_predictions(self, batch_ids):
        batch_ids = torch.LongTensor(batch_ids).to(self.device)
        batch_conf = self.model(batch_ids)
        
        if isinstance(batch_conf, tuple):
            return batch_conf[0].data.cpu()
        else:
            if not isinstance(batch_conf, torch.Tensor):
                batch_conf = batch_conf.logits
            return batch_conf.data.cpu()
        return batch_conf

    def __call__(self, batch_ids):
        batch_predictions = self.get_predictions(batch_ids)
        if self.merge_logits:
            batch_predictions2 = (
                (batch_predictions[:, 1] - batch_predictions[:, 0]).unsqueeze(1).numpy()
            )
            return batch_predictions2
        else:
            return batch_predictions.numpy()

In [55]:
from exlib.explainers.libs.archipelago.explainer import Archipelago
# from exlib.explainers.libs.archipelago.application_utils.text_utils_torch import *
# from exlib.explainers.libs.archipelago.application_utils.utils_torch import ModelWrapperTorch

model_wrapper = BertWrapperTorch(backbone_model, device)
inputs_np = inputs.cpu().numpy()[0]
baseline = np.zeros_like(inputs_np)
xf = TextXformer(inputs_np, baseline, segs)

class_idx = labels
apgo = Archipelago(model_wrapper, data_xformer=xf, output_indices=class_idx)

In [ ]:
explanation = apgo.explain(top_k=3)


## Lime

In [13]:
from exlib.explainers.lime import LimeTextCls

eik = {
    "top_labels": 2, 
    "num_samples": 1000
}

def split_expression(x):
    # return processor.tokenize(x)
    # import pdb; pdb.set_trace()
    tokens = x.split()
    # tokens = [' ' + a for a in x.split()]
    # tokens[0] = tokens[0][1:]
    # tokens = processor.tokenize(x)
    # import pdb; pdb.set_trace()
    return tokens
    
ltek = {
    "mask_string": "[MASK]",
    "split_expression": split_expression
}

lime_explainer = LimeTextCls(original_model, processor,
                             LimeTextExplainerKwargs=ltek,
                             explain_instance_kwargs=eik)

lime_expln = lime_explainer(inputs, labels)

In [14]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

def show_attrs(tokens, masks):
    print_str = ""
    for i in range(len(tokens)):
        expln_val = masks[i].item()
        if expln_val == 0:
            print_str += f" {tokens[i]}"
        else:
            print_str += f"<span style='background-color:rgb({255 * (1-expln_val)},{255 * expln_val},{255 * expln_val})'> {tokens[i]}</span>"
    return print_str

In [15]:
tokens = processor.convert_ids_to_tokens(inputs[0])

In [16]:
printmd(show_attrs(tokens, lime_expln.attributions[0]))

 [CLS]<span style='background-color:rgb(286.6798348724842,-31.679834872484207,-31.679834872484207)'> allan</span> sat down at his desk and pulled<span style='background-color:rgb(175.02955198287964,79.97044801712036,79.97044801712036)'> the</span> chair in close<span style='background-color:rgb(288.3886259794235,-33.38862597942352,-33.38862597942352)'> .</span> opening a side drawer , he took out a piece of paper and his ink ##pot<span style='background-color:rgb(288.3886259794235,-33.38862597942352,-33.38862597942352)'> .</span> after filling his pen ,<span style='background-color:rgb(286.6798348724842,-31.679834872484207,-31.679834872484207)'> allan</span> looked at his paper in<span style='background-color:rgb(175.02955198287964,79.97044801712036,79.97044801712036)'> the</span> orange glow from<span style='background-color:rgb(175.02955198287964,79.97044801712036,79.97044801712036)'> the</span> lantern set back in<span style='background-color:rgb(175.02955198287964,79.97044801712036,79.97044801712036)'> the</span> desk ' s right - hand corner<span style='background-color:rgb(288.3886259794235,-33.38862597942352,-33.38862597942352)'> .</span> his pen cast a forbid ##ding line of shadow slant ##ing across<span style='background-color:rgb(175.02955198287964,79.97044801712036,79.97044801712036)'> the</span> page , echoing<span style='background-color:rgb(175.02955198287964,79.97044801712036,79.97044801712036)'> the</span> ink ##y darkness crouch ##ing in<span style='background-color:rgb(175.02955198287964,79.97044801712036,79.97044801712036)'> the</span> edges of<span style='background-color:rgb(175.02955198287964,79.97044801712036,79.97044801712036)'> the</span> lantern ' s struggling glow<span style='background-color:rgb(288.3886259794235,-33.38862597942352,-33.38862597942352)'> .</span><span style='background-color:rgb(175.02955198287964,79.97044801712036,79.97044801712036)'> the</span> only other illumination came from a lu ##rid moonlight filtered through thin branches and clouds , casting its bone - pale glow onto<span style='background-color:rgb(175.02955198287964,79.97044801712036,79.97044801712036)'> the</span> pine floor ##boards<span style='background-color:rgb(288.3886259794235,-33.38862597942352,-33.38862597942352)'> .</span><span style='background-color:rgb(286.6798348724842,-31.679834872484207,-31.679834872484207)'> allan</span> unfolded another page , this one crowded with ranks of letters in tight formation from left to right<span style='background-color:rgb(288.3886259794235,-33.38862597942352,-33.38862597942352)'> .</span><span style='background-color:rgb(175.02955198287964,79.97044801712036,79.97044801712036)'> the</span> lines of letters stepped into their divisions , in<span style='background-color:rgb(175.02955198287964,79.97044801712036,79.97044801712036)'> the</span> shape of a story ' s outline :<span style='background-color:rgb(175.02955198287964,79.97044801712036,79.97044801712036)'> the</span> loose , dry skeleton of a tale lay exposed beneath their feet , awaiting tend ##ons , muscle and blushing skin<span style='background-color:rgb(288.3886259794235,-33.38862597942352,-33.38862597942352)'> .</span><span style='background-color:rgb(286.6798348724842,-31.679834872484207,-31.679834872484207)'> allan</span> reviewed<span style='background-color:rgb(175.02955198287964,79.97044801712036,79.97044801712036)'> the</span> troops , all prepared to di ##se ##mba ##rk , their task to form<span style='background-color:rgb(175.02955198287964,79.97044801712036,79.97044801712036)'> the</span> tale of a young man returning home from life abroad to find his childhood friend a bride to - be , thus upset ##ting<span style='background-color:rgb(175.02955198287964,79.97044801712036,79.97044801712036)'> the</span> apple cart of his life ' s plan , clarified<span style='background-color:rgb(204.11696456372738,50.88303543627262,50.88303543627262)'> –</span> of course<span style='background-color:rgb(204.11696456372738,50.88303543627262,50.88303543627262)'> –</span> by his very time away from her he loved best<span style='background-color:rgb(288.3886259794235,-33.38862597942352,-33.38862597942352)'> .</span> although<span style='background-color:rgb(175.02955198287964,79.97044801712036,79.97044801712036)'> the</span> concept was a simple one ,<span style='background-color:rgb(286.6798348724842,-31.679834872484207,-31.679834872484207)'> allan</span> thought it had potential<span style='background-color:rgb(288.3886259794235,-33.38862597942352,-33.38862597942352)'> .</span> besides ,<span style='background-color:rgb(175.02955198287964,79.97044801712036,79.97044801712036)'> the</span> public liked a good , simple romance<span style='background-color:rgb(288.3886259794235,-33.38862597942352,-33.38862597942352)'> .</span> perhaps this will be more sale ##able , he thought and began to write<span style='background-color:rgb(288.3886259794235,-33.38862597942352,-33.38862597942352)'> .</span> they gazed at each other , lost in<span style='background-color:rgb(175.02955198287964,79.97044801712036,79.97044801712036)'> the</span> rap ##ture of love based<span style='background-color:rgb(274.06952753663063,-19.06952753663063,-19.06952753663063)'> so</span> deeply within their hearts that they had never seen it before<span style='background-color:rgb(288.3886259794235,-33.38862597942352,-33.38862597942352)'> .</span> " what about roger ? " she asked , knowing that<span style='background-color:rgb(175.02955198287964,79.97044801712036,79.97044801712036)'> the</span> answer no longer mattered<span style='background-color:rgb(288.3886259794235,-33.38862597942352,-33.38862597942352)'> .</span><span style='background-color:rgb(191.58834524452686,63.41165475547314,63.41165475547314)'> [SEP]</span> name few objects said to be in<span style='background-color:rgb(212.77649469673634,42.223505303263664,42.223505303263664)'> or</span> on<span style='background-color:rgb(286.6798348724842,-31.679834872484207,-31.679834872484207)'> allan</span> ' s desk<span style='background-color:rgb(292.9408995062113,-37.94089950621128,-37.94089950621128)'> |</span><span style='background-color:rgb(292.9408995062113,-37.94089950621128,-37.94089950621128)'> |</span><span style='background-color:rgb(287.4654997140169,-32.465499714016914,-32.465499714016914)'> erase</span> ##r<span style='background-color:rgb(191.58834524452686,63.41165475547314,63.41165475547314)'> [SEP]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span><span style='background-color:rgb(107.93662816286087,147.06337183713913,147.06337183713913)'> [PAD]</span>

In [17]:
tokens1 = processor.convert_ids_to_tokens(inputs[1])
printmd(show_attrs(tokens1, lime_expln.attributions[1]))

 [CLS] allan sat down at his desk and pulled<span style='background-color:rgb(169.60283517837524,85.39716482162476,85.39716482162476)'> the</span> chair in close . opening a side drawer , he took out a piece of paper and his ink<span style='background-color:rgb(216.44729152321815,38.552708476781845,38.552708476781845)'> ##pot</span> . after filling his pen , allan looked at his paper in<span style='background-color:rgb(169.60283517837524,85.39716482162476,85.39716482162476)'> the</span> orange glow from<span style='background-color:rgb(169.60283517837524,85.39716482162476,85.39716482162476)'> the</span> lantern set back in<span style='background-color:rgb(169.60283517837524,85.39716482162476,85.39716482162476)'> the</span> desk ' s right - hand corner . his pen cast a forbid ##ding line of shadow slant ##ing across<span style='background-color:rgb(169.60283517837524,85.39716482162476,85.39716482162476)'> the</span> page , echoing<span style='background-color:rgb(169.60283517837524,85.39716482162476,85.39716482162476)'> the</span> ink ##y darkness crouch ##ing in<span style='background-color:rgb(169.60283517837524,85.39716482162476,85.39716482162476)'> the</span> edges of<span style='background-color:rgb(169.60283517837524,85.39716482162476,85.39716482162476)'> the</span> lantern ' s struggling glow .<span style='background-color:rgb(169.60283517837524,85.39716482162476,85.39716482162476)'> the</span> only other illumination came from a lu ##rid moonlight filtered through thin branches and clouds , casting its bone - pale glow onto<span style='background-color:rgb(169.60283517837524,85.39716482162476,85.39716482162476)'> the</span> pine floor ##boards . allan unfolded another page , this one crowded with ranks of letters in tight formation from left to right .<span style='background-color:rgb(169.60283517837524,85.39716482162476,85.39716482162476)'> the</span> lines of letters stepped into their divisions , in<span style='background-color:rgb(169.60283517837524,85.39716482162476,85.39716482162476)'> the</span> shape of a<span style='background-color:rgb(260.22963515482843,-5.229635154828429,-5.229635154828429)'> story</span> ' s<span style='background-color:rgb(266.8004472926259,-11.800447292625904,-11.800447292625904)'> outline</span> :<span style='background-color:rgb(169.60283517837524,85.39716482162476,85.39716482162476)'> the</span> loose , dry skeleton of a tale lay exposed beneath their feet , awaiting tend ##ons , muscle and blushing skin . allan reviewed<span style='background-color:rgb(169.60283517837524,85.39716482162476,85.39716482162476)'> the</span> troops , all<span style='background-color:rgb(287.1209456026554,-32.12094560265541,-32.12094560265541)'> prepared</span> to di ##se ##mba ##rk , their task to form<span style='background-color:rgb(169.60283517837524,85.39716482162476,85.39716482162476)'> the</span> tale of a young man returning home from life abroad to find his childhood friend a bride to - be , thus upset ##ting<span style='background-color:rgb(169.60283517837524,85.39716482162476,85.39716482162476)'> the</span> apple cart of his life ' s plan , clarified – of course – by his very time away from her he loved best . although<span style='background-color:rgb(169.60283517837524,85.39716482162476,85.39716482162476)'> the</span> concept was a simple one , allan thought it had potential . besides ,<span style='background-color:rgb(169.60283517837524,85.39716482162476,85.39716482162476)'> the</span> public liked a good , simple romance . perhaps this will be more sale ##able , he thought and began to write . they gazed at each other , lost in<span style='background-color:rgb(169.60283517837524,85.39716482162476,85.39716482162476)'> the</span><span style='background-color:rgb(205.5294019728899,49.4705980271101,49.4705980271101)'> rap</span> ##ture of love based so deeply within their hearts that they had never seen it before .<span style='background-color:rgb(204.05126608908176,50.948733910918236,50.948733910918236)'> "</span> what about roger<span style='background-color:rgb(191.6387153416872,63.3612846583128,63.3612846583128)'> ?</span><span style='background-color:rgb(204.05126608908176,50.948733910918236,50.948733910918236)'> "</span> she asked , knowing that<span style='background-color:rgb(169.60283517837524,85.39716482162476,85.39716482162476)'> the</span> answer no longer mattered .<span style='background-color:rgb(153.63403245806694,101.36596754193306,101.36596754193306)'> [SEP]</span> name few objects said to be in or on allan ' s desk | | pencil<span style='background-color:rgb(153.63403245806694,101.36596754193306,101.36596754193306)'> [SEP]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span><span style='background-color:rgb(180.530676394701,74.469323605299,74.469323605299)'> [PAD]</span>

In [20]:
lime_expln.explainer_output[0].as_list()

[('[PAD]', 0.5767190980600844),
 ('the', 0.31360959520663556),
 ('[SEP]', 0.24867316109376783),
 ('–', 0.19954130976062953),
 ('or', 0.16558237185584104),
 ('|', -0.1487878343240845),
 ('.', -0.1309357914639342),
 ('erase', -0.12731568318035072),
 ('allan', -0.12423464436928029),
 ('so', -0.07478245937713171)]

In [21]:
lime_expln.explainer_output[0].local_exp

{0: [(208, -0.7893760245796537),
  (198, -0.30193107376986145),
  (9, -0.30047980664344687),
  (141, -0.2145088359383795),
  (203, -0.21344021701543137),
  (96, -0.21216588209013354),
  (1, 0.13338063608778614),
  (207, 0.12451006101765646),
  (205, 0.11947097009523455),
  (178, 0.08199956342861209)],
 1: [(208, 0.5767190980600844),
  (9, 0.31360959520663556),
  (198, 0.24867316109376783),
  (141, 0.19954130976062953),
  (203, 0.16558237185584104),
  (205, -0.1487878343240845),
  (13, -0.1309357914639342),
  (206, -0.12731568318035072),
  (1, -0.12423464436928029),
  (178, -0.07478245937713171)]}

In [22]:
j = 0
grouped_attrs = []
for i in tqdm(range(2)):
    grouped_attrs.append([v for k, v in lime_expln.explainer_output[j].local_exp[i]])
grouped_attrs_aggr = torch.tensor([sum(ga) for ga in grouped_attrs]).to(device)
aggr_pred = torch.argmax(grouped_attrs_aggr)

  0%|          | 0/2 [00:00<?, ?it/s]

In [23]:
aggr_pred

tensor(1, device='cuda:0')

## SHAP

In [18]:
from exlib.explainers.shap import ShapTextCls

shap_explainer = ShapTextCls(original_model, processor).to(device)



In [19]:
inputs_dict = {
    'input_ids': inputs,
    'token_type_ids': token_type_ids,
    'attention_mask': attention_mask
}

inputs_raw = [processor.decode(input_ids_i).replace('[CLS]', '').replace('[PAD]', '').strip() 
              for input_ids_i in inputs]

shap_expln = shap_explainer(inputs_raw, labels)

PartitionExplainer explainer: 3it [00:11, 11.69s/it]               


In [20]:
shap_expln.explainer_output.values[0].shape

(356, 2)

In [21]:
# bsz = len(shap_expln.explainer_output.values)
j = 0
grouped_attrs_aggr = torch.tensor(shap_expln.explainer_output.values[j]).mean(0)
aggr_pred = torch.argmax(grouped_attrs_aggr)

In [22]:
aggr_pred

tensor(0)

In [23]:
labels

tensor([0, 0], device='cuda:0')

In [24]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

def show_attrs(tokens, masks):
    print_str = ""
    for i in range(len(tokens)):
        expln_val = masks[i].item()
        if expln_val == 0:
            print_str += f" {tokens[i]}"
        else:
            print_str += f"<span style='background-color:rgb({255 * (1-expln_val)},{255 * expln_val},{255 * expln_val})'> {tokens[i]}</span>"
    return print_str

# tokens = processor.convert_ids_to_tokens(inputs[0])
tokens = shap_expln.explainer_output.data[0]
printmd(show_attrs(tokens, shap_expln.attributions[0]))

<span style='background-color:rgb(256.24154931286944,-1.2415493128694244,-1.2415493128694244)'> </span><span style='background-color:rgb(256.24154931286944,-1.2415493128694244,-1.2415493128694244)'> allan </span><span style='background-color:rgb(256.24154931286944,-1.2415493128694244,-1.2415493128694244)'> sat </span><span style='background-color:rgb(256.24154931286944,-1.2415493128694244,-1.2415493128694244)'> down </span><span style='background-color:rgb(256.24154931286944,-1.2415493128694242,-1.2415493128694242)'> at </span><span style='background-color:rgb(256.24154931286944,-1.2415493128694242,-1.2415493128694242)'> his </span><span style='background-color:rgb(256.24154931286944,-1.2415493128694242,-1.2415493128694242)'> desk </span><span style='background-color:rgb(256.2640343349034,-1.264034334903388,-1.264034334903388)'> and </span><span style='background-color:rgb(256.2640343349034,-1.264034334903388,-1.264034334903388)'> pulled </span><span style='background-color:rgb(256.2640343349034,-1.264034334903388,-1.264034334903388)'> the </span><span style='background-color:rgb(256.2640343349034,-1.264034334903388,-1.264034334903388)'> chair </span><span style='background-color:rgb(256.2640343349034,-1.264034334903388,-1.264034334903388)'> in </span><span style='background-color:rgb(256.2640343349034,-1.264034334903388,-1.264034334903388)'> close</span><span style='background-color:rgb(256.2640343349034,-1.264034334903388,-1.264034334903388)'> . </span><span style='background-color:rgb(256.51691728605675,-1.5169172860567472,-1.5169172860567472)'> opening </span><span style='background-color:rgb(256.51691728605675,-1.5169172860567472,-1.5169172860567472)'> a </span><span style='background-color:rgb(256.51691728605675,-1.5169172860567472,-1.5169172860567472)'> side </span><span style='background-color:rgb(256.51691728605675,-1.5169172860567472,-1.5169172860567472)'> drawer</span><span style='background-color:rgb(256.51691728605675,-1.5169172860567472,-1.5169172860567472)'> , </span><span style='background-color:rgb(256.45892905684906,-1.4589290568490527,-1.4589290568490527)'> he </span><span style='background-color:rgb(256.45892905684906,-1.4589290568490527,-1.4589290568490527)'> took </span><span style='background-color:rgb(256.45892905684906,-1.4589290568490527,-1.4589290568490527)'> out </span><span style='background-color:rgb(256.45892905684906,-1.4589290568490527,-1.4589290568490527)'> a </span><span style='background-color:rgb(256.45892905684906,-1.4589290568490527,-1.4589290568490527)'> piece </span><span style='background-color:rgb(256.45892905684906,-1.4589290568490527,-1.4589290568490527)'> of </span><span style='background-color:rgb(256.45892905684906,-1.4589290568490527,-1.4589290568490527)'> paper </span><span style='background-color:rgb(256.45892905684906,-1.4589290568490527,-1.4589290568490527)'> and </span><span style='background-color:rgb(256.45892905684906,-1.4589290568490527,-1.4589290568490527)'> his </span><span style='background-color:rgb(256.45892905684906,-1.4589290568490527,-1.4589290568490527)'> ink</span><span style='background-color:rgb(256.45892905684906,-1.4589290568490527,-1.4589290568490527)'> pot</span><span style='background-color:rgb(256.45892905684906,-1.4589290568490527,-1.4589290568490527)'> . </span><span style='background-color:rgb(255.85419593894218,-0.8541959389421684,-0.8541959389421684)'> after </span><span style='background-color:rgb(255.85419593894218,-0.8541959389421684,-0.8541959389421684)'> filling </span><span style='background-color:rgb(255.85419593894218,-0.8541959389421684,-0.8541959389421684)'> his </span><span style='background-color:rgb(255.85419593894218,-0.8541959389421684,-0.8541959389421684)'> pen</span><span style='background-color:rgb(255.85419593894218,-0.8541959389421684,-0.8541959389421684)'> , </span><span style='background-color:rgb(256.0840176767978,-1.0840176767978413,-1.0840176767978413)'> allan </span><span style='background-color:rgb(256.0840176767978,-1.0840176767978413,-1.0840176767978413)'> looked </span><span style='background-color:rgb(256.0840176767978,-1.0840176767978413,-1.0840176767978413)'> at </span><span style='background-color:rgb(256.0840176767978,-1.0840176767978413,-1.0840176767978413)'> his </span><span style='background-color:rgb(256.0840176767978,-1.0840176767978413,-1.0840176767978413)'> paper </span><span style='background-color:rgb(256.0840176767978,-1.0840176767978413,-1.0840176767978413)'> in </span><span style='background-color:rgb(256.0840176767978,-1.0840176767978413,-1.0840176767978413)'> the </span><span style='background-color:rgb(256.0840176767978,-1.0840176767978413,-1.0840176767978413)'> orange </span><span style='background-color:rgb(255.96144209802554,-0.9614420980255216,-0.9614420980255216)'> glow </span><span style='background-color:rgb(255.96144209802554,-0.9614420980255216,-0.9614420980255216)'> from </span><span style='background-color:rgb(255.96144209802554,-0.9614420980255216,-0.9614420980255216)'> the </span><span style='background-color:rgb(255.96144209802554,-0.9614420980255216,-0.9614420980255216)'> lantern </span><span style='background-color:rgb(255.96144209802554,-0.9614420980255215,-0.9614420980255215)'> set </span><span style='background-color:rgb(255.96144209802554,-0.9614420980255215,-0.9614420980255215)'> back </span><span style='background-color:rgb(255.96144209802554,-0.9614420980255215,-0.9614420980255215)'> in </span><span style='background-color:rgb(255.96144209802554,-0.9614420980255215,-0.9614420980255215)'> the </span><span style='background-color:rgb(255.96144209802554,-0.9614420980255215,-0.9614420980255215)'> desk</span><span style='background-color:rgb(255.96144209802554,-0.9614420980255215,-0.9614420980255215)'> '</span><span style='background-color:rgb(255.96144209802554,-0.9614420980255215,-0.9614420980255215)'> s </span><span style='background-color:rgb(255.96144209802554,-0.9614420980255217,-0.9614420980255217)'> right </span><span style='background-color:rgb(255.96144209802554,-0.9614420980255217,-0.9614420980255217)'> - </span><span style='background-color:rgb(255.96144209802554,-0.9614420980255217,-0.9614420980255217)'> hand </span><span style='background-color:rgb(255.96144209802554,-0.9614420980255217,-0.9614420980255217)'> corner</span><span style='background-color:rgb(255.96144209802554,-0.9614420980255217,-0.9614420980255217)'> . </span><span style='background-color:rgb(255.12493128484732,-0.12493128484735884,-0.12493128484735884)'> his </span><span style='background-color:rgb(255.12493128484732,-0.12493128484735884,-0.12493128484735884)'> pen </span><span style='background-color:rgb(255.12493128484732,-0.12493128484735884,-0.12493128484735884)'> cast </span><span style='background-color:rgb(255.12493128484732,-0.12493128484735884,-0.12493128484735884)'> a </span><span style='background-color:rgb(255.12493128484732,-0.12493128484735884,-0.12493128484735884)'> forbid</span><span style='background-color:rgb(255.12493128484732,-0.12493128484735884,-0.12493128484735884)'> ding </span><span style='background-color:rgb(255.12493128484732,-0.12493128484735884,-0.12493128484735884)'> line </span><span style='background-color:rgb(255.12493128484732,-0.12493128484735884,-0.12493128484735884)'> of </span><span style='background-color:rgb(255.12493128484732,-0.12493128484735884,-0.12493128484735884)'> shadow </span><span style='background-color:rgb(254.9495958155021,0.050404184497892864,0.050404184497892864)'> slant</span><span style='background-color:rgb(254.9495958155021,0.050404184497892864,0.050404184497892864)'> ing </span><span style='background-color:rgb(254.9495958155021,0.05040418449789286,0.05040418449789286)'> across </span><span style='background-color:rgb(254.9495958155021,0.05040418449789286,0.05040418449789286)'> the </span><span style='background-color:rgb(254.9495958155021,0.05040418449789286,0.05040418449789286)'> page</span><span style='background-color:rgb(254.9495958155021,0.050404184497892864,0.050404184497892864)'> , </span><span style='background-color:rgb(255.4279830811235,-0.42798308112348127,-0.42798308112348127)'> echoing </span><span style='background-color:rgb(255.4279830811235,-0.42798308112348127,-0.42798308112348127)'> the </span><span style='background-color:rgb(255.4279830811235,-0.4279830811234812,-0.4279830811234812)'> ink</span><span style='background-color:rgb(255.4279830811235,-0.4279830811234812,-0.4279830811234812)'> y </span><span style='background-color:rgb(255.4279830811235,-0.4279830811234812,-0.4279830811234812)'> darkness </span><span style='background-color:rgb(255.4279830811235,-0.4279830811234812,-0.4279830811234812)'> crouch</span><span style='background-color:rgb(255.4279830811235,-0.4279830811234812,-0.4279830811234812)'> ing </span><span style='background-color:rgb(255.4279830811235,-0.4279830811234812,-0.4279830811234812)'> in </span><span style='background-color:rgb(255.4279830811235,-0.4279830811234812,-0.4279830811234812)'> the </span><span style='background-color:rgb(255.69554772926497,-0.6955477292649448,-0.6955477292649448)'> edges </span><span style='background-color:rgb(255.69554772926497,-0.6955477292649448,-0.6955477292649448)'> of </span><span style='background-color:rgb(255.69554772926497,-0.6955477292649448,-0.6955477292649448)'> the </span><span style='background-color:rgb(255.69554772926497,-0.6955477292649448,-0.6955477292649448)'> lantern</span><span style='background-color:rgb(255.69554772926497,-0.6955477292649448,-0.6955477292649448)'> '</span><span style='background-color:rgb(255.69554772926497,-0.6955477292649448,-0.6955477292649448)'> s </span><span style='background-color:rgb(255.69554772926497,-0.6955477292649448,-0.6955477292649448)'> struggling </span><span style='background-color:rgb(255.69554772926497,-0.6955477292649448,-0.6955477292649448)'> glow</span><span style='background-color:rgb(255.69554772926497,-0.6955477292649448,-0.6955477292649448)'> . </span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> the </span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> only </span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> other </span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> illumination </span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> came </span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> from </span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> a </span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> lu</span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> rid </span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> moonlight </span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> filtered </span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> through </span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> thin </span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> branches </span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> and </span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> clouds</span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> , </span><span style='background-color:rgb(255.93035074847145,-0.9303507484714544,-0.9303507484714544)'> casting </span><span style='background-color:rgb(255.93035074847145,-0.9303507484714544,-0.9303507484714544)'> its </span><span style='background-color:rgb(255.93035074847145,-0.9303507484714544,-0.9303507484714544)'> bone </span><span style='background-color:rgb(255.93035074847145,-0.9303507484714544,-0.9303507484714544)'> - </span><span style='background-color:rgb(255.93035074847145,-0.9303507484714544,-0.9303507484714544)'> pale </span><span style='background-color:rgb(255.93035074847145,-0.9303507484714544,-0.9303507484714544)'> glow </span><span style='background-color:rgb(255.93035074847145,-0.9303507484714544,-0.9303507484714544)'> onto </span><span style='background-color:rgb(255.93035074847145,-0.9303507484714544,-0.9303507484714544)'> the </span><span style='background-color:rgb(255.93035074847145,-0.9303507484714544,-0.9303507484714544)'> pine </span><span style='background-color:rgb(255.93035074847145,-0.9303507484714544,-0.9303507484714544)'> floor</span><span style='background-color:rgb(255.93035074847145,-0.9303507484714544,-0.9303507484714544)'> boards</span><span style='background-color:rgb(255.93035074847145,-0.9303507484714544,-0.9303507484714544)'> . </span><span style='background-color:rgb(254.7436338079124,0.25636619208760714,0.25636619208760714)'> allan </span><span style='background-color:rgb(254.7436338079124,0.25636619208760714,0.25636619208760714)'> unfolded </span><span style='background-color:rgb(254.7436338079124,0.25636619208760714,0.25636619208760714)'> another </span><span style='background-color:rgb(254.7436338079124,0.25636619208760714,0.25636619208760714)'> page</span><span style='background-color:rgb(254.7436338079124,0.25636619208760714,0.25636619208760714)'> , </span><span style='background-color:rgb(254.98819142421326,0.011808575786726196,0.011808575786726196)'> this </span><span style='background-color:rgb(254.98819142421326,0.011808575786726196,0.011808575786726196)'> one </span><span style='background-color:rgb(254.98819142421326,0.011808575786726196,0.011808575786726196)'> crowded </span><span style='background-color:rgb(254.98819142421326,0.011808575786726196,0.011808575786726196)'> with </span><span style='background-color:rgb(254.98819142421326,0.011808575786726196,0.011808575786726196)'> ranks </span><span style='background-color:rgb(254.98819142421326,0.011808575786726196,0.011808575786726196)'> of </span><span style='background-color:rgb(254.98819142421326,0.011808575786726196,0.011808575786726196)'> letters </span><span style='background-color:rgb(254.98819142421326,0.011808575786726196,0.011808575786726196)'> in </span><span style='background-color:rgb(254.98819142421326,0.011808575786726196,0.011808575786726196)'> tight </span><span style='background-color:rgb(254.98819142421326,0.011808575786726196,0.011808575786726196)'> formation </span><span style='background-color:rgb(254.98819142421326,0.011808575786726196,0.011808575786726196)'> from </span><span style='background-color:rgb(254.98819142421326,0.011808575786726196,0.011808575786726196)'> left </span><span style='background-color:rgb(254.98819142421326,0.011808575786726198,0.011808575786726198)'> to </span><span style='background-color:rgb(254.98819142421326,0.011808575786726198,0.011808575786726198)'> right</span><span style='background-color:rgb(254.98819142421326,0.011808575786726198,0.011808575786726198)'> . </span><span style='background-color:rgb(254.22774283754643,0.7722571624535663,0.7722571624535663)'> the </span><span style='background-color:rgb(254.22774283754643,0.7722571624535663,0.7722571624535663)'> lines </span><span style='background-color:rgb(254.22774283754643,0.7722571624535663,0.7722571624535663)'> of </span><span style='background-color:rgb(254.22774283754643,0.7722571624535663,0.7722571624535663)'> letters </span><span style='background-color:rgb(254.22774283754643,0.7722571624535663,0.7722571624535663)'> stepped </span><span style='background-color:rgb(254.22774283754643,0.7722571624535663,0.7722571624535663)'> into </span><span style='background-color:rgb(254.22774283754643,0.7722571624535663,0.7722571624535663)'> their </span><span style='background-color:rgb(254.22774283754643,0.7722571624535663,0.7722571624535663)'> divisions</span><span style='background-color:rgb(254.22774283754643,0.7722571624535663,0.7722571624535663)'> , </span><span style='background-color:rgb(254.31400109144099,0.6859989085590111,0.6859989085590111)'> in </span><span style='background-color:rgb(254.31400109144099,0.6859989085590111,0.6859989085590111)'> the </span><span style='background-color:rgb(254.31400109144099,0.6859989085590111,0.6859989085590111)'> shape </span><span style='background-color:rgb(254.31400109144099,0.6859989085590111,0.6859989085590111)'> of </span><span style='background-color:rgb(254.31400109144099,0.6859989085590111,0.6859989085590111)'> a </span><span style='background-color:rgb(254.31400109144099,0.6859989085590111,0.6859989085590111)'> story</span><span style='background-color:rgb(254.31400109144099,0.6859989085590111,0.6859989085590111)'> '</span><span style='background-color:rgb(254.31400109144099,0.6859989085590111,0.6859989085590111)'> s </span><span style='background-color:rgb(254.31400109144099,0.6859989085590112,0.6859989085590112)'> outline </span><span style='background-color:rgb(254.31400109144099,0.6859989085590112,0.6859989085590112)'> : </span><span style='background-color:rgb(254.31400109144099,0.6859989085590112,0.6859989085590112)'> the </span><span style='background-color:rgb(254.31400109144099,0.6859989085590112,0.6859989085590112)'> loose</span><span style='background-color:rgb(254.31400109144099,0.6859989085590112,0.6859989085590112)'> , </span><span style='background-color:rgb(253.83068933855296,1.169310661447053,1.169310661447053)'> dry </span><span style='background-color:rgb(253.83068933855296,1.169310661447053,1.169310661447053)'> skeleton </span><span style='background-color:rgb(253.83068933855296,1.169310661447053,1.169310661447053)'> of </span><span style='background-color:rgb(253.83068933855296,1.169310661447053,1.169310661447053)'> a </span><span style='background-color:rgb(253.83068933855296,1.1693106614470532,1.1693106614470532)'> tale </span><span style='background-color:rgb(253.83068933855296,1.1693106614470532,1.1693106614470532)'> lay </span><span style='background-color:rgb(253.83068933855296,1.1693106614470532,1.1693106614470532)'> exposed </span><span style='background-color:rgb(253.83068933855296,1.1693106614470532,1.1693106614470532)'> beneath </span><span style='background-color:rgb(253.83068933855296,1.1693106614470532,1.1693106614470532)'> their </span><span style='background-color:rgb(253.83068933855296,1.1693106614470532,1.1693106614470532)'> feet</span><span style='background-color:rgb(253.83068933855296,1.1693106614470532,1.1693106614470532)'> , </span><span style='background-color:rgb(253.6649409946653,1.3350590053347071,1.3350590053347071)'> awaiting </span><span style='background-color:rgb(253.6649409946653,1.3350590053347071,1.3350590053347071)'> tend</span><span style='background-color:rgb(253.6649409946653,1.3350590053347071,1.3350590053347071)'> ons</span><span style='background-color:rgb(253.6649409946653,1.3350590053347071,1.3350590053347071)'> , </span><span style='background-color:rgb(253.6649409946653,1.3350590053347071,1.3350590053347071)'> muscle </span><span style='background-color:rgb(253.6649409946653,1.3350590053347071,1.3350590053347071)'> and </span><span style='background-color:rgb(253.6649409946653,1.3350590053347071,1.3350590053347071)'> blushing </span><span style='background-color:rgb(253.6649409946653,1.3350590053347071,1.3350590053347071)'> skin</span><span style='background-color:rgb(253.6649409946653,1.3350590053347071,1.3350590053347071)'> . </span><span style='background-color:rgb(255.10901550547433,-0.10901550547434735,-0.10901550547434735)'> allan </span><span style='background-color:rgb(255.10901550547433,-0.10901550547434735,-0.10901550547434735)'> reviewed </span><span style='background-color:rgb(255.10901550547433,-0.10901550547434735,-0.10901550547434735)'> the </span><span style='background-color:rgb(255.10901550547433,-0.10901550547434735,-0.10901550547434735)'> troops</span><span style='background-color:rgb(255.10901550547433,-0.10901550547434735,-0.10901550547434735)'> , </span><span style='background-color:rgb(255.10901550547433,-0.10901550547434735,-0.10901550547434735)'> all </span><span style='background-color:rgb(255.10901550547433,-0.10901550547434735,-0.10901550547434735)'> prepared </span><span style='background-color:rgb(255.10901550547433,-0.10901550547434735,-0.10901550547434735)'> to </span><span style='background-color:rgb(255.10901550547433,-0.10901550547434735,-0.10901550547434735)'> di</span><span style='background-color:rgb(255.10901550547433,-0.10901550547434735,-0.10901550547434735)'> se</span><span style='background-color:rgb(255.10901550547433,-0.10901550547434735,-0.10901550547434735)'> mba</span><span style='background-color:rgb(255.10901550547433,-0.10901550547434735,-0.10901550547434735)'> rk</span><span style='background-color:rgb(255.10901550547433,-0.10901550547434735,-0.10901550547434735)'> , </span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> their </span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> task </span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> to </span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> form </span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> the </span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> tale </span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> of </span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> a </span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> young </span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> man </span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> returning </span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> home </span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> from </span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> life </span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> abroad </span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> to </span><span style='background-color:rgb(255.55388782553086,-0.5538878255308821,-0.5538878255308821)'> find </span><span style='background-color:rgb(255.55388782553086,-0.5538878255308821,-0.5538878255308821)'> his </span><span style='background-color:rgb(255.55388782553086,-0.5538878255308821,-0.5538878255308821)'> childhood </span><span style='background-color:rgb(255.55388782553086,-0.5538878255308821,-0.5538878255308821)'> friend </span><span style='background-color:rgb(255.55388782553086,-0.5538878255308822,-0.5538878255308822)'> a </span><span style='background-color:rgb(255.55388782553086,-0.5538878255308822,-0.5538878255308822)'> bride </span><span style='background-color:rgb(255.55388782553086,-0.5538878255308821,-0.5538878255308821)'> to </span><span style='background-color:rgb(255.55388782553086,-0.5538878255308821,-0.5538878255308821)'> - </span><span style='background-color:rgb(255.55388782553086,-0.5538878255308821,-0.5538878255308821)'> be</span><span style='background-color:rgb(255.55388782553086,-0.5538878255308821,-0.5538878255308821)'> , </span><span style='background-color:rgb(255.1190420788582,-0.11904207885818709,-0.11904207885818709)'> thus </span><span style='background-color:rgb(255.1190420788582,-0.11904207885818709,-0.11904207885818709)'> upset</span><span style='background-color:rgb(255.1190420788582,-0.11904207885818709,-0.11904207885818709)'> ting </span><span style='background-color:rgb(255.1190420788582,-0.11904207885818707,-0.11904207885818707)'> the </span><span style='background-color:rgb(255.1190420788582,-0.11904207885818707,-0.11904207885818707)'> apple </span><span style='background-color:rgb(255.1190420788582,-0.11904207885818709,-0.11904207885818709)'> cart </span><span style='background-color:rgb(255.1190420788582,-0.11904207885818709,-0.11904207885818709)'> of </span><span style='background-color:rgb(255.1190420788582,-0.11904207885818709,-0.11904207885818709)'> his </span><span style='background-color:rgb(255.1190420788582,-0.1190420788581871,-0.1190420788581871)'> life</span><span style='background-color:rgb(255.1190420788582,-0.1190420788581871,-0.1190420788581871)'> '</span><span style='background-color:rgb(255.1190420788582,-0.1190420788581871,-0.1190420788581871)'> s </span><span style='background-color:rgb(255.1190420788582,-0.1190420788581871,-0.1190420788581871)'> plan</span><span style='background-color:rgb(255.1190420788582,-0.1190420788581871,-0.1190420788581871)'> , </span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> clarified </span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> – </span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> of </span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> course </span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> – </span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> by </span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> his </span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> very </span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> time </span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> away </span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> from </span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> her </span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> he </span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> loved </span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> best</span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> . </span><span style='background-color:rgb(254.9246505937672,0.07534940623279128,0.07534940623279128)'> although </span><span style='background-color:rgb(254.9246505937672,0.07534940623279128,0.07534940623279128)'> the </span><span style='background-color:rgb(254.9246505937672,0.07534940623279128,0.07534940623279128)'> concept </span><span style='background-color:rgb(254.9246505937672,0.07534940623279128,0.07534940623279128)'> was </span><span style='background-color:rgb(254.9246505937672,0.07534940623279128,0.07534940623279128)'> a </span><span style='background-color:rgb(254.9246505937672,0.07534940623279128,0.07534940623279128)'> simple </span><span style='background-color:rgb(254.9246505937672,0.07534940623279128,0.07534940623279128)'> one</span><span style='background-color:rgb(254.9246505937672,0.07534940623279128,0.07534940623279128)'> , </span><span style='background-color:rgb(254.9246505937672,0.07534940623279127,0.07534940623279127)'> allan </span><span style='background-color:rgb(254.9246505937672,0.07534940623279127,0.07534940623279127)'> thought </span><span style='background-color:rgb(254.9246505937672,0.07534940623279127,0.07534940623279127)'> it </span><span style='background-color:rgb(254.9246505937672,0.07534940623279127,0.07534940623279127)'> had </span><span style='background-color:rgb(254.9246505937672,0.07534940623279127,0.07534940623279127)'> potential</span><span style='background-color:rgb(254.9246505937672,0.07534940623279127,0.07534940623279127)'> . </span><span style='background-color:rgb(255.37261852712808,-0.3726185271280588,-0.3726185271280588)'> besides</span><span style='background-color:rgb(255.37261852712808,-0.3726185271280588,-0.3726185271280588)'> , </span><span style='background-color:rgb(255.37261852712808,-0.3726185271280588,-0.3726185271280588)'> the </span><span style='background-color:rgb(255.37261852712808,-0.3726185271280588,-0.3726185271280588)'> public </span><span style='background-color:rgb(255.37261852712808,-0.37261852712805876,-0.37261852712805876)'> liked </span><span style='background-color:rgb(255.37261852712808,-0.37261852712805876,-0.37261852712805876)'> a </span><span style='background-color:rgb(255.37261852712808,-0.37261852712805876,-0.37261852712805876)'> good</span><span style='background-color:rgb(255.37261852712808,-0.37261852712805876,-0.37261852712805876)'> , </span><span style='background-color:rgb(255.1774729725203,-0.17747297252032468,-0.17747297252032468)'> simple </span><span style='background-color:rgb(255.1774729725203,-0.17747297252032468,-0.17747297252032468)'> romance</span><span style='background-color:rgb(255.1774729725203,-0.17747297252032468,-0.17747297252032468)'> . </span><span style='background-color:rgb(256.12629468123697,-1.126294681236946,-1.126294681236946)'> perhaps </span><span style='background-color:rgb(256.12629468123697,-1.126294681236946,-1.126294681236946)'> this </span><span style='background-color:rgb(256.12629468123697,-1.126294681236946,-1.126294681236946)'> will </span><span style='background-color:rgb(256.12629468123697,-1.126294681236946,-1.126294681236946)'> be </span><span style='background-color:rgb(256.12629468123697,-1.126294681236946,-1.126294681236946)'> more </span><span style='background-color:rgb(255.67398949842237,-0.6739894984223479,-0.6739894984223479)'> sale</span><span style='background-color:rgb(255.67398949842237,-0.6739894984223479,-0.6739894984223479)'> able</span><span style='background-color:rgb(255.67398949842237,-0.6739894984223479,-0.6739894984223479)'> , </span><span style='background-color:rgb(255.17045418831654,-0.17045418831652828,-0.17045418831652828)'> he </span><span style='background-color:rgb(255.17045418831654,-0.17045418831652828,-0.17045418831652828)'> thought </span><span style='background-color:rgb(255.17045418831654,-0.17045418831652828,-0.17045418831652828)'> and </span><span style='background-color:rgb(255.17045418831654,-0.17045418831652828,-0.17045418831652828)'> began </span><span style='background-color:rgb(255.17045418831654,-0.17045418831652828,-0.17045418831652828)'> to </span><span style='background-color:rgb(255.17045418831654,-0.17045418831652828,-0.17045418831652828)'> write</span><span style='background-color:rgb(255.17045418831654,-0.17045418831652828,-0.17045418831652828)'> . </span><span style='background-color:rgb(253.2792412531055,1.7207587468944756,1.7207587468944756)'> they </span><span style='background-color:rgb(253.2792412531055,1.7207587468944756,1.7207587468944756)'> gazed </span><span style='background-color:rgb(253.2792412531055,1.7207587468944754,1.7207587468944754)'> at </span><span style='background-color:rgb(253.2792412531055,1.7207587468944754,1.7207587468944754)'> each </span><span style='background-color:rgb(253.2792412531055,1.7207587468944754,1.7207587468944754)'> other</span><span style='background-color:rgb(253.2792412531055,1.7207587468944754,1.7207587468944754)'> , </span><span style='background-color:rgb(253.52325625965358,1.4767437403464156,1.4767437403464156)'> lost </span><span style='background-color:rgb(253.52325625965358,1.4767437403464156,1.4767437403464156)'> in </span><span style='background-color:rgb(253.52325625965358,1.4767437403464156,1.4767437403464156)'> the </span><span style='background-color:rgb(253.52325625965358,1.4767437403464154,1.4767437403464154)'> rap</span><span style='background-color:rgb(253.52325625965358,1.4767437403464154,1.4767437403464154)'> ture </span><span style='background-color:rgb(253.52325625965358,1.4767437403464154,1.4767437403464154)'> of </span><span style='background-color:rgb(253.52325625965358,1.4767437403464154,1.4767437403464154)'> love </span><span style='background-color:rgb(253.86358484507656,1.1364151549234185,1.1364151549234185)'> based </span><span style='background-color:rgb(253.86358484507656,1.1364151549234185,1.1364151549234185)'> so </span><span style='background-color:rgb(253.86358484507656,1.1364151549234185,1.1364151549234185)'> deeply </span><span style='background-color:rgb(253.86358484507656,1.1364151549234185,1.1364151549234185)'> within </span><span style='background-color:rgb(253.86358484507656,1.1364151549234185,1.1364151549234185)'> their </span><span style='background-color:rgb(253.86358484507656,1.1364151549234185,1.1364151549234185)'> hearts </span><span style='background-color:rgb(253.86358484507656,1.1364151549234185,1.1364151549234185)'> that </span><span style='background-color:rgb(253.86358484507656,1.1364151549234185,1.1364151549234185)'> they </span><span style='background-color:rgb(253.86358484507656,1.1364151549234183,1.1364151549234183)'> had </span><span style='background-color:rgb(253.86358484507656,1.1364151549234183,1.1364151549234183)'> never </span><span style='background-color:rgb(253.86358484507656,1.1364151549234185,1.1364151549234185)'> seen </span><span style='background-color:rgb(253.86358484507656,1.1364151549234185,1.1364151549234185)'> it </span><span style='background-color:rgb(253.86358484507656,1.1364151549234185,1.1364151549234185)'> before</span><span style='background-color:rgb(253.86358484507656,1.1364151549234185,1.1364151549234185)'> . </span><span style='background-color:rgb(250.22749400586997,4.772505994130028,4.772505994130028)'> " </span><span style='background-color:rgb(250.22749400586997,4.772505994130028,4.772505994130028)'> what </span><span style='background-color:rgb(250.22749400586997,4.772505994130028,4.772505994130028)'> about </span><span style='background-color:rgb(250.22749400586997,4.772505994130028,4.772505994130028)'> roger</span><span style='background-color:rgb(250.22749400586997,4.772505994130028,4.772505994130028)'> ? </span><span style='background-color:rgb(247.2031556009855,7.796844399014503,7.796844399014503)'> " </span><span style='background-color:rgb(247.2031556009855,7.796844399014503,7.796844399014503)'> she </span><span style='background-color:rgb(247.2031556009855,7.796844399014503,7.796844399014503)'> asked</span><span style='background-color:rgb(247.2031556009855,7.796844399014502,7.796844399014502)'> , </span><span style='background-color:rgb(250.14096433744578,4.859035662554219,4.859035662554219)'> knowing </span><span style='background-color:rgb(250.14096433744578,4.859035662554219,4.859035662554219)'> that </span><span style='background-color:rgb(250.14096433744578,4.859035662554219,4.859035662554219)'> the </span><span style='background-color:rgb(250.14096433744578,4.859035662554219,4.859035662554219)'> answer </span><span style='background-color:rgb(250.14096433744578,4.859035662554219,4.859035662554219)'> no </span><span style='background-color:rgb(250.14096433744578,4.859035662554219,4.859035662554219)'> longer </span><span style='background-color:rgb(250.14096433744578,4.859035662554219,4.859035662554219)'> mattered</span><span style='background-color:rgb(250.14096433744578,4.859035662554219,4.859035662554219)'> . </span><span style='background-color:rgb(180.79991474747658,74.20008525252342,74.20008525252342)'> [SEP] </span><span style='background-color:rgb(252.53304625558357,2.466953744416449,2.466953744416449)'> name </span><span style='background-color:rgb(252.53304625558357,2.466953744416449,2.466953744416449)'> few </span><span style='background-color:rgb(257.6414919008935,-2.6414919008935356,-2.6414919008935356)'> objects </span><span style='background-color:rgb(257.6414919008935,-2.6414919008935356,-2.6414919008935356)'> said </span><span style='background-color:rgb(255.21320026141308,-0.21320026141308648,-0.21320026141308648)'> to </span><span style='background-color:rgb(255.21320026141308,-0.21320026141308648,-0.21320026141308648)'> be </span><span style='background-color:rgb(255.21320026141308,-0.21320026141308648,-0.21320026141308648)'> in </span><span style='background-color:rgb(261.44247199830716,-6.4424719983071554,-6.4424719983071554)'> or </span><span style='background-color:rgb(259.5920876642049,-4.5920876642048825,-4.5920876642048825)'> on </span><span style='background-color:rgb(255.37087327291374,-0.37087327291374095,-0.37087327291374095)'> allan</span><span style='background-color:rgb(255.37087327291374,-0.37087327291374095,-0.37087327291374095)'> '</span><span style='background-color:rgb(255.37087327291374,-0.37087327291374095,-0.37087327291374095)'> s </span><span style='background-color:rgb(250.95823206673958,4.041767933260417,4.041767933260417)'> desk </span><span style='background-color:rgb(250.95823206673958,4.041767933260417,4.041767933260417)'> | </span><span style='background-color:rgb(253.25270082769566,1.7472991723043376,1.7472991723043376)'> | </span><span style='background-color:rgb(225.7652162591403,29.23478374085971,29.23478374085971)'> erase</span><span style='background-color:rgb(225.7652162591403,29.23478374085971,29.23478374085971)'> r </span><span style='background-color:rgb(255.000018524006,-1.8524006009101868e-05,-1.8524006009101868e-05)'> [SEP]</span><span style='background-color:rgb(255.000018524006,-1.8524006009101868e-05,-1.8524006009101868e-05)'> </span>

In [25]:
shap_expln.explainer_output.data[0]

array(['', 'allan ', 'sat ', 'down ', 'at ', 'his ', 'desk ', 'and ',
       'pulled ', 'the ', 'chair ', 'in ', 'close', '. ', 'opening ',
       'a ', 'side ', 'drawer', ', ', 'he ', 'took ', 'out ', 'a ',
       'piece ', 'of ', 'paper ', 'and ', 'his ', 'ink', 'pot', '. ',
       'after ', 'filling ', 'his ', 'pen', ', ', 'allan ', 'looked ',
       'at ', 'his ', 'paper ', 'in ', 'the ', 'orange ', 'glow ',
       'from ', 'the ', 'lantern ', 'set ', 'back ', 'in ', 'the ',
       'desk', "'", 's ', 'right ', '- ', 'hand ', 'corner', '. ', 'his ',
       'pen ', 'cast ', 'a ', 'forbid', 'ding ', 'line ', 'of ',
       'shadow ', 'slant', 'ing ', 'across ', 'the ', 'page', ', ',
       'echoing ', 'the ', 'ink', 'y ', 'darkness ', 'crouch', 'ing ',
       'in ', 'the ', 'edges ', 'of ', 'the ', 'lantern', "'", 's ',
       'struggling ', 'glow', '. ', 'the ', 'only ', 'other ',
       'illumination ', 'came ', 'from ', 'a ', 'lu', 'rid ',
       'moonlight ', 'filtered ', 'through 

In [128]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

def show_attrs(tokens, masks):
    print_str = ""
    for i in range(len(tokens)):
        expln_val = masks[i].item()
        if expln_val == 0:
            print_str += f" {tokens[i]}"
        else:
            print_str += f"<span style='background-color:rgb({255 * (1-expln_val)},{255 * expln_val},{255 * expln_val})'> {tokens[i]}</span>"
    return print_str

tokens = processor.convert_ids_to_tokens(inputs[0])
# tokens = shap_expln.explainer_output.data[0]
printmd(show_attrs(tokens, shap_expln.attributions[0]))

<span style='background-color:rgb(256.24154931286944,-1.2415493128694244,-1.2415493128694244)'> [CLS]</span><span style='background-color:rgb(256.24154931286944,-1.2415493128694244,-1.2415493128694244)'> allan</span><span style='background-color:rgb(256.24154931286944,-1.2415493128694244,-1.2415493128694244)'> sat</span><span style='background-color:rgb(256.24154931286944,-1.2415493128694244,-1.2415493128694244)'> down</span><span style='background-color:rgb(256.24154931286944,-1.2415493128694242,-1.2415493128694242)'> at</span><span style='background-color:rgb(256.24154931286944,-1.2415493128694242,-1.2415493128694242)'> his</span><span style='background-color:rgb(256.24154931286944,-1.2415493128694242,-1.2415493128694242)'> desk</span><span style='background-color:rgb(256.2640343349034,-1.264034334903388,-1.264034334903388)'> and</span><span style='background-color:rgb(256.2640343349034,-1.264034334903388,-1.264034334903388)'> pulled</span><span style='background-color:rgb(256.2640343349034,-1.264034334903388,-1.264034334903388)'> the</span><span style='background-color:rgb(256.2640343349034,-1.264034334903388,-1.264034334903388)'> chair</span><span style='background-color:rgb(256.2640343349034,-1.264034334903388,-1.264034334903388)'> in</span><span style='background-color:rgb(256.2640343349034,-1.264034334903388,-1.264034334903388)'> close</span><span style='background-color:rgb(256.2640343349034,-1.264034334903388,-1.264034334903388)'> .</span><span style='background-color:rgb(256.51691728605675,-1.5169172860567472,-1.5169172860567472)'> opening</span><span style='background-color:rgb(256.51691728605675,-1.5169172860567472,-1.5169172860567472)'> a</span><span style='background-color:rgb(256.51691728605675,-1.5169172860567472,-1.5169172860567472)'> side</span><span style='background-color:rgb(256.51691728605675,-1.5169172860567472,-1.5169172860567472)'> drawer</span><span style='background-color:rgb(256.51691728605675,-1.5169172860567472,-1.5169172860567472)'> ,</span><span style='background-color:rgb(256.45892905684906,-1.4589290568490527,-1.4589290568490527)'> he</span><span style='background-color:rgb(256.45892905684906,-1.4589290568490527,-1.4589290568490527)'> took</span><span style='background-color:rgb(256.45892905684906,-1.4589290568490527,-1.4589290568490527)'> out</span><span style='background-color:rgb(256.45892905684906,-1.4589290568490527,-1.4589290568490527)'> a</span><span style='background-color:rgb(256.45892905684906,-1.4589290568490527,-1.4589290568490527)'> piece</span><span style='background-color:rgb(256.45892905684906,-1.4589290568490527,-1.4589290568490527)'> of</span><span style='background-color:rgb(256.45892905684906,-1.4589290568490527,-1.4589290568490527)'> paper</span><span style='background-color:rgb(256.45892905684906,-1.4589290568490527,-1.4589290568490527)'> and</span><span style='background-color:rgb(256.45892905684906,-1.4589290568490527,-1.4589290568490527)'> his</span><span style='background-color:rgb(256.45892905684906,-1.4589290568490527,-1.4589290568490527)'> ink</span><span style='background-color:rgb(256.45892905684906,-1.4589290568490527,-1.4589290568490527)'> ##pot</span><span style='background-color:rgb(256.45892905684906,-1.4589290568490527,-1.4589290568490527)'> .</span><span style='background-color:rgb(255.85419593894218,-0.8541959389421684,-0.8541959389421684)'> after</span><span style='background-color:rgb(255.85419593894218,-0.8541959389421684,-0.8541959389421684)'> filling</span><span style='background-color:rgb(255.85419593894218,-0.8541959389421684,-0.8541959389421684)'> his</span><span style='background-color:rgb(255.85419593894218,-0.8541959389421684,-0.8541959389421684)'> pen</span><span style='background-color:rgb(255.85419593894218,-0.8541959389421684,-0.8541959389421684)'> ,</span><span style='background-color:rgb(256.0840176767978,-1.0840176767978413,-1.0840176767978413)'> allan</span><span style='background-color:rgb(256.0840176767978,-1.0840176767978413,-1.0840176767978413)'> looked</span><span style='background-color:rgb(256.0840176767978,-1.0840176767978413,-1.0840176767978413)'> at</span><span style='background-color:rgb(256.0840176767978,-1.0840176767978413,-1.0840176767978413)'> his</span><span style='background-color:rgb(256.0840176767978,-1.0840176767978413,-1.0840176767978413)'> paper</span><span style='background-color:rgb(256.0840176767978,-1.0840176767978413,-1.0840176767978413)'> in</span><span style='background-color:rgb(256.0840176767978,-1.0840176767978413,-1.0840176767978413)'> the</span><span style='background-color:rgb(256.0840176767978,-1.0840176767978413,-1.0840176767978413)'> orange</span><span style='background-color:rgb(255.96144209802554,-0.9614420980255216,-0.9614420980255216)'> glow</span><span style='background-color:rgb(255.96144209802554,-0.9614420980255216,-0.9614420980255216)'> from</span><span style='background-color:rgb(255.96144209802554,-0.9614420980255216,-0.9614420980255216)'> the</span><span style='background-color:rgb(255.96144209802554,-0.9614420980255216,-0.9614420980255216)'> lantern</span><span style='background-color:rgb(255.96144209802554,-0.9614420980255215,-0.9614420980255215)'> set</span><span style='background-color:rgb(255.96144209802554,-0.9614420980255215,-0.9614420980255215)'> back</span><span style='background-color:rgb(255.96144209802554,-0.9614420980255215,-0.9614420980255215)'> in</span><span style='background-color:rgb(255.96144209802554,-0.9614420980255215,-0.9614420980255215)'> the</span><span style='background-color:rgb(255.96144209802554,-0.9614420980255215,-0.9614420980255215)'> desk</span><span style='background-color:rgb(255.96144209802554,-0.9614420980255215,-0.9614420980255215)'> '</span><span style='background-color:rgb(255.96144209802554,-0.9614420980255215,-0.9614420980255215)'> s</span><span style='background-color:rgb(255.96144209802554,-0.9614420980255217,-0.9614420980255217)'> right</span><span style='background-color:rgb(255.96144209802554,-0.9614420980255217,-0.9614420980255217)'> -</span><span style='background-color:rgb(255.96144209802554,-0.9614420980255217,-0.9614420980255217)'> hand</span><span style='background-color:rgb(255.96144209802554,-0.9614420980255217,-0.9614420980255217)'> corner</span><span style='background-color:rgb(255.96144209802554,-0.9614420980255217,-0.9614420980255217)'> .</span><span style='background-color:rgb(255.12493128484732,-0.12493128484735884,-0.12493128484735884)'> his</span><span style='background-color:rgb(255.12493128484732,-0.12493128484735884,-0.12493128484735884)'> pen</span><span style='background-color:rgb(255.12493128484732,-0.12493128484735884,-0.12493128484735884)'> cast</span><span style='background-color:rgb(255.12493128484732,-0.12493128484735884,-0.12493128484735884)'> a</span><span style='background-color:rgb(255.12493128484732,-0.12493128484735884,-0.12493128484735884)'> forbid</span><span style='background-color:rgb(255.12493128484732,-0.12493128484735884,-0.12493128484735884)'> ##ding</span><span style='background-color:rgb(255.12493128484732,-0.12493128484735884,-0.12493128484735884)'> line</span><span style='background-color:rgb(255.12493128484732,-0.12493128484735884,-0.12493128484735884)'> of</span><span style='background-color:rgb(255.12493128484732,-0.12493128484735884,-0.12493128484735884)'> shadow</span><span style='background-color:rgb(254.9495958155021,0.050404184497892864,0.050404184497892864)'> slant</span><span style='background-color:rgb(254.9495958155021,0.050404184497892864,0.050404184497892864)'> ##ing</span><span style='background-color:rgb(254.9495958155021,0.05040418449789286,0.05040418449789286)'> across</span><span style='background-color:rgb(254.9495958155021,0.05040418449789286,0.05040418449789286)'> the</span><span style='background-color:rgb(254.9495958155021,0.05040418449789286,0.05040418449789286)'> page</span><span style='background-color:rgb(254.9495958155021,0.050404184497892864,0.050404184497892864)'> ,</span><span style='background-color:rgb(255.4279830811235,-0.42798308112348127,-0.42798308112348127)'> echoing</span><span style='background-color:rgb(255.4279830811235,-0.42798308112348127,-0.42798308112348127)'> the</span><span style='background-color:rgb(255.4279830811235,-0.4279830811234812,-0.4279830811234812)'> ink</span><span style='background-color:rgb(255.4279830811235,-0.4279830811234812,-0.4279830811234812)'> ##y</span><span style='background-color:rgb(255.4279830811235,-0.4279830811234812,-0.4279830811234812)'> darkness</span><span style='background-color:rgb(255.4279830811235,-0.4279830811234812,-0.4279830811234812)'> crouch</span><span style='background-color:rgb(255.4279830811235,-0.4279830811234812,-0.4279830811234812)'> ##ing</span><span style='background-color:rgb(255.4279830811235,-0.4279830811234812,-0.4279830811234812)'> in</span><span style='background-color:rgb(255.4279830811235,-0.4279830811234812,-0.4279830811234812)'> the</span><span style='background-color:rgb(255.69554772926497,-0.6955477292649448,-0.6955477292649448)'> edges</span><span style='background-color:rgb(255.69554772926497,-0.6955477292649448,-0.6955477292649448)'> of</span><span style='background-color:rgb(255.69554772926497,-0.6955477292649448,-0.6955477292649448)'> the</span><span style='background-color:rgb(255.69554772926497,-0.6955477292649448,-0.6955477292649448)'> lantern</span><span style='background-color:rgb(255.69554772926497,-0.6955477292649448,-0.6955477292649448)'> '</span><span style='background-color:rgb(255.69554772926497,-0.6955477292649448,-0.6955477292649448)'> s</span><span style='background-color:rgb(255.69554772926497,-0.6955477292649448,-0.6955477292649448)'> struggling</span><span style='background-color:rgb(255.69554772926497,-0.6955477292649448,-0.6955477292649448)'> glow</span><span style='background-color:rgb(255.69554772926497,-0.6955477292649448,-0.6955477292649448)'> .</span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> the</span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> only</span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> other</span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> illumination</span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> came</span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> from</span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> a</span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> lu</span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> ##rid</span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> moonlight</span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> filtered</span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> through</span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> thin</span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> branches</span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> and</span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> clouds</span><span style='background-color:rgb(255.6920718305032,-0.6920718305031811,-0.6920718305031811)'> ,</span><span style='background-color:rgb(255.93035074847145,-0.9303507484714544,-0.9303507484714544)'> casting</span><span style='background-color:rgb(255.93035074847145,-0.9303507484714544,-0.9303507484714544)'> its</span><span style='background-color:rgb(255.93035074847145,-0.9303507484714544,-0.9303507484714544)'> bone</span><span style='background-color:rgb(255.93035074847145,-0.9303507484714544,-0.9303507484714544)'> -</span><span style='background-color:rgb(255.93035074847145,-0.9303507484714544,-0.9303507484714544)'> pale</span><span style='background-color:rgb(255.93035074847145,-0.9303507484714544,-0.9303507484714544)'> glow</span><span style='background-color:rgb(255.93035074847145,-0.9303507484714544,-0.9303507484714544)'> onto</span><span style='background-color:rgb(255.93035074847145,-0.9303507484714544,-0.9303507484714544)'> the</span><span style='background-color:rgb(255.93035074847145,-0.9303507484714544,-0.9303507484714544)'> pine</span><span style='background-color:rgb(255.93035074847145,-0.9303507484714544,-0.9303507484714544)'> floor</span><span style='background-color:rgb(255.93035074847145,-0.9303507484714544,-0.9303507484714544)'> ##boards</span><span style='background-color:rgb(255.93035074847145,-0.9303507484714544,-0.9303507484714544)'> .</span><span style='background-color:rgb(254.7436338079124,0.25636619208760714,0.25636619208760714)'> allan</span><span style='background-color:rgb(254.7436338079124,0.25636619208760714,0.25636619208760714)'> unfolded</span><span style='background-color:rgb(254.7436338079124,0.25636619208760714,0.25636619208760714)'> another</span><span style='background-color:rgb(254.7436338079124,0.25636619208760714,0.25636619208760714)'> page</span><span style='background-color:rgb(254.7436338079124,0.25636619208760714,0.25636619208760714)'> ,</span><span style='background-color:rgb(254.98819142421326,0.011808575786726196,0.011808575786726196)'> this</span><span style='background-color:rgb(254.98819142421326,0.011808575786726196,0.011808575786726196)'> one</span><span style='background-color:rgb(254.98819142421326,0.011808575786726196,0.011808575786726196)'> crowded</span><span style='background-color:rgb(254.98819142421326,0.011808575786726196,0.011808575786726196)'> with</span><span style='background-color:rgb(254.98819142421326,0.011808575786726196,0.011808575786726196)'> ranks</span><span style='background-color:rgb(254.98819142421326,0.011808575786726196,0.011808575786726196)'> of</span><span style='background-color:rgb(254.98819142421326,0.011808575786726196,0.011808575786726196)'> letters</span><span style='background-color:rgb(254.98819142421326,0.011808575786726196,0.011808575786726196)'> in</span><span style='background-color:rgb(254.98819142421326,0.011808575786726196,0.011808575786726196)'> tight</span><span style='background-color:rgb(254.98819142421326,0.011808575786726196,0.011808575786726196)'> formation</span><span style='background-color:rgb(254.98819142421326,0.011808575786726196,0.011808575786726196)'> from</span><span style='background-color:rgb(254.98819142421326,0.011808575786726196,0.011808575786726196)'> left</span><span style='background-color:rgb(254.98819142421326,0.011808575786726198,0.011808575786726198)'> to</span><span style='background-color:rgb(254.98819142421326,0.011808575786726198,0.011808575786726198)'> right</span><span style='background-color:rgb(254.98819142421326,0.011808575786726198,0.011808575786726198)'> .</span><span style='background-color:rgb(254.22774283754643,0.7722571624535663,0.7722571624535663)'> the</span><span style='background-color:rgb(254.22774283754643,0.7722571624535663,0.7722571624535663)'> lines</span><span style='background-color:rgb(254.22774283754643,0.7722571624535663,0.7722571624535663)'> of</span><span style='background-color:rgb(254.22774283754643,0.7722571624535663,0.7722571624535663)'> letters</span><span style='background-color:rgb(254.22774283754643,0.7722571624535663,0.7722571624535663)'> stepped</span><span style='background-color:rgb(254.22774283754643,0.7722571624535663,0.7722571624535663)'> into</span><span style='background-color:rgb(254.22774283754643,0.7722571624535663,0.7722571624535663)'> their</span><span style='background-color:rgb(254.22774283754643,0.7722571624535663,0.7722571624535663)'> divisions</span><span style='background-color:rgb(254.22774283754643,0.7722571624535663,0.7722571624535663)'> ,</span><span style='background-color:rgb(254.31400109144099,0.6859989085590111,0.6859989085590111)'> in</span><span style='background-color:rgb(254.31400109144099,0.6859989085590111,0.6859989085590111)'> the</span><span style='background-color:rgb(254.31400109144099,0.6859989085590111,0.6859989085590111)'> shape</span><span style='background-color:rgb(254.31400109144099,0.6859989085590111,0.6859989085590111)'> of</span><span style='background-color:rgb(254.31400109144099,0.6859989085590111,0.6859989085590111)'> a</span><span style='background-color:rgb(254.31400109144099,0.6859989085590111,0.6859989085590111)'> story</span><span style='background-color:rgb(254.31400109144099,0.6859989085590111,0.6859989085590111)'> '</span><span style='background-color:rgb(254.31400109144099,0.6859989085590111,0.6859989085590111)'> s</span><span style='background-color:rgb(254.31400109144099,0.6859989085590112,0.6859989085590112)'> outline</span><span style='background-color:rgb(254.31400109144099,0.6859989085590112,0.6859989085590112)'> :</span><span style='background-color:rgb(254.31400109144099,0.6859989085590112,0.6859989085590112)'> the</span><span style='background-color:rgb(254.31400109144099,0.6859989085590112,0.6859989085590112)'> loose</span><span style='background-color:rgb(254.31400109144099,0.6859989085590112,0.6859989085590112)'> ,</span><span style='background-color:rgb(253.83068933855296,1.169310661447053,1.169310661447053)'> dry</span><span style='background-color:rgb(253.83068933855296,1.169310661447053,1.169310661447053)'> skeleton</span><span style='background-color:rgb(253.83068933855296,1.169310661447053,1.169310661447053)'> of</span><span style='background-color:rgb(253.83068933855296,1.169310661447053,1.169310661447053)'> a</span><span style='background-color:rgb(253.83068933855296,1.1693106614470532,1.1693106614470532)'> tale</span><span style='background-color:rgb(253.83068933855296,1.1693106614470532,1.1693106614470532)'> lay</span><span style='background-color:rgb(253.83068933855296,1.1693106614470532,1.1693106614470532)'> exposed</span><span style='background-color:rgb(253.83068933855296,1.1693106614470532,1.1693106614470532)'> beneath</span><span style='background-color:rgb(253.83068933855296,1.1693106614470532,1.1693106614470532)'> their</span><span style='background-color:rgb(253.83068933855296,1.1693106614470532,1.1693106614470532)'> feet</span><span style='background-color:rgb(253.83068933855296,1.1693106614470532,1.1693106614470532)'> ,</span><span style='background-color:rgb(253.6649409946653,1.3350590053347071,1.3350590053347071)'> awaiting</span><span style='background-color:rgb(253.6649409946653,1.3350590053347071,1.3350590053347071)'> tend</span><span style='background-color:rgb(253.6649409946653,1.3350590053347071,1.3350590053347071)'> ##ons</span><span style='background-color:rgb(253.6649409946653,1.3350590053347071,1.3350590053347071)'> ,</span><span style='background-color:rgb(253.6649409946653,1.3350590053347071,1.3350590053347071)'> muscle</span><span style='background-color:rgb(253.6649409946653,1.3350590053347071,1.3350590053347071)'> and</span><span style='background-color:rgb(253.6649409946653,1.3350590053347071,1.3350590053347071)'> blushing</span><span style='background-color:rgb(253.6649409946653,1.3350590053347071,1.3350590053347071)'> skin</span><span style='background-color:rgb(253.6649409946653,1.3350590053347071,1.3350590053347071)'> .</span><span style='background-color:rgb(255.10901550547433,-0.10901550547434735,-0.10901550547434735)'> allan</span><span style='background-color:rgb(255.10901550547433,-0.10901550547434735,-0.10901550547434735)'> reviewed</span><span style='background-color:rgb(255.10901550547433,-0.10901550547434735,-0.10901550547434735)'> the</span><span style='background-color:rgb(255.10901550547433,-0.10901550547434735,-0.10901550547434735)'> troops</span><span style='background-color:rgb(255.10901550547433,-0.10901550547434735,-0.10901550547434735)'> ,</span><span style='background-color:rgb(255.10901550547433,-0.10901550547434735,-0.10901550547434735)'> all</span><span style='background-color:rgb(255.10901550547433,-0.10901550547434735,-0.10901550547434735)'> prepared</span><span style='background-color:rgb(255.10901550547433,-0.10901550547434735,-0.10901550547434735)'> to</span><span style='background-color:rgb(255.10901550547433,-0.10901550547434735,-0.10901550547434735)'> di</span><span style='background-color:rgb(255.10901550547433,-0.10901550547434735,-0.10901550547434735)'> ##se</span><span style='background-color:rgb(255.10901550547433,-0.10901550547434735,-0.10901550547434735)'> ##mba</span><span style='background-color:rgb(255.10901550547433,-0.10901550547434735,-0.10901550547434735)'> ##rk</span><span style='background-color:rgb(255.10901550547433,-0.10901550547434735,-0.10901550547434735)'> ,</span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> their</span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> task</span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> to</span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> form</span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> the</span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> tale</span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> of</span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> a</span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> young</span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> man</span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> returning</span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> home</span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> from</span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> life</span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> abroad</span><span style='background-color:rgb(255.50408818463512,-0.5040881846350833,-0.5040881846350833)'> to</span><span style='background-color:rgb(255.55388782553086,-0.5538878255308821,-0.5538878255308821)'> find</span><span style='background-color:rgb(255.55388782553086,-0.5538878255308821,-0.5538878255308821)'> his</span><span style='background-color:rgb(255.55388782553086,-0.5538878255308821,-0.5538878255308821)'> childhood</span><span style='background-color:rgb(255.55388782553086,-0.5538878255308821,-0.5538878255308821)'> friend</span><span style='background-color:rgb(255.55388782553086,-0.5538878255308822,-0.5538878255308822)'> a</span><span style='background-color:rgb(255.55388782553086,-0.5538878255308822,-0.5538878255308822)'> bride</span><span style='background-color:rgb(255.55388782553086,-0.5538878255308821,-0.5538878255308821)'> to</span><span style='background-color:rgb(255.55388782553086,-0.5538878255308821,-0.5538878255308821)'> -</span><span style='background-color:rgb(255.55388782553086,-0.5538878255308821,-0.5538878255308821)'> be</span><span style='background-color:rgb(255.55388782553086,-0.5538878255308821,-0.5538878255308821)'> ,</span><span style='background-color:rgb(255.1190420788582,-0.11904207885818709,-0.11904207885818709)'> thus</span><span style='background-color:rgb(255.1190420788582,-0.11904207885818709,-0.11904207885818709)'> upset</span><span style='background-color:rgb(255.1190420788582,-0.11904207885818709,-0.11904207885818709)'> ##ting</span><span style='background-color:rgb(255.1190420788582,-0.11904207885818707,-0.11904207885818707)'> the</span><span style='background-color:rgb(255.1190420788582,-0.11904207885818707,-0.11904207885818707)'> apple</span><span style='background-color:rgb(255.1190420788582,-0.11904207885818709,-0.11904207885818709)'> cart</span><span style='background-color:rgb(255.1190420788582,-0.11904207885818709,-0.11904207885818709)'> of</span><span style='background-color:rgb(255.1190420788582,-0.11904207885818709,-0.11904207885818709)'> his</span><span style='background-color:rgb(255.1190420788582,-0.1190420788581871,-0.1190420788581871)'> life</span><span style='background-color:rgb(255.1190420788582,-0.1190420788581871,-0.1190420788581871)'> '</span><span style='background-color:rgb(255.1190420788582,-0.1190420788581871,-0.1190420788581871)'> s</span><span style='background-color:rgb(255.1190420788582,-0.1190420788581871,-0.1190420788581871)'> plan</span><span style='background-color:rgb(255.1190420788582,-0.1190420788581871,-0.1190420788581871)'> ,</span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> clarified</span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> –</span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> of</span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> course</span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> –</span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> by</span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> his</span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> very</span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> time</span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> away</span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> from</span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> her</span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> he</span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> loved</span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> best</span><span style='background-color:rgb(255.24599156775992,-0.2459915677599352,-0.2459915677599352)'> .</span><span style='background-color:rgb(254.9246505937672,0.07534940623279128,0.07534940623279128)'> although</span><span style='background-color:rgb(254.9246505937672,0.07534940623279128,0.07534940623279128)'> the</span><span style='background-color:rgb(254.9246505937672,0.07534940623279128,0.07534940623279128)'> concept</span><span style='background-color:rgb(254.9246505937672,0.07534940623279128,0.07534940623279128)'> was</span><span style='background-color:rgb(254.9246505937672,0.07534940623279128,0.07534940623279128)'> a</span><span style='background-color:rgb(254.9246505937672,0.07534940623279128,0.07534940623279128)'> simple</span><span style='background-color:rgb(254.9246505937672,0.07534940623279128,0.07534940623279128)'> one</span><span style='background-color:rgb(254.9246505937672,0.07534940623279128,0.07534940623279128)'> ,</span><span style='background-color:rgb(254.9246505937672,0.07534940623279127,0.07534940623279127)'> allan</span><span style='background-color:rgb(254.9246505937672,0.07534940623279127,0.07534940623279127)'> thought</span><span style='background-color:rgb(254.9246505937672,0.07534940623279127,0.07534940623279127)'> it</span><span style='background-color:rgb(254.9246505937672,0.07534940623279127,0.07534940623279127)'> had</span><span style='background-color:rgb(254.9246505937672,0.07534940623279127,0.07534940623279127)'> potential</span><span style='background-color:rgb(254.9246505937672,0.07534940623279127,0.07534940623279127)'> .</span><span style='background-color:rgb(255.37261852712808,-0.3726185271280588,-0.3726185271280588)'> besides</span><span style='background-color:rgb(255.37261852712808,-0.3726185271280588,-0.3726185271280588)'> ,</span><span style='background-color:rgb(255.37261852712808,-0.3726185271280588,-0.3726185271280588)'> the</span><span style='background-color:rgb(255.37261852712808,-0.3726185271280588,-0.3726185271280588)'> public</span><span style='background-color:rgb(255.37261852712808,-0.37261852712805876,-0.37261852712805876)'> liked</span><span style='background-color:rgb(255.37261852712808,-0.37261852712805876,-0.37261852712805876)'> a</span><span style='background-color:rgb(255.37261852712808,-0.37261852712805876,-0.37261852712805876)'> good</span><span style='background-color:rgb(255.37261852712808,-0.37261852712805876,-0.37261852712805876)'> ,</span><span style='background-color:rgb(255.1774729725203,-0.17747297252032468,-0.17747297252032468)'> simple</span><span style='background-color:rgb(255.1774729725203,-0.17747297252032468,-0.17747297252032468)'> romance</span><span style='background-color:rgb(255.1774729725203,-0.17747297252032468,-0.17747297252032468)'> .</span><span style='background-color:rgb(256.12629468123697,-1.126294681236946,-1.126294681236946)'> perhaps</span><span style='background-color:rgb(256.12629468123697,-1.126294681236946,-1.126294681236946)'> this</span><span style='background-color:rgb(256.12629468123697,-1.126294681236946,-1.126294681236946)'> will</span><span style='background-color:rgb(256.12629468123697,-1.126294681236946,-1.126294681236946)'> be</span><span style='background-color:rgb(256.12629468123697,-1.126294681236946,-1.126294681236946)'> more</span><span style='background-color:rgb(255.67398949842237,-0.6739894984223479,-0.6739894984223479)'> sale</span><span style='background-color:rgb(255.67398949842237,-0.6739894984223479,-0.6739894984223479)'> ##able</span><span style='background-color:rgb(255.67398949842237,-0.6739894984223479,-0.6739894984223479)'> ,</span><span style='background-color:rgb(255.17045418831654,-0.17045418831652828,-0.17045418831652828)'> he</span><span style='background-color:rgb(255.17045418831654,-0.17045418831652828,-0.17045418831652828)'> thought</span><span style='background-color:rgb(255.17045418831654,-0.17045418831652828,-0.17045418831652828)'> and</span><span style='background-color:rgb(255.17045418831654,-0.17045418831652828,-0.17045418831652828)'> began</span><span style='background-color:rgb(255.17045418831654,-0.17045418831652828,-0.17045418831652828)'> to</span><span style='background-color:rgb(255.17045418831654,-0.17045418831652828,-0.17045418831652828)'> write</span><span style='background-color:rgb(255.17045418831654,-0.17045418831652828,-0.17045418831652828)'> .</span><span style='background-color:rgb(253.2792412531055,1.7207587468944756,1.7207587468944756)'> they</span><span style='background-color:rgb(253.2792412531055,1.7207587468944756,1.7207587468944756)'> gazed</span><span style='background-color:rgb(253.2792412531055,1.7207587468944754,1.7207587468944754)'> at</span><span style='background-color:rgb(253.2792412531055,1.7207587468944754,1.7207587468944754)'> each</span><span style='background-color:rgb(253.2792412531055,1.7207587468944754,1.7207587468944754)'> other</span><span style='background-color:rgb(253.2792412531055,1.7207587468944754,1.7207587468944754)'> ,</span><span style='background-color:rgb(253.52325625965358,1.4767437403464156,1.4767437403464156)'> lost</span><span style='background-color:rgb(253.52325625965358,1.4767437403464156,1.4767437403464156)'> in</span><span style='background-color:rgb(253.52325625965358,1.4767437403464156,1.4767437403464156)'> the</span><span style='background-color:rgb(253.52325625965358,1.4767437403464154,1.4767437403464154)'> rap</span><span style='background-color:rgb(253.52325625965358,1.4767437403464154,1.4767437403464154)'> ##ture</span><span style='background-color:rgb(253.52325625965358,1.4767437403464154,1.4767437403464154)'> of</span><span style='background-color:rgb(253.52325625965358,1.4767437403464154,1.4767437403464154)'> love</span><span style='background-color:rgb(253.86358484507656,1.1364151549234185,1.1364151549234185)'> based</span><span style='background-color:rgb(253.86358484507656,1.1364151549234185,1.1364151549234185)'> so</span><span style='background-color:rgb(253.86358484507656,1.1364151549234185,1.1364151549234185)'> deeply</span><span style='background-color:rgb(253.86358484507656,1.1364151549234185,1.1364151549234185)'> within</span><span style='background-color:rgb(253.86358484507656,1.1364151549234185,1.1364151549234185)'> their</span><span style='background-color:rgb(253.86358484507656,1.1364151549234185,1.1364151549234185)'> hearts</span><span style='background-color:rgb(253.86358484507656,1.1364151549234185,1.1364151549234185)'> that</span><span style='background-color:rgb(253.86358484507656,1.1364151549234185,1.1364151549234185)'> they</span><span style='background-color:rgb(253.86358484507656,1.1364151549234183,1.1364151549234183)'> had</span><span style='background-color:rgb(253.86358484507656,1.1364151549234183,1.1364151549234183)'> never</span><span style='background-color:rgb(253.86358484507656,1.1364151549234185,1.1364151549234185)'> seen</span><span style='background-color:rgb(253.86358484507656,1.1364151549234185,1.1364151549234185)'> it</span><span style='background-color:rgb(253.86358484507656,1.1364151549234185,1.1364151549234185)'> before</span><span style='background-color:rgb(253.86358484507656,1.1364151549234185,1.1364151549234185)'> .</span><span style='background-color:rgb(250.22749400586997,4.772505994130028,4.772505994130028)'> "</span><span style='background-color:rgb(250.22749400586997,4.772505994130028,4.772505994130028)'> what</span><span style='background-color:rgb(250.22749400586997,4.772505994130028,4.772505994130028)'> about</span><span style='background-color:rgb(250.22749400586997,4.772505994130028,4.772505994130028)'> roger</span><span style='background-color:rgb(250.22749400586997,4.772505994130028,4.772505994130028)'> ?</span><span style='background-color:rgb(247.2031556009855,7.796844399014503,7.796844399014503)'> "</span><span style='background-color:rgb(247.2031556009855,7.796844399014503,7.796844399014503)'> she</span><span style='background-color:rgb(247.2031556009855,7.796844399014503,7.796844399014503)'> asked</span><span style='background-color:rgb(247.2031556009855,7.796844399014502,7.796844399014502)'> ,</span><span style='background-color:rgb(250.14096433744578,4.859035662554219,4.859035662554219)'> knowing</span><span style='background-color:rgb(250.14096433744578,4.859035662554219,4.859035662554219)'> that</span><span style='background-color:rgb(250.14096433744578,4.859035662554219,4.859035662554219)'> the</span><span style='background-color:rgb(250.14096433744578,4.859035662554219,4.859035662554219)'> answer</span><span style='background-color:rgb(250.14096433744578,4.859035662554219,4.859035662554219)'> no</span><span style='background-color:rgb(250.14096433744578,4.859035662554219,4.859035662554219)'> longer</span><span style='background-color:rgb(250.14096433744578,4.859035662554219,4.859035662554219)'> mattered</span><span style='background-color:rgb(250.14096433744578,4.859035662554219,4.859035662554219)'> .</span><span style='background-color:rgb(180.79991474747658,74.20008525252342,74.20008525252342)'> [SEP]</span><span style='background-color:rgb(252.53304625558357,2.466953744416449,2.466953744416449)'> name</span><span style='background-color:rgb(252.53304625558357,2.466953744416449,2.466953744416449)'> few</span><span style='background-color:rgb(257.6414919008935,-2.6414919008935356,-2.6414919008935356)'> objects</span><span style='background-color:rgb(257.6414919008935,-2.6414919008935356,-2.6414919008935356)'> said</span><span style='background-color:rgb(255.21320026141308,-0.21320026141308648,-0.21320026141308648)'> to</span><span style='background-color:rgb(255.21320026141308,-0.21320026141308648,-0.21320026141308648)'> be</span><span style='background-color:rgb(255.21320026141308,-0.21320026141308648,-0.21320026141308648)'> in</span><span style='background-color:rgb(261.44247199830716,-6.4424719983071554,-6.4424719983071554)'> or</span><span style='background-color:rgb(259.5920876642049,-4.5920876642048825,-4.5920876642048825)'> on</span><span style='background-color:rgb(255.37087327291374,-0.37087327291374095,-0.37087327291374095)'> allan</span><span style='background-color:rgb(255.37087327291374,-0.37087327291374095,-0.37087327291374095)'> '</span><span style='background-color:rgb(255.37087327291374,-0.37087327291374095,-0.37087327291374095)'> s</span><span style='background-color:rgb(250.95823206673958,4.041767933260417,4.041767933260417)'> desk</span><span style='background-color:rgb(250.95823206673958,4.041767933260417,4.041767933260417)'> |</span><span style='background-color:rgb(253.25270082769566,1.7472991723043376,1.7472991723043376)'> |</span><span style='background-color:rgb(225.7652162591403,29.23478374085971,29.23478374085971)'> erase</span><span style='background-color:rgb(225.7652162591403,29.23478374085971,29.23478374085971)'> ##r</span><span style='background-color:rgb(255.000018524006,-1.8524006009101868e-05,-1.8524006009101868e-05)'> [SEP]</span><span style='background-color:rgb(255.000018524006,-1.8524006009101868e-05,-1.8524006009101868e-05)'> [PAD]</span> [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

## RISE

In [15]:
class WrappedBackboneModel2(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
    
    def forward(self, inputs=None, **kwargs):
        outputs = self.model(inputs, output_hidden_states=True, **kwargs)
        # import pdb; pdb.set_trace()
        return torch.softmax(outputs.logits, dim=-1)

In [16]:
original_model2 = WrappedBackboneModel2(backbone_model)

In [78]:
from exlib.explainers.common import FeatureAttrMethod, FeatureAttrOutput
from skimage.transform import resize

class RiseTextCls(FeatureAttrMethod):
    def __init__(self, model, input_size=512, \
                 gpu_batch=100, N=500, \
                 s=8, p1=0.5, seed=42, mask_combine=None,
                normalize=False):
        super(RiseTextCls, self).__init__(model)
        self.input_size = input_size  # only one number for text
        self.gpu_batch = gpu_batch
        self.generate_masks(N, s, p1)
        self.mask_combine = mask_combine
        self.normalize = normalize

    def generate_masks(self, N, s, p1, savepath='masks.npy'):
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        cell_size = np.ceil(np.array(self.input_size) / s)
        up_size = (s + 1) * cell_size

        grid = np.random.rand(N, s) < p1
        grid = grid.astype('float32')

        self.masks = np.empty((N, self.input_size))

        for i in range(N):
            # Random shifts
            x = np.random.randint(0, cell_size)
            # Linear upsampling and cropping
            self.masks[i, :] = resize(grid[i], tuple([up_size]), order=1, mode='reflect', 
                                        anti_aliasing=False)[x:x + self.input_size]
        self.masks = (self.masks.reshape(-1, self.input_size) >= 0.5)
        # np.save(savepath, self.masks)
        self.masks = torch.from_numpy(self.masks).long()
        self.masks = self.masks.to(device)
        self.N = N
        self.p1 = p1

    def forward(self, x, label=None, kwargs=None):
        # Apply array of filters to the image]
        # print('RISE')
        self.model.eval()
        with torch.no_grad():
            N = self.N
            B, L = x.size()
            if self.mask_combine:
                stack = self.mask_combine(x, self.masks)
                stack = stack.reshape(N * B, L, -1)
            else:
                stack = torch.mul(self.masks.view(N, 1, L), x.data.view(B, L))
                # import pdb; pdb.set_trace()
                stack = stack.reshape(N * B, L)
            kwargs_new = {}
            for k, v in kwargs.items():
                if v is not None:
                    kwargs_new[k] = v.unsqueeze(0).expand(N, B, L).reshape(N * B, L)
            # stack = stack

            #p = nn.Softmax(dim=1)(model(stack)) in batches
            p = []
            for i in tqdm(range(0, N*B, self.gpu_batch)):
                kwargs_curr = {k: v[i:min(i + self.gpu_batch, N*B)] for k, v in kwargs_new.items()}
                if self.mask_combine:
                    pred = self.model(inputs_embeds=stack[i:min(i + self.gpu_batch, N*B)], **kwargs_curr)
                else:
                    pred = self.model(stack[i:min(i + self.gpu_batch, N*B)], **kwargs_curr)
                p.append(pred)
            p = torch.cat(p)
            if label is None:
                # if no label, then explain the top class
                pred_x = self.model(x, **kwargs)
                label = torch.argmax(pred_x, dim=-1)
            CL = p.size(1)
            p = p.view(N, B, CL)
            sal = torch.matmul(p.permute(1, 2, 0), self.masks.float().view(N, L))
            # import pdb; pdb.set_trace()
            if self.normalize:
                sal = sal / L
            sal = sal.view(B, CL, L)
        
        return FeatureAttrOutput(sal[range(B), label], sal)


In [82]:
# from exlib.explainers.rise import RiseTextCls

rise_explainer = RiseTextCls(original_model2).to(device)
rise_expln = rise_explainer(inputs, labels, kwargs=kwargs)

  0%|          | 0/10 [00:00<?, ?it/s]

In [86]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

def show_attrs(tokens, masks, normalize=True):
    if normalize:
        masks_max = masks.max(-1).values
        masks_min = masks.min(-1).values
        # import pdb; pdb.set_trace()
        masks = (masks - masks_min) / (masks_max - masks_min)
    print_str = ""
    for i in range(len(tokens)):
        expln_val = masks[i].item()
        if expln_val == 0:
            print_str += f" {tokens[i]}"
        else:
            print_str += f"<span style='background-color:rgb({255 * (1-expln_val)},{255 * expln_val},{255 * expln_val})'> {tokens[i]}</span>"
    return print_str

tokens = processor.convert_ids_to_tokens(inputs[0])
# tokens = shap_expln.explainer_output.data[0]
printmd(show_attrs(tokens, rise_expln.attributions[0]))

<span style='background-color:rgb(238.8148406893015,16.18515931069851,16.18515931069851)'> [CLS]</span><span style='background-color:rgb(238.8148406893015,16.18515931069851,16.18515931069851)'> allan</span><span style='background-color:rgb(238.8148406893015,16.18515931069851,16.18515931069851)'> sat</span><span style='background-color:rgb(238.8148406893015,16.18515931069851,16.18515931069851)'> down</span><span style='background-color:rgb(238.8148406893015,16.18515931069851,16.18515931069851)'> at</span><span style='background-color:rgb(238.8148406893015,16.18515931069851,16.18515931069851)'> his</span><span style='background-color:rgb(238.8148406893015,16.18515931069851,16.18515931069851)'> desk</span><span style='background-color:rgb(238.8148406893015,16.18515931069851,16.18515931069851)'> and</span><span style='background-color:rgb(238.8148406893015,16.18515931069851,16.18515931069851)'> pulled</span><span style='background-color:rgb(239.05032735317945,15.949672646820545,15.949672646820545)'> the</span><span style='background-color:rgb(238.56281351298094,16.437186487019062,16.437186487019062)'> chair</span><span style='background-color:rgb(240.37113362923265,14.628866370767355,14.628866370767355)'> in</span><span style='background-color:rgb(240.08213253691792,14.917867463082075,14.917867463082075)'> close</span><span style='background-color:rgb(240.31039768829942,14.689602311700583,14.689602311700583)'> .</span><span style='background-color:rgb(241.54580734670162,13.454192653298378,13.454192653298378)'> opening</span><span style='background-color:rgb(237.18802828341722,17.811971716582775,17.811971716582775)'> a</span><span style='background-color:rgb(236.88218649476767,18.117813505232334,18.117813505232334)'> side</span><span style='background-color:rgb(239.54216916114092,15.457830838859081,15.457830838859081)'> drawer</span><span style='background-color:rgb(234.3964909762144,20.60350902378559,20.60350902378559)'> ,</span><span style='background-color:rgb(234.2841386049986,20.71586139500141,20.71586139500141)'> he</span><span style='background-color:rgb(240.9393845871091,14.060615412890911,14.060615412890911)'> took</span><span style='background-color:rgb(234.96054980903864,20.03945019096136,20.03945019096136)'> out</span><span style='background-color:rgb(232.37581450492144,22.624185495078564,22.624185495078564)'> a</span><span style='background-color:rgb(232.15826477855444,22.84173522144556,22.84173522144556)'> piece</span><span style='background-color:rgb(231.23348750174046,23.766512498259544,23.766512498259544)'> of</span><span style='background-color:rgb(230.2177906036377,24.782209396362305,24.782209396362305)'> paper</span><span style='background-color:rgb(228.52032478898764,26.479675211012363,26.479675211012363)'> and</span><span style='background-color:rgb(228.07790502905846,26.922094970941544,26.922094970941544)'> his</span><span style='background-color:rgb(223.72059144079685,31.279408559203148,31.279408559203148)'> ink</span><span style='background-color:rgb(221.895929351449,33.10407064855099,33.10407064855099)'> ##pot</span><span style='background-color:rgb(222.2952537238598,32.70474627614021,32.70474627614021)'> .</span><span style='background-color:rgb(221.4844912290573,33.51550877094269,33.51550877094269)'> after</span><span style='background-color:rgb(225.60621935874224,29.393780641257763,29.393780641257763)'> filling</span><span style='background-color:rgb(225.0268853455782,29.973114654421806,29.973114654421806)'> his</span><span style='background-color:rgb(225.81458307802677,29.18541692197323,29.18541692197323)'> pen</span><span style='background-color:rgb(227.2758972644806,27.72410273551941,27.72410273551941)'> ,</span><span style='background-color:rgb(227.29177091270685,27.708229087293148,27.708229087293148)'> allan</span><span style='background-color:rgb(227.47620731592178,27.523792684078217,27.523792684078217)'> looked</span><span style='background-color:rgb(218.09501990675926,36.90498009324074,36.90498009324074)'> at</span><span style='background-color:rgb(220.64218282699585,34.35781717300415,34.35781717300415)'> his</span><span style='background-color:rgb(222.8205071389675,32.179492861032486,32.179492861032486)'> paper</span><span style='background-color:rgb(214.9123790860176,40.08762091398239,40.08762091398239)'> in</span><span style='background-color:rgb(214.09443117678165,40.905568823218346,40.905568823218346)'> the</span><span style='background-color:rgb(213.3463691174984,41.6536308825016,41.6536308825016)'> orange</span><span style='background-color:rgb(210.39875328540802,44.60124671459198,44.60124671459198)'> glow</span><span style='background-color:rgb(210.4980267584324,44.50197324156761,44.50197324156761)'> from</span><span style='background-color:rgb(214.15965847671032,40.84034152328968,40.84034152328968)'> the</span><span style='background-color:rgb(216.51093810796738,38.48906189203262,38.48906189203262)'> lantern</span><span style='background-color:rgb(213.77601206302643,41.22398793697357,41.22398793697357)'> set</span><span style='background-color:rgb(215.50702057778835,39.49297942221165,39.49297942221165)'> back</span><span style='background-color:rgb(211.51272431015968,43.48727568984032,43.48727568984032)'> in</span><span style='background-color:rgb(205.65688893198967,49.34311106801033,49.34311106801033)'> the</span><span style='background-color:rgb(208.31500969827175,46.68499030172825,46.68499030172825)'> desk</span><span style='background-color:rgb(204.97735239565372,50.022647604346275,50.022647604346275)'> '</span><span style='background-color:rgb(202.79842771589756,52.20157228410244,52.20157228410244)'> s</span><span style='background-color:rgb(206.73006154596806,48.269938454031944,48.269938454031944)'> right</span><span style='background-color:rgb(209.32514749467373,45.67485250532627,45.67485250532627)'> -</span><span style='background-color:rgb(209.2138286679983,45.786171332001686,45.786171332001686)'> hand</span><span style='background-color:rgb(211.4594093710184,43.54059062898159,43.54059062898159)'> corner</span><span style='background-color:rgb(216.85039289295673,38.149607107043266,38.149607107043266)'> .</span><span style='background-color:rgb(216.5876141935587,38.41238580644131,38.41238580644131)'> his</span><span style='background-color:rgb(218.05438488721848,36.945615112781525,36.945615112781525)'> pen</span><span style='background-color:rgb(223.2751413434744,31.724858656525612,31.724858656525612)'> cast</span><span style='background-color:rgb(225.45326616615057,29.54673383384943,29.54673383384943)'> a</span><span style='background-color:rgb(218.73076379299164,36.26923620700836,36.26923620700836)'> forbid</span><span style='background-color:rgb(219.4447796791792,35.55522032082081,35.55522032082081)'> ##ding</span><span style='background-color:rgb(222.72878766059875,32.271212339401245,32.271212339401245)'> line</span><span style='background-color:rgb(217.83161044120789,37.168389558792114,37.168389558792114)'> of</span><span style='background-color:rgb(215.65235517919064,39.347644820809364,39.347644820809364)'> shadow</span><span style='background-color:rgb(219.77960631251335,35.22039368748665,35.22039368748665)'> slant</span><span style='background-color:rgb(226.28446016460657,28.71553983539343,28.71553983539343)'> ##ing</span><span style='background-color:rgb(226.13490018993616,28.86509981006384,28.86509981006384)'> across</span><span style='background-color:rgb(225.75032092630863,29.249679073691368,29.249679073691368)'> the</span><span style='background-color:rgb(225.75032092630863,29.249679073691368,29.249679073691368)'> page</span><span style='background-color:rgb(225.75032092630863,29.249679073691368,29.249679073691368)'> ,</span><span style='background-color:rgb(225.75032092630863,29.249679073691368,29.249679073691368)'> echoing</span><span style='background-color:rgb(225.75032092630863,29.249679073691368,29.249679073691368)'> the</span><span style='background-color:rgb(225.75032092630863,29.249679073691368,29.249679073691368)'> ink</span><span style='background-color:rgb(225.75032092630863,29.249679073691368,29.249679073691368)'> ##y</span><span style='background-color:rgb(225.75032092630863,29.249679073691368,29.249679073691368)'> darkness</span><span style='background-color:rgb(225.75032092630863,29.249679073691368,29.249679073691368)'> crouch</span><span style='background-color:rgb(227.92707972228527,27.07292027771473,27.07292027771473)'> ##ing</span><span style='background-color:rgb(225.29844917356968,29.70155082643032,29.70155082643032)'> in</span><span style='background-color:rgb(224.4621292501688,30.5378707498312,30.5378707498312)'> the</span><span style='background-color:rgb(224.46542367339134,30.534576326608658,30.534576326608658)'> edges</span><span style='background-color:rgb(224.0126532688737,30.98734673112631,30.98734673112631)'> of</span><span style='background-color:rgb(224.13079652935266,30.869203470647335,30.869203470647335)'> the</span><span style='background-color:rgb(223.90056688338518,31.09943311661482,31.09943311661482)'> lantern</span><span style='background-color:rgb(227.8220495581627,27.17795044183731,27.17795044183731)'> '</span><span style='background-color:rgb(226.9133796170354,28.08662038296461,28.08662038296461)'> s</span><span style='background-color:rgb(233.22248317301273,21.777516826987267,21.777516826987267)'> struggling</span><span style='background-color:rgb(232.6533668115735,22.346633188426495,22.346633188426495)'> glow</span><span style='background-color:rgb(227.92012609541416,27.07987390458584,27.07987390458584)'> .</span><span style='background-color:rgb(225.74080243706703,29.259197562932968,29.259197562932968)'> the</span><span style='background-color:rgb(230.94674918800592,24.053250811994076,24.053250811994076)'> only</span><span style='background-color:rgb(230.94674918800592,24.053250811994076,24.053250811994076)'> other</span><span style='background-color:rgb(230.96072673797607,24.039273262023926,24.039273262023926)'> illumination</span><span style='background-color:rgb(234.6109799668193,20.389020033180714,20.389020033180714)'> came</span><span style='background-color:rgb(232.34769221395254,22.65230778604746,22.65230778604746)'> from</span><span style='background-color:rgb(234.91182882338762,20.088171176612377,20.088171176612377)'> a</span><span style='background-color:rgb(238.9669617265463,16.033038273453712,16.033038273453712)'> lu</span><span style='background-color:rgb(236.29400085657835,18.70599914342165,18.70599914342165)'> ##rid</span><span style='background-color:rgb(237.949135042727,17.050864957273006,17.050864957273006)'> moonlight</span><span style='background-color:rgb(234.4561629742384,20.543837025761604,20.543837025761604)'> filtered</span><span style='background-color:rgb(229.95048824697733,25.04951175302267,25.04951175302267)'> through</span><span style='background-color:rgb(228.4090382605791,26.59096173942089,26.59096173942089)'> thin</span><span style='background-color:rgb(227.36931335180998,27.63068664819002,27.63068664819002)'> branches</span><span style='background-color:rgb(229.9918870255351,25.008112974464893,25.008112974464893)'> and</span><span style='background-color:rgb(227.79579296708107,27.20420703291893,27.20420703291893)'> clouds</span><span style='background-color:rgb(227.53883935511112,27.461160644888878,27.461160644888878)'> ,</span><span style='background-color:rgb(231.5320735797286,23.467926420271397,23.467926420271397)'> casting</span><span style='background-color:rgb(228.9858416095376,26.0141583904624,26.0141583904624)'> its</span><span style='background-color:rgb(228.7048276886344,26.295172311365604,26.295172311365604)'> bone</span><span style='background-color:rgb(231.5696782618761,23.430321738123894,23.430321738123894)'> -</span><span style='background-color:rgb(235.56191504001617,19.438084959983826,19.438084959983826)'> pale</span><span style='background-color:rgb(236.0330555588007,18.966944441199303,18.966944441199303)'> glow</span><span style='background-color:rgb(236.26880820840597,18.73119179159403,18.73119179159403)'> onto</span><span style='background-color:rgb(238.34815923124552,16.651840768754482,16.651840768754482)'> the</span><span style='background-color:rgb(238.3444981276989,16.6555018723011,16.6555018723011)'> pine</span><span style='background-color:rgb(233.9514759555459,21.048524044454098,21.048524044454098)'> floor</span><span style='background-color:rgb(236.1279022693634,18.872097730636597,18.872097730636597)'> ##boards</span><span style='background-color:rgb(232.21790067851543,22.782099321484566,22.782099321484566)'> .</span><span style='background-color:rgb(234.161570481956,20.838429518043995,20.838429518043995)'> allan</span><span style='background-color:rgb(236.34332221001387,18.656677789986134,18.656677789986134)'> unfolded</span><span style='background-color:rgb(234.16596304625273,20.834036953747272,20.834036953747272)'> another</span><span style='background-color:rgb(239.97580379247665,15.024196207523346,15.024196207523346)'> page</span><span style='background-color:rgb(244.29617671296,10.703823287039995,10.703823287039995)'> ,</span><span style='background-color:rgb(239.85526476055384,15.144735239446163,15.144735239446163)'> this</span><span style='background-color:rgb(241.61479644477367,13.385203555226326,13.385203555226326)'> one</span><span style='background-color:rgb(238.18701937794685,16.812980622053146,16.812980622053146)'> crowded</span><span style='background-color:rgb(238.08544892817736,16.914551071822643,16.914551071822643)'> with</span><span style='background-color:rgb(238.4376159310341,16.562384068965912,16.562384068965912)'> ranks</span><span style='background-color:rgb(238.74442096799612,16.25557903200388,16.25557903200388)'> of</span><span style='background-color:rgb(241.26878701150417,13.731212988495827,13.731212988495827)'> letters</span><span style='background-color:rgb(240.57167736813426,14.42832263186574,14.42832263186574)'> in</span><span style='background-color:rgb(238.39461743831635,16.605382561683655,16.605382561683655)'> tight</span><span style='background-color:rgb(240.45336786657572,14.546632133424282,14.546632133424282)'> formation</span><span style='background-color:rgb(239.8110693320632,15.188930667936802,15.188930667936802)'> from</span><span style='background-color:rgb(239.8110693320632,15.188930667936802,15.188930667936802)'> left</span><span style='background-color:rgb(238.25298003852367,16.747019961476326,16.747019961476326)'> to</span><span style='background-color:rgb(240.51070868968964,14.489291310310364,14.489291310310364)'> right</span><span style='background-color:rgb(229.49668619781733,25.503313802182674,25.503313802182674)'> .</span><span style='background-color:rgb(225.00219237059355,29.997807629406452,29.997807629406452)'> the</span><span style='background-color:rgb(224.91326954215765,30.08673045784235,30.08673045784235)'> lines</span><span style='background-color:rgb(227.4733118712902,27.526688128709793,27.526688128709793)'> of</span><span style='background-color:rgb(227.4733118712902,27.526688128709793,27.526688128709793)'> letters</span><span style='background-color:rgb(227.4733118712902,27.526688128709793,27.526688128709793)'> stepped</span><span style='background-color:rgb(227.4733118712902,27.526688128709793,27.526688128709793)'> into</span><span style='background-color:rgb(227.4733118712902,27.526688128709793,27.526688128709793)'> their</span><span style='background-color:rgb(227.4733118712902,27.526688128709793,27.526688128709793)'> divisions</span><span style='background-color:rgb(227.4733118712902,27.526688128709793,27.526688128709793)'> ,</span><span style='background-color:rgb(227.4733118712902,27.526688128709793,27.526688128709793)'> in</span><span style='background-color:rgb(227.4733118712902,27.526688128709793,27.526688128709793)'> the</span><span style='background-color:rgb(231.04232925921679,23.957670740783215,23.957670740783215)'> shape</span><span style='background-color:rgb(233.70880007743835,21.291199922561646,21.291199922561646)'> of</span><span style='background-color:rgb(232.0658803358674,22.934119664132595,22.934119664132595)'> a</span><span style='background-color:rgb(234.2390464246273,20.760953575372696,20.760953575372696)'> story</span><span style='background-color:rgb(234.69181682914495,20.308183170855045,20.308183170855045)'> '</span><span style='background-color:rgb(234.69181682914495,20.308183170855045,20.308183170855045)'> s</span><span style='background-color:rgb(237.22936056554317,17.770639434456825,17.770639434456825)'> outline</span><span style='background-color:rgb(233.22308164089918,21.77691835910082,21.77691835910082)'> :</span><span style='background-color:rgb(231.1536157876253,23.846384212374687,23.846384212374687)'> the</span><span style='background-color:rgb(225.6213292479515,29.378670752048492,29.378670752048492)'> loose</span><span style='background-color:rgb(225.96204556524754,29.037954434752464,29.037954434752464)'> ,</span><span style='background-color:rgb(230.23672688752413,24.763273112475872,24.763273112475872)'> dry</span><span style='background-color:rgb(234.02545608580112,20.974543914198875,20.974543914198875)'> skeleton</span><span style='background-color:rgb(233.1605825945735,21.839417405426502,21.839417405426502)'> of</span><span style='background-color:rgb(232.87897020578384,22.121029794216156,22.121029794216156)'> a</span><span style='background-color:rgb(234.12010330706835,20.879896692931652,20.879896692931652)'> tale</span><span style='background-color:rgb(232.64661077409983,22.353389225900173,22.353389225900173)'> lay</span><span style='background-color:rgb(239.1263717226684,15.87362827733159,15.87362827733159)'> exposed</span><span style='background-color:rgb(239.4784712791443,15.521528720855713,15.521528720855713)'> beneath</span><span style='background-color:rgb(239.76933617144823,15.23066382855177,15.23066382855177)'> their</span><span style='background-color:rgb(243.95129961892962,11.048700381070375,11.048700381070375)'> feet</span><span style='background-color:rgb(236.70716978609562,18.29283021390438,18.29283021390438)'> ,</span><span style='background-color:rgb(238.7083115056157,16.29168849438429,16.29168849438429)'> awaiting</span><span style='background-color:rgb(236.7869085073471,18.213091492652893,18.213091492652893)'> tend</span><span style='background-color:rgb(238.49252678453922,16.507473215460777,16.507473215460777)'> ##ons</span><span style='background-color:rgb(240.11631075292826,14.883689247071743,14.883689247071743)'> ,</span><span style='background-color:rgb(240.37975251674652,14.620247483253479,14.620247483253479)'> muscle</span><span style='background-color:rgb(240.37902105599642,14.620978944003582,14.620978944003582)'> and</span><span style='background-color:rgb(240.64419362694025,14.35580637305975,14.35580637305975)'> blushing</span><span style='background-color:rgb(242.61827984824777,12.381720151752234,12.381720151752234)'> skin</span><span style='background-color:rgb(243.03088165819645,11.96911834180355,11.96911834180355)'> .</span><span style='background-color:rgb(240.48222161829472,14.517778381705284,14.517778381705284)'> allan</span><span style='background-color:rgb(244.09796319901943,10.902036800980568,10.902036800980568)'> reviewed</span><span style='background-color:rgb(242.34565302729607,12.654346972703934,12.654346972703934)'> the</span><span style='background-color:rgb(240.32018216326833,14.679817836731672,14.679817836731672)'> troops</span><span style='background-color:rgb(240.32018216326833,14.679817836731672,14.679817836731672)'> ,</span><span style='background-color:rgb(238.2408320903778,16.759167909622192,16.759167909622192)'> all</span><span style='background-color:rgb(233.88608146458864,21.113918535411358,21.113918535411358)'> prepared</span><span style='background-color:rgb(236.23646054416895,18.76353945583105,18.76353945583105)'> to</span><span style='background-color:rgb(236.0796144604683,18.920385539531708,18.920385539531708)'> di</span><span style='background-color:rgb(231.6441599652171,23.355840034782887,23.355840034782887)'> ##se</span><span style='background-color:rgb(229.3445309624076,25.65546903759241,25.65546903759241)'> ##mba</span><span style='background-color:rgb(235.86698926985264,19.13301073014736,19.13301073014736)'> ##rk</span><span style='background-color:rgb(238.04378226399422,16.956217736005783,16.956217736005783)'> ,</span><span style='background-color:rgb(230.11156160384417,24.888438396155834,24.888438396155834)'> their</span><span style='background-color:rgb(232.3241961747408,22.67580382525921,22.67580382525921)'> task</span><span style='background-color:rgb(234.68685809522867,20.313141904771328,20.313141904771328)'> to</span><span style='background-color:rgb(232.35378328710794,22.646216712892056,22.646216712892056)'> form</span><span style='background-color:rgb(236.9253844767809,18.07461552321911,18.07461552321911)'> the</span><span style='background-color:rgb(236.6703288629651,18.329671137034893,18.329671137034893)'> tale</span><span style='background-color:rgb(234.83511853963137,20.164881460368633,20.164881460368633)'> of</span><span style='background-color:rgb(236.87722966074944,18.122770339250565,18.122770339250565)'> a</span><span style='background-color:rgb(235.74911199510098,19.250888004899025,19.250888004899025)'> young</span><span style='background-color:rgb(242.72557374089956,12.274426259100437,12.274426259100437)'> man</span><span style='background-color:rgb(242.7252080105245,12.274791989475489,12.274791989475489)'> returning</span><span style='background-color:rgb(236.53747659176588,18.46252340823412,18.46252340823412)'> home</span><span style='background-color:rgb(234.50664706528187,20.493352934718132,20.493352934718132)'> from</span><span style='background-color:rgb(227.97410409897566,27.02589590102434,27.02589590102434)'> life</span><span style='background-color:rgb(231.79292008280754,23.20707991719246,23.20707991719246)'> abroad</span><span style='background-color:rgb(231.82047430425882,23.179525695741177,23.179525695741177)'> to</span><span style='background-color:rgb(236.1786200478673,18.821379952132702,18.821379952132702)'> find</span><span style='background-color:rgb(240.67281369119883,14.327186308801174,14.327186308801174)'> his</span><span style='background-color:rgb(240.60099659487605,14.399003405123949,14.399003405123949)'> childhood</span><span style='background-color:rgb(235.54021630436182,19.45978369563818,19.45978369563818)'> friend</span><span style='background-color:rgb(235.54021630436182,19.45978369563818,19.45978369563818)'> a</span><span style='background-color:rgb(235.54021630436182,19.45978369563818,19.45978369563818)'> bride</span><span style='background-color:rgb(235.54021630436182,19.45978369563818,19.45978369563818)'> to</span><span style='background-color:rgb(235.54021630436182,19.45978369563818,19.45978369563818)'> -</span><span style='background-color:rgb(235.54021630436182,19.45978369563818,19.45978369563818)'> be</span><span style='background-color:rgb(235.54021630436182,19.45978369563818,19.45978369563818)'> ,</span><span style='background-color:rgb(235.54021630436182,19.45978369563818,19.45978369563818)'> thus</span><span style='background-color:rgb(235.54021630436182,19.45978369563818,19.45978369563818)'> upset</span><span style='background-color:rgb(235.9937846660614,19.0062153339386,19.0062153339386)'> ##ting</span><span style='background-color:rgb(233.8148295879364,21.1851704120636,21.1851704120636)'> the</span><span style='background-color:rgb(236.28192130476236,18.718078695237637,18.718078695237637)'> apple</span><span style='background-color:rgb(238.14502023160458,16.854979768395424,16.854979768395424)'> cart</span><span style='background-color:rgb(238.0439494550228,16.956050544977188,16.956050544977188)'> of</span><span style='background-color:rgb(238.0439494550228,16.956050544977188,16.956050544977188)'> his</span><span style='background-color:rgb(234.9916321411729,20.008367858827114,20.008367858827114)'> life</span><span style='background-color:rgb(237.9959922283888,17.004007771611214,17.004007771611214)'> '</span><span style='background-color:rgb(235.8200978860259,19.179902113974094,19.179902113974094)'> s</span><span style='background-color:rgb(237.0056228712201,17.994377128779888,17.994377128779888)'> plan</span><span style='background-color:rgb(235.16738411039114,19.83261588960886,19.83261588960886)'> ,</span><span style='background-color:rgb(228.73318176716566,26.26681823283434,26.26681823283434)'> clarified</span><span style='background-color:rgb(229.110440723598,25.889559276401997,25.889559276401997)'> –</span><span style='background-color:rgb(225.1910213381052,29.8089786618948,29.8089786618948)'> of</span><span style='background-color:rgb(225.4085710644722,29.5914289355278,29.5914289355278)'> course</span><span style='background-color:rgb(222.0722132921219,32.92778670787811,32.92778670787811)'> –</span><span style='background-color:rgb(219.90919835865498,35.090801641345024,35.090801641345024)'> by</span><span style='background-color:rgb(215.94981081783772,39.050189182162285,39.050189182162285)'> his</span><span style='background-color:rgb(215.6097898632288,39.3902101367712,39.3902101367712)'> very</span><span style='background-color:rgb(215.32005541026592,39.67994458973408,39.67994458973408)'> time</span><span style='background-color:rgb(208.9627305418253,46.037269458174706,46.037269458174706)'> away</span><span style='background-color:rgb(214.87457491457462,40.12542508542538,40.12542508542538)'> from</span><span style='background-color:rgb(212.4018234014511,42.59817659854889,42.59817659854889)'> her</span><span style='background-color:rgb(216.88430227339268,38.11569772660732,38.11569772660732)'> he</span><span style='background-color:rgb(216.77870593965054,38.221294060349464,38.221294060349464)'> loved</span><span style='background-color:rgb(217.02371299266815,37.97628700733185,37.97628700733185)'> best</span><span style='background-color:rgb(216.57559923827648,38.42440076172352,38.42440076172352)'> .</span><span style='background-color:rgb(218.7565530091524,36.24344699084759,36.24344699084759)'> although</span><span style='background-color:rgb(216.5697095543146,38.43029044568539,38.43029044568539)'> the</span><span style='background-color:rgb(208.02451148629189,46.975488513708115,46.975488513708115)'> concept</span><span style='background-color:rgb(209.96498376131058,45.03501623868942,45.03501623868942)'> was</span><span style='background-color:rgb(209.96498376131058,45.03501623868942,45.03501623868942)'> a</span><span style='background-color:rgb(208.00783798098564,46.99216201901436,46.99216201901436)'> simple</span><span style='background-color:rgb(209.76131089031696,45.23868910968304,45.23868910968304)'> one</span><span style='background-color:rgb(213.43492716550827,41.56507283449173,41.56507283449173)'> ,</span><span style='background-color:rgb(215.31133867800236,39.68866132199764,39.68866132199764)'> allan</span><span style='background-color:rgb(215.21203100681305,39.78796899318695,39.78796899318695)'> thought</span><span style='background-color:rgb(211.54500357806683,43.454996421933174,43.454996421933174)'> it</span><span style='background-color:rgb(211.26339308917522,43.736606910824776,43.736606910824776)'> had</span><span style='background-color:rgb(206.8249424546957,48.1750575453043,48.1750575453043)'> potential</span><span style='background-color:rgb(215.76467335224152,39.235326647758484,39.235326647758484)'> .</span><span style='background-color:rgb(220.4708918184042,34.5291081815958,34.5291081815958)'> besides</span><span style='background-color:rgb(215.13099655508995,39.86900344491005,39.86900344491005)'> ,</span><span style='background-color:rgb(217.30832152068615,37.69167847931385,37.69167847931385)'> the</span><span style='background-color:rgb(217.3394038528204,37.660596147179604,37.660596147179604)'> public</span><span style='background-color:rgb(212.98342019319534,42.01657980680466,42.01657980680466)'> liked</span><span style='background-color:rgb(211.12937428057194,43.87062571942806,43.87062571942806)'> a</span><span style='background-color:rgb(208.69376577436924,46.30623422563076,46.30623422563076)'> good</span><span style='background-color:rgb(206.61860972642899,48.381390273571014,48.381390273571014)'> ,</span><span style='background-color:rgb(207.09414280951023,47.90585719048977,47.90585719048977)'> simple</span><span style='background-color:rgb(205.4281260073185,49.5718739926815,49.5718739926815)'> romance</span><span style='background-color:rgb(203.29489767551422,51.70510232448578,51.70510232448578)'> .</span><span style='background-color:rgb(198.00821095705032,56.99178904294968,56.99178904294968)'> perhaps</span><span style='background-color:rgb(193.52942928671837,61.47057071328163,61.47057071328163)'> this</span><span style='background-color:rgb(193.5286959260702,61.47130407392979,61.47130407392979)'> will</span><span style='background-color:rgb(201.9247367978096,53.0752632021904,53.0752632021904)'> be</span><span style='background-color:rgb(197.67501443624496,57.324985563755035,57.324985563755035)'> more</span><span style='background-color:rgb(197.67501443624496,57.324985563755035,57.324985563755035)'> sale</span><span style='background-color:rgb(193.85623455047607,61.143765449523926,61.143765449523926)'> ##able</span><span style='background-color:rgb(189.49998870491982,65.50001129508018,65.50001129508018)'> ,</span><span style='background-color:rgb(183.31235513091087,71.68764486908913,71.68764486908913)'> he</span><span style='background-color:rgb(177.23946779966354,77.76053220033646,77.76053220033646)'> thought</span><span style='background-color:rgb(175.677120834589,79.322879165411,79.322879165411)'> and</span><span style='background-color:rgb(176.3834497332573,78.6165502667427,78.6165502667427)'> began</span><span style='background-color:rgb(176.3834497332573,78.6165502667427,78.6165502667427)'> to</span><span style='background-color:rgb(176.3834497332573,78.6165502667427,78.6165502667427)'> write</span><span style='background-color:rgb(176.3834497332573,78.6165502667427,78.6165502667427)'> .</span><span style='background-color:rgb(176.3834497332573,78.6165502667427,78.6165502667427)'> they</span><span style='background-color:rgb(176.3834497332573,78.6165502667427,78.6165502667427)'> gazed</span><span style='background-color:rgb(176.3834497332573,78.6165502667427,78.6165502667427)'> at</span><span style='background-color:rgb(176.3834497332573,78.6165502667427,78.6165502667427)'> each</span><span style='background-color:rgb(176.3834497332573,78.6165502667427,78.6165502667427)'> other</span><span style='background-color:rgb(170.1841051876545,84.8158948123455,84.8158948123455)'> ,</span><span style='background-color:rgb(170.63374266028404,84.36625733971596,84.36625733971596)'> lost</span><span style='background-color:rgb(163.21449637413025,91.78550362586975,91.78550362586975)'> in</span><span style='background-color:rgb(152.92710319161415,102.07289680838585,102.07289680838585)'> the</span><span style='background-color:rgb(153.15537214279175,101.84462785720825,101.84462785720825)'> rap</span><span style='background-color:rgb(151.91992923617363,103.08007076382637,103.08007076382637)'> ##ture</span><span style='background-color:rgb(148.43567579984665,106.56432420015335,106.56432420015335)'> of</span><span style='background-color:rgb(146.25751867890358,108.74248132109642,108.74248132109642)'> love</span><span style='background-color:rgb(147.16592073440552,107.83407926559448,107.83407926559448)'> based</span><span style='background-color:rgb(144.0287710726261,110.97122892737389,110.97122892737389)'> so</span><span style='background-color:rgb(144.25713881850243,110.74286118149757,110.74286118149757)'> deeply</span><span style='background-color:rgb(138.43578279018402,116.56421720981598,116.56421720981598)'> within</span><span style='background-color:rgb(134.4784927368164,120.5215072631836,120.5215072631836)'> their</span><span style='background-color:rgb(132.756599932909,122.24340006709099,122.24340006709099)'> hearts</span><span style='background-color:rgb(133.03818002343178,121.96181997656822,121.96181997656822)'> that</span><span style='background-color:rgb(132.0277078449726,122.97229215502739,122.97229215502739)'> they</span><span style='background-color:rgb(131.32287606596947,123.67712393403053,123.67712393403053)'> had</span><span style='background-color:rgb(128.75880405306816,126.24119594693184,126.24119594693184)'> never</span><span style='background-color:rgb(117.08342134952545,137.91657865047455,137.91657865047455)'> seen</span><span style='background-color:rgb(108.67020159959793,146.32979840040207,146.32979840040207)'> it</span><span style='background-color:rgb(109.16121125221252,145.83878874778748,145.83878874778748)'> before</span><span style='background-color:rgb(104.92346584796906,150.07653415203094,150.07653415203094)'> .</span><span style='background-color:rgb(105.78840583562851,149.2115941643715,149.2115941643715)'> "</span><span style='background-color:rgb(99.6609154343605,155.3390845656395,155.3390845656395)'> what</span><span style='background-color:rgb(98.0609580874443,156.9390419125557,156.9390419125557)'> about</span><span style='background-color:rgb(94.42385405302048,160.57614594697952,160.57614594697952)'> roger</span><span style='background-color:rgb(90.64600557088852,164.35399442911148,164.35399442911148)'> ?</span><span style='background-color:rgb(84.26684707403183,170.73315292596817,170.73315292596817)'> "</span><span style='background-color:rgb(84.66097712516785,170.33902287483215,170.33902287483215)'> she</span><span style='background-color:rgb(83.72099876403809,171.2790012359619,171.2790012359619)'> asked</span><span style='background-color:rgb(79.19009149074554,175.80990850925446,175.80990850925446)'> ,</span><span style='background-color:rgb(79.8943305015564,175.1056694984436,175.1056694984436)'> knowing</span><span style='background-color:rgb(75.43347626924515,179.56652373075485,179.56652373075485)'> that</span><span style='background-color:rgb(72.25806266069412,182.74193733930588,182.74193733930588)'> the</span><span style='background-color:rgb(68.90556335449219,186.0944366455078,186.0944366455078)'> answer</span><span style='background-color:rgb(67.56203144788742,187.43796855211258,187.43796855211258)'> no</span><span style='background-color:rgb(65.48524528741837,189.51475471258163,189.51475471258163)'> longer</span><span style='background-color:rgb(61.56029909849167,193.43970090150833,193.43970090150833)'> mattered</span><span style='background-color:rgb(57.51620650291443,197.48379349708557,197.48379349708557)'> .</span><span style='background-color:rgb(58.9472097158432,196.0527902841568,196.0527902841568)'> [SEP]</span><span style='background-color:rgb(50.23248374462128,204.76751625537872,204.76751625537872)'> name</span><span style='background-color:rgb(43.587081134319305,211.4129188656807,211.4129188656807)'> few</span><span style='background-color:rgb(36.54069364070892,218.45930635929108,218.45930635929108)'> objects</span><span style='background-color:rgb(32.186583280563354,222.81341671943665,222.81341671943665)'> said</span><span style='background-color:rgb(33.49723935127258,221.50276064872742,221.50276064872742)'> to</span><span style='background-color:rgb(33.46352756023407,221.53647243976593,221.53647243976593)'> be</span><span style='background-color:rgb(35.03389209508896,219.96610790491104,219.96610790491104)'> in</span><span style='background-color:rgb(34.25721377134323,220.74278622865677,220.74278622865677)'> or</span><span style='background-color:rgb(33.22520434856415,221.77479565143585,221.77479565143585)'> on</span><span style='background-color:rgb(31.219535171985626,223.78046482801437,223.78046482801437)'> allan</span><span style='background-color:rgb(27.113247513771057,227.88675248622894,227.88675248622894)'> '</span><span style='background-color:rgb(23.109235167503357,231.89076483249664,231.89076483249664)'> s</span><span style='background-color:rgb(22.15913414955139,232.8408658504486,232.8408658504486)'> desk</span><span style='background-color:rgb(11.813566088676453,243.18643391132355,243.18643391132355)'> |</span><span style='background-color:rgb(7.470596730709076,247.52940326929092,247.52940326929092)'> |</span><span style='background-color:rgb(0.01793503761291504,254.98206496238708,254.98206496238708)'> erase</span><span style='background-color:rgb(0.0,255.0,255.0)'> ##r</span><span style='background-color:rgb(3.2447370886802673,251.75526291131973,251.75526291131973)'> [SEP]</span><span style='background-color:rgb(7.063547372817993,247.936452627182,247.936452627182)'> [PAD]</span><span style='background-color:rgb(9.242107272148132,245.75789272785187,245.75789272785187)'> [PAD]</span><span style='background-color:rgb(15.19971638917923,239.80028361082077,239.80028361082077)'> [PAD]</span><span style='background-color:rgb(23.20815145969391,231.7918485403061,231.7918485403061)'> [PAD]</span><span style='background-color:rgb(24.859345257282257,230.14065474271774,230.14065474271774)'> [PAD]</span><span style='background-color:rgb(28.507468700408936,226.49253129959106,226.49253129959106)'> [PAD]</span><span style='background-color:rgb(28.507468700408936,226.49253129959106,226.49253129959106)'> [PAD]</span><span style='background-color:rgb(28.507468700408936,226.49253129959106,226.49253129959106)'> [PAD]</span><span style='background-color:rgb(28.507468700408936,226.49253129959106,226.49253129959106)'> [PAD]</span><span style='background-color:rgb(28.507468700408936,226.49253129959106,226.49253129959106)'> [PAD]</span><span style='background-color:rgb(28.507468700408936,226.49253129959106,226.49253129959106)'> [PAD]</span><span style='background-color:rgb(28.507468700408936,226.49253129959106,226.49253129959106)'> [PAD]</span><span style='background-color:rgb(28.507468700408936,226.49253129959106,226.49253129959106)'> [PAD]</span><span style='background-color:rgb(28.507468700408936,226.49253129959106,226.49253129959106)'> [PAD]</span><span style='background-color:rgb(30.173025727272034,224.82697427272797,224.82697427272797)'> [PAD]</span><span style='background-color:rgb(31.902343332767487,223.0976566672325,223.0976566672325)'> [PAD]</span><span style='background-color:rgb(32.589772045612335,222.41022795438766,222.41022795438766)'> [PAD]</span><span style='background-color:rgb(43.19874197244644,211.80125802755356,211.80125802755356)'> [PAD]</span><span style='background-color:rgb(42.84704804420471,212.1529519557953,212.1529519557953)'> [PAD]</span><span style='background-color:rgb(43.96434009075165,211.03565990924835,211.03565990924835)'> [PAD]</span><span style='background-color:rgb(54.342981576919556,200.65701842308044,200.65701842308044)'> [PAD]</span><span style='background-color:rgb(55.34396946430206,199.65603053569794,199.65603053569794)'> [PAD]</span><span style='background-color:rgb(61.27978295087814,193.72021704912186,193.72021704912186)'> [PAD]</span><span style='background-color:rgb(66.81255966424942,188.18744033575058,188.18744033575058)'> [PAD]</span><span style='background-color:rgb(68.08172196149826,186.91827803850174,186.91827803850174)'> [PAD]</span><span style='background-color:rgb(76.06925815343857,178.93074184656143,178.93074184656143)'> [PAD]</span><span style='background-color:rgb(77.48171836137772,177.51828163862228,177.51828163862228)'> [PAD]</span><span style='background-color:rgb(81.79343819618225,173.20656180381775,173.20656180381775)'> [PAD]</span><span style='background-color:rgb(81.29429697990417,173.70570302009583,173.70570302009583)'> [PAD]</span><span style='background-color:rgb(84.65708613395691,170.3429138660431,170.3429138660431)'> [PAD]</span><span style='background-color:rgb(86.36393934488297,168.63606065511703,168.63606065511703)'> [PAD]</span><span style='background-color:rgb(87.05095767974854,167.94904232025146,167.94904232025146)'> [PAD]</span><span style='background-color:rgb(98.29159051179886,156.70840948820114,156.70840948820114)'> [PAD]</span><span style='background-color:rgb(108.88153105974197,146.11846894025803,146.11846894025803)'> [PAD]</span><span style='background-color:rgb(110.56595027446747,144.43404972553253,144.43404972553253)'> [PAD]</span><span style='background-color:rgb(115.39892613887787,139.60107386112213,139.60107386112213)'> [PAD]</span><span style='background-color:rgb(119.44437146186829,135.5556285381317,135.5556285381317)'> [PAD]</span><span style='background-color:rgb(119.0378388762474,135.9621611237526,135.9621611237526)'> [PAD]</span><span style='background-color:rgb(118.65171879529953,136.34828120470047,136.34828120470047)'> [PAD]</span><span style='background-color:rgb(122.18802183866501,132.811978161335,132.811978161335)'> [PAD]</span><span style='background-color:rgb(127.87712216377258,127.12287783622742,127.12287783622742)'> [PAD]</span><span style='background-color:rgb(132.09413588047028,122.90586411952972,122.90586411952972)'> [PAD]</span><span style='background-color:rgb(131.70822098851204,123.29177901148796,123.29177901148796)'> [PAD]</span><span style='background-color:rgb(142.78557658195496,112.21442341804504,112.21442341804504)'> [PAD]</span><span style='background-color:rgb(145.7580202817917,109.24197971820831,109.24197971820831)'> [PAD]</span><span style='background-color:rgb(145.38764655590057,109.61235344409943,109.61235344409943)'> [PAD]</span><span style='background-color:rgb(147.46017694473267,107.53982305526733,107.53982305526733)'> [PAD]</span><span style='background-color:rgb(150.60817122459412,104.39182877540588,104.39182877540588)'> [PAD]</span><span style='background-color:rgb(149.59796503186226,105.40203496813774,105.40203496813774)'> [PAD]</span><span style='background-color:rgb(151.00825935602188,103.99174064397812,103.99174064397812)'> [PAD]</span><span style='background-color:rgb(150.9089858829975,104.09101411700249,104.09101411700249)'> [PAD]</span><span style='background-color:rgb(156.66658133268356,98.33341866731644,98.33341866731644)'> [PAD]</span><span style='background-color:rgb(160.8556665480137,94.14433345198631,94.14433345198631)'> [PAD]</span><span style='background-color:rgb(163.86039331555367,91.13960668444633,91.13960668444633)'> [PAD]</span><span style='background-color:rgb(165.89181751012802,89.10818248987198,89.10818248987198)'> [PAD]</span><span style='background-color:rgb(174.29807603359222,80.70192396640778,80.70192396640778)'> [PAD]</span><span style='background-color:rgb(181.31078213453293,73.68921786546707,73.68921786546707)'> [PAD]</span><span style='background-color:rgb(185.17715066671371,69.82284933328629,69.82284933328629)'> [PAD]</span><span style='background-color:rgb(192.46727228164673,62.53272771835327,62.53272771835327)'> [PAD]</span><span style='background-color:rgb(192.46727228164673,62.53272771835327,62.53272771835327)'> [PAD]</span><span style='background-color:rgb(192.56711192429066,62.43288807570934,62.43288807570934)'> [PAD]</span><span style='background-color:rgb(193.34369525313377,61.656304746866226,61.656304746866226)'> [PAD]</span><span style='background-color:rgb(195.52235014736652,59.477649852633476,59.477649852633476)'> [PAD]</span><span style='background-color:rgb(199.15968976914883,55.84031023085117,55.84031023085117)'> [PAD]</span><span style='background-color:rgb(206.67867690324783,48.32132309675217,48.32132309675217)'> [PAD]</span><span style='background-color:rgb(212.99743384122849,42.002566158771515,42.002566158771515)'> [PAD]</span><span style='background-color:rgb(211.413184851408,43.586815148591995,43.586815148591995)'> [PAD]</span><span style='background-color:rgb(222.15484745800495,32.84515254199505,32.84515254199505)'> [PAD]</span><span style='background-color:rgb(226.26409325748682,28.73590674251318,28.73590674251318)'> [PAD]</span><span style='background-color:rgb(231.33013151586056,23.669868484139442,23.669868484139442)'> [PAD]</span><span style='background-color:rgb(237.52658251672983,17.47341748327017,17.47341748327017)'> [PAD]</span><span style='background-color:rgb(234.2818435281515,20.718156471848488,20.718156471848488)'> [PAD]</span><span style='background-color:rgb(238.55825375765562,16.44174624234438,16.44174624234438)'> [PAD]</span><span style='background-color:rgb(240.70918913930655,14.290810860693455,14.290810860693455)'> [PAD]</span><span style='background-color:rgb(243.1184745579958,11.881525442004204,11.881525442004204)'> [PAD]</span><span style='background-color:rgb(241.49109598249197,13.50890401750803,13.50890401750803)'> [PAD]</span><span style='background-color:rgb(239.1206482797861,15.87935172021389,15.87935172021389)'> [PAD]</span><span style='background-color:rgb(236.94165900349617,18.05834099650383,18.05834099650383)'> [PAD]</span><span style='background-color:rgb(236.94165900349617,18.05834099650383,18.05834099650383)'> [PAD]</span><span style='background-color:rgb(236.94165900349617,18.05834099650383,18.05834099650383)'> [PAD]</span><span style='background-color:rgb(236.94165900349617,18.05834099650383,18.05834099650383)'> [PAD]</span><span style='background-color:rgb(236.94165900349617,18.05834099650383,18.05834099650383)'> [PAD]</span><span style='background-color:rgb(236.94165900349617,18.05834099650383,18.05834099650383)'> [PAD]</span><span style='background-color:rgb(236.94165900349617,18.05834099650383,18.05834099650383)'> [PAD]</span><span style='background-color:rgb(236.94165900349617,18.05834099650383,18.05834099650383)'> [PAD]</span><span style='background-color:rgb(236.94165900349617,18.05834099650383,18.05834099650383)'> [PAD]</span><span style='background-color:rgb(245.8903089724481,9.10969102755189,9.10969102755189)'> [PAD]</span><span style='background-color:rgb(246.3399511948228,8.660048805177212,8.660048805177212)'> [PAD]</span> [PAD]<span style='background-color:rgb(248.46189935691655,6.538100643083453,6.538100643083453)'> [PAD]</span><span style='background-color:rgb(248.68643785826862,6.313562141731381,6.313562141731381)'> [PAD]</span><span style='background-color:rgb(248.68643785826862,6.313562141731381,6.313562141731381)'> [PAD]</span><span style='background-color:rgb(248.32894686609507,6.671053133904934,6.671053133904934)'> [PAD]</span><span style='background-color:rgb(248.68084693327546,6.319153066724539,6.319153066724539)'> [PAD]</span><span style='background-color:rgb(241.96260318160057,13.03739681839943,13.03739681839943)'> [PAD]</span><span style='background-color:rgb(243.6968757212162,11.303124278783798,11.303124278783798)'> [PAD]</span><span style='background-color:rgb(243.92524156719446,11.074758432805538,11.074758432805538)'> [PAD]</span><span style='background-color:rgb(243.14786028116941,11.852139718830585,11.852139718830585)'> [PAD]</span><span style='background-color:rgb(245.51907364279032,9.480926357209682,9.480926357209682)'> [PAD]</span><span style='background-color:rgb(243.37163312360644,11.628366876393557,11.628366876393557)'> [PAD]</span><span style='background-color:rgb(243.6532132141292,11.34678678587079,11.34678678587079)'> [PAD]</span><span style='background-color:rgb(242.38465603441,12.61534396559,12.61534396559)'> [PAD]</span><span style='background-color:rgb(246.19558364152908,8.804416358470917,8.804416358470917)'> [PAD]</span><span style='background-color:rgb(248.3747396338731,6.625260366126895,6.625260366126895)'> [PAD]</span><span style='background-color:rgb(244.70112478360534,10.298875216394663,10.298875216394663)'> [PAD]</span><span style='background-color:rgb(240.34427667036653,14.655723329633474,14.655723329633474)'> [PAD]</span><span style='background-color:rgb(240.16889233142138,14.831107668578625,14.831107668578625)'> [PAD]</span><span style='background-color:rgb(235.80146178603172,19.198538213968277,19.198538213968277)'> [PAD]</span><span style='background-color:rgb(235.97890846431255,19.021091535687447,19.021091535687447)'> [PAD]</span><span style='background-color:rgb(239.95377257466316,15.046227425336838,15.046227425336838)'> [PAD]</span><span style='background-color:rgb(242.46070040389895,12.539299596101046,12.539299596101046)'> [PAD]</span><span style='background-color:rgb(238.49142864346504,16.508571356534958,16.508571356534958)'> [PAD]</span><span style='background-color:rgb(234.79115679860115,20.20884320139885,20.20884320139885)'> [PAD]</span><span style='background-color:rgb(238.99168889969587,16.008311100304127,16.008311100304127)'> [PAD]</span><span style='background-color:rgb(240.51986049860716,14.480139501392841,14.480139501392841)'> [PAD]</span><span style='background-color:rgb(232.5617480278015,22.438251972198486,22.438251972198486)'> [PAD]</span><span style='background-color:rgb(231.38510886579752,23.61489113420248,23.61489113420248)'> [PAD]</span><span style='background-color:rgb(231.36873554438353,23.631264455616474,23.631264455616474)'> [PAD]</span><span style='background-color:rgb(238.75077802687883,16.249221973121166,16.249221973121166)'> [PAD]</span><span style='background-color:rgb(244.31727603077888,10.682723969221115,10.682723969221115)'> [PAD]</span><span style='background-color:rgb(246.9949296489358,8.005070351064205,8.005070351064205)'> [PAD]</span><span style='background-color:rgb(245.28208890929818,9.717911090701818,9.717911090701818)'> [PAD]</span><span style='background-color:rgb(247.557422965765,7.442577034235001,7.442577034235001)'> [PAD]</span><span style='background-color:rgb(249.73667917773128,5.263320822268724,5.263320822268724)'> [PAD]</span><span style='background-color:rgb(247.58893894962966,7.411061050370336,7.411061050370336)'> [PAD]</span><span style='background-color:rgb(249.92986793629825,5.070132063701749,5.070132063701749)'> [PAD]</span><span style='background-color:rgb(251.8863827129826,3.113617287017405,3.113617287017405)'> [PAD]</span><span style='background-color:rgb(245.53621262311935,9.463787376880646,9.463787376880646)'> [PAD]</span><span style='background-color:rgb(251.5135500440374,3.4864499559625983,3.4864499559625983)'> [PAD]</span><span style='background-color:rgb(249.58359251730144,5.41640748269856,5.41640748269856)'> [PAD]</span><span style='background-color:rgb(244.18535565957427,10.81464434042573,10.81464434042573)'> [PAD]</span><span style='background-color:rgb(244.21913489699364,10.780865103006363,10.780865103006363)'> [PAD]</span><span style='background-color:rgb(244.21913489699364,10.780865103006363,10.780865103006363)'> [PAD]</span><span style='background-color:rgb(247.64258589595556,7.357414104044437,7.357414104044437)'> [PAD]</span><span style='background-color:rgb(244.214309155941,10.78569084405899,10.78569084405899)'> [PAD]</span><span style='background-color:rgb(240.50634937360883,14.493650626391172,14.493650626391172)'> [PAD]</span><span style='background-color:rgb(236.17559161037207,18.824408389627934,18.824408389627934)'> [PAD]</span><span style='background-color:rgb(235.51625479012728,19.483745209872723,19.483745209872723)'> [PAD]</span><span style='background-color:rgb(243.42963986098766,11.570360139012337,11.570360139012337)'> [PAD]</span><span style='background-color:rgb(242.73203054443002,12.267969455569983,12.267969455569983)'> [PAD]</span><span style='background-color:rgb(240.55353999137878,14.446460008621216,14.446460008621216)'> [PAD]</span><span style='background-color:rgb(244.83823662623763,10.16176337376237,10.16176337376237)'> [PAD]</span><span style='background-color:rgb(247.51046556048095,7.489534439519048,7.489534439519048)'> [PAD]</span><span style='background-color:rgb(247.51046556048095,7.489534439519048,7.489534439519048)'> [PAD]</span><span style='background-color:rgb(245.87057378143072,9.129426218569279,9.129426218569279)'> [PAD]</span><span style='background-color:rgb(245.62370482832193,9.376295171678066,9.376295171678066)'> [PAD]</span><span style='background-color:rgb(243.44514777883887,11.554852221161127,11.554852221161127)'> [PAD]</span><span style='background-color:rgb(238.9019339159131,16.098066084086895,16.098066084086895)'> [PAD]</span><span style='background-color:rgb(239.2381921224296,15.76180787757039,15.76180787757039)'> [PAD]</span><span style='background-color:rgb(239.24191972240806,15.758080277591944,15.758080277591944)'> [PAD]</span><span style='background-color:rgb(239.24191972240806,15.758080277591944,15.758080277591944)'> [PAD]</span><span style='background-color:rgb(239.24191972240806,15.758080277591944,15.758080277591944)'> [PAD]</span><span style='background-color:rgb(239.24191972240806,15.758080277591944,15.758080277591944)'> [PAD]</span><span style='background-color:rgb(239.24191972240806,15.758080277591944,15.758080277591944)'> [PAD]</span><span style='background-color:rgb(239.24191972240806,15.758080277591944,15.758080277591944)'> [PAD]</span><span style='background-color:rgb(239.24191972240806,15.758080277591944,15.758080277591944)'> [PAD]</span><span style='background-color:rgb(239.24191972240806,15.758080277591944,15.758080277591944)'> [PAD]</span>

In [87]:
rise_expln.attributions[0][:100]

tensor([143.8457, 143.8457, 143.8457, 143.8457, 143.8457, 143.8457, 143.8457,
        143.8457, 143.8457, 143.7377, 143.9612, 143.1321, 143.2646, 143.1600,
        142.5935, 144.5916, 144.7318, 143.5122, 145.8715, 145.9230, 142.8716,
        145.6129, 146.7980, 146.8977, 147.3217, 147.7874, 148.5657, 148.7686,
        150.7664, 151.6030, 151.4199, 151.7917, 149.9019, 150.1675, 149.8063,
        149.1363, 149.1290, 149.0445, 153.3457, 152.1779, 151.1791, 154.8050,
        155.1800, 155.5230, 156.8745, 156.8290, 155.1501, 154.0721, 155.3260,
        154.5323, 156.3637, 159.0486, 157.8299, 159.3602, 160.3593, 158.5566,
        157.3667, 157.4178, 156.3882, 153.9164, 154.0369, 153.3644, 150.9707,
        149.9720, 153.0543, 152.7269, 151.2212, 153.4665, 154.4657, 152.5734,
        149.5909, 149.6595, 149.8358, 149.8358, 149.8358, 149.8358, 149.8358,
        149.8358, 149.8358, 149.8358, 149.8358, 148.8377, 150.0430, 150.4264,
        150.4249, 150.6325, 150.5783, 150.6839, 148.8859, 149.30

In [89]:
rise_expln.explainer_output.shape

torch.Size([2, 2, 512])

In [90]:
rise_expln.explainer_output

tensor([[[143.8457, 143.8457, 143.8457,  ..., 143.6499, 143.6499, 143.6499],
         [108.1543, 108.1543, 108.1543,  ..., 105.3501, 105.3501, 105.3501]],

        [[143.6210, 143.6210, 143.6210,  ..., 143.3065, 143.3065, 143.3065],
         [108.3790, 108.3790, 108.3790,  ..., 105.6935, 105.6935, 105.6935]]],
       device='cuda:0')

In [92]:
j = 0
grouped_attrs_aggr = rise_expln.explainer_output[j].mean(-1)
aggr_pred = torch.argmax(grouped_attrs_aggr)
aggr_pred

tensor(0, device='cuda:0')

## IntGrad

In [ ]:
from exlib.explainers.intgrad import IntGradTextCls

intgrad_explainer = IntGradTextCls(original_model2, projection_layer=projection_layer).to(device)
intgrad_expln = intgrad_explainer(inputs, labels, x_kwargs=kwargs)

In [21]:
intgrad_expln.attributions.shape

torch.Size([2, 512])

In [22]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

def show_attrs(tokens, masks, normalize=True):
    if normalize:
        masks_max = masks.max(-1).values
        masks_min = masks.min(-1).values
        # import pdb; pdb.set_trace()
        masks = (masks - masks_min) / (masks_max - masks_min)
    print_str = ""
    for i in range(len(tokens)):
        expln_val = masks[i].item()
        if expln_val == 0:
            print_str += f" {tokens[i]}"
        else:
            print_str += f"<span style='background-color:rgb({255 * (1-expln_val)},{255 * expln_val},{255 * expln_val})'> {tokens[i]}</span>"
    return print_str

tokens = processor.convert_ids_to_tokens(inputs[0])
# tokens = shap_expln.explainer_output.data[0]
printmd(show_attrs(tokens, intgrad_expln.attributions[0]))

<span style='background-color:rgb(144.50095653533936,110.49904346466064,110.49904346466064)'> [CLS]</span><span style='background-color:rgb(132.0989692211151,122.90103077888489,122.90103077888489)'> allan</span><span style='background-color:rgb(134.04495880007744,120.95504119992256,120.95504119992256)'> sat</span><span style='background-color:rgb(136.14323660731316,118.85676339268684,118.85676339268684)'> down</span><span style='background-color:rgb(137.06120654940605,117.93879345059395,117.93879345059395)'> at</span><span style='background-color:rgb(130.0081466138363,124.99185338616371,124.99185338616371)'> his</span><span style='background-color:rgb(133.96893247961998,121.03106752038002,121.03106752038002)'> desk</span><span style='background-color:rgb(129.3670678138733,125.63293218612671,125.63293218612671)'> and</span><span style='background-color:rgb(132.7328284084797,122.26717159152031,122.26717159152031)'> pulled</span><span style='background-color:rgb(134.1080506145954,120.89194938540459,120.89194938540459)'> the</span><span style='background-color:rgb(133.95312532782555,121.04687467217445,121.04687467217445)'> chair</span><span style='background-color:rgb(133.92766669392586,121.07233330607414,121.07233330607414)'> in</span><span style='background-color:rgb(133.9499182999134,121.0500817000866,121.0500817000866)'> close</span><span style='background-color:rgb(131.36246994137764,123.63753005862236,123.63753005862236)'> .</span><span style='background-color:rgb(132.32935085892677,122.67064914107323,122.67064914107323)'> opening</span><span style='background-color:rgb(134.33515682816505,120.66484317183495,120.66484317183495)'> a</span><span style='background-color:rgb(132.4335716664791,122.56642833352089,122.56642833352089)'> side</span><span style='background-color:rgb(128.74072462320328,126.25927537679672,126.25927537679672)'> drawer</span><span style='background-color:rgb(129.97862979769707,125.02137020230293,125.02137020230293)'> ,</span><span style='background-color:rgb(132.79221922159195,122.20778077840805,122.20778077840805)'> he</span><span style='background-color:rgb(131.6463679075241,123.35363209247589,123.35363209247589)'> took</span><span style='background-color:rgb(130.3556987643242,124.64430123567581,124.64430123567581)'> out</span><span style='background-color:rgb(129.25416067242622,125.74583932757378,125.74583932757378)'> a</span><span style='background-color:rgb(134.20214876532555,120.79785123467445,120.79785123467445)'> piece</span><span style='background-color:rgb(129.80765417218208,125.19234582781792,125.19234582781792)'> of</span><span style='background-color:rgb(132.20857053995132,122.79142946004868,122.79142946004868)'> paper</span><span style='background-color:rgb(135.06886705756187,119.93113294243813,119.93113294243813)'> and</span><span style='background-color:rgb(133.08496311306953,121.91503688693047,121.91503688693047)'> his</span><span style='background-color:rgb(133.75678226351738,121.24321773648262,121.24321773648262)'> ink</span><span style='background-color:rgb(133.7756596505642,121.2243403494358,121.2243403494358)'> ##pot</span><span style='background-color:rgb(134.6235689520836,120.37643104791641,120.37643104791641)'> .</span><span style='background-color:rgb(135.5117864906788,119.48821350932121,119.48821350932121)'> after</span><span style='background-color:rgb(132.23716020584106,122.76283979415894,122.76283979415894)'> filling</span><span style='background-color:rgb(134.2694203555584,120.7305796444416,120.7305796444416)'> his</span><span style='background-color:rgb(130.6332890689373,124.3667109310627,124.3667109310627)'> pen</span><span style='background-color:rgb(130.93391373753548,124.06608626246452,124.06608626246452)'> ,</span><span style='background-color:rgb(133.3642253279686,121.6357746720314,121.6357746720314)'> allan</span><span style='background-color:rgb(130.6434117257595,124.3565882742405,124.3565882742405)'> looked</span><span style='background-color:rgb(132.90014103055,122.09985896945,122.09985896945)'> at</span><span style='background-color:rgb(135.57951405644417,119.42048594355583,119.42048594355583)'> his</span><span style='background-color:rgb(133.40020939707756,121.59979060292244,121.59979060292244)'> paper</span><span style='background-color:rgb(131.76960289478302,123.23039710521698,123.23039710521698)'> in</span><span style='background-color:rgb(133.53413701057434,121.46586298942566,121.46586298942566)'> the</span><span style='background-color:rgb(132.54353016614914,122.45646983385086,122.45646983385086)'> orange</span><span style='background-color:rgb(131.68466985225677,123.31533014774323,123.31533014774323)'> glow</span><span style='background-color:rgb(131.87836065888405,123.12163934111595,123.12163934111595)'> from</span><span style='background-color:rgb(136.04892566800117,118.95107433199883,118.95107433199883)'> the</span><span style='background-color:rgb(129.82375010848045,125.17624989151955,125.17624989151955)'> lantern</span><span style='background-color:rgb(132.27803081274033,122.72196918725967,122.72196918725967)'> set</span><span style='background-color:rgb(131.9969978928566,123.0030021071434,123.0030021071434)'> back</span><span style='background-color:rgb(131.11679032444954,123.88320967555046,123.88320967555046)'> in</span><span style='background-color:rgb(130.8625991642475,124.13740083575249,124.13740083575249)'> the</span><span style='background-color:rgb(131.66409015655518,123.33590984344482,123.33590984344482)'> desk</span><span style='background-color:rgb(133.6851941049099,121.3148058950901,121.3148058950901)'> '</span><span style='background-color:rgb(135.07064536213875,119.92935463786125,119.92935463786125)'> s</span><span style='background-color:rgb(128.94370213150978,126.05629786849022,126.05629786849022)'> right</span><span style='background-color:rgb(130.16802683472633,124.83197316527367,124.83197316527367)'> -</span><span style='background-color:rgb(132.1432976424694,122.8567023575306,122.8567023575306)'> hand</span><span style='background-color:rgb(131.40995979309082,123.59004020690918,123.59004020690918)'> corner</span><span style='background-color:rgb(136.63932278752327,118.36067721247673,118.36067721247673)'> .</span><span style='background-color:rgb(130.72190031409264,124.27809968590736,124.27809968590736)'> his</span><span style='background-color:rgb(132.33286947011948,122.66713052988052,122.66713052988052)'> pen</span><span style='background-color:rgb(128.76153230667114,126.23846769332886,126.23846769332886)'> cast</span><span style='background-color:rgb(126.10863626003265,128.89136373996735,128.89136373996735)'> a</span><span style='background-color:rgb(133.6700101196766,121.32998988032341,121.32998988032341)'> forbid</span><span style='background-color:rgb(131.6218440234661,123.37815597653389,123.37815597653389)'> ##ding</span><span style='background-color:rgb(134.3740439414978,120.6259560585022,120.6259560585022)'> line</span><span style='background-color:rgb(132.88569420576096,122.11430579423904,122.11430579423904)'> of</span><span style='background-color:rgb(133.07834386825562,121.92165613174438,121.92165613174438)'> shadow</span><span style='background-color:rgb(132.71240070462227,122.28759929537773,122.28759929537773)'> slant</span><span style='background-color:rgb(132.42502212524414,122.57497787475586,122.57497787475586)'> ##ing</span><span style='background-color:rgb(131.60954028367996,123.39045971632004,123.39045971632004)'> across</span><span style='background-color:rgb(141.18972301483154,113.81027698516846,113.81027698516846)'> the</span><span style='background-color:rgb(136.10852167010307,118.89147832989693,118.89147832989693)'> page</span><span style='background-color:rgb(132.42572888731956,122.57427111268044,122.57427111268044)'> ,</span><span style='background-color:rgb(132.40804463624954,122.59195536375046,122.59195536375046)'> echoing</span><span style='background-color:rgb(131.8108230829239,123.18917691707611,123.18917691707611)'> the</span><span style='background-color:rgb(130.5015653371811,124.49843466281891,124.49843466281891)'> ink</span><span style='background-color:rgb(133.0512437224388,121.94875627756119,121.94875627756119)'> ##y</span><span style='background-color:rgb(135.38370296359062,119.61629703640938,119.61629703640938)'> darkness</span><span style='background-color:rgb(131.84945940971375,123.15054059028625,123.15054059028625)'> crouch</span><span style='background-color:rgb(132.49707385897636,122.50292614102364,122.50292614102364)'> ##ing</span><span style='background-color:rgb(134.95611190795898,120.04388809204102,120.04388809204102)'> in</span><span style='background-color:rgb(130.02491131424904,124.97508868575096,124.97508868575096)'> the</span><span style='background-color:rgb(132.95976743102074,122.04023256897926,122.04023256897926)'> edges</span><span style='background-color:rgb(132.78088063001633,122.21911936998367,122.21911936998367)'> of</span><span style='background-color:rgb(131.76053658127785,123.23946341872215,123.23946341872215)'> the</span><span style='background-color:rgb(132.2156685590744,122.7843314409256,122.7843314409256)'> lantern</span><span style='background-color:rgb(131.87594398856163,123.12405601143837,123.12405601143837)'> '</span><span style='background-color:rgb(129.80236485600471,125.19763514399529,125.19763514399529)'> s</span><span style='background-color:rgb(135.6096464395523,119.39035356044769,119.39035356044769)'> struggling</span><span style='background-color:rgb(132.6569464802742,122.3430535197258,122.3430535197258)'> glow</span><span style='background-color:rgb(132.46335446834564,122.53664553165436,122.53664553165436)'> .</span><span style='background-color:rgb(134.61318790912628,120.38681209087372,120.38681209087372)'> the</span><span style='background-color:rgb(132.7177508175373,122.28224918246269,122.28224918246269)'> only</span><span style='background-color:rgb(134.28724139928818,120.71275860071182,120.71275860071182)'> other</span><span style='background-color:rgb(130.28572171926498,124.71427828073502,124.71427828073502)'> illumination</span><span style='background-color:rgb(131.0872583091259,123.9127416908741,123.9127416908741)'> came</span><span style='background-color:rgb(133.99908006191254,121.00091993808746,121.00091993808746)'> from</span><span style='background-color:rgb(130.3141137957573,124.6858862042427,124.6858862042427)'> a</span><span style='background-color:rgb(132.40248173475266,122.59751826524734,122.59751826524734)'> lu</span><span style='background-color:rgb(126.8193045258522,128.1806954741478,128.1806954741478)'> ##rid</span><span style='background-color:rgb(128.57219606637955,126.42780393362045,126.42780393362045)'> moonlight</span><span style='background-color:rgb(130.96448689699173,124.03551310300827,124.03551310300827)'> filtered</span><span style='background-color:rgb(129.19280156493187,125.80719843506813,125.80719843506813)'> through</span><span style='background-color:rgb(137.8474147617817,117.15258523821831,117.15258523821831)'> thin</span><span style='background-color:rgb(133.92716512084007,121.07283487915993,121.07283487915993)'> branches</span><span style='background-color:rgb(137.64791786670685,117.35208213329315,117.35208213329315)'> and</span><span style='background-color:rgb(133.43407317996025,121.56592682003975,121.56592682003975)'> clouds</span><span style='background-color:rgb(137.60823279619217,117.39176720380783,117.39176720380783)'> ,</span><span style='background-color:rgb(132.9293918609619,122.07060813903809,122.07060813903809)'> casting</span><span style='background-color:rgb(130.53174331784248,124.46825668215752,124.46825668215752)'> its</span><span style='background-color:rgb(133.6068271100521,121.39317288994789,121.39317288994789)'> bone</span><span style='background-color:rgb(132.138152718544,122.861847281456,122.861847281456)'> -</span><span style='background-color:rgb(134.43108648061752,120.56891351938248,120.56891351938248)'> pale</span><span style='background-color:rgb(129.45682659745216,125.54317340254784,125.54317340254784)'> glow</span><span style='background-color:rgb(134.9840708076954,120.01592919230461,120.01592919230461)'> onto</span><span style='background-color:rgb(131.62857726216316,123.37142273783684,123.37142273783684)'> the</span><span style='background-color:rgb(133.55128929018974,121.44871070981026,121.44871070981026)'> pine</span><span style='background-color:rgb(131.5256331861019,123.47436681389809,123.47436681389809)'> floor</span><span style='background-color:rgb(132.15037286281586,122.84962713718414,122.84962713718414)'> ##boards</span><span style='background-color:rgb(135.9889116883278,119.01108831167221,119.01108831167221)'> .</span><span style='background-color:rgb(129.24618110060692,125.75381889939308,125.75381889939308)'> allan</span><span style='background-color:rgb(131.47058174014091,123.52941825985909,123.52941825985909)'> unfolded</span><span style='background-color:rgb(128.6462540924549,126.35374590754509,126.35374590754509)'> another</span><span style='background-color:rgb(131.8999207019806,123.10007929801941,123.10007929801941)'> page</span><span style='background-color:rgb(131.53143927454948,123.46856072545052,123.46856072545052)'> ,</span><span style='background-color:rgb(131.39980673789978,123.60019326210022,123.60019326210022)'> this</span><span style='background-color:rgb(136.1343678832054,118.86563211679459,118.86563211679459)'> one</span><span style='background-color:rgb(131.1370812356472,123.8629187643528,123.8629187643528)'> crowded</span><span style='background-color:rgb(131.82556629180908,123.17443370819092,123.17443370819092)'> with</span><span style='background-color:rgb(132.96952530741692,122.03047469258308,122.03047469258308)'> ranks</span><span style='background-color:rgb(139.12917718291283,115.87082281708717,115.87082281708717)'> of</span><span style='background-color:rgb(138.56072008609772,116.43927991390228,116.43927991390228)'> letters</span><span style='background-color:rgb(128.0450503528118,126.95494964718819,126.95494964718819)'> in</span><span style='background-color:rgb(130.51590576767921,124.48409423232079,124.48409423232079)'> tight</span><span style='background-color:rgb(131.5129266679287,123.4870733320713,123.4870733320713)'> formation</span><span style='background-color:rgb(129.54650178551674,125.45349821448326,125.45349821448326)'> from</span><span style='background-color:rgb(137.9438155889511,117.05618441104889,117.05618441104889)'> left</span><span style='background-color:rgb(136.49572849273682,118.50427150726318,118.50427150726318)'> to</span><span style='background-color:rgb(131.42978712916374,123.57021287083626,123.57021287083626)'> right</span><span style='background-color:rgb(130.02261623740196,124.97738376259804,124.97738376259804)'> .</span><span style='background-color:rgb(129.44893062114716,125.55106937885284,125.55106937885284)'> the</span><span style='background-color:rgb(127.09307223558426,127.90692776441574,127.90692776441574)'> lines</span><span style='background-color:rgb(125.79323798418045,129.20676201581955,129.20676201581955)'> of</span><span style='background-color:rgb(132.68273949623108,122.31726050376892,122.31726050376892)'> letters</span><span style='background-color:rgb(132.40582555532455,122.59417444467545,122.59417444467545)'> stepped</span><span style='background-color:rgb(130.1944962143898,124.8055037856102,124.8055037856102)'> into</span><span style='background-color:rgb(144.69441175460815,110.30558824539185,110.30558824539185)'> their</span><span style='background-color:rgb(125.81538319587708,129.18461680412292,129.18461680412292)'> divisions</span><span style='background-color:rgb(125.71724206209183,129.28275793790817,129.28275793790817)'> ,</span><span style='background-color:rgb(129.50560837984085,125.49439162015915,125.49439162015915)'> in</span><span style='background-color:rgb(134.69764977693558,120.30235022306442,120.30235022306442)'> the</span><span style='background-color:rgb(129.68232169747353,125.31767830252647,125.31767830252647)'> shape</span><span style='background-color:rgb(136.04170605540276,118.95829394459724,118.95829394459724)'> of</span><span style='background-color:rgb(136.11183509230614,118.88816490769386,118.88816490769386)'> a</span><span style='background-color:rgb(133.04527804255486,121.95472195744514,121.95472195744514)'> story</span><span style='background-color:rgb(132.5428006052971,122.45719939470291,122.45719939470291)'> '</span><span style='background-color:rgb(134.25273925065994,120.74726074934006,120.74726074934006)'> s</span><span style='background-color:rgb(128.6859391629696,126.31406083703041,126.31406083703041)'> outline</span><span style='background-color:rgb(132.9425923526287,122.05740764737129,122.05740764737129)'> :</span><span style='background-color:rgb(138.39187234640121,116.60812765359879,116.60812765359879)'> the</span><span style='background-color:rgb(132.04907029867172,122.95092970132828,122.95092970132828)'> loose</span><span style='background-color:rgb(133.3613982796669,121.6386017203331,121.6386017203331)'> ,</span><span style='background-color:rgb(134.5080779492855,120.49192205071449,120.49192205071449)'> dry</span><span style='background-color:rgb(132.22130745649338,122.77869254350662,122.77869254350662)'> skeleton</span><span style='background-color:rgb(126.33010357618332,128.66989642381668,128.66989642381668)'> of</span><span style='background-color:rgb(134.90585580468178,120.09414419531822,120.09414419531822)'> a</span><span style='background-color:rgb(125.69261938333511,129.3073806166649,129.3073806166649)'> tale</span><span style='background-color:rgb(135.72417229413986,119.27582770586014,119.27582770586014)'> lay</span><span style='background-color:rgb(131.8586777150631,123.1413222849369,123.1413222849369)'> exposed</span><span style='background-color:rgb(129.77450475096703,125.22549524903297,125.22549524903297)'> beneath</span><span style='background-color:rgb(130.80403670668602,124.19596329331398,124.19596329331398)'> their</span><span style='background-color:rgb(131.655533015728,123.344466984272,123.344466984272)'> feet</span><span style='background-color:rgb(137.13094800710678,117.86905199289322,117.86905199289322)'> ,</span><span style='background-color:rgb(128.24213057756424,126.75786942243576,126.75786942243576)'> awaiting</span><span style='background-color:rgb(133.78297805786133,121.21702194213867,121.21702194213867)'> tend</span><span style='background-color:rgb(128.78415629267693,126.21584370732307,126.21584370732307)'> ##ons</span><span style='background-color:rgb(134.38102796673775,120.61897203326225,120.61897203326225)'> ,</span><span style='background-color:rgb(131.24334633350372,123.75665366649628,123.75665366649628)'> muscle</span><span style='background-color:rgb(141.3388878107071,113.66111218929291,113.66111218929291)'> and</span><span style='background-color:rgb(130.1298388838768,124.8701611161232,124.8701611161232)'> blushing</span><span style='background-color:rgb(136.2596395611763,118.7403604388237,118.7403604388237)'> skin</span><span style='background-color:rgb(129.07530426979065,125.92469573020935,125.92469573020935)'> .</span><span style='background-color:rgb(128.2886552810669,126.7113447189331,126.7113447189331)'> allan</span><span style='background-color:rgb(130.6482830643654,124.35171693563461,124.35171693563461)'> reviewed</span><span style='background-color:rgb(131.84080347418785,123.15919652581215,123.15919652581215)'> the</span><span style='background-color:rgb(135.0533562898636,119.94664371013641,119.94664371013641)'> troops</span><span style='background-color:rgb(135.77743023633957,119.22256976366043,119.22256976366043)'> ,</span><span style='background-color:rgb(132.1288812160492,122.8711187839508,122.8711187839508)'> all</span><span style='background-color:rgb(133.67358952760696,121.32641047239304,121.32641047239304)'> prepared</span><span style='background-color:rgb(131.61178216338158,123.38821783661842,123.38821783661842)'> to</span><span style='background-color:rgb(133.71766716241837,121.28233283758163,121.28233283758163)'> di</span><span style='background-color:rgb(132.0082224905491,122.99177750945091,122.99177750945091)'> ##se</span><span style='background-color:rgb(133.40486034750938,121.59513965249062,121.59513965249062)'> ##mba</span><span style='background-color:rgb(133.2115875184536,121.7884124815464,121.7884124815464)'> ##rk</span><span style='background-color:rgb(131.06917887926102,123.93082112073898,123.93082112073898)'> ,</span><span style='background-color:rgb(132.19829589128494,122.80170410871506,122.80170410871506)'> their</span><span style='background-color:rgb(133.46827894449234,121.53172105550766,121.53172105550766)'> task</span><span style='background-color:rgb(132.85390511155128,122.14609488844872,122.14609488844872)'> to</span><span style='background-color:rgb(131.92148834466934,123.07851165533066,123.07851165533066)'> form</span><span style='background-color:rgb(133.96383315324783,121.03616684675217,121.03616684675217)'> the</span><span style='background-color:rgb(132.9094885289669,122.0905114710331,122.0905114710331)'> tale</span><span style='background-color:rgb(133.51079866290092,121.48920133709908,121.48920133709908)'> of</span><span style='background-color:rgb(133.58291879296303,121.41708120703697,121.41708120703697)'> a</span><span style='background-color:rgb(132.50656574964523,122.49343425035477,122.49343425035477)'> young</span><span style='background-color:rgb(133.0388107895851,121.96118921041489,121.96118921041489)'> man</span><span style='background-color:rgb(133.5865893959999,121.41341060400009,121.41341060400009)'> returning</span><span style='background-color:rgb(136.46173551678658,118.53826448321342,118.53826448321342)'> home</span><span style='background-color:rgb(135.4451684653759,119.5548315346241,119.5548315346241)'> from</span><span style='background-color:rgb(133.58034253120422,121.41965746879578,121.41965746879578)'> life</span><span style='background-color:rgb(135.02684131264687,119.97315868735313,119.97315868735313)'> abroad</span><span style='background-color:rgb(134.42861661314964,120.57138338685036,120.57138338685036)'> to</span><span style='background-color:rgb(129.8851092159748,125.11489078402519,125.11489078402519)'> find</span><span style='background-color:rgb(130.72065398097038,124.27934601902962,124.27934601902962)'> his</span><span style='background-color:rgb(131.9753086566925,123.0246913433075,123.0246913433075)'> childhood</span><span style='background-color:rgb(131.49563759565353,123.50436240434647,123.50436240434647)'> friend</span><span style='background-color:rgb(131.19533210992813,123.80466789007187,123.80466789007187)'> a</span><span style='background-color:rgb(131.92603290081024,123.07396709918976,123.07396709918976)'> bride</span><span style='background-color:rgb(132.37547278404236,122.62452721595764,122.62452721595764)'> to</span><span style='background-color:rgb(135.66532105207443,119.33467894792557,119.33467894792557)'> -</span><span style='background-color:rgb(132.4174529314041,122.58254706859589,122.58254706859589)'> be</span><span style='background-color:rgb(131.2878343462944,123.7121656537056,123.7121656537056)'> ,</span><span style='background-color:rgb(132.26261124014854,122.73738875985146,122.73738875985146)'> thus</span><span style='background-color:rgb(133.01875546574593,121.98124453425407,121.98124453425407)'> upset</span><span style='background-color:rgb(132.55157813429832,122.44842186570168,122.44842186570168)'> ##ting</span><span style='background-color:rgb(127.3177769780159,127.6822230219841,127.6822230219841)'> the</span><span style='background-color:rgb(134.24294337630272,120.75705662369728,120.75705662369728)'> apple</span><span style='background-color:rgb(129.43829119205475,125.56170880794525,125.56170880794525)'> cart</span><span style='background-color:rgb(128.35900470614433,126.64099529385567,126.64099529385567)'> of</span><span style='background-color:rgb(135.8044771850109,119.19552281498909,119.19552281498909)'> his</span><span style='background-color:rgb(124.5052894949913,130.4947105050087,130.4947105050087)'> life</span><span style='background-color:rgb(133.37592110037804,121.62407889962196,121.62407889962196)'> '</span><span style='background-color:rgb(128.28543305397034,126.71456694602966,126.71456694602966)'> s</span><span style='background-color:rgb(134.20111522078514,120.79888477921486,120.79888477921486)'> plan</span><span style='background-color:rgb(127.60661467909813,127.39338532090187,127.39338532090187)'> ,</span><span style='background-color:rgb(137.63917833566666,117.36082166433334,117.36082166433334)'> clarified</span><span style='background-color:rgb(132.17766299843788,122.82233700156212,122.82233700156212)'> –</span><span style='background-color:rgb(132.56169319152832,122.43830680847168,122.43830680847168)'> of</span><span style='background-color:rgb(131.3668929040432,123.6331070959568,123.6331070959568)'> course</span><span style='background-color:rgb(128.2727113366127,126.7272886633873,126.7272886633873)'> –</span><span style='background-color:rgb(134.51582953333855,120.48417046666145,120.48417046666145)'> by</span><span style='background-color:rgb(137.71594181656837,117.28405818343163,117.28405818343163)'> his</span><span style='background-color:rgb(135.08615612983704,119.91384387016296,119.91384387016296)'> very</span><span style='background-color:rgb(130.70252135396004,124.29747864603996,124.29747864603996)'> time</span><span style='background-color:rgb(133.48500564694405,121.51499435305595,121.51499435305595)'> away</span><span style='background-color:rgb(138.2489924132824,116.7510075867176,116.7510075867176)'> from</span><span style='background-color:rgb(130.68682059645653,124.31317940354347,124.31317940354347)'> her</span><span style='background-color:rgb(122.33151733875275,132.66848266124725,132.66848266124725)'> he</span><span style='background-color:rgb(126.65910512208939,128.3408948779106,128.3408948779106)'> loved</span><span style='background-color:rgb(141.84273317456245,113.15726682543755,113.15726682543755)'> best</span><span style='background-color:rgb(130.60313388705254,124.39686611294746,124.39686611294746)'> .</span><span style='background-color:rgb(133.5105250775814,121.4894749224186,121.4894749224186)'> although</span><span style='background-color:rgb(127.26514220237732,127.73485779762268,127.73485779762268)'> the</span><span style='background-color:rgb(136.1700403690338,118.82995963096619,118.82995963096619)'> concept</span><span style='background-color:rgb(128.54692742228508,126.45307257771492,126.45307257771492)'> was</span><span style='background-color:rgb(131.86535775661469,123.13464224338531,123.13464224338531)'> a</span><span style='background-color:rgb(135.1399004459381,119.86009955406189,119.86009955406189)'> simple</span><span style='background-color:rgb(132.67093732953072,122.32906267046928,122.32906267046928)'> one</span><span style='background-color:rgb(137.09942489862442,117.90057510137558,117.90057510137558)'> ,</span><span style='background-color:rgb(129.07910406589508,125.92089593410492,125.92089593410492)'> allan</span><span style='background-color:rgb(123.67196381092072,131.32803618907928,131.32803618907928)'> thought</span><span style='background-color:rgb(125.12926161289215,129.87073838710785,129.87073838710785)'> it</span><span style='background-color:rgb(127.42018908262253,127.57981091737747,127.57981091737747)'> had</span><span style='background-color:rgb(138.38059455156326,116.61940544843674,116.61940544843674)'> potential</span><span style='background-color:rgb(140.4421891272068,114.5578108727932,114.5578108727932)'> .</span><span style='background-color:rgb(130.11476129293442,124.88523870706558,124.88523870706558)'> besides</span><span style='background-color:rgb(129.99335780739784,125.00664219260216,125.00664219260216)'> ,</span><span style='background-color:rgb(138.73054057359695,116.26945942640305,116.26945942640305)'> the</span><span style='background-color:rgb(132.44841367006302,122.55158632993698,122.55158632993698)'> public</span><span style='background-color:rgb(137.6709370315075,117.32906296849251,117.32906296849251)'> liked</span><span style='background-color:rgb(134.76598531007767,120.23401468992233,120.23401468992233)'> a</span><span style='background-color:rgb(128.74347567558289,126.25652432441711,126.25652432441711)'> good</span><span style='background-color:rgb(131.79861053824425,123.20138946175575,123.20138946175575)'> ,</span><span style='background-color:rgb(143.91147136688232,111.08852863311768,111.08852863311768)'> simple</span><span style='background-color:rgb(143.83301317691803,111.16698682308197,111.16698682308197)'> romance</span><span style='background-color:rgb(124.36565458774567,130.63434541225433,130.63434541225433)'> .</span><span style='background-color:rgb(132.82302036881447,122.17697963118553,122.17697963118553)'> perhaps</span><span style='background-color:rgb(142.2997422516346,112.7002577483654,112.7002577483654)'> this</span><span style='background-color:rgb(129.62633550167084,125.37366449832916,125.37366449832916)'> will</span><span style='background-color:rgb(133.4459513425827,121.5540486574173,121.5540486574173)'> be</span><span style='background-color:rgb(133.47505018115044,121.52494981884956,121.52494981884956)'> more</span><span style='background-color:rgb(138.1804744899273,116.81952551007271,116.81952551007271)'> sale</span><span style='background-color:rgb(132.8973975777626,122.1026024222374,122.1026024222374)'> ##able</span><span style='background-color:rgb(132.0041263103485,122.99587368965149,122.99587368965149)'> ,</span><span style='background-color:rgb(132.22694635391235,122.77305364608765,122.77305364608765)'> he</span><span style='background-color:rgb(130.74608221650124,124.25391778349876,124.25391778349876)'> thought</span><span style='background-color:rgb(125.75524002313614,129.24475997686386,129.24475997686386)'> and</span><span style='background-color:rgb(138.88747975230217,116.11252024769783,116.11252024769783)'> began</span><span style='background-color:rgb(132.26346239447594,122.73653760552406,122.73653760552406)'> to</span><span style='background-color:rgb(136.53392404317856,118.46607595682144,118.46607595682144)'> write</span><span style='background-color:rgb(126.23921245336533,128.76078754663467,128.76078754663467)'> .</span><span style='background-color:rgb(128.23273748159409,126.76726251840591,126.76726251840591)'> they</span><span style='background-color:rgb(132.40609154105186,122.59390845894814,122.59390845894814)'> gazed</span><span style='background-color:rgb(132.83840954303741,122.16159045696259,122.16159045696259)'> at</span><span style='background-color:rgb(127.86185458302498,127.13814541697502,127.13814541697502)'> each</span><span style='background-color:rgb(137.89215356111526,117.10784643888474,117.10784643888474)'> other</span><span style='background-color:rgb(142.4053005874157,112.5946994125843,112.5946994125843)'> ,</span><span style='background-color:rgb(134.0552182495594,120.9447817504406,120.9447817504406)'> lost</span><span style='background-color:rgb(122.54102289676666,132.45897710323334,132.45897710323334)'> in</span><span style='background-color:rgb(138.7263683974743,116.27363160252571,116.27363160252571)'> the</span><span style='background-color:rgb(134.63708102703094,120.36291897296906,120.36291897296906)'> rap</span><span style='background-color:rgb(129.68948811292648,125.31051188707352,125.31051188707352)'> ##ture</span><span style='background-color:rgb(132.3191674053669,122.6808325946331,122.6808325946331)'> of</span><span style='background-color:rgb(139.59080681204796,115.40919318795204,115.40919318795204)'> love</span><span style='background-color:rgb(130.3494442999363,124.6505557000637,124.6505557000637)'> based</span><span style='background-color:rgb(130.37161231040955,124.62838768959045,124.62838768959045)'> so</span><span style='background-color:rgb(137.1583293378353,117.84167066216469,117.84167066216469)'> deeply</span><span style='background-color:rgb(138.87294933199883,116.12705066800117,116.12705066800117)'> within</span><span style='background-color:rgb(140.15841275453568,114.84158724546432,114.84158724546432)'> their</span><span style='background-color:rgb(133.50664168596268,121.49335831403732,121.49335831403732)'> hearts</span><span style='background-color:rgb(124.71351832151413,130.28648167848587,130.28648167848587)'> that</span><span style='background-color:rgb(135.59880942106247,119.40119057893753,119.40119057893753)'> they</span><span style='background-color:rgb(139.29450631141663,115.70549368858337,115.70549368858337)'> had</span><span style='background-color:rgb(135.75622737407684,119.24377262592316,119.24377262592316)'> never</span><span style='background-color:rgb(131.81415930390358,123.18584069609642,123.18584069609642)'> seen</span><span style='background-color:rgb(131.13539412617683,123.86460587382317,123.86460587382317)'> it</span><span style='background-color:rgb(133.35477143526077,121.64522856473923,121.64522856473923)'> before</span><span style='background-color:rgb(146.41305953264236,108.58694046735764,108.58694046735764)'> .</span><span style='background-color:rgb(130.67922860383987,124.32077139616013,124.32077139616013)'> "</span><span style='background-color:rgb(118.18852365016937,136.81147634983063,136.81147634983063)'> what</span><span style='background-color:rgb(142.50476405024529,112.49523594975471,112.49523594975471)'> about</span><span style='background-color:rgb(145.48425257205963,109.51574742794037,109.51574742794037)'> roger</span><span style='background-color:rgb(135.4119050502777,119.58809494972229,119.58809494972229)'> ?</span><span style='background-color:rgb(143.11068713665009,111.88931286334991,111.88931286334991)'> "</span><span style='background-color:rgb(121.95623427629471,133.0437657237053,133.0437657237053)'> she</span><span style='background-color:rgb(144.03162091970444,110.96837908029556,110.96837908029556)'> asked</span><span style='background-color:rgb(154.84071850776672,100.15928149223328,100.15928149223328)'> ,</span><span style='background-color:rgb(150.9725260734558,104.02747392654419,104.02747392654419)'> knowing</span><span style='background-color:rgb(125.64952969551086,129.35047030448914,129.35047030448914)'> that</span><span style='background-color:rgb(159.3455895781517,95.6544104218483,95.6544104218483)'> the</span><span style='background-color:rgb(133.43104854226112,121.56895145773888,121.56895145773888)'> answer</span><span style='background-color:rgb(145.9639160335064,109.0360839664936,109.0360839664936)'> no</span><span style='background-color:rgb(117.72753238677979,137.27246761322021,137.27246761322021)'> longer</span><span style='background-color:rgb(117.43216663599014,137.56783336400986,137.56783336400986)'> mattered</span><span style='background-color:rgb(111.52178138494492,143.47821861505508,143.47821861505508)'> .</span><span style='background-color:rgb(122.26651042699814,132.73348957300186,132.73348957300186)'> [SEP]</span><span style='background-color:rgb(123.1787958741188,131.8212041258812,131.8212041258812)'> name</span><span style='background-color:rgb(75.37806004285812,179.62193995714188,179.62193995714188)'> few</span><span style='background-color:rgb(147.19997450709343,107.80002549290657,107.80002549290657)'> objects</span><span style='background-color:rgb(143.42519626021385,111.57480373978615,111.57480373978615)'> said</span><span style='background-color:rgb(135.34220159053802,119.65779840946198,119.65779840946198)'> to</span><span style='background-color:rgb(133.0127213895321,121.98727861046791,121.98727861046791)'> be</span><span style='background-color:rgb(114.97486889362335,140.02513110637665,140.02513110637665)'> in</span><span style='background-color:rgb(148.5510604083538,106.4489395916462,106.4489395916462)'> or</span><span style='background-color:rgb(133.619776815176,121.38022318482399,121.38022318482399)'> on</span><span style='background-color:rgb(136.02782920002937,118.97217079997063,118.97217079997063)'> allan</span><span style='background-color:rgb(155.26934310793877,99.73065689206123,99.73065689206123)'> '</span><span style='background-color:rgb(177.60538056492805,77.39461943507195,77.39461943507195)'> s</span><span style='background-color:rgb(98.37439566850662,156.62560433149338,156.62560433149338)'> desk</span><span style='background-color:rgb(155.05663812160492,99.94336187839508,99.94336187839508)'> |</span><span style='background-color:rgb(93.02367478609085,161.97632521390915,161.97632521390915)'> |</span><span style='background-color:rgb(0.0,255.0,255.0)'> erase</span> ##r<span style='background-color:rgb(231.040917634964,23.95908236503601,23.95908236503601)'> [SEP]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span><span style='background-color:rgb(132.72946938872337,122.27053061127663,122.27053061127663)'> [PAD]</span>

In [87]:
j = 0
grouped_attrs_intgrad = []
for i in tqdm(range(2)):
    intgrad_expln = intgrad_explainer(inputs[j][None], torch.tensor([i]))
    print('intgrad_expln.attributions', intgrad_expln.attributions.shape)
    grouped_attrs_intgrad.append(intgrad_expln.attributions)
print('torch.stack(grouped_attrs_intgrad)', torch.cat(grouped_attrs_intgrad).shape)
grouped_attrs_intgrad_aggr = torch.cat(grouped_attrs_intgrad).sum(-1)
grouped_attrs_intgrad_aggr, torch.argmax(grouped_attrs_intgrad_aggr)

  0%|          | 0/2 [00:00<?, ?it/s]

intgrad_expln.attributions torch.Size([1, 512])
intgrad_expln.attributions torch.Size([1, 512])
torch.stack(grouped_attrs_intgrad) torch.Size([2, 512])


(tensor([[-1.7239e-07],
         [-4.0127e-07]], device='cuda:0'),
 tensor(0, device='cuda:0'))

## GradCAM

In [28]:
!pip uninstall -y grad-cam

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Found existing installation: grad-cam 1.4.8
Uninstalling grad-cam-1.4.8:
  Successfully uninstalled grad-cam-1.4.8


In [16]:

import sys
sys.path.insert(0, '/shared_data0/weiqiuy/github/pytorch-grad-cam')
import pytorch_grad_cam
pytorch_grad_cam.__file__

'/shared_data0/weiqiuy/github/pytorch-grad-cam/pytorch_grad_cam/__init__.py'

In [74]:
import numpy as np
import torch
import ttach as tta
from typing import Callable, List, Tuple

from pytorch_grad_cam.activations_and_gradients import ActivationsAndGradients
from pytorch_grad_cam.utils.svd_on_activations import get_2d_projection
from pytorch_grad_cam.utils.image import scale_cam_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget

class BaseCAMText:
    def __init__(self,
                 model: torch.nn.Module,
                 target_layers: List[torch.nn.Module],
                 use_cuda: bool = False,
                 reshape_transform: Callable = None,
                 compute_input_gradient: bool = False,
                 uses_gradients: bool = True) -> None:
        self.model = model.eval()
        self.target_layers = target_layers
        self.cuda = use_cuda
        if self.cuda:
            self.model = model.cuda()
        self.reshape_transform = reshape_transform
        self.compute_input_gradient = compute_input_gradient
        self.uses_gradients = uses_gradients
        self.activations_and_grads = ActivationsAndGradients(
            self.model, target_layers, reshape_transform)

    """ Get a vector of weights for every channel in the target layer.
        Methods that return weights channels,
        will typically need to only implement this function. """

    def get_cam_weights(self,
                        input_tensor: torch.Tensor,
                        target_layers: List[torch.nn.Module],
                        targets: List[torch.nn.Module],
                        activations: torch.Tensor,
                        grads: torch.Tensor) -> np.ndarray:
        raise Exception("Not Implemented")

    def get_cam_image(self,
                      input_tensor: torch.Tensor,
                      target_layer: torch.nn.Module,
                      targets: List[torch.nn.Module],
                      activations: torch.Tensor,
                      grads: torch.Tensor,
                      eigen_smooth: bool = False) -> np.ndarray:

        weights = self.get_cam_weights(input_tensor,
                                       target_layer,
                                       targets,
                                       activations,
                                       grads)
        import pdb; pdb.set_trace()
        if len(activations.shape) == 3:
            weighted_activations = weights[:, :, None] * activations
        elif len(activations.shape) == 2:
            weighted_activations = weights[:, None] * activations
        else:
            raise Exception("Unsupported shape for activations: " + activations.shape)
        # cam = weighted_activations.sum(axis=-1)
        # if eigen_smooth:
        #     cam = get_2d_projection(weighted_activations)
        # else:
        #     cam = weighted_activations.sum(axis=1)
        cam = weighted_activations[:,0]
        return cam

    def forward(self,
                input_tensor: torch.Tensor,
                targets: List[torch.nn.Module],
                eigen_smooth: bool = False) -> np.ndarray:

        if self.cuda:
            input_tensor = input_tensor.cuda()

        if self.compute_input_gradient:
            input_tensor = torch.autograd.Variable(input_tensor,
                                                   requires_grad=True)

        outputs = self.activations_and_grads(input_tensor)
        if targets is None:
            target_categories = np.argmax(outputs.cpu().data.numpy(), axis=-1)
            targets = [ClassifierOutputTarget(
                category) for category in target_categories]

        if self.uses_gradients:
            self.model.zero_grad()
            loss = sum([target(output)
                       for target, output in zip(targets, outputs)])
            loss.backward(retain_graph=True)

        # In most of the saliency attribution papers, the saliency is
        # computed with a single target layer.
        # Commonly it is the last convolutional layer.
        # Here we support passing a list with multiple target layers.
        # It will compute the saliency image for every image,
        # and then aggregate them (with a default mean aggregation).
        # This gives you more flexibility in case you just want to
        # use all conv layers for example, all Batchnorm layers,
        # or something else.
        cam_per_layer = self.compute_cam_per_layer(input_tensor,
                                                   targets,
                                                   eigen_smooth)
        return self.aggregate_multi_layers(cam_per_layer)

    def get_target_width_height(self,
                                input_tensor: torch.Tensor) -> Tuple[int, int]:
        width, height = input_tensor.size(-1), input_tensor.size(-2)
        return width, height

    def compute_cam_per_layer(
            self,
            input_tensor: torch.Tensor,
            targets: List[torch.nn.Module],
            eigen_smooth: bool) -> np.ndarray:
        activations_list = [a.cpu().data.numpy()
                            for a in self.activations_and_grads.activations]
        grads_list = [g.cpu().data.numpy()
                      for g in self.activations_and_grads.gradients]
        target_size = self.get_target_width_height(input_tensor)

        cam_per_target_layer = []
        # Loop over the saliency image from every layer
        for i in range(len(self.target_layers)):
            target_layer = self.target_layers[i]
            layer_activations = None
            layer_grads = None
            if i < len(activations_list):
                layer_activations = activations_list[i]
            if i < len(grads_list):
                layer_grads = grads_list[i]

            cam = self.get_cam_image(input_tensor,
                                     target_layer,
                                     targets,
                                     layer_activations,
                                     layer_grads,
                                     eigen_smooth)
            cam = np.maximum(cam, 0)
            scaled = scale_cam_image(cam, target_size)
            cam_per_target_layer.append(scaled[:, None, :])

        return cam_per_target_layer

    def aggregate_multi_layers(
            self,
            cam_per_target_layer: np.ndarray) -> np.ndarray:
        cam_per_target_layer = np.concatenate(cam_per_target_layer, axis=1)
        cam_per_target_layer = np.maximum(cam_per_target_layer, 0)
        result = np.mean(cam_per_target_layer, axis=1)
        return scale_cam_image(result)

    def forward_augmentation_smoothing(self,
                                       input_tensor: torch.Tensor,
                                       targets: List[torch.nn.Module],
                                       eigen_smooth: bool = False) -> np.ndarray:
        transforms = tta.Compose(
            [
                tta.HorizontalFlip(),
                tta.Multiply(factors=[0.9, 1, 1.1]),
            ]
        )
        cams = []
        for transform in transforms:
            augmented_tensor = transform.augment_image(input_tensor)
            cam = self.forward(augmented_tensor,
                               targets,
                               eigen_smooth)

            # The ttach library expects a tensor of size BxCxHxW
            cam = cam[:, None, :, :]
            cam = torch.from_numpy(cam)
            cam = transform.deaugment_mask(cam)

            # Back to numpy float32, HxW
            cam = cam.numpy()
            cam = cam[:, 0, :, :]
            cams.append(cam)

        cam = np.mean(np.float32(cams), axis=0)
        return cam

    def __call__(self,
                 input_tensor: torch.Tensor,
                 targets: List[torch.nn.Module] = None,
                 aug_smooth: bool = False,
                 eigen_smooth: bool = False) -> np.ndarray:

        # Smooth the CAM result with test time augmentation
        if aug_smooth is True:
            return self.forward_augmentation_smoothing(
                input_tensor, targets, eigen_smooth)

        return self.forward(input_tensor,
                            targets, eigen_smooth)

    def __del__(self):
        self.activations_and_grads.release()

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_value, exc_tb):
        self.activations_and_grads.release()
        if isinstance(exc_value, IndexError):
            # Handle IndexError here...
            print(
                f"An exception occurred in CAM with block: {exc_type}. Message: {exc_value}")
            return True


In [75]:
from exlib.explainers.common import *
from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget, ClassifierOutputSoftmaxTarget

class WrappedModelGradCAM(torch.nn.Module):
    def __init__(self, model): 
        super(WrappedModelGradCAM, self).__init__()
        self.model = model
        for name, param in self.model.named_parameters():
            param.requires_grad = True
            
    def forward(self, x):
        return self.model(x)

class GradCAMCls(FeatureAttrMethod):
    def __init__(self, model, target_layers, reshape_transform=None):
        
        model = WrappedModelGradCAM(model)

        super().__init__(model)
        
        self.target_layers = target_layers
        with torch.enable_grad():
            self.grad_cam = GradCAM(model=model, target_layers=self.target_layers,
                                    reshape_transform=reshape_transform,
                                    use_cuda=True if torch.cuda.is_available() else False)

    def forward(self, X, label=None, target_func=ClassifierOutputSoftmaxTarget):
        with torch.enable_grad():
            grad_cam_result = self.grad_cam(input_tensor=X, targets=[target_func(label) for label in label])
            grad_cam_result = torch.tensor(grad_cam_result)

        return FeatureAttrOutput(grad_cam_result.unsqueeze(1), grad_cam_result)

    
class GradCAMText(BaseCAMText):
    def __init__(self, model, target_layers, use_cuda=False,
                 reshape_transform=None):
        super(
            GradCAMText,
            self).__init__(
            model,
            target_layers,
            use_cuda,
            reshape_transform)

    def get_cam_weights(self,
                        input_tensor,
                        target_layer,
                        target_category,
                        activations,
                        grads):
        return np.mean(grads, axis=(-1))

class GradCAMTextCls(FeatureAttrMethod):
    def __init__(self, model, target_layers, reshape_transform=None):
        model = WrappedModelGradCAM(model)

        super().__init__(model)
        
        self.target_layers = target_layers
        with torch.enable_grad():
            self.grad_cam = GradCAMText(model=model, target_layers=self.target_layers,
                                    reshape_transform=reshape_transform,
                                    use_cuda=True if torch.cuda.is_available() else False)
 
    def forward(self, X, label=None, target_func=ClassifierOutputSoftmaxTarget):
        grad_cam_results = []
        for i in range(len(label)):
            with torch.enable_grad():
                grad_cam_result = self.grad_cam(input_tensor=X[i:i+1], 
                                                targets=[target_func(label[i:i+1])])
                # import pdb; pdb.set_trace()
                grad_cam_result = torch.tensor(grad_cam_result)
                grad_cam_results.append(grad_cam_result)
        grad_cam_results = torch.cat(grad_cam_results)

        return FeatureAttrOutput(grad_cam_results.squeeze(1), {})

In [76]:
gradcam_explainer = GradCAMTextCls(original_model2, 
                                    [original_model2.model.bert.encoder.layer[-1].output.LayerNorm]).to(device)

gradcam_expln = gradcam_explainer(inputs, labels)

> /tmp/ipykernel_3796/2149128867.py(56)get_cam_image()
     54                                        grads)
     55         import pdb; pdb.set_trace()
---> 56         if len(activations.shape) == 3:
     57             weighted_activations = weights[:, :, None] * activations
     58         elif len(activations.shape) == 2:



ipdb>  c


> /tmp/ipykernel_3796/2149128867.py(56)get_cam_image()
     54                                        grads)
     55         import pdb; pdb.set_trace()
---> 56         if len(activations.shape) == 3:
     57             weighted_activations = weights[:, :, None] * activations
     58         elif len(activations.shape) == 2:



ipdb>  c


In [77]:
gradcam_expln.attributions.shape

torch.Size([2, 512])

In [78]:
gradcam_expln.attributions, gradcam_expln.attributions.sum()

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor(0.))

In [47]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

def show_attrs(tokens, masks, normalize=True):
    if normalize:
        masks_max = masks.max(-1).values
        masks_min = masks.min(-1).values
        # import pdb; pdb.set_trace()
        masks = (masks - masks_min) / (masks_max - masks_min)
    print_str = ""
    for i in range(len(tokens)):
        expln_val = masks[i].item()
        if expln_val == 0:
            print_str += f" {tokens[i]}"
        else:
            print_str += f"<span style='background-color:rgb({255 * (1-expln_val)},{255 * expln_val},{255 * expln_val})'> {tokens[i]}</span>"
    return print_str

tokens = processor.convert_ids_to_tokens(inputs[0])
# tokens = shap_expln.explainer_output.data[0]
printmd(show_attrs(tokens, gradcam_expln.attributions[0]))

<span style='background-color:rgb(nan,nan,nan)'> [CLS]</span><span style='background-color:rgb(nan,nan,nan)'> allan</span><span style='background-color:rgb(nan,nan,nan)'> sat</span><span style='background-color:rgb(nan,nan,nan)'> down</span><span style='background-color:rgb(nan,nan,nan)'> at</span><span style='background-color:rgb(nan,nan,nan)'> his</span><span style='background-color:rgb(nan,nan,nan)'> desk</span><span style='background-color:rgb(nan,nan,nan)'> and</span><span style='background-color:rgb(nan,nan,nan)'> pulled</span><span style='background-color:rgb(nan,nan,nan)'> the</span><span style='background-color:rgb(nan,nan,nan)'> chair</span><span style='background-color:rgb(nan,nan,nan)'> in</span><span style='background-color:rgb(nan,nan,nan)'> close</span><span style='background-color:rgb(nan,nan,nan)'> .</span><span style='background-color:rgb(nan,nan,nan)'> opening</span><span style='background-color:rgb(nan,nan,nan)'> a</span><span style='background-color:rgb(nan,nan,nan)'> side</span><span style='background-color:rgb(nan,nan,nan)'> drawer</span><span style='background-color:rgb(nan,nan,nan)'> ,</span><span style='background-color:rgb(nan,nan,nan)'> he</span><span style='background-color:rgb(nan,nan,nan)'> took</span><span style='background-color:rgb(nan,nan,nan)'> out</span><span style='background-color:rgb(nan,nan,nan)'> a</span><span style='background-color:rgb(nan,nan,nan)'> piece</span><span style='background-color:rgb(nan,nan,nan)'> of</span><span style='background-color:rgb(nan,nan,nan)'> paper</span><span style='background-color:rgb(nan,nan,nan)'> and</span><span style='background-color:rgb(nan,nan,nan)'> his</span><span style='background-color:rgb(nan,nan,nan)'> ink</span><span style='background-color:rgb(nan,nan,nan)'> ##pot</span><span style='background-color:rgb(nan,nan,nan)'> .</span><span style='background-color:rgb(nan,nan,nan)'> after</span><span style='background-color:rgb(nan,nan,nan)'> filling</span><span style='background-color:rgb(nan,nan,nan)'> his</span><span style='background-color:rgb(nan,nan,nan)'> pen</span><span style='background-color:rgb(nan,nan,nan)'> ,</span><span style='background-color:rgb(nan,nan,nan)'> allan</span><span style='background-color:rgb(nan,nan,nan)'> looked</span><span style='background-color:rgb(nan,nan,nan)'> at</span><span style='background-color:rgb(nan,nan,nan)'> his</span><span style='background-color:rgb(nan,nan,nan)'> paper</span><span style='background-color:rgb(nan,nan,nan)'> in</span><span style='background-color:rgb(nan,nan,nan)'> the</span><span style='background-color:rgb(nan,nan,nan)'> orange</span><span style='background-color:rgb(nan,nan,nan)'> glow</span><span style='background-color:rgb(nan,nan,nan)'> from</span><span style='background-color:rgb(nan,nan,nan)'> the</span><span style='background-color:rgb(nan,nan,nan)'> lantern</span><span style='background-color:rgb(nan,nan,nan)'> set</span><span style='background-color:rgb(nan,nan,nan)'> back</span><span style='background-color:rgb(nan,nan,nan)'> in</span><span style='background-color:rgb(nan,nan,nan)'> the</span><span style='background-color:rgb(nan,nan,nan)'> desk</span><span style='background-color:rgb(nan,nan,nan)'> '</span><span style='background-color:rgb(nan,nan,nan)'> s</span><span style='background-color:rgb(nan,nan,nan)'> right</span><span style='background-color:rgb(nan,nan,nan)'> -</span><span style='background-color:rgb(nan,nan,nan)'> hand</span><span style='background-color:rgb(nan,nan,nan)'> corner</span><span style='background-color:rgb(nan,nan,nan)'> .</span><span style='background-color:rgb(nan,nan,nan)'> his</span><span style='background-color:rgb(nan,nan,nan)'> pen</span><span style='background-color:rgb(nan,nan,nan)'> cast</span><span style='background-color:rgb(nan,nan,nan)'> a</span><span style='background-color:rgb(nan,nan,nan)'> forbid</span><span style='background-color:rgb(nan,nan,nan)'> ##ding</span><span style='background-color:rgb(nan,nan,nan)'> line</span><span style='background-color:rgb(nan,nan,nan)'> of</span><span style='background-color:rgb(nan,nan,nan)'> shadow</span><span style='background-color:rgb(nan,nan,nan)'> slant</span><span style='background-color:rgb(nan,nan,nan)'> ##ing</span><span style='background-color:rgb(nan,nan,nan)'> across</span><span style='background-color:rgb(nan,nan,nan)'> the</span><span style='background-color:rgb(nan,nan,nan)'> page</span><span style='background-color:rgb(nan,nan,nan)'> ,</span><span style='background-color:rgb(nan,nan,nan)'> echoing</span><span style='background-color:rgb(nan,nan,nan)'> the</span><span style='background-color:rgb(nan,nan,nan)'> ink</span><span style='background-color:rgb(nan,nan,nan)'> ##y</span><span style='background-color:rgb(nan,nan,nan)'> darkness</span><span style='background-color:rgb(nan,nan,nan)'> crouch</span><span style='background-color:rgb(nan,nan,nan)'> ##ing</span><span style='background-color:rgb(nan,nan,nan)'> in</span><span style='background-color:rgb(nan,nan,nan)'> the</span><span style='background-color:rgb(nan,nan,nan)'> edges</span><span style='background-color:rgb(nan,nan,nan)'> of</span><span style='background-color:rgb(nan,nan,nan)'> the</span><span style='background-color:rgb(nan,nan,nan)'> lantern</span><span style='background-color:rgb(nan,nan,nan)'> '</span><span style='background-color:rgb(nan,nan,nan)'> s</span><span style='background-color:rgb(nan,nan,nan)'> struggling</span><span style='background-color:rgb(nan,nan,nan)'> glow</span><span style='background-color:rgb(nan,nan,nan)'> .</span><span style='background-color:rgb(nan,nan,nan)'> the</span><span style='background-color:rgb(nan,nan,nan)'> only</span><span style='background-color:rgb(nan,nan,nan)'> other</span><span style='background-color:rgb(nan,nan,nan)'> illumination</span><span style='background-color:rgb(nan,nan,nan)'> came</span><span style='background-color:rgb(nan,nan,nan)'> from</span><span style='background-color:rgb(nan,nan,nan)'> a</span><span style='background-color:rgb(nan,nan,nan)'> lu</span><span style='background-color:rgb(nan,nan,nan)'> ##rid</span><span style='background-color:rgb(nan,nan,nan)'> moonlight</span><span style='background-color:rgb(nan,nan,nan)'> filtered</span><span style='background-color:rgb(nan,nan,nan)'> through</span><span style='background-color:rgb(nan,nan,nan)'> thin</span><span style='background-color:rgb(nan,nan,nan)'> branches</span><span style='background-color:rgb(nan,nan,nan)'> and</span><span style='background-color:rgb(nan,nan,nan)'> clouds</span><span style='background-color:rgb(nan,nan,nan)'> ,</span><span style='background-color:rgb(nan,nan,nan)'> casting</span><span style='background-color:rgb(nan,nan,nan)'> its</span><span style='background-color:rgb(nan,nan,nan)'> bone</span><span style='background-color:rgb(nan,nan,nan)'> -</span><span style='background-color:rgb(nan,nan,nan)'> pale</span><span style='background-color:rgb(nan,nan,nan)'> glow</span><span style='background-color:rgb(nan,nan,nan)'> onto</span><span style='background-color:rgb(nan,nan,nan)'> the</span><span style='background-color:rgb(nan,nan,nan)'> pine</span><span style='background-color:rgb(nan,nan,nan)'> floor</span><span style='background-color:rgb(nan,nan,nan)'> ##boards</span><span style='background-color:rgb(nan,nan,nan)'> .</span><span style='background-color:rgb(nan,nan,nan)'> allan</span><span style='background-color:rgb(nan,nan,nan)'> unfolded</span><span style='background-color:rgb(nan,nan,nan)'> another</span><span style='background-color:rgb(nan,nan,nan)'> page</span><span style='background-color:rgb(nan,nan,nan)'> ,</span><span style='background-color:rgb(nan,nan,nan)'> this</span><span style='background-color:rgb(nan,nan,nan)'> one</span><span style='background-color:rgb(nan,nan,nan)'> crowded</span><span style='background-color:rgb(nan,nan,nan)'> with</span><span style='background-color:rgb(nan,nan,nan)'> ranks</span><span style='background-color:rgb(nan,nan,nan)'> of</span><span style='background-color:rgb(nan,nan,nan)'> letters</span><span style='background-color:rgb(nan,nan,nan)'> in</span><span style='background-color:rgb(nan,nan,nan)'> tight</span><span style='background-color:rgb(nan,nan,nan)'> formation</span><span style='background-color:rgb(nan,nan,nan)'> from</span><span style='background-color:rgb(nan,nan,nan)'> left</span><span style='background-color:rgb(nan,nan,nan)'> to</span><span style='background-color:rgb(nan,nan,nan)'> right</span><span style='background-color:rgb(nan,nan,nan)'> .</span><span style='background-color:rgb(nan,nan,nan)'> the</span><span style='background-color:rgb(nan,nan,nan)'> lines</span><span style='background-color:rgb(nan,nan,nan)'> of</span><span style='background-color:rgb(nan,nan,nan)'> letters</span><span style='background-color:rgb(nan,nan,nan)'> stepped</span><span style='background-color:rgb(nan,nan,nan)'> into</span><span style='background-color:rgb(nan,nan,nan)'> their</span><span style='background-color:rgb(nan,nan,nan)'> divisions</span><span style='background-color:rgb(nan,nan,nan)'> ,</span><span style='background-color:rgb(nan,nan,nan)'> in</span><span style='background-color:rgb(nan,nan,nan)'> the</span><span style='background-color:rgb(nan,nan,nan)'> shape</span><span style='background-color:rgb(nan,nan,nan)'> of</span><span style='background-color:rgb(nan,nan,nan)'> a</span><span style='background-color:rgb(nan,nan,nan)'> story</span><span style='background-color:rgb(nan,nan,nan)'> '</span><span style='background-color:rgb(nan,nan,nan)'> s</span><span style='background-color:rgb(nan,nan,nan)'> outline</span><span style='background-color:rgb(nan,nan,nan)'> :</span><span style='background-color:rgb(nan,nan,nan)'> the</span><span style='background-color:rgb(nan,nan,nan)'> loose</span><span style='background-color:rgb(nan,nan,nan)'> ,</span><span style='background-color:rgb(nan,nan,nan)'> dry</span><span style='background-color:rgb(nan,nan,nan)'> skeleton</span><span style='background-color:rgb(nan,nan,nan)'> of</span><span style='background-color:rgb(nan,nan,nan)'> a</span><span style='background-color:rgb(nan,nan,nan)'> tale</span><span style='background-color:rgb(nan,nan,nan)'> lay</span><span style='background-color:rgb(nan,nan,nan)'> exposed</span><span style='background-color:rgb(nan,nan,nan)'> beneath</span><span style='background-color:rgb(nan,nan,nan)'> their</span><span style='background-color:rgb(nan,nan,nan)'> feet</span><span style='background-color:rgb(nan,nan,nan)'> ,</span><span style='background-color:rgb(nan,nan,nan)'> awaiting</span><span style='background-color:rgb(nan,nan,nan)'> tend</span><span style='background-color:rgb(nan,nan,nan)'> ##ons</span><span style='background-color:rgb(nan,nan,nan)'> ,</span><span style='background-color:rgb(nan,nan,nan)'> muscle</span><span style='background-color:rgb(nan,nan,nan)'> and</span><span style='background-color:rgb(nan,nan,nan)'> blushing</span><span style='background-color:rgb(nan,nan,nan)'> skin</span><span style='background-color:rgb(nan,nan,nan)'> .</span><span style='background-color:rgb(nan,nan,nan)'> allan</span><span style='background-color:rgb(nan,nan,nan)'> reviewed</span><span style='background-color:rgb(nan,nan,nan)'> the</span><span style='background-color:rgb(nan,nan,nan)'> troops</span><span style='background-color:rgb(nan,nan,nan)'> ,</span><span style='background-color:rgb(nan,nan,nan)'> all</span><span style='background-color:rgb(nan,nan,nan)'> prepared</span><span style='background-color:rgb(nan,nan,nan)'> to</span><span style='background-color:rgb(nan,nan,nan)'> di</span><span style='background-color:rgb(nan,nan,nan)'> ##se</span><span style='background-color:rgb(nan,nan,nan)'> ##mba</span><span style='background-color:rgb(nan,nan,nan)'> ##rk</span><span style='background-color:rgb(nan,nan,nan)'> ,</span><span style='background-color:rgb(nan,nan,nan)'> their</span><span style='background-color:rgb(nan,nan,nan)'> task</span><span style='background-color:rgb(nan,nan,nan)'> to</span><span style='background-color:rgb(nan,nan,nan)'> form</span><span style='background-color:rgb(nan,nan,nan)'> the</span><span style='background-color:rgb(nan,nan,nan)'> tale</span><span style='background-color:rgb(nan,nan,nan)'> of</span><span style='background-color:rgb(nan,nan,nan)'> a</span><span style='background-color:rgb(nan,nan,nan)'> young</span><span style='background-color:rgb(nan,nan,nan)'> man</span><span style='background-color:rgb(nan,nan,nan)'> returning</span><span style='background-color:rgb(nan,nan,nan)'> home</span><span style='background-color:rgb(nan,nan,nan)'> from</span><span style='background-color:rgb(nan,nan,nan)'> life</span><span style='background-color:rgb(nan,nan,nan)'> abroad</span><span style='background-color:rgb(nan,nan,nan)'> to</span><span style='background-color:rgb(nan,nan,nan)'> find</span><span style='background-color:rgb(nan,nan,nan)'> his</span><span style='background-color:rgb(nan,nan,nan)'> childhood</span><span style='background-color:rgb(nan,nan,nan)'> friend</span><span style='background-color:rgb(nan,nan,nan)'> a</span><span style='background-color:rgb(nan,nan,nan)'> bride</span><span style='background-color:rgb(nan,nan,nan)'> to</span><span style='background-color:rgb(nan,nan,nan)'> -</span><span style='background-color:rgb(nan,nan,nan)'> be</span><span style='background-color:rgb(nan,nan,nan)'> ,</span><span style='background-color:rgb(nan,nan,nan)'> thus</span><span style='background-color:rgb(nan,nan,nan)'> upset</span><span style='background-color:rgb(nan,nan,nan)'> ##ting</span><span style='background-color:rgb(nan,nan,nan)'> the</span><span style='background-color:rgb(nan,nan,nan)'> apple</span><span style='background-color:rgb(nan,nan,nan)'> cart</span><span style='background-color:rgb(nan,nan,nan)'> of</span><span style='background-color:rgb(nan,nan,nan)'> his</span><span style='background-color:rgb(nan,nan,nan)'> life</span><span style='background-color:rgb(nan,nan,nan)'> '</span><span style='background-color:rgb(nan,nan,nan)'> s</span><span style='background-color:rgb(nan,nan,nan)'> plan</span><span style='background-color:rgb(nan,nan,nan)'> ,</span><span style='background-color:rgb(nan,nan,nan)'> clarified</span><span style='background-color:rgb(nan,nan,nan)'> –</span><span style='background-color:rgb(nan,nan,nan)'> of</span><span style='background-color:rgb(nan,nan,nan)'> course</span><span style='background-color:rgb(nan,nan,nan)'> –</span><span style='background-color:rgb(nan,nan,nan)'> by</span><span style='background-color:rgb(nan,nan,nan)'> his</span><span style='background-color:rgb(nan,nan,nan)'> very</span><span style='background-color:rgb(nan,nan,nan)'> time</span><span style='background-color:rgb(nan,nan,nan)'> away</span><span style='background-color:rgb(nan,nan,nan)'> from</span><span style='background-color:rgb(nan,nan,nan)'> her</span><span style='background-color:rgb(nan,nan,nan)'> he</span><span style='background-color:rgb(nan,nan,nan)'> loved</span><span style='background-color:rgb(nan,nan,nan)'> best</span><span style='background-color:rgb(nan,nan,nan)'> .</span><span style='background-color:rgb(nan,nan,nan)'> although</span><span style='background-color:rgb(nan,nan,nan)'> the</span><span style='background-color:rgb(nan,nan,nan)'> concept</span><span style='background-color:rgb(nan,nan,nan)'> was</span><span style='background-color:rgb(nan,nan,nan)'> a</span><span style='background-color:rgb(nan,nan,nan)'> simple</span><span style='background-color:rgb(nan,nan,nan)'> one</span><span style='background-color:rgb(nan,nan,nan)'> ,</span><span style='background-color:rgb(nan,nan,nan)'> allan</span><span style='background-color:rgb(nan,nan,nan)'> thought</span><span style='background-color:rgb(nan,nan,nan)'> it</span><span style='background-color:rgb(nan,nan,nan)'> had</span><span style='background-color:rgb(nan,nan,nan)'> potential</span><span style='background-color:rgb(nan,nan,nan)'> .</span><span style='background-color:rgb(nan,nan,nan)'> besides</span><span style='background-color:rgb(nan,nan,nan)'> ,</span><span style='background-color:rgb(nan,nan,nan)'> the</span><span style='background-color:rgb(nan,nan,nan)'> public</span><span style='background-color:rgb(nan,nan,nan)'> liked</span><span style='background-color:rgb(nan,nan,nan)'> a</span><span style='background-color:rgb(nan,nan,nan)'> good</span><span style='background-color:rgb(nan,nan,nan)'> ,</span><span style='background-color:rgb(nan,nan,nan)'> simple</span><span style='background-color:rgb(nan,nan,nan)'> romance</span><span style='background-color:rgb(nan,nan,nan)'> .</span><span style='background-color:rgb(nan,nan,nan)'> perhaps</span><span style='background-color:rgb(nan,nan,nan)'> this</span><span style='background-color:rgb(nan,nan,nan)'> will</span><span style='background-color:rgb(nan,nan,nan)'> be</span><span style='background-color:rgb(nan,nan,nan)'> more</span><span style='background-color:rgb(nan,nan,nan)'> sale</span><span style='background-color:rgb(nan,nan,nan)'> ##able</span><span style='background-color:rgb(nan,nan,nan)'> ,</span><span style='background-color:rgb(nan,nan,nan)'> he</span><span style='background-color:rgb(nan,nan,nan)'> thought</span><span style='background-color:rgb(nan,nan,nan)'> and</span><span style='background-color:rgb(nan,nan,nan)'> began</span><span style='background-color:rgb(nan,nan,nan)'> to</span><span style='background-color:rgb(nan,nan,nan)'> write</span><span style='background-color:rgb(nan,nan,nan)'> .</span><span style='background-color:rgb(nan,nan,nan)'> they</span><span style='background-color:rgb(nan,nan,nan)'> gazed</span><span style='background-color:rgb(nan,nan,nan)'> at</span><span style='background-color:rgb(nan,nan,nan)'> each</span><span style='background-color:rgb(nan,nan,nan)'> other</span><span style='background-color:rgb(nan,nan,nan)'> ,</span><span style='background-color:rgb(nan,nan,nan)'> lost</span><span style='background-color:rgb(nan,nan,nan)'> in</span><span style='background-color:rgb(nan,nan,nan)'> the</span><span style='background-color:rgb(nan,nan,nan)'> rap</span><span style='background-color:rgb(nan,nan,nan)'> ##ture</span><span style='background-color:rgb(nan,nan,nan)'> of</span><span style='background-color:rgb(nan,nan,nan)'> love</span><span style='background-color:rgb(nan,nan,nan)'> based</span><span style='background-color:rgb(nan,nan,nan)'> so</span><span style='background-color:rgb(nan,nan,nan)'> deeply</span><span style='background-color:rgb(nan,nan,nan)'> within</span><span style='background-color:rgb(nan,nan,nan)'> their</span><span style='background-color:rgb(nan,nan,nan)'> hearts</span><span style='background-color:rgb(nan,nan,nan)'> that</span><span style='background-color:rgb(nan,nan,nan)'> they</span><span style='background-color:rgb(nan,nan,nan)'> had</span><span style='background-color:rgb(nan,nan,nan)'> never</span><span style='background-color:rgb(nan,nan,nan)'> seen</span><span style='background-color:rgb(nan,nan,nan)'> it</span><span style='background-color:rgb(nan,nan,nan)'> before</span><span style='background-color:rgb(nan,nan,nan)'> .</span><span style='background-color:rgb(nan,nan,nan)'> "</span><span style='background-color:rgb(nan,nan,nan)'> what</span><span style='background-color:rgb(nan,nan,nan)'> about</span><span style='background-color:rgb(nan,nan,nan)'> roger</span><span style='background-color:rgb(nan,nan,nan)'> ?</span><span style='background-color:rgb(nan,nan,nan)'> "</span><span style='background-color:rgb(nan,nan,nan)'> she</span><span style='background-color:rgb(nan,nan,nan)'> asked</span><span style='background-color:rgb(nan,nan,nan)'> ,</span><span style='background-color:rgb(nan,nan,nan)'> knowing</span><span style='background-color:rgb(nan,nan,nan)'> that</span><span style='background-color:rgb(nan,nan,nan)'> the</span><span style='background-color:rgb(nan,nan,nan)'> answer</span><span style='background-color:rgb(nan,nan,nan)'> no</span><span style='background-color:rgb(nan,nan,nan)'> longer</span><span style='background-color:rgb(nan,nan,nan)'> mattered</span><span style='background-color:rgb(nan,nan,nan)'> .</span><span style='background-color:rgb(nan,nan,nan)'> [SEP]</span><span style='background-color:rgb(nan,nan,nan)'> name</span><span style='background-color:rgb(nan,nan,nan)'> few</span><span style='background-color:rgb(nan,nan,nan)'> objects</span><span style='background-color:rgb(nan,nan,nan)'> said</span><span style='background-color:rgb(nan,nan,nan)'> to</span><span style='background-color:rgb(nan,nan,nan)'> be</span><span style='background-color:rgb(nan,nan,nan)'> in</span><span style='background-color:rgb(nan,nan,nan)'> or</span><span style='background-color:rgb(nan,nan,nan)'> on</span><span style='background-color:rgb(nan,nan,nan)'> allan</span><span style='background-color:rgb(nan,nan,nan)'> '</span><span style='background-color:rgb(nan,nan,nan)'> s</span><span style='background-color:rgb(nan,nan,nan)'> desk</span><span style='background-color:rgb(nan,nan,nan)'> |</span><span style='background-color:rgb(nan,nan,nan)'> |</span><span style='background-color:rgb(nan,nan,nan)'> erase</span><span style='background-color:rgb(nan,nan,nan)'> ##r</span><span style='background-color:rgb(nan,nan,nan)'> [SEP]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span><span style='background-color:rgb(nan,nan,nan)'> [PAD]</span>

In [39]:
j = 0
grouped_attrs_gradcam = []
for i in tqdm(range(2)):
    gradcam_expln = gradcam_explainer(inputs[j][None], torch.tensor([i]))
    grouped_attrs_gradcam.append(gradcam_expln.attributions.sum(1).view(-1))
grouped_attrs_gradcam_aggr = torch.stack(grouped_attrs_gradcam).sum(-1)
grouped_attrs_gradcam_aggr, torch.argmax(grouped_attrs_gradcam_aggr)

  0%|          | 0/2 [00:00<?, ?it/s]

(tensor([25.0217, 90.4046]), tensor(1))

## MFABA

## PLS